# Coordinate Agents, API Integration, Agent Communication and Validate Coordination

#### Milestone 2: Week 3-4
1. Develop the Planning Module to generate and coordinate specialized agents
2. Implement API integration for contract upload and domain classification.
3. Design basic prompt templates for agent communication.
4. Validate inter-agent coordination using LangGraph.

#### 1. Load Agent Outputs

In [1]:
import warnings
warnings.filterwarnings("ignore")
import re
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import re
from datetime import datetime
from collections import defaultdict
from pinecone import Pinecone, ServerlessSpec
import requests


In [2]:
print("LOADING AGENT OUTPUTS")

LEGAL_AGENT_OUTPUT = "../Data/Results/Legal_Agent"
COMPLIANCE_AGENT_OUTPUT = "../Data/Results/Compliance_Agent"
FINANCE_AGENT_OUTPUT = "../Data/Results/Finance_Agent"
OPERATIONS_AGENT_OUTPUT = "../Data/Results/Operations_Agent"
COORDINATOR_OUTPUT = "../Data/Results/Coordinator"

os.makedirs(COORDINATOR_OUTPUT, exist_ok=True)

def load_latest_agent_output(output_dir, agent_name):
    
    if not os.path.exists(output_dir):
        print(f"  Directory not found: {output_dir}")
        return None, None
    
    json_files = [f for f in os.listdir(output_dir) if f.endswith('.json')]
    if not json_files:
        print(f"  No JSON files found in {output_dir}")
        return None, None
    
    base_files = [f for f in json_files 
                  if "ENHANCED" not in f.upper() 
                  and "student" not in f.lower()
                  and "consolidated" in f.lower()] 
    
    if not base_files:
        base_files = [f for f in json_files 
                      if "ENHANCED" not in f.upper() 
                      and "student" not in f.lower()]
    
    if not base_files:
        base_files = json_files  
    
    latest_file = max(base_files, key=lambda f: os.path.getctime(os.path.join(output_dir, f)))
    filepath = os.path.join(output_dir, latest_file)
    
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"  Loaded PRE-COMPUTED results: {latest_file}")
        return data, latest_file
    except Exception as e:
        print(f"  Error loading {latest_file}: {str(e)[:50]}")
        return None, None

print("\nLoading Agent Outputs")

agent_outputs = {}

print("\n1. Legal Agent")
legal_output, legal_file = load_latest_agent_output(LEGAL_AGENT_OUTPUT, "Legal")
if legal_output:
    contracts_count = len(legal_output.get("contracts", [legal_output])) if legal_output.get("contracts") else 1
    total_clauses = legal_output.get("metadata", {}).get("total_clauses_extracted", 
                     len(legal_output.get("output", {}).get("extracted_clauses", [])))
    
    agent_outputs["legal"] = {
        "data": legal_output,
        "file": legal_file,
        "contracts_count": contracts_count,
        "total_clauses": total_clauses,
        "source": "From Legal Agent"
    }
    print(f"     Contracts: {agent_outputs['legal']['contracts_count']}")
    print(f"     Total Clauses: {agent_outputs['legal']['total_clauses']}")
    print(f"     Source: {agent_outputs['legal']['source']}")

print("\n2. Compliance Agent")
compliance_output, compliance_file = load_latest_agent_output(COMPLIANCE_AGENT_OUTPUT, "Compliance")
if compliance_output:
    contracts_count = len(compliance_output.get("contracts", [compliance_output])) if compliance_output.get("contracts") else 1
    total_clauses = compliance_output.get("metadata", {}).get("total_clauses_extracted",
                     len(compliance_output.get("output", {}).get("extracted_clauses", [])))
    
    agent_outputs["compliance"] = {
        "data": compliance_output,
        "file": compliance_file,
        "contracts_count": contracts_count,
        "total_clauses": total_clauses,
        "source": "From Compliance Agent"
    }
    print(f"     Contracts: {agent_outputs['compliance']['contracts_count']}")
    print(f"     Total Clauses: {agent_outputs['compliance']['total_clauses']}")
    print(f"     Source: {agent_outputs['compliance']['source']}")

print("\n3. Finance Agent")
finance_output, finance_file = load_latest_agent_output(FINANCE_AGENT_OUTPUT, "Finance")
if finance_output:
    contracts_count = len(finance_output.get("contracts", [finance_output])) if finance_output.get("contracts") else 1
    total_clauses = finance_output.get("metadata", {}).get("total_clauses_extracted",
                     len(finance_output.get("output", {}).get("extracted_clauses", [])))
    
    agent_outputs["finance"] = {
        "data": finance_output,
        "file": finance_file,
        "contracts_count": contracts_count,
        "total_clauses": total_clauses,
        "source": "From Finance Agent"
    }
    print(f"     Contracts: {agent_outputs['finance']['contracts_count']}")
    print(f"     Total Clauses: {agent_outputs['finance']['total_clauses']}")
    print(f"     Source: {agent_outputs['finance']['source']}")

print("\n4. Operations Agent")
operations_output, operations_file = load_latest_agent_output(OPERATIONS_AGENT_OUTPUT, "Operations")
if operations_output:
    contracts_count = len(operations_output.get("contracts", [operations_output])) if operations_output.get("contracts") else 1
    total_clauses = operations_output.get("metadata", {}).get("total_clauses_extracted",
                     len(operations_output.get("output", {}).get("extracted_clauses", [])))
    
    agent_outputs["operations"] = {
        "data": operations_output,
        "file": operations_file,
        "contracts_count": contracts_count,
        "total_clauses": total_clauses,
        "source": "From Operations Agent"
    }
    print(f"     Contracts: {agent_outputs['operations']['contracts_count']}")
    print(f"     Total Clauses: {agent_outputs['operations']['total_clauses']}")
    print(f"     Source: {agent_outputs['operations']['source']}")

print(f"\nTotal Agents Loaded: {len(agent_outputs)}")
print(f"Total Pre-Computed Contracts: {sum(a['contracts_count'] for a in agent_outputs.values())}")
print(f"Total Pre-Computed Clauses: {sum(a['total_clauses'] for a in agent_outputs.values())}")

if len(agent_outputs) < 4:
    print("\nWARNING: Not all agents loaded. Some routing may fail.")
    print("   Run agents first to generate outputs, then run coordinator.")

LOADING AGENT OUTPUTS

Loading Agent Outputs

1. Legal Agent
  Loaded PRE-COMPUTED results: legal_agent_output.json
     Contracts: 1
     Total Clauses: 2
     Source: From Legal Agent

2. Compliance Agent
  Loaded PRE-COMPUTED results: compliance_agent_output.json
     Contracts: 1
     Total Clauses: 1
     Source: From Compliance Agent

3. Finance Agent
  Loaded PRE-COMPUTED results: finance_agent_output.json
     Contracts: 1
     Total Clauses: 3
     Source: From Finance Agent

4. Operations Agent
  Loaded PRE-COMPUTED results: operations_agent_output.json
     Contracts: 1
     Total Clauses: 2
     Source: From Operations Agent

Total Agents Loaded: 4
Total Pre-Computed Contracts: 4
Total Pre-Computed Clauses: 8


#### 1. Pinecone Setup and Agent Output Upload

In [23]:
print("PINECONE SETUP AND DATA UPLOAD")

PINECONE_API_KEY = 'pcsk_3ftgmC_GzUZkRCnxa2jmDu7TTjnGWjC3QaN8c2PcQ5KN5PUSyQaEmmcdGUGu2BLd4Y7TRn'
pc = Pinecone(api_key=PINECONE_API_KEY)

INDEX_NAME = 'contract-agents'
DIMENSION = 384

existing_indexes = [idx.name for idx in pc.list_indexes()]
if INDEX_NAME not in existing_indexes:
    print(f"Creating Pinecone index: {INDEX_NAME}")
    pc.create_index(
        name=INDEX_NAME,
        dimension=DIMENSION,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    print(f"Index '{INDEX_NAME}' created successfully")
else:
    print(f"Index '{INDEX_NAME}' already exists")

index = pc.Index(INDEX_NAME)
print(f"Connected to index: {INDEX_NAME}")

PINECONE SETUP AND DATA UPLOAD
Creating Pinecone index: contract-agents
Index 'contract-agents' created successfully
Connected to index: contract-agents


In [24]:
print("INSTALLING SENTENCE TRANSFORMERS FOR PROPER EMBEDDINGS")

try:
    from sentence_transformers import SentenceTransformer
    print("sentence-transformers already installed")
except ImportError:
    print("Installing sentence-transformers...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'sentence-transformers', '-q'])
    from sentence_transformers import SentenceTransformer
    print("sentence-transformers installed successfully")

print("\nLoading embedding model...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("Embedding model loaded: all-MiniLM-L6-v2 (384 dimensions)")

def create_embedding(text):
    embedding = embedding_model.encode(text, convert_to_numpy=True)
    return embedding.tolist()

print("Embedding function ready\n")

INSTALLING SENTENCE TRANSFORMERS FOR PROPER EMBEDDINGS
sentence-transformers already installed

Loading embedding model...
Embedding model loaded: all-MiniLM-L6-v2 (384 dimensions)
Embedding function ready



In [25]:
print("LOADING AGENT OUTPUT JSON FILES")

agent_file_paths = {
    'legal': '../Data/Results/Legal_Agent',
    'compliance': '../Data/Results/Compliance_Agent',
    'finance': '../Data/Results/Finance_Agent',
    'operations': '../Data/Results/Operations_Agent'
}

if not os.path.exists('../Data/Results/Legal_Agent'):
    print("Using files from current directory...")
    agent_file_paths = {
        'legal': '.',
        'compliance': '.',
        'finance': '.',
        'operations': '.'
    }

def find_agent_file(directory, agent_name):
    if not os.path.exists(directory):
        return None
    
    json_files = [f for f in os.listdir(directory) if f.endswith('.json')]
    
    for filename in json_files:
        if agent_name in filename.lower() and 'output' in filename.lower():
            return os.path.join(directory, filename)
    
    if json_files:
        return os.path.join(directory, json_files[0])
    
    return None

LOADING AGENT OUTPUT JSON FILES


In [26]:
print("CREATING VECTORS WITH SEMANTIC EMBEDDINGS")

vectors_to_upsert = []
agent_summary = {}

for agent_name, directory in agent_file_paths.items():
    agent_file = find_agent_file(directory, agent_name)
    
    if not agent_file:
        direct_file = f"{agent_name}_agent_output.json"
        if os.path.exists(direct_file):
            agent_file = direct_file
    
    if agent_file and os.path.exists(agent_file):
        try:
            with open(agent_file, 'r', encoding='utf-8') as f:
                agent_data = json.load(f)
            
            output_data = agent_data.get('output', {})
            clauses = output_data.get('extracted_clauses', [])
            risk_level = output_data.get('risk_level', 'unknown')
            confidence = output_data.get('confidence', 0.0)
            timestamp = agent_data.get('timestamp', '')
            
            for idx, clause in enumerate(clauses):
                if clause and len(clause.strip()) > 0:
                    vector_id = f"{agent_name}_{idx}"
                    
                    embedding = create_embedding(clause)
                    
                    metadata = {
                        'agent': agent_name,
                        'clause': clause[:1000],
                        'clause_full': clause,
                        'risk_level': risk_level,
                        'confidence': float(confidence),
                        'timestamp': timestamp,
                        'clause_index': idx
                    }
                    vectors_to_upsert.append((vector_id, embedding, metadata))
            
            agent_summary[agent_name] = {
                'file': os.path.basename(agent_file),
                'clauses': len(clauses),
                'risk_level': risk_level,
                'confidence': confidence
            }
            
            print(f"  {agent_name.upper()}: {len(clauses)} clauses from {os.path.basename(agent_file)}")
            
        except Exception as e:
            print(f"  {agent_name.upper()}: Error loading file - {str(e)}")
    else:
        print(f"  {agent_name.upper()}: File not found in {directory}")

CREATING VECTORS WITH SEMANTIC EMBEDDINGS
  LEGAL: 2 clauses from legal_agent_output.json
  COMPLIANCE: 1 clauses from compliance_agent_output.json
  FINANCE: 3 clauses from finance_agent_output.json
  OPERATIONS: 2 clauses from operations_agent_output.json


In [27]:
if vectors_to_upsert:
    print(f"UPLOADING {len(vectors_to_upsert)} VECTORS TO PINECONE")

    batch_size = 100
    for i in range(0, len(vectors_to_upsert), batch_size):
        batch = vectors_to_upsert[i:i+batch_size]
        index.upsert(vectors=batch)
        print(f"  Uploaded batch {i//batch_size + 1}/{(len(vectors_to_upsert)-1)//batch_size + 1}")
    
    print(f"Successfully uploaded {len(vectors_to_upsert)} vectors to Pinecone")
    
    import time
    time.sleep(2)  
    stats = index.describe_index_stats()
    
    print(f"PINECONE INDEX STATS")
    print(f"  Total vectors: {stats.total_vector_count}")
    print(f"  Dimension: {stats.dimension}")
    print(f"  Index fullness: {stats.index_fullness}")
    
    print(f"AGENT DATA SUMMARY")
    for agent, info in agent_summary.items():
        print(f"\n{agent.upper()}:")
        print(f"  File: {info['file']}")
        print(f"  Clauses: {info['clauses']}")
        print(f"  Risk Level: {info['risk_level']}")
        print(f"  Confidence: {info['confidence']}")
else:
    print(f"\n{'='*80}")
    print("ERROR: No vectors to upload!")
    print(f"{'='*80}")
    print("Please ensure agent output JSON files are available.")
    print(f"\nSearched in:")
    for agent, path in agent_file_paths.items():
        print(f"  {agent}: {path}")

print("PINECONE SETUP AND DATA UPLOAD COMPLETE!")

UPLOADING 8 VECTORS TO PINECONE
  Uploaded batch 1/1
Successfully uploaded 8 vectors to Pinecone
PINECONE INDEX STATS
  Total vectors: 8
  Dimension: 384
  Index fullness: 0.0
AGENT DATA SUMMARY

LEGAL:
  File: legal_agent_output.json
  Clauses: 2
  Risk Level: low
  Confidence: 0.85

COMPLIANCE:
  File: compliance_agent_output.json
  Clauses: 1
  Risk Level: high
  Confidence: 1.0

FINANCE:
  File: finance_agent_output.json
  Clauses: 3
  Risk Level: medium
  Confidence: 0.7

OPERATIONS:
  File: operations_agent_output.json
  Clauses: 2
  Risk Level: medium
  Confidence: 0.8
PINECONE SETUP AND DATA UPLOAD COMPLETE!


##### All the Agents are loaded - along with the outputs

#### 2. Defining Coordinator Routing Rules

In [106]:
print("DYNAMIC COORDINATOR WITH GEMMA-2-9B-IT")

OLLAMA_MODEL = "gemma-2-9b-it"
OLLAMA_URL = "http://localhost:11434/api/generate"

def query_ollama(prompt, model=OLLAMA_MODEL):
    try:
        response = requests.post(
            OLLAMA_URL,
            json={
                "model": model,
                "prompt": prompt,
                "stream": False
            },
            timeout=30
        )
        if response.status_code == 200:
            return response.json().get('response', '')
        else:
            return None
    except Exception as e:
        print(f"Ollama error: {str(e)}")
        return None

def dynamic_route_query(query):
    
    coordinator_prompt = f"""You are a contract analysis coordinator. Given a user query about a contract, 
determine which specialized agents should analyze it.

Available agents:
- Legal: termination, liability, jurisdiction, governing law, indemnity
- Compliance: data protection, GDPR, privacy, audit, regulatory requirements
- Finance: payment terms, fees, penalties, costs, invoicing
- Operations: deliverables, timelines, SLAs, performance, milestones

User Query: "{query}"

Respond ONLY with a JSON object in this exact format:
{{
  "agents": ["agent1", "agent2"],
  "reasoning": "brief explanation"
}}

JSON Response:"""
    
    response = query_ollama(coordinator_prompt)
    
    if response:
        try:
            import re
            json_match = re.search(r'\{[^}]+\}', response, re.DOTALL)
            if json_match:
                result = json.loads(json_match.group())
                return {
                    'agents': result.get('agents', []),
                    'reasoning': result.get('reasoning', ''),
                    'approach': 'dynamic_llm_coordinator'
                }
        except:
            pass
    
    return {
        'agents': ['legal', 'compliance', 'finance', 'operations'],
        'reasoning': 'Fallback: routing to all agents',
        'approach': 'fallback'
    }

print("\nDynamic Coordinator initialized")
print(f"Model: {OLLAMA_MODEL}")
print(f"Ollama URL: {OLLAMA_URL}")
print("\nCoordinator will use LLM reasoning to route queries")


DYNAMIC COORDINATOR WITH GEMMA-2-9B-IT

Dynamic Coordinator initialized
Model: gemma-2-9b-it
Ollama URL: http://localhost:11434/api/generate

Coordinator will use LLM reasoning to route queries


##### Keywords for Each Agent
1. Legal - termination, terminate, cancel, cancellation, governing law, jurisdiction, venue, applicable law, liability, liable, damages, limitation of liability 
2. Compliance - gdpr, data protection, privacy, personal data, audit, auditing, inspection, review, regulatory, regulation, compliance, legal requirement
3. Finance - payment, pay, paid, payable, remittance, fee, fees, charge, charges, cost, penalty, penalties, late fee, late payment
4. Operations - deliverable, deliver, delivery, output, timeline, deadline, schedule, due date, sla, service level, performance, kpi, metrics

#### 3. Defining Routing Function

In [126]:
print("DYNAMIC COORDINATOR WITH GEMMA-2-9B-IT")

OLLAMA_MODEL = "gemma2:9b" 
OLLAMA_URL = "http://localhost:11434/api/generate"

def check_ollama_models():
    """Check available Ollama models"""
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=5)
        if response.status_code == 200:
            models = response.json().get('models', [])
            return [m['name'] for m in models]
        return []
    except:
        return []

def query_ollama(prompt, model=OLLAMA_MODEL):
    try:
        response = requests.post(
            OLLAMA_URL,
            json={
                "model": model,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": 0.3,
                    "top_p": 0.9,
                    "num_predict": 150
                }
            },
            timeout=60
        )
        if response.status_code == 200:
            return response.json().get('response', '')
        else:
            print(f"Ollama HTTP error: {response.status_code}")
            print(f"Response: {response.text[:200]}")
            return None
    except requests.exceptions.ConnectionError:
        print("ERROR: Cannot connect to Ollama. Make sure Ollama is running (ollama serve)")
        return None
    except Exception as e:
        print(f"Ollama error: {str(e)}")
        return None


def dynamic_route_query(query):
    
    coordinator_prompt = f"""You are an intelligent contract analysis coordinator. Analyze the user's query and determine which specialized agents should handle it.

Available Agents:
1. LEGAL - Handles: termination clauses, liability, indemnification, jurisdiction, governing law, dispute resolution, breach of contract, legal rights
2. COMPLIANCE - Handles: data protection, GDPR, privacy regulations, audit requirements, regulatory compliance, confidentiality, certification standards
3. FINANCE - Handles: payment terms, fees, invoicing, costs, penalties, late payments, reimbursement, financial obligations
4. OPERATIONS - Handles: deliverables, timelines, milestones, SLAs, service levels, performance metrics, KPIs, project schedules

User Query: "{query}"

Based on the query, select the most relevant agent(s). You can select multiple agents if the query covers multiple domains.

Respond with ONLY a valid JSON object (no other text):
{{
  "agents": ["agent1", "agent2"],
  "reasoning": "brief explanation of why these agents were selected"
}}

Valid agent names: legal, compliance, finance, operations

JSON Response:"""
    
    print(f"\nQuerying {OLLAMA_MODEL} coordinator...")
    response = query_ollama(coordinator_prompt)
    
    if response:
        try:
            import re
            
            json_match = re.search(r'\{[^{}]*"agents"[^{}]*\}', response, re.DOTALL)
            if json_match:
                json_str = json_match.group()
                result = json.loads(json_str)
                
                valid_agents = ['legal', 'compliance', 'finance', 'operations']
                selected_agents = [a.lower() for a in result.get('agents', []) if a.lower() in valid_agents]
                
                if selected_agents:
                    return {
                        'agents': selected_agents,
                        'reasoning': result.get('reasoning', 'LLM-based routing'),
                        'approach': 'dynamic_llm_coordinator',
                        'model': OLLAMA_MODEL,
                        'scores': {agent: 1 for agent in selected_agents}
                    }
            
            agents_found = []
            for agent in ['legal', 'compliance', 'finance', 'operations']:
                if agent in response.lower():
                    agents_found.append(agent)
            
            if agents_found:
                return {
                    'agents': agents_found,
                    'reasoning': 'Extracted from LLM response',
                    'approach': 'dynamic_llm_coordinator_parsed',
                    'model': OLLAMA_MODEL,
                    'scores': {agent: 1 for agent in agents_found}
                }
                
        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {str(e)}")
            print(f"Response was: {response[:200]}")
        except Exception as e:
            print(f"Error processing LLM response: {str(e)}")
    
    print("Falling back to keyword-based routing...")
    return fallback_keyword_routing(query)


def fallback_keyword_routing(query):
    query_lower = query.lower()
    
    keyword_rules = {
        'legal': ['termination', 'terminate', 'cancel', 'liability', 'liable', 
                  'jurisdiction', 'governing law', 'indemnif', 'breach', 'dispute'],
        'compliance': ['gdpr', 'data protection', 'privacy', 'audit', 'compliance', 
                       'regulation', 'confidential', 'security', 'certification'],
        'finance': ['payment', 'fee', 'cost', 'invoice', 'penalty', 'charge', 
                    'financial', 'reimburs', 'budget', 'expense'],
        'operations': ['deliverable', 'timeline', 'milestone', 'sla', 'performance', 
                       'schedule', 'deadline', 'kpi', 'service level']
    }
    
    agent_scores = defaultdict(int)
    for agent, keywords in keyword_rules.items():
        for keyword in keywords:
            if keyword in query_lower:
                agent_scores[agent] += 1
    
    if agent_scores:
        routed_agents = sorted(agent_scores.keys(), key=lambda a: agent_scores[a], reverse=True)
        return {
            'agents': routed_agents,
            'reasoning': 'Keyword-based fallback routing',
            'approach': 'fallback_keyword',
            'scores': dict(agent_scores)
        }
    else:
        return {
            'agents': ['legal', 'compliance', 'finance', 'operations'],
            'reasoning': 'No specific matches - routing to all agents',
            'approach': 'fallback_all',
            'scores': {}
        }


def route_query_verbose(query):
   
    print("DYNAMIC ROUTING WITH GEMMA-2-9B-IT")

    if len(query) > 100:
        print(f"Query: \"{query[:100]}...\"")
    else:
        print(f"Query: \"{query}\"")
    
    result = dynamic_route_query(query)
    
    print(f"\nApproach: {result['approach'].upper().replace('_', ' ')}")
    print(f"Model: {result.get('model', 'N/A')}")
    print(f"Agents Routed: {', '.join(result['agents']) if result['agents'] else 'None'}")
    print(f"Reasoning: {result['reasoning']}")
    
    if result.get('scores'):
        print(f"\nAgent Scores:")
        for agent in result['agents']:
            score = result['scores'].get(agent, 0)
            print(f"   {agent.upper()}: {score}")
    
    print(f"\nAction: Retrieve relevant data from Pinecone vector store")
    
    return result

print(f"\nChecking Ollama setup...")
print(f"URL: {OLLAMA_URL}")

available_models = check_ollama_models()
if available_models:
    print(f"Ollama is running")
    print(f"  Available models: {', '.join(available_models)}")
    
    gemma_models = [m for m in available_models if 'gemma' in m.lower() and '9b' in m]
    if gemma_models:
        OLLAMA_MODEL = gemma_models[0]
        print(f"  Using model: {OLLAMA_MODEL}")
    else:
        print(f"  WARNING: gemma-2-9b not found in available models")
        print(f"  Please install it: ollama pull gemma2:9b")
        print(f"  Current model setting: {OLLAMA_MODEL}")
else:
    print("Cannot connect to Ollama or no models installed")


print(f"\nTesting model connection...")
test_response = query_ollama("Hello", model=OLLAMA_MODEL)
if test_response:
    print(f"Model {OLLAMA_MODEL} is responding")
    print(f"  Test response: {test_response[:100]}...")
else:
    print(f"Model {OLLAMA_MODEL} is not responding")

print("Dynamic coordinator initialized!")

DYNAMIC COORDINATOR WITH GEMMA-2-9B-IT

Checking Ollama setup...
URL: http://localhost:11434/api/generate
Ollama is running
  Available models: gemma2:9b
  Using model: gemma2:9b

Testing model connection...
Model gemma2:9b is responding
  Test response: Hello! 👋

How can I help you today? 😊...
Dynamic coordinator initialized!


#### 4. Testing Routing Logic

In [30]:
print("TESTING ROUTING LOGIC")

test_queries = [
    "What are the termination and liability clauses?",
    "Does this contract comply with GDPR and data protection regulations?",
    "What are the payment terms and late fee penalties?",
    "What are the deliverable timelines and milestones?",
    
    "Show me termination, GDPR compliance, and payment terms",
    "What are the service level agreements and performance standards?",
    
    "Indemnification and breach of contract details",
    "ISO 27001 certification requirements and audit obligations",
    "Invoice payment schedule with penalty interest",
    "Deliverable acceptance criteria and performance KPIs",
    
    "What is in this contract?", 
    "Jurisdiction and applicable governing law",
]

print(f"\nTesting Routing with {len(test_queries)} Sample Queries:")
print("="*80)

test_results = []

for idx, query in enumerate(test_queries, 1):
    print(f"\nTest {idx}/{len(test_queries)}")
    result = route_query_verbose(query)
    test_results.append({
        "query": query,
        "result": result
    })

agent_routing_counts = defaultdict(int)
multi_agent_queries = 0
fallback_queries = 0
approach_counts = defaultdict(int)

for test in test_results:
    agents = test["result"]["agents"]
    approach = test["result"].get("approach", "unknown")
    
    if len(agents) > 1:
        multi_agent_queries += 1
    
    if approach in ["fallback_keyword", "fallback_all"]:
        fallback_queries += 1
    
    approach_counts[approach] += 1
    
    for agent in agents:
        agent_routing_counts[agent] += 1

print("ROUTING STATISTICS")
print(f"Total Test Queries: {len(test_results)}")
print(f"Single-Agent Queries: {len(test_results) - multi_agent_queries} ({((len(test_results) - multi_agent_queries)/len(test_results)*100):.1f}%)")
print(f"Multi-Agent Queries: {multi_agent_queries} ({(multi_agent_queries/len(test_results)*100):.1f}%)")
print(f"Fallback Queries: {fallback_queries} ({(fallback_queries/len(test_results)*100):.1f}%)")

print("ROUTING APPROACH BREAKDOWN")
for approach, count in sorted(approach_counts.items(), key=lambda x: x[1], reverse=True):
    percentage = (count / len(test_results)) * 100
    approach_name = approach.replace('_', ' ').title()
    bar = "█" * int(percentage / 5)
    print(f"{approach_name:30} : {count:2} ({percentage:5.1f}%) {bar}")

print("AGENT ROUTING FREQUENCY")
for agent, count in sorted(agent_routing_counts.items(), key=lambda x: x[1], reverse=True):
    percentage = (count / len(test_results)) * 100
    bar = "█" * int(percentage / 5)
    print(f"{agent.upper():12} : {count:2} queries ({percentage:5.1f}%) {bar}")

TESTING ROUTING LOGIC

Testing Routing with 12 Sample Queries:

Test 1/12
DYNAMIC ROUTING WITH GEMMA-2-9B-IT
Query: "What are the termination and liability clauses?"

Querying gemma2:9b coordinator...

Approach: DYNAMIC LLM COORDINATOR
Model: gemma2:9b
Agents Routed: legal
Reasoning: The query specifically asks about termination and liability clauses, which fall under the domain of contract law handled by the LEGAL agent.

Agent Scores:
   LEGAL: 1

Action: Retrieve relevant data from Pinecone vector store

Test 2/12
DYNAMIC ROUTING WITH GEMMA-2-9B-IT
Query: "Does this contract comply with GDPR and data protection regulations?"

Querying gemma2:9b coordinator...

Approach: DYNAMIC LLM COORDINATOR
Model: gemma2:9b
Agents Routed: compliance
Reasoning: The query specifically asks about GDPR and data protection regulations, which fall under the purview of the COMPLIANCE agent.

Agent Scores:
   COMPLIANCE: 1

Action: Retrieve relevant data from Pinecone vector store

Test 3/12
DYNAMIC ROUT

##### Queries - Routing to Agent as per Keywords
1. Operations - 4 matches
2. Legal - 6 matches
3. Compliance - 4 matches
4. Finance - 4 matches

#### 5. Defining Coordinator Logic

In [31]:
print("DYNAMIC COORDINATOR EXECUTION")

def retrieve_from_pinecone(query, agent_filter=None, top_k=3):

    try:
        query_embedding = create_embedding(query)
        
        filter_dict = None
        if agent_filter:
            if isinstance(agent_filter, list):
                filter_dict = {"agent": {"$in": agent_filter}}
            else:
                filter_dict = {"agent": {"$eq": agent_filter}}
        
        results = index.query(
            vector=query_embedding,
            top_k=top_k,
            filter=filter_dict,
            include_metadata=True
        )
        
        return results
        
    except Exception as e:
        print(f"  Pinecone retrieval error: {str(e)}")
        return None


def route_query_with_retrieval(query):
    
    print("ROUTING AND RETRIEVAL")
   
    routing_result = dynamic_route_query(query)
    
    print(f"\nCoordinator Decision:")
    print(f"  Agents: {', '.join(routing_result['agents'])}")
    print(f"  Reasoning: {routing_result['reasoning']}")
    print(f"  Approach: {routing_result['approach']}")
    
    print(f"\nRetrieving from Pinecone...")
    retrieved_data = {}
    
    for agent in routing_result['agents']:
        results = retrieve_from_pinecone(query, agent_filter=agent, top_k=3)
        
        if results and results.matches:
            filtered_matches = [m for m in results.matches if m.score > 0.3]
            
            clauses = [match.metadata.get('clause_full', match.metadata.get('clause', '')) 
                      for match in filtered_matches]
            scores = [match.score for match in filtered_matches]
            
            retrieved_data[agent] = {
                'clauses': clauses,
                'scores': scores,
                'count': len(clauses)
            }
            
            print(f"  {agent.upper()}: {len(clauses)} clauses (scores: {[f'{s:.3f}' for s in scores]})")
        else:
            retrieved_data[agent] = {
                'clauses': [],
                'scores': [],
                'count': 0
            }
            print(f"  {agent.upper()}: No relevant clauses found")
    
    routing_result['retrieved_data'] = retrieved_data
    return routing_result


def coordinator_execute_dynamic(query):
    print(f"PROCESSING QUERY")
    print(f"Query: \"{query}\"")
    
    routing_result = route_query_with_retrieval(query)
    
    results = {
        'query': query,
        'coordinator_model': OLLAMA_MODEL,
        'approach': 'dynamic_llm_with_pinecone',
        'routing': routing_result,
        'agent_results': {},
        'summary': {
            'total_agents_routed': len(routing_result['agents']),
            'total_clauses_retrieved': 0,
            'routing_approach': routing_result['approach']
        }
    }
    
    print("AGENT RESULTS")
 
    for agent_name in routing_result['agents']:
        retrieved = routing_result['retrieved_data'].get(agent_name, {})
        clauses = retrieved.get('clauses', [])
        scores = retrieved.get('scores', [])
        
        results['agent_results'][agent_name] = {
            'agent_name': agent_name,
            'source': 'pinecone_vector_store',
            'clauses_retrieved': len(clauses),
            'clauses': clauses,
            'relevance_scores': scores
        }
        
        results['summary']['total_clauses_retrieved'] += len(clauses)
        
        print(f"\n{agent_name.upper()} Agent:")
        print(f"  Source: Pinecone Vector Store")
        print(f"  Clauses Retrieved: {len(clauses)}")
        
        if scores:
            avg_score = sum(scores) / len(scores)
            max_score = max(scores)
            print(f"  Relevance - Avg: {avg_score:.3f}, Max: {max_score:.3f}")
        
        if clauses:
            preview = clauses[0][:120] + "..." if len(clauses[0]) > 120 else clauses[0]
            print(f"  Sample Clause: {preview}")
    
    return results


def coordinator_execute_verbose_dynamic(query):

    print(f"\n{'#'*80}")
    print("#" + " "*78 + "#")
    print("#" + "DYNAMIC COORDINATOR EXECUTION".center(78) + "#")
    print("#" + " "*78 + "#")
    print(f"{'#'*80}\n")
    
    results = coordinator_execute_dynamic(query)
    
    print("EXECUTION SUMMARY")
    print(f"Coordinator Model: {results['coordinator_model']}")
    print(f"Routing Approach: {results['summary']['routing_approach']}")
    print(f"Overall Approach: {results['approach']}")
    print(f"Agents Routed: {results['summary']['total_agents_routed']}")
    print(f"Total Clauses Retrieved: {results['summary']['total_clauses_retrieved']}")
    
    if results['summary']['total_clauses_retrieved'] > 0:
        avg_per_agent = results['summary']['total_clauses_retrieved'] / results['summary']['total_agents_routed']
        print(f"Average Clauses per Agent: {avg_per_agent:.1f}")
    
  
    print("DETAILED CLAUSE RETRIEVAL")

    for agent_name, agent_result in results['agent_results'].items():
        clauses = agent_result.get('clauses', [])
        scores = agent_result.get('relevance_scores', [])
        
        print(f"\n{agent_name.upper()} Agent - {len(clauses)} clause(s):")
        
        if clauses:
            for idx, (clause, score) in enumerate(zip(clauses, scores), 1):
                print(f"\n  [{idx}] Relevance: {score:.3f}")
                print(f"      {clause[:200]}{'...' if len(clause) > 200 else ''}")
        else:
            print("  No relevant clauses found")
    
    
    return results


print("\nDynamic coordinator execution functions defined")

DYNAMIC COORDINATOR EXECUTION

Dynamic coordinator execution functions defined


#### 6. Running Coordinator

In [36]:
print("RUNNING DYNAMIC COORDINATOR")

try:
    stats = index.describe_index_stats()
    print(f"\nPinecone connected: {stats.total_vector_count} vectors available")
except Exception as e:
    print(f"\nPinecone connection error: {str(e)}")
    print("  Please run Task 1 to set up Pinecone and upload agent outputs")

sample_queries = [
    "What are the termination clauses and governing law provisions?",
    "What are the data protection and privacy obligations?",
    "Payment terms, fees, and late payment penalties",
    "Deliverable timelines and milestone requirements",
    "Complete contract analysis covering legal, compliance, finance, and operations"
]

print(f"Running Dynamic Coordinator on {len(sample_queries)} Sample Queries")
print(f"Coordinator Model: {OLLAMA_MODEL}")
print(f"Vector Store: Pinecone")
print(f"Approach: Dynamic LLM-based routing with vector retrieval")

coordinator_results = []

for idx, query in enumerate(sample_queries, 1):
    
    print(f"COORDINATOR RUN {idx}/{len(sample_queries)}")
    
    try:
        result = coordinator_execute_verbose_dynamic(query)
        coordinator_results.append(result)
    except Exception as e:
        print(f"\nError processing query: {str(e)}")
       
        coordinator_results.append({
            'query': query,
            'error': str(e),
            'summary': {
                'total_agents_routed': 0,
                'total_clauses_retrieved': 0
            },
            'agent_results': {}
        })
    
    if idx < len(sample_queries):
        print(f"\n{'.'*80}\n")


print("OVERALL COORDINATOR STATISTICS")

total_agents_routed = sum(r.get("summary", {}).get("total_agents_routed", 0) for r in coordinator_results)
total_clauses_retrieved = sum(r.get("summary", {}).get("total_clauses_retrieved", 0) for r in coordinator_results)
successful_queries = len([r for r in coordinator_results if 'error' not in r])

avg_agents_per_query = total_agents_routed / successful_queries if successful_queries > 0 else 0
avg_clauses_per_query = total_clauses_retrieved / successful_queries if successful_queries > 0 else 0

print(f"\nQuery Processing:")
print(f"  Total Queries Processed: {len(coordinator_results)}")
print(f"  Successful: {successful_queries}")
print(f"  Failed: {len(coordinator_results) - successful_queries}")

print(f"\nRouting Statistics:")
print(f"  Total Agents Routed: {total_agents_routed}")
print(f"  Average Agents per Query: {avg_agents_per_query:.1f}")

print(f"\nRetrieval Statistics:")
print(f"  Total Clauses Retrieved: {total_clauses_retrieved}")
print(f"  Average Clauses per Query: {avg_clauses_per_query:.1f}")

agent_usage = defaultdict(int)
agent_clause_contribution = defaultdict(int)
agent_relevance_scores = defaultdict(list)

for result in coordinator_results:
    if 'error' not in result:
        for agent, agent_result in result.get("agent_results", {}).items():
            if "error" not in agent_result:
                agent_usage[agent] += 1
                clauses_count = agent_result.get("clauses_retrieved", 0)
                agent_clause_contribution[agent] += clauses_count
                
                scores = agent_result.get("relevance_scores", [])
                if scores:
                    agent_relevance_scores[agent].extend(scores)


print("AGENT USAGE FREQUENCY")

if agent_usage:
    for agent, count in sorted(agent_usage.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / successful_queries) * 100 if successful_queries > 0 else 0
        clauses = agent_clause_contribution[agent]
        avg_clauses = clauses / count if count > 0 else 0
        bar = "█" * int(percentage / 5)
        
        print(f"\n{agent.upper():12}")
        print(f"  Queries: {count}/{successful_queries} ({percentage:5.1f}%) {bar}")
        print(f"  Clauses: {clauses} total ({avg_clauses:.1f} avg per query)")
        
        if agent in agent_relevance_scores and agent_relevance_scores[agent]:
            avg_relevance = sum(agent_relevance_scores[agent]) / len(agent_relevance_scores[agent])
            print(f"  Avg Relevance: {avg_relevance:.3f}")
else:
    print("  No agent usage data available")


print("ROUTING APPROACH BREAKDOWN")

approach_counts = defaultdict(int)
for result in coordinator_results:
    if 'error' not in result:
        approach = result.get('summary', {}).get('routing_approach', 'unknown')
        approach_counts[approach] += 1

for approach, count in sorted(approach_counts.items(), key=lambda x: x[1], reverse=True):
    percentage = (count / successful_queries) * 100 if successful_queries > 0 else 0
    approach_name = approach.replace('_', ' ').title()
    bar = "█" * int(percentage / 5)
    print(f"{approach_name:30} : {count:2} ({percentage:5.1f}%) {bar}")

print("TOP RELEVANCE MATCHES")

all_matches = []
for result in coordinator_results:
    if 'error' not in result:
        query = result['query']
        for agent, agent_result in result.get('agent_results', {}).items():
            clauses = agent_result.get('clauses', [])
            scores = agent_result.get('relevance_scores', [])
            for clause, score in zip(clauses, scores):
                all_matches.append({
                    'query': query,
                    'agent': agent,
                    'score': score,
                    'clause': clause
                })

all_matches.sort(key=lambda x: x['score'], reverse=True)
for idx, match in enumerate(all_matches[:3], 1):
    print(f"\n[{idx}] Score: {match['score']:.3f} | Agent: {match['agent'].upper()}")
    print(f"    Query: {match['query'][:60]}...")
    print(f"    Clause: {match['clause'][:150]}...")

print("COORDINATOR EXECUTION COMPLETE")

RUNNING DYNAMIC COORDINATOR

Pinecone connected: 8 vectors available
Running Dynamic Coordinator on 5 Sample Queries
Coordinator Model: gemma2:9b
Vector Store: Pinecone
Approach: Dynamic LLM-based routing with vector retrieval
COORDINATOR RUN 1/5

################################################################################
#                                                                              #
#                        DYNAMIC COORDINATOR EXECUTION                         #
#                                                                              #
################################################################################

PROCESSING QUERY
Query: "What are the termination clauses and governing law provisions?"
ROUTING AND RETRIEVAL

Querying gemma2:9b coordinator...

Coordinator Decision:
  Agents: legal
  Reasoning: The query specifically asks about termination clauses and governing law provisions, which fall under the domain of contract law handled by the LEGAL

#### 7. Saving Coordinator Output

In [37]:
print("SAVING COORDINATOR OUTPUT")

if coordinator_results:
    sample_result = coordinator_results[0]
    
    print(f"\nSample Result Preview:")
    print(f"{'─'*80}")
    print(f"Query: \"{sample_result['query']}\"")
    print(f"Coordinator Model: {sample_result.get('coordinator_model', 'N/A')}")
    print(f"Approach: {sample_result['approach'].upper().replace('_', ' ')}")
    
    print(f"\nRouting Decision:")
    routing = sample_result.get('routing', {})
    print(f"  Method: Dynamic LLM-based coordination")
    print(f"  Approach: {routing.get('approach', 'N/A')}")
    print(f"  Agents: {', '.join(routing.get('agents', []))}")
    print(f"  Reasoning: {routing.get('reasoning', 'N/A')}")
    
    print(f"\nAgent Results (from Pinecone):")
    for agent_name, agent_result in sample_result.get("agent_results", {}).items():
        print(f"\n  {agent_name.upper()}:")
        if "error" in agent_result:
            print(f"    Error: {agent_result['error']}")
        else:
            print(f"    Source: {agent_result.get('source', 'N/A')}")
            print(f"    Clauses Retrieved: {agent_result.get('clauses_retrieved', 0)}")
            
            scores = agent_result.get('relevance_scores', [])
            if scores:
                print(f"    Avg Relevance: {sum(scores)/len(scores):.3f}")
                print(f"    Max Relevance: {max(scores):.3f}")
            
            clauses = agent_result.get('clauses', [])
            if clauses:
                print(f"    Sample Clauses:")
                for i, clause in enumerate(clauses[:2], 1):
                    preview = clause[:100] + "..." if len(clause) > 100 else clause
                    print(f"       {i}. {preview}")
    
    print(f"\n{'─'*80}")
    print(f"Summary:")
    summary = sample_result.get('summary', {})
    print(f"  Agents Routed: {summary.get('total_agents_routed', 0)}")
    print(f"  Total Clauses Retrieved: {summary.get('total_clauses_retrieved', 0)}")
    print(f"  Routing Approach: {summary.get('routing_approach', 'N/A')}")

COORDINATOR_OUTPUT = "../Data/Results/Coordinator"
os.makedirs(COORDINATOR_OUTPUT, exist_ok=True)

print(f"\n{'='*80}")
print("SAVING COORDINATOR RESULTS")
print(f"{'='*80}")

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

successful_results = [r for r in coordinator_results if 'error' not in r]
total_agents = sum(r.get("summary", {}).get("total_agents_routed", 0) for r in successful_results)
total_clauses = sum(r.get("summary", {}).get("total_clauses_retrieved", 0) for r in successful_results)
avg_agents = total_agents / len(successful_results) if successful_results else 0
avg_clauses = total_clauses / len(successful_results) if successful_results else 0

approach_counts = defaultdict(int)
for result in successful_results:
    approach = result.get('summary', {}).get('routing_approach', 'unknown')
    approach_counts[approach] += 1

try:
    pinecone_stats = index.describe_index_stats()
    vector_count = pinecone_stats.total_vector_count
except:
    vector_count = 0

coordinator_output = {
    "metadata": {
        "coordinator_name": "Dynamic LLM Coordinator",
        "coordinator_model": OLLAMA_MODEL,
        "coordinator_approach": "Dynamic LLM routing with Pinecone retrieval",
        "execution_timestamp": timestamp,
        "description": "Uses Gemma-2-9b-it for intelligent query routing and Pinecone for vector-based clause retrieval",
        "total_queries_processed": len(coordinator_results),
        "successful_queries": len(successful_results),
        "failed_queries": len(coordinator_results) - len(successful_results),
        "total_agents_routed": total_agents,
        "total_clauses_retrieved": total_clauses,
        "avg_agents_per_query": round(avg_agents, 2),
        "avg_clauses_per_query": round(avg_clauses, 2),
        "routing_approaches": dict(approach_counts),
        "vector_store": {
            "type": "Pinecone",
            "index_name": INDEX_NAME,
            "total_vectors": vector_count,
            "dimension": DIMENSION
        },
        "processing_details": {
            "data_source": "Pinecone Vector Store",
            "routing_method": "LLM-based dynamic coordination",
            "retrieval_method": "Vector similarity search",
            "model_provider": "Ollama (local)",
            "response_time": "Dynamic (LLM + vector search)"
        }
    },
    "queries": coordinator_results
}

output_filename = f"coordinator_results_dynamic_{timestamp}.json"
output_path = os.path.join(COORDINATOR_OUTPUT, output_filename)

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(coordinator_output, f, indent=2, ensure_ascii=False)

print(f"\nCoordinator results saved!")
print(f"   File: {output_filename}")
print(f"   Location: {COORDINATOR_OUTPUT}")
print(f"\nOutput Statistics:")
print(f"   Approach: {coordinator_output['metadata']['coordinator_approach']}")
print(f"   Model: {coordinator_output['metadata']['coordinator_model']}")
print(f"   Queries: {len(coordinator_results)} ({len(successful_results)} successful)")
print(f"   Total Agents Routed: {total_agents}")
print(f"   Total Clauses Retrieved: {total_clauses}")
print(f"   Avg Agents/Query: {avg_agents:.2f}")
print(f"   Avg Clauses/Query: {avg_clauses:.2f}")

summary_lines = []
summary_lines.append("=" * 100)
summary_lines.append("DYNAMIC COORDINATOR EXECUTION SUMMARY REPORT")
summary_lines.append("=" * 100)
summary_lines.append(f"\nGenerated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
summary_lines.append(f"Coordinator Model: {OLLAMA_MODEL}")
summary_lines.append(f"Vector Store: Pinecone (Index: {INDEX_NAME})")
summary_lines.append(f"\n{'=' * 100}\n")

summary_lines.append("COORDINATOR CONFIGURATION")
summary_lines.append("─" * 100)
summary_lines.append(f"Name: Dynamic LLM Coordinator")
summary_lines.append(f"Model: {OLLAMA_MODEL} (via Ollama)")
summary_lines.append(f"Approach: LLM-based intelligent routing")
summary_lines.append(f"Data Source: Pinecone Vector Store")
summary_lines.append(f"Retrieval Method: Semantic similarity search")
summary_lines.append(f"Total Vectors in Store: {vector_count}")

summary_lines.append(f"\n\n{'=' * 100}\n")
summary_lines.append("OVERALL STATISTICS")
summary_lines.append("─" * 100)
summary_lines.append(f"Total Queries Processed: {len(coordinator_results)}")
summary_lines.append(f"  Successful: {len(successful_results)}")
summary_lines.append(f"  Failed: {len(coordinator_results) - len(successful_results)}")
summary_lines.append(f"Total Agents Routed: {total_agents}")
summary_lines.append(f"Total Clauses Retrieved: {total_clauses}")
summary_lines.append(f"Average Agents per Query: {avg_agents:.2f}")
summary_lines.append(f"Average Clauses per Query: {avg_clauses:.2f}")

summary_lines.append(f"\n\n{'=' * 100}\n")
summary_lines.append("ROUTING APPROACH BREAKDOWN")
summary_lines.append("─" * 100)
for approach, count in sorted(approach_counts.items(), key=lambda x: x[1], reverse=True):
    percentage = (count / len(successful_results)) * 100 if successful_results else 0
    approach_name = approach.replace('_', ' ').title()
    summary_lines.append(f"{approach_name:40} : {count:3} queries ({percentage:5.1f}%)")

agent_usage = defaultdict(int)
agent_clause_count = defaultdict(int)
agent_relevance = defaultdict(list)

for result in successful_results:
    for agent, agent_result in result.get("agent_results", {}).items():
        if "error" not in agent_result:
            agent_usage[agent] += 1
            agent_clause_count[agent] += agent_result.get("clauses_retrieved", 0)
            scores = agent_result.get("relevance_scores", [])
            if scores:
                agent_relevance[agent].extend(scores)

summary_lines.append(f"\n\n{'=' * 100}\n")
summary_lines.append("AGENT USAGE ANALYSIS")
summary_lines.append("─" * 100)

for agent in sorted(agent_usage.keys()):
    count = agent_usage[agent]
    clauses = agent_clause_count[agent]
    avg_clauses = clauses / count if count > 0 else 0
    percentage = (count / len(successful_results)) * 100 if successful_results else 0
    
    summary_lines.append(f"\n{agent.upper()}:")
    summary_lines.append(f"  Queries Handled: {count} ({percentage:.1f}%)")
    summary_lines.append(f"  Total Clauses Retrieved: {clauses}")
    summary_lines.append(f"  Average Clauses per Query: {avg_clauses:.2f}")
    
    if agent in agent_relevance and agent_relevance[agent]:
        avg_rel = sum(agent_relevance[agent]) / len(agent_relevance[agent])
        max_rel = max(agent_relevance[agent])
        summary_lines.append(f"  Average Relevance Score: {avg_rel:.3f}")
        summary_lines.append(f"  Maximum Relevance Score: {max_rel:.3f}")

summary_lines.append(f"\n\n{'=' * 100}\n")
summary_lines.append("QUERY EXECUTION DETAILS")
summary_lines.append("─" * 100)

for idx, result in enumerate(coordinator_results, 1):
    summary_lines.append(f"\n\nQuery {idx}: \"{result.get('query', 'N/A')}\"")
    
    if 'error' in result:
        summary_lines.append(f"  Status: FAILED")
        summary_lines.append(f"  Error: {result['error']}")
    else:
        routing = result.get('routing', {})
        summary = result.get('summary', {})
        
        summary_lines.append(f"  Status: SUCCESS")
        summary_lines.append(f"  Routing Approach: {summary.get('routing_approach', 'N/A')}")
        summary_lines.append(f"  Agents Routed: {', '.join(routing.get('agents', []))}")
        summary_lines.append(f"  Reasoning: {routing.get('reasoning', 'N/A')}")
        summary_lines.append(f"  Clauses Retrieved: {summary.get('total_clauses_retrieved', 0)}")
        
        for agent, agent_result in result.get('agent_results', {}).items():
            clauses_count = agent_result.get('clauses_retrieved', 0)
            if clauses_count > 0:
                scores = agent_result.get('relevance_scores', [])
                avg_score = sum(scores) / len(scores) if scores else 0
                summary_lines.append(f"    {agent.upper()}: {clauses_count} clauses (avg relevance: {avg_score:.3f})")

summary_lines.append("END OF REPORT")

summary_content = "\n".join(summary_lines)
summary_file = f"coordinator_summary_dynamic_{timestamp}.txt"
summary_path = os.path.join(COORDINATOR_OUTPUT, summary_file)

with open(summary_path, 'w', encoding='utf-8') as f:
    f.write(summary_content)

print(f"\nSummary report saved!")
print(f"   File: {summary_file}")
print(f"   Location: {COORDINATOR_OUTPUT}")

print("FILES GENERATED (DYNAMIC COORDINATOR)")

print(f"\n1. {output_filename}")


print(f"\n2. {summary_file}")

print("All coordinator outputs saved successfully!")

SAVING COORDINATOR OUTPUT

Sample Result Preview:
────────────────────────────────────────────────────────────────────────────────
Query: "What are the termination clauses and governing law provisions?"
Coordinator Model: gemma2:9b
Approach: DYNAMIC LLM WITH PINECONE

Routing Decision:
  Method: Dynamic LLM-based coordination
  Approach: dynamic_llm_coordinator
  Agents: legal
  Reasoning: The query specifically asks about termination clauses and governing law provisions, which fall under the domain of contract law handled by the LEGAL agent.

Agent Results (from Pinecone):

  LEGAL:
    Source: pinecone_vector_store
    Clauses Retrieved: 2
    Avg Relevance: 0.540
    Max Relevance: 0.631
    Sample Clauses:
       1. The other party shall give notice of termination in writing to the other party, which notice shall s...
       2. The other party asserts any rights in or to the terminating party's intellectual property in violati...

───────────────────────────────────────────────────

##### Coordinator Result is stored in JSON format alonng with the keywords specified for each agent and the queries

#### 8. Adding New Keyword - 'Indemnity'

In [38]:
print("TESTING COORDINATOR WITH NEW KEYWORD: 'INDEMNITY'")

indemnity_test_queries = [
    "What are the indemnity provisions?",
    "Show indemnity and liability clauses",
    "Indemnification and indemnity obligations",
    "Does the contract include indemnity protections?",
    "Indemnity caps and financial liability limits",
    "Payment terms with indemnity requirements"
]

print(f"TESTING DYNAMIC COORDINATOR WITH INDEMNITY QUERIES")

print(f"Total Test Queries: {len(indemnity_test_queries)}")
print(f"Coordinator Model: {OLLAMA_MODEL}")
print(f"Expected Primary Agent: Legal")

indemnity_test_results = []

for idx, query in enumerate(indemnity_test_queries, 1):

    print(f"TEST {idx}/{len(indemnity_test_queries)}")
    print(f"Query: \"{query}\"")
    
    routing_result = dynamic_route_query(query)
    
    print("COORDINATOR DECISION:")
    print(f"  Approach: {routing_result['approach']}")
    print(f"  Agents Routed: {', '.join(routing_result['agents'])}")
    print(f"  Reasoning: {routing_result['reasoning']}")
    
    is_legal_selected = 'legal' in routing_result['agents']
    is_legal_primary = routing_result['agents'][0] == 'legal' if routing_result['agents'] else False
    
    print(f"\nANALYSIS:")
    if is_legal_primary:
        print(f"  CORRECT: Legal agent selected as PRIMARY agent")
    elif is_legal_selected:
        print(f"  PARTIAL: Legal agent selected but not primary")
    else:
        print(f"  INCORRECT: Legal agent NOT selected")
    

    print("RETRIEVING FROM PINECONE:")

    agent_results = {}
    total_clauses = 0
    
    for agent_name in routing_result['agents']:
        try:
            results = retrieve_from_pinecone(query, agent_filter=agent_name, top_k=3)
            
            if results and results.matches:
                filtered_matches = [m for m in results.matches if m.score > 0.3]
                clauses = [match.metadata.get('clause_full', match.metadata.get('clause', '')) 
                          for match in filtered_matches]
                scores = [match.score for match in filtered_matches]
                
                agent_results[agent_name] = {
                    'clauses_count': len(clauses),
                    'clauses': clauses,
                    'scores': scores
                }
                
                total_clauses += len(clauses)
                
                print(f"  {agent_name.upper()}: {len(clauses)} clauses retrieved")
                if scores:
                    print(f"    Avg relevance: {sum(scores)/len(scores):.3f}")
            else:
                agent_results[agent_name] = {'clauses_count': 0, 'clauses': [], 'scores': []}
                print(f"  {agent_name.upper()}: No relevant clauses found")
                
        except Exception as e:
            print(f"  {agent_name.upper()}: Error - {str(e)}")
            agent_results[agent_name] = {'clauses_count': 0, 'clauses': [], 'scores': []}
    
    indemnity_test_results.append({
        "query": query,
        "routing": routing_result,
        "agent_results": agent_results,
        "legal_selected": is_legal_selected,
        "legal_primary": is_legal_primary,
        "total_clauses": total_clauses
    })

print("INDEMNITY KEYWORD ROUTING ANALYSIS")

legal_as_primary = sum(1 for test in indemnity_test_results if test['legal_primary'])
legal_selected = sum(1 for test in indemnity_test_results if test['legal_selected'])
total_tests = len(indemnity_test_results)

print(f"\nRouting Accuracy:")
print(f"  Total Queries Tested: {total_tests}")
print(f"  Legal as Primary Agent: {legal_as_primary}/{total_tests} ({legal_as_primary/total_tests*100:.1f}%)")
print(f"  Legal Selected (any position): {legal_selected}/{total_tests} ({legal_selected/total_tests*100:.1f}%)")

approach_counts = defaultdict(int)
for test in indemnity_test_results:
    approach = test['routing']['approach']
    approach_counts[approach] += 1

print(f"\nRouting Approach Breakdown:")
for approach, count in sorted(approach_counts.items(), key=lambda x: x[1], reverse=True):
    percentage = (count / total_tests) * 100
    approach_name = approach.replace('_', ' ').title()
    bar = "█" * int(percentage / 5)
    print(f"  {approach_name:30} : {count:2} ({percentage:5.1f}%) {bar}")

total_clauses_retrieved = sum(test['total_clauses'] for test in indemnity_test_results)
queries_with_clauses = sum(1 for test in indemnity_test_results if test['total_clauses'] > 0)

print(f"\nRetrieval Performance:")
print(f"  Total Clauses Retrieved: {total_clauses_retrieved}")
print(f"  Queries with Results: {queries_with_clauses}/{total_tests} ({queries_with_clauses/total_tests*100:.1f}%)")
print(f"  Average Clauses per Query: {total_clauses_retrieved/total_tests:.2f}")


print("DETAILED QUERY-BY-QUERY RESULTS")

for idx, test in enumerate(indemnity_test_results, 1):
    print(f"\n[{idx}] Query: \"{test['query']}\"")
    print(f"    Primary Agent: {test['routing']['agents'][0] if test['routing']['agents'] else 'None'}")
    print(f"    All Agents: {', '.join(test['routing']['agents'])}")
    print(f"    Reasoning: {test['routing']['reasoning']}")
    print(f"    Clauses Retrieved: {test['total_clauses']}")
    print(f"    Legal Selected: {'✓' if test['legal_selected'] else '✗'}")


print("SAVING TEST RESULTS")
COORDINATOR_OUTPUT = "../Data/Results/Coordinator"
os.makedirs(COORDINATOR_OUTPUT, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

indemnity_output = {
    "task": "Test Coordinator with 'Indemnity' Keyword",
    "coordinator_model": OLLAMA_MODEL,
    "approach": "Dynamic LLM-based routing with Pinecone retrieval",
    "decision": "Indemnity queries should route to Legal Agent",
    "reasoning": "Indemnity is a legal protection concept related to liability and indemnification",
    "test_timestamp": timestamp,
    "test_queries": indemnity_test_queries,
    "results": indemnity_test_results,
    "metrics": {
        "total_queries": total_tests,
        "legal_as_primary": legal_as_primary,
        "legal_as_primary_rate": f"{legal_as_primary/total_tests*100:.1f}%",
        "legal_selected": legal_selected,
        "legal_selected_rate": f"{legal_selected/total_tests*100:.1f}%",
        "total_clauses_retrieved": total_clauses_retrieved,
        "avg_clauses_per_query": round(total_clauses_retrieved/total_tests, 2),
        "queries_with_results": queries_with_clauses,
        "retrieval_success_rate": f"{queries_with_clauses/total_tests*100:.1f}%"
    },
    "routing_approaches": dict(approach_counts),
    "note": "Dynamic LLM coordinator automatically understands 'indemnity' without explicit rules"
}

indemnity_file = f"coordinator_indemnity_test_{timestamp}.json"
indemnity_path = os.path.join(COORDINATOR_OUTPUT, indemnity_file)

with open(indemnity_path, 'w', encoding='utf-8') as f:
    json.dump(indemnity_output, f, indent=2, ensure_ascii=False)

print(f"\nIndemnity test results saved!")
print(f"   File: {indemnity_file}")
print(f"   Location: {COORDINATOR_OUTPUT}")

print("INDEMNITY KEYWORD TEST COMPLETE!")

TESTING COORDINATOR WITH NEW KEYWORD: 'INDEMNITY'
TESTING DYNAMIC COORDINATOR WITH INDEMNITY QUERIES
Total Test Queries: 6
Coordinator Model: gemma2:9b
Expected Primary Agent: Legal
TEST 1/6
Query: "What are the indemnity provisions?"

Querying gemma2:9b coordinator...
COORDINATOR DECISION:
  Approach: dynamic_llm_coordinator
  Agents Routed: legal
  Reasoning: The query specifically asks about 'indemnity provisions', which fall under the domain of contract law and are handled by the LEGAL agent.

ANALYSIS:
  CORRECT: Legal agent selected as PRIMARY agent
RETRIEVING FROM PINECONE:
  LEGAL: 2 clauses retrieved
    Avg relevance: 0.399
TEST 2/6
Query: "Show indemnity and liability clauses"

Querying gemma2:9b coordinator...
COORDINATOR DECISION:
  Approach: dynamic_llm_coordinator
  Agents Routed: legal
  Reasoning: The query specifically requests 'indemnity and liability clauses', which fall under the domain of contract law handled by the LEGAL agent.

ANALYSIS:
  CORRECT: Legal agent s

# LanGraph

#### 1. LanGraph Setup

In [39]:
import sys
import subprocess

print("LANGGRAPH SETUP")

print("\nInstalling LangGraph and dependencies...")

packages = [
    "langgraph",
    "langchain-core",
    "langchain-community"
]

for package in packages:
    try:
        print(f"\nInstalling {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"{package} installed successfully")
    except Exception as e:
        print(f"{package} installation note: {str(e)[:100]}")

print("Importing LangGraph components...")

try:
    from langgraph.graph import StateGraph, END
    from typing import TypedDict, Dict, Any, List, Annotated
    import operator
    import os
    import json
    from datetime import datetime
    from collections import defaultdict
    
    print("LangGraph imported successfully")
    print("Core dependencies imported")
    
    print("\nTesting LangGraph setup...")
    
    class TestState(TypedDict):
        messages: List[str]
    
    workflow = StateGraph(TestState)
    print("StateGraph initialized successfully")
    
except ImportError as e:
    print(f"Import error: {e}")
    print("\nAttempting alternative import method...")
    
    try:
        from langgraph.graph import Graph
        from typing import TypedDict, Dict, Any, List, Annotated
        import operator
        import os
        import json
        from datetime import datetime
        from collections import defaultdict
        
        StateGraph = Graph
        END = "END"
        
        print("LangGraph imported (compatibility mode)")
        print("Using older Graph API")
        
    except Exception as e2:
        print(f"Alternative import also failed: {e2}")
        print("\nInstalling latest LangGraph version...")
        
        try:
            subprocess.check_call([
                sys.executable, "-m", "pip", "install", 
                "--upgrade", "langgraph", "langchain-core"
            ])
            
            from langgraph.graph import StateGraph, END
            from typing import TypedDict, Dict, Any, List, Annotated
            import operator
            import os
            import json
            from datetime import datetime
            from collections import defaultdict
            
            print("LangGraph reinstalled and imported successfully")
            
        except Exception as e3:
            print(f"Critical error: {e3}")
            print("\nPlease install manually: pip install langgraph langchain-core")

LANGGRAPH SETUP

Installing LangGraph and dependencies...

Installing langgraph...
langgraph installed successfully

Installing langchain-core...
langchain-core installed successfully

Installing langchain-community...
langchain-community installed successfully
Importing LangGraph components...
LangGraph imported successfully
Core dependencies imported

Testing LangGraph setup...
StateGraph initialized successfully


In [40]:
print("DEFINING COORDINATOR STATE")

class CoordinatorState(TypedDict):
    query: str  
    routed_agents: List[str]  
    routing_reasoning: str  
    agent_results: Dict[str, Any]  
    retrieved_clauses: List[Dict[str, Any]]  
    final_response: str  
    metadata: Dict[str, Any]  

print("CoordinatorState defined")

print("LANGGRAPH SETUP COMPLETE")

DEFINING COORDINATOR STATE
CoordinatorState defined
LANGGRAPH SETUP COMPLETE


#### 2. Defining Shared Graph State

In [41]:
print("DEFINING SHARED GRAPH STATE FOR DYNAMIC COORDINATOR")
class GraphState(TypedDict):
   
    query: str  
    
    routed_agents: List[str]  
    routing_reasoning: str  
    routing_approach: str  
    
    legal: Dict[str, Any]  
    compliance: Dict[str, Any]  
    finance: Dict[str, Any]  
    operations: Dict[str, Any] 
    
    all_clauses: List[Dict[str, Any]]  
    total_clauses_retrieved: int  
    
    execution_order: List[str]  
    timestamp: str  
    
    coordinator_model: str  
    vector_store: str  
    execution_status: str  

print("GraphState defined with TypedDict")
example_state = {
    "query": "Analyze contract termination and payment terms",
    "routed_agents": [],
    "routing_reasoning": "",
    "routing_approach": "",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "all_clauses": [],
    "total_clauses_retrieved": 0,
    "execution_order": [],
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "coordinator_model": "gemma2:9b",
    "vector_store": "Pinecone",
    "execution_status": "initialized"
}

print("EXAMPLE INITIAL STATE:")

print(json.dumps(example_state, indent=2))

print("✓ GRAPH STATE DEFINITION COMPLETE")

DEFINING SHARED GRAPH STATE FOR DYNAMIC COORDINATOR
GraphState defined with TypedDict
EXAMPLE INITIAL STATE:
{
  "query": "Analyze contract termination and payment terms",
  "routed_agents": [],
  "routing_reasoning": "",
  "routing_approach": "",
  "legal": {},
  "compliance": {},
  "finance": {},
  "operations": {},
  "all_clauses": [],
  "total_clauses_retrieved": 0,
  "execution_order": [],
  "timestamp": "20260115_142328",
  "coordinator_model": "gemma2:9b",
  "vector_store": "Pinecone",
  "execution_status": "initialized"
}
✓ GRAPH STATE DEFINITION COMPLETE


#### 3. Defining Agent Nodes

In [56]:
print("DEFINING AGENT NODES FOR DYNAMIC COORDINATOR")

print("DEFINING DYNAMIC AGENT NODES")

def legal_node(state: GraphState) -> GraphState:
    
    print(f"\n  → LEGAL AGENT executing")
    print(f"     Query: {state['query'][:60]}...")
    
    try:
        results = retrieve_from_pinecone(state['query'], agent_filter='legal', top_k=5)
        
        if results and results.matches:
    
            filtered_matches = [m for m in results.matches if m.score > 0.1]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["legal"] = {
                "agent": "Legal",
                "status": "completed",
                "source": "pinecone_vector_store",
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "relevance_scores": scores,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "max_relevance": max(scores) if scores else 0
            }
            
            for clause in clauses:
                clause['agent'] = 'legal'
                state["all_clauses"].append(clause)
            
            state["total_clauses_retrieved"] += len(clauses)
            
            print(f"     Retrieved: {len(clauses)} clauses")
            print(f"     Avg relevance: {state['legal']['avg_relevance']:.3f}")
        else:
            state["legal"] = {
                "agent": "Legal",
                "status": "no_results",
                "source": "pinecone_vector_store",
                "clauses_retrieved": 0,
                "clauses": [],
                "message": "No relevant legal clauses found"
            }
            print(f"     No relevant clauses found")
            
    except Exception as e:
        state["legal"] = {
            "agent": "Legal",
            "status": "error",
            "error": str(e)
        }
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("legal")
    return state


def compliance_node(state: GraphState) -> GraphState:

    print(f"\n  → COMPLIANCE AGENT executing")
    print(f"     Query: {state['query'][:60]}...")
    
    try:
        results = retrieve_from_pinecone(state['query'], agent_filter='compliance', top_k=5)
        
        if results and results.matches:
            filtered_matches = [m for m in results.matches if m.score > 0.1]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["compliance"] = {
                "agent": "Compliance",
                "status": "completed",
                "source": "pinecone_vector_store",
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "relevance_scores": scores,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "max_relevance": max(scores) if scores else 0
            }
            
            for clause in clauses:
                clause['agent'] = 'compliance'
                state["all_clauses"].append(clause)
            
            state["total_clauses_retrieved"] += len(clauses)
            
            print(f"     Retrieved: {len(clauses)} clauses")
            print(f"     Avg relevance: {state['compliance']['avg_relevance']:.3f}")
        else:
            state["compliance"] = {
                "agent": "Compliance",
                "status": "no_results",
                "source": "pinecone_vector_store",
                "clauses_retrieved": 0,
                "clauses": [],
                "message": "No relevant compliance clauses found"
            }
            print(f"      No relevant clauses found")
            
    except Exception as e:
        state["compliance"] = {
            "agent": "Compliance",
            "status": "error",
            "error": str(e)
        }
        print(f"      Error: {str(e)[:50]}")
    
    state["execution_order"].append("compliance")
    return state


def finance_node(state: GraphState) -> GraphState:
   
    print(f"\n  → FINANCE AGENT executing")
    print(f"     Query: {state['query'][:60]}...")
    
    try:
        results = retrieve_from_pinecone(state['query'], agent_filter='finance', top_k=5)
        
        if results and results.matches:
            filtered_matches = [m for m in results.matches if m.score > 0.1]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["finance"] = {
                "agent": "Finance",
                "status": "completed",
                "source": "pinecone_vector_store",
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "relevance_scores": scores,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "max_relevance": max(scores) if scores else 0
            }
            
            for clause in clauses:
                clause['agent'] = 'finance'
                state["all_clauses"].append(clause)
            
            state["total_clauses_retrieved"] += len(clauses)
            
            print(f"     Retrieved: {len(clauses)} clauses")
            print(f"     Avg relevance: {state['finance']['avg_relevance']:.3f}")
        else:
            state["finance"] = {
                "agent": "Finance",
                "status": "no_results",
                "source": "pinecone_vector_store",
                "clauses_retrieved": 0,
                "clauses": [],
                "message": "No relevant finance clauses found"
            }
            print(f"     No relevant clauses found")
            
    except Exception as e:
        state["finance"] = {
            "agent": "Finance",
            "status": "error",
            "error": str(e)
        }
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("finance")
    return state


def operations_node(state: GraphState) -> GraphState:
   
    print(f"\n  → OPERATIONS AGENT executing")
    print(f"     Query: {state['query'][:60]}...")
    
    try:
        results = retrieve_from_pinecone(state['query'], agent_filter='operations', top_k=5)
        
        if results and results.matches:
            filtered_matches = [m for m in results.matches if m.score > 0.1]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["operations"] = {
                "agent": "Operations",
                "status": "completed",
                "source": "pinecone_vector_store",
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "relevance_scores": scores,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "max_relevance": max(scores) if scores else 0
            }
            
            for clause in clauses:
                clause['agent'] = 'operations'
                state["all_clauses"].append(clause)
            
            state["total_clauses_retrieved"] += len(clauses)
            
            print(f"     Retrieved: {len(clauses)} clauses")
            print(f"     Avg relevance: {state['operations']['avg_relevance']:.3f}")
        else:
            state["operations"] = {
                "agent": "Operations",
                "status": "no_results",
                "source": "pinecone_vector_store",
                "clauses_retrieved": 0,
                "clauses": [],
                "message": "No relevant operations clauses found"
            }
            print(f"     No relevant clauses found")
            
    except Exception as e:
        state["operations"] = {
            "agent": "Operations",
            "status": "error",
            "error": str(e)
        }
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("operations")
    return state

print("AGENT NODES DEFINED")


DEFINING AGENT NODES FOR DYNAMIC COORDINATOR
DEFINING DYNAMIC AGENT NODES
AGENT NODES DEFINED


#### 4. Building Graph Skeleton

In [57]:
print("BUILDING GRAPH SKELETON FOR DYNAMIC COORDINATOR")

workflow = StateGraph(GraphState)

print("\nStateGraph initialized")

print("GRAPH PROPERTIES")

print(f"  Graph Type: {type(workflow).__name__}")

print("GRAPH WORKFLOW ARCHITECTURE")

print("GRAPH SKELETON READY")

BUILDING GRAPH SKELETON FOR DYNAMIC COORDINATOR

StateGraph initialized
GRAPH PROPERTIES
  Graph Type: StateGraph
GRAPH WORKFLOW ARCHITECTURE
GRAPH SKELETON READY


#### 5. Defining Edges - Simple Flow

In [58]:
print("DEFINING DYNAMIC WORKFLOW WITH CONDITIONAL ROUTING")

print("\nDefining routing node...")

def routing_node(state: GraphState) -> GraphState:

    print("ROUTING NODE: Dynamic LLM Coordinator")

    print(f"Query: \"{state['query']}\"")
    
    routing_result = dynamic_route_query(state['query'])
    
    state['routed_agents'] = routing_result['agents']
    state['routing_reasoning'] = routing_result['reasoning']
    state['routing_approach'] = routing_result['approach']
    
    print(f"\nRouting Decision:")
    print(f"  Agents: {', '.join(state['routed_agents'])}")
    print(f"  Reasoning: {state['routing_reasoning']}")
    print(f"  Approach: {state['routing_approach']}")
    
    state['execution_order'].append("routing")
    
    return state

print("Defining aggregation node...")

def aggregation_node(state: GraphState) -> GraphState:

    print("AGGREGATION NODE: Consolidating Results")

    if state['all_clauses']:
        state['all_clauses'].sort(key=lambda x: x.get('relevance_score', 0), reverse=True)
        
        print(f"\nTotal clauses retrieved: {state['total_clauses_retrieved']}")
        print(f"Top relevance score: {state['all_clauses'][0]['relevance_score']:.3f}")
        print(f"Agents executed: {', '.join([a for a in state['execution_order'] if a != 'routing' and a != 'aggregation'])}")
        
        state['execution_status'] = 'success'
    else:
        print(f"\nNo clauses retrieved")
        state['execution_status'] = 'no_results'
    
    state['execution_order'].append("aggregation")
    
    return state


print("Defining conditional routing logic...")

def should_invoke_agent(agent_name: str):

    def check(state: GraphState) -> str:
        if agent_name in state['routed_agents']:
            return agent_name
        else:
            return "skip"
    return check

print("ADDING NODES TO GRAPH")

workflow.add_node("routing", routing_node)
print("Added: routing (Dynamic LLM Coordinator)")

workflow.add_node("legal_agent", legal_node)
print("Added: legal_agent")

workflow.add_node("compliance_agent", compliance_node)
print("Added: compliance_agent")

workflow.add_node("finance_agent", finance_node)
print("Added: finance_agent")

workflow.add_node("operations_agent", operations_node)
print("Added: operations_agent")

workflow.add_node("aggregation", aggregation_node)
print("Added: aggregation")

print("DEFINING EXECUTION FLOW")

workflow.set_entry_point("routing")
print("Entry Point: routing")

print("\nAdding conditional edges from routing:")

workflow.add_conditional_edges(
    "routing",
    lambda state: "legal_agent" if "legal" in state['routed_agents'] else "aggregation",
    {
        "legal_agent": "legal_agent",
        "aggregation": "aggregation"
    }
)
print("  - routing → legal_agent (if legal selected)")

workflow.add_conditional_edges(
    "legal_agent",
    lambda state: "compliance_agent" if "compliance" in state['routed_agents'] else "aggregation",
    {
        "compliance_agent": "compliance_agent",
        "aggregation": "aggregation"
    }
)
print("  - legal_agent → compliance_agent (if compliance selected)")

workflow.add_conditional_edges(
    "compliance_agent",
    lambda state: "finance_agent" if "finance" in state['routed_agents'] else "aggregation",
    {
        "finance_agent": "finance_agent",
        "aggregation": "aggregation"
    }
)
print("  - compliance_agent → finance_agent (if finance selected)")

workflow.add_conditional_edges(
    "finance_agent",
    lambda state: "operations_agent" if "operations" in state['routed_agents'] else "aggregation",
    {
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("  - finance_agent → operations_agent (if operations selected)")

workflow.add_edge("operations_agent", "aggregation")
print("  - operations_agent → aggregation")

workflow.add_edge("aggregation", END)
print("  - aggregation → END")

print("DYNAMIC WORKFLOW DEFINED")

DEFINING DYNAMIC WORKFLOW WITH CONDITIONAL ROUTING

Defining routing node...
Defining aggregation node...
Defining conditional routing logic...
ADDING NODES TO GRAPH
Added: routing (Dynamic LLM Coordinator)
Added: legal_agent
Added: compliance_agent
Added: finance_agent
Added: operations_agent
Added: aggregation
DEFINING EXECUTION FLOW
Entry Point: routing

Adding conditional edges from routing:
  - routing → legal_agent (if legal selected)
  - legal_agent → compliance_agent (if compliance selected)
  - compliance_agent → finance_agent (if finance selected)
  - finance_agent → operations_agent (if operations selected)
  - operations_agent → aggregation
  - aggregation → END
DYNAMIC WORKFLOW DEFINED


#### 6. Compiling Graph

In [59]:
print("COMPILING DYNAMIC WORKFLOW GRAPH")

print("\nCompiling graph with conditional routing...")

try:
    app = workflow.compile()
    print("Graph compiled successfully!")
    
    print("COMPILED GRAPH DETAILS")

    print(f"  Type: {type(app).__name__}")
   
    
    print("GRAPH STRUCTURE VALIDATION")
   
    print("WORKFLOW EXECUTION PATH")
   
    print("READY FOR EXECUTION")
   
    
except Exception as e:
    print(f"\nCompilation failed: {str(e)}")
    print("\nDebug info:")
    print(f"  Error type: {type(e).__name__}")
    print(f"  Error message: {str(e)}")

print("GRAPH COMPILATION COMPLETE")

COMPILING DYNAMIC WORKFLOW GRAPH

Compiling graph with conditional routing...
Graph compiled successfully!
COMPILED GRAPH DETAILS
  Type: CompiledStateGraph
GRAPH STRUCTURE VALIDATION
WORKFLOW EXECUTION PATH
READY FOR EXECUTION
GRAPH COMPILATION COMPLETE


#### 7. Executing Graph

In [60]:
print("EXECUTING DYNAMIC WORKFLOW GRAPH")

print("\nEnsuring graph is compiled...")

try:
    if 'app' not in locals():
        print("Compiling workflow...")
        app = workflow.compile()
        print("Workflow compiled successfully")
    else:
        print("Using existing compiled workflow")
except Exception as e:
    print(f"Compilation error: {str(e)}")
    print("\nAttempting to rebuild workflow...")
    
    workflow = StateGraph(GraphState)
    
    workflow.add_node("routing", routing_node)
    workflow.add_node("legal_agent", legal_node)
    workflow.add_node("compliance_agent", compliance_node)
    workflow.add_node("finance_agent", finance_node)
    workflow.add_node("operations_agent", operations_node)
    workflow.add_node("aggregation", aggregation_node)
    
    workflow.set_entry_point("routing")
    
    workflow.add_conditional_edges(
        "routing",
        lambda state: "legal_agent" if "legal" in state['routed_agents'] else "aggregation",
        {"legal_agent": "legal_agent", "aggregation": "aggregation"}
    )
    workflow.add_conditional_edges(
        "legal_agent",
        lambda state: "compliance_agent" if "compliance" in state['routed_agents'] else "aggregation",
        {"compliance_agent": "compliance_agent", "aggregation": "aggregation"}
    )
    workflow.add_conditional_edges(
        "compliance_agent",
        lambda state: "finance_agent" if "finance" in state['routed_agents'] else "aggregation",
        {"finance_agent": "finance_agent", "aggregation": "aggregation"}
    )
    workflow.add_conditional_edges(
        "finance_agent",
        lambda state: "operations_agent" if "operations" in state['routed_agents'] else "aggregation",
        {"operations_agent": "operations_agent", "aggregation": "aggregation"}
    )
    workflow.add_edge("operations_agent", "aggregation")
    workflow.add_edge("aggregation", END)
    
    app = workflow.compile()
    print("Workflow rebuilt and compiled successfully")

print("PREPARING INITIAL STATE")

input_state = {
    "query": "Analyze contract for termination, GDPR compliance, data protection, privacy obligations, regulatory requirements, audit and reporting requirements, confidentiality and non-disclosure obligations, payment terms, deliverables, project outputs, timelines and milestones for delivery, performance standards and obligations, operational requirements and responsibilities, service level agreements and SLAs.",
    "routed_agents": [],
    "routing_reasoning": "",
    "routing_approach": "",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "all_clauses": [],
    "total_clauses_retrieved": 0,
    "execution_order": [],
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "coordinator_model": "gemma2:9b",
    "vector_store": "Pinecone",
    "execution_status": "initialized"
}

print("\nInitial State Created:")
print(json.dumps({
    "query": input_state["query"],
    "coordinator_model": input_state["coordinator_model"],
    "vector_store": input_state["vector_store"],
    "timestamp": input_state["timestamp"],
    "status": input_state["execution_status"]
}, indent=2))

print("EXECUTING MULTI-AGENT WORKFLOW")
print(f"Query: \"{input_state['query']}\"")
print("\nStarting execution...")

try:
    result = app.invoke(input_state)
    
    print("✓ EXECUTION COMPLETED SUCCESSFULLY")
  
    print("\nExecution Summary:")
    print(f"  Status: {result.get('execution_status', 'unknown')}")
    print(f"  Agents Routed: {', '.join(result.get('routed_agents', []))}")
    print(f"  Routing Reasoning: {result.get('routing_reasoning', 'N/A')}")
    print(f"  Execution Order: {' → '.join(result.get('execution_order', []))}")
    print(f"  Total Clauses Retrieved: {result.get('total_clauses_retrieved', 0)}")
    
    print("AGENT RESULTS")

    for agent_name in ['legal', 'compliance', 'finance', 'operations']:
        agent_result = result.get(agent_name, {})
        if agent_result:
            print(f"\n{agent_name.upper()} Agent:")
            print(f"  Status: {agent_result.get('status', 'unknown')}")
            
            if agent_result.get('status') == 'completed':
                print(f"  Clauses Retrieved: {agent_result.get('clauses_retrieved', 0)}")
                if agent_result.get('avg_relevance'):
                    print(f"  Avg Relevance: {agent_result['avg_relevance']:.3f}")
                    print(f"  Max Relevance: {agent_result['max_relevance']:.3f}")
    
    if result.get('all_clauses'):
   
        print("TOP RETRIEVED CLAUSES")

        
        for idx, clause in enumerate(result['all_clauses'][:5], 1):
            print(f"\n[{idx}] Agent: {clause['agent'].upper()} | Relevance: {clause['relevance_score']:.3f}")
            clause_text = clause['clause'][:150] + "..." if len(clause['clause']) > 150 else clause['clause']
            print(f"    {clause_text}")
    
    
    
except Exception as e:
    print(f"\nExecution failed: {str(e)}")
    print("ERROR DETAILS")
    print(f"  Error Type: {type(e).__name__}")
    print(f"  Error Message: {str(e)}")
    
    import traceback
    print("\nFull Traceback:")
    traceback.print_exc()

print("WORKFLOW EXECUTION COMPLETE")

EXECUTING DYNAMIC WORKFLOW GRAPH

Ensuring graph is compiled...
Using existing compiled workflow
PREPARING INITIAL STATE

Initial State Created:
{
  "query": "Analyze contract for termination, GDPR compliance, data protection, privacy obligations, regulatory requirements, audit and reporting requirements, confidentiality and non-disclosure obligations, payment terms, deliverables, project outputs, timelines and milestones for delivery, performance standards and obligations, operational requirements and responsibilities, service level agreements and SLAs.",
  "coordinator_model": "gemma2:9b",
  "vector_store": "Pinecone",
  "timestamp": "20260115_145315",
  "status": "initialized"
}
EXECUTING MULTI-AGENT WORKFLOW
Query: "Analyze contract for termination, GDPR compliance, data protection, privacy obligations, regulatory requirements, audit and reporting requirements, confidentiality and non-disclosure obligations, payment terms, deliverables, project outputs, timelines and milestones for

#### 8. Inspecting Output

In [61]:
print("INSPECTING WORKFLOW OUTPUT")

print("\nInspecting Graph Execution Results...")

print("STATE STRUCTURE")

print(f"State Keys: {list(result.keys())}")


print("ROUTING INFORMATION")

print(f"Query: \"{result['query']}\"")
print(f"Coordinator Model: {result.get('coordinator_model', 'N/A')}")
print(f"Vector Store: {result.get('vector_store', 'N/A')}")
print(f"Routing Approach: {result.get('routing_approach', 'N/A')}")
print(f"Agents Routed: {', '.join(result.get('routed_agents', []))}")
print(f"Routing Reasoning: {result.get('routing_reasoning', 'N/A')}")


print("EXECUTION ORDER")

for idx, node in enumerate(result['execution_order'], 1):
    print(f"  {idx}. {node}")

print("AGENT RESULTS INSPECTION")

print(f"\n1. LEGAL AGENT:")
if result['legal']:
    print(f"   Status: {result['legal'].get('status', 'unknown')}")
    print(f"   Source: {result['legal'].get('source', 'N/A')}")
    print(f"   Clauses Retrieved: {result['legal'].get('clauses_retrieved', 0)}")
    
    if result['legal'].get('status') == 'completed':
        print(f"   Avg Relevance: {result['legal'].get('avg_relevance', 0):.3f}")
        print(f"   Max Relevance: {result['legal'].get('max_relevance', 0):.3f}")
        
        clauses = result['legal'].get('clauses', [])
        if clauses:
            print(f"   Sample Clauses:")
            for i, clause_data in enumerate(clauses[:2], 1):
                clause_text = clause_data['clause'][:100] + "..." if len(clause_data['clause']) > 100 else clause_data['clause']
                print(f"      {i}. [{clause_data['relevance_score']:.3f}] {clause_text}")
else:
    print("   No data")

print(f"\n2. COMPLIANCE AGENT:")
if result['compliance']:
    print(f"   Status: {result['compliance'].get('status', 'unknown')}")
    print(f"   Source: {result['compliance'].get('source', 'N/A')}")
    print(f"   Clauses Retrieved: {result['compliance'].get('clauses_retrieved', 0)}")
    
    if result['compliance'].get('status') == 'completed':
        print(f"   Avg Relevance: {result['compliance'].get('avg_relevance', 0):.3f}")
        print(f"   Max Relevance: {result['compliance'].get('max_relevance', 0):.3f}")
        
        clauses = result['compliance'].get('clauses', [])
        if clauses:
            print(f"   Sample Clauses:")
            for i, clause_data in enumerate(clauses[:2], 1):
                clause_text = clause_data['clause'][:100] + "..." if len(clause_data['clause']) > 100 else clause_data['clause']
                print(f"      {i}. [{clause_data['relevance_score']:.3f}] {clause_text}")
else:
    print("   No data")

print(f"\n3. FINANCE AGENT:")
if result['finance']:
    print(f"   Status: {result['finance'].get('status', 'unknown')}")
    print(f"   Source: {result['finance'].get('source', 'N/A')}")
    print(f"   Clauses Retrieved: {result['finance'].get('clauses_retrieved', 0)}")
    
    if result['finance'].get('status') == 'completed':
        print(f"   Avg Relevance: {result['finance'].get('avg_relevance', 0):.3f}")
        print(f"   Max Relevance: {result['finance'].get('max_relevance', 0):.3f}")
        
        clauses = result['finance'].get('clauses', [])
        if clauses:
            print(f"   Sample Clauses:")
            for i, clause_data in enumerate(clauses[:2], 1):
                clause_text = clause_data['clause'][:100] + "..." if len(clause_data['clause']) > 100 else clause_data['clause']
                print(f"      {i}. [{clause_data['relevance_score']:.3f}] {clause_text}")
else:
    print("   No data")

print(f"\n4. OPERATIONS AGENT:")
if result['operations']:
    print(f"   Status: {result['operations'].get('status', 'unknown')}")
    print(f"   Source: {result['operations'].get('source', 'N/A')}")
    print(f"   Clauses Retrieved: {result['operations'].get('clauses_retrieved', 0)}")
    
    if result['operations'].get('status') == 'completed':
        print(f"   Avg Relevance: {result['operations'].get('avg_relevance', 0):.3f}")
        print(f"   Max Relevance: {result['operations'].get('max_relevance', 0):.3f}")
        
        clauses = result['operations'].get('clauses', [])
        if clauses:
            print(f"   Sample Clauses:")
            for i, clause_data in enumerate(clauses[:2], 1):
                clause_text = clause_data['clause'][:100] + "..." if len(clause_data['clause']) > 100 else clause_data['clause']
                print(f"      {i}. [{clause_data['relevance_score']:.3f}] {clause_text}")
else:
    print("   No data")

total_clauses = result.get('total_clauses_retrieved', 0)
total_agents_executed = len([a for a in result['execution_order'] if a not in ['routing', 'aggregation']])

print("OVERALL STATISTICS")
print(f"Total Agents Executed: {total_agents_executed}")
print(f"Total Clauses Retrieved: {total_clauses}")
print(f"Execution Status: {result.get('execution_status', 'unknown')}")

print("SAVING RESULTS")

LANGGRAPH_OUTPUT = "../Data/Results/LangGraph"
os.makedirs(LANGGRAPH_OUTPUT, exist_ok=True)

timestamp = result.get('timestamp', datetime.now().strftime("%Y%m%d_%H%M%S"))
output_file = f"langgraph_execution_{timestamp}.json"
output_path = os.path.join(LANGGRAPH_OUTPUT, output_file)

output_data = {
    "execution_metadata": {
        "query": result['query'],
        "coordinator_model": result.get('coordinator_model', 'N/A'),
        "vector_store": result.get('vector_store', 'N/A'),
        "timestamp": result['timestamp'],
        "execution_status": result.get('execution_status', 'unknown')
    },
    "routing": {
        "approach": result.get('routing_approach', 'N/A'),
        "agents_routed": result.get('routed_agents', []),
        "reasoning": result.get('routing_reasoning', 'N/A')
    },
    "execution_order": result['execution_order'],
    "agent_results": {
        "legal": {
            "status": result['legal'].get('status', 'unknown'),
            "source": result['legal'].get('source', 'N/A'),
            "clauses_retrieved": result['legal'].get('clauses_retrieved', 0),
            "avg_relevance": result['legal'].get('avg_relevance', 0),
            "max_relevance": result['legal'].get('max_relevance', 0)
        },
        "compliance": {
            "status": result['compliance'].get('status', 'unknown'),
            "source": result['compliance'].get('source', 'N/A'),
            "clauses_retrieved": result['compliance'].get('clauses_retrieved', 0),
            "avg_relevance": result['compliance'].get('avg_relevance', 0),
            "max_relevance": result['compliance'].get('max_relevance', 0)
        },
        "finance": {
            "status": result['finance'].get('status', 'unknown'),
            "source": result['finance'].get('source', 'N/A'),
            "clauses_retrieved": result['finance'].get('clauses_retrieved', 0),
            "avg_relevance": result['finance'].get('avg_relevance', 0),
            "max_relevance": result['finance'].get('max_relevance', 0)
        },
        "operations": {
            "status": result['operations'].get('status', 'unknown'),
            "source": result['operations'].get('source', 'N/A'),
            "clauses_retrieved": result['operations'].get('clauses_retrieved', 0),
            "avg_relevance": result['operations'].get('avg_relevance', 0),
            "max_relevance": result['operations'].get('max_relevance', 0)
        }
    },
    "all_clauses": result.get('all_clauses', []),
    "summary": {
        "total_agents_executed": total_agents_executed,
        "total_clauses_retrieved": total_clauses,
        "workflow_type": "Dynamic LLM-based coordination with Pinecone retrieval"
    }
}

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"\nResults saved successfully!")
print(f"  File: {output_file}")
print(f"  Location: {LANGGRAPH_OUTPUT}")
print(f"  Size: {len(json.dumps(output_data))} bytes")

print("INSPECTION AND SAVE COMPLETE")

INSPECTING WORKFLOW OUTPUT

Inspecting Graph Execution Results...
STATE STRUCTURE
State Keys: ['query', 'routed_agents', 'routing_reasoning', 'routing_approach', 'legal', 'compliance', 'finance', 'operations', 'all_clauses', 'total_clauses_retrieved', 'execution_order', 'timestamp', 'coordinator_model', 'vector_store', 'execution_status']
ROUTING INFORMATION
Query: "Analyze contract for termination, GDPR compliance, data protection, privacy obligations, regulatory requirements, audit and reporting requirements, confidentiality and non-disclosure obligations, payment terms, deliverables, project outputs, timelines and milestones for delivery, performance standards and obligations, operational requirements and responsibilities, service level agreements and SLAs."
Coordinator Model: gemma2:9b
Vector Store: Pinecone
Routing Approach: dynamic_llm_coordinator
Agents Routed: legal, compliance, finance, operations
Routing Reasoning: The query touches upon legal clauses (termination), data prot

#### 9. Changing Execution Order

In [62]:
print("MODIFIED CONDITIONAL BRANCHING ORDER")


workflow_reordered = StateGraph(GraphState)

workflow_reordered.add_node("routing", routing_node)
workflow_reordered.add_node("legal_agent", legal_node)
workflow_reordered.add_node("compliance_agent", compliance_node)
workflow_reordered.add_node("finance_agent", finance_node)
workflow_reordered.add_node("operations_agent", operations_node)
workflow_reordered.add_node("aggregation", aggregation_node)

workflow_reordered.set_entry_point("routing")

print("\nDefining new conditional flow:")

workflow_reordered.add_conditional_edges(
    "routing",
    lambda state: "compliance_agent" if "compliance" in state['routed_agents'] else "finance_agent" if "finance" in state['routed_agents'] else "legal_agent" if "legal" in state['routed_agents'] else "operations_agent" if "operations" in state['routed_agents'] else "aggregation",
    {
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "legal_agent": "legal_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("  routing → compliance (priority 1)")

workflow_reordered.add_conditional_edges(
    "compliance_agent",
    lambda state: "finance_agent" if "finance" in state['routed_agents'] else "legal_agent" if "legal" in state['routed_agents'] else "operations_agent" if "operations" in state['routed_agents'] else "aggregation",
    {
        "finance_agent": "finance_agent",
        "legal_agent": "legal_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("  compliance → finance (priority 2)")

workflow_reordered.add_conditional_edges(
    "finance_agent",
    lambda state: "legal_agent" if "legal" in state['routed_agents'] else "operations_agent" if "operations" in state['routed_agents'] else "aggregation",
    {
        "legal_agent": "legal_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("  finance → legal (priority 3)")

workflow_reordered.add_conditional_edges(
    "legal_agent",
    lambda state: "operations_agent" if "operations" in state['routed_agents'] else "aggregation",
    {
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("  legal → operations (priority 4)")

workflow_reordered.add_edge("operations_agent", "aggregation")
workflow_reordered.add_edge("aggregation", END)
print("  operations → aggregation → END")

print("\nCompiling reordered workflow...")
app_reordered = workflow_reordered.compile()
print("Graph compiled successfully")


print("EXECUTING REORDERED WORKFLOW")

input_state_reordered = {
    "query": "Analyze contract for GDPR compliance, payment terms, termination, and SLAs",
    "routed_agents": [],
    "routing_reasoning": "",
    "routing_approach": "",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "all_clauses": [],
    "total_clauses_retrieved": 0,
    "execution_order": [],
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "coordinator_model": "gemma2:9b",
    "vector_store": "Pinecone",
    "execution_status": "initialized"
}

print(f"Query: \"{input_state_reordered['query']}\"")
print("\nExpected order: Compliance → Finance → Legal → Operations")
print("(Based on LLM routing + new conditional priority)")

try:
    result_reordered = app_reordered.invoke(input_state_reordered)
    
    print("EXECUTION ORDER COMPARISON")

    
    print(f"\nORIGINAL ORDER (from earlier execution):")
    print(f"  Legal → Compliance → Finance → Operations")
    
    print(f"\nNEW ORDER (reordered workflow):")
    print(f"  {' → '.join(result_reordered['execution_order'])}")
    
    print(f"\nRouted Agents:")
    print(f"  {', '.join(result_reordered.get('routed_agents', []))}")
    
    print(f"\nRouting Reasoning:")
    print(f"  {result_reordered.get('routing_reasoning', 'N/A')}")
    
    execution_agents = [node for node in result_reordered['execution_order'] if node not in ['routing', 'aggregation']]
    
    print("ORDER CHANGE VERIFICATION")

    if execution_agents and execution_agents[0] == 'compliance':
        print("SUCCESS: Execution order changed!")
        print(f"  First agent: Compliance (as intended)")
        print(f"  Full agent order: {' → '.join(execution_agents)}")
    else:
        print("Note: Actual order depends on which agents LLM selected")
        print(f"  Agents executed: {' → '.join(execution_agents) if execution_agents else 'None'}")
    

    print("RESULTS SUMMARY")

    print(f"Total Clauses Retrieved: {result_reordered.get('total_clauses_retrieved', 0)}")
    print(f"Execution Status: {result_reordered.get('execution_status', 'unknown')}")
    
    print("\nAgent Results:")
    for agent_name in execution_agents:
        agent_result = result_reordered.get(agent_name, {})
        if agent_result:
            status = agent_result.get('status', 'unknown')
            clauses = agent_result.get('clauses_retrieved', 0)
            print(f"  {agent_name.upper()}: {status} - {clauses} clauses")
    
except Exception as e:
    print(f"\n✗ Execution failed: {str(e)}")
    import traceback
    traceback.print_exc()

print("EXECUTION ORDER CHANGE DEMONSTRATION COMPLETE")

MODIFIED CONDITIONAL BRANCHING ORDER

Defining new conditional flow:
  routing → compliance (priority 1)
  compliance → finance (priority 2)
  finance → legal (priority 3)
  legal → operations (priority 4)
  operations → aggregation → END

Compiling reordered workflow...
Graph compiled successfully
EXECUTING REORDERED WORKFLOW
Query: "Analyze contract for GDPR compliance, payment terms, termination, and SLAs"

Expected order: Compliance → Finance → Legal → Operations
(Based on LLM routing + new conditional priority)
ROUTING NODE: Dynamic LLM Coordinator
Query: "Analyze contract for GDPR compliance, payment terms, termination, and SLAs"

Querying gemma2:9b coordinator...

Routing Decision:
  Agents: compliance, finance, legal, operations
  Reasoning: GDPR compliance falls under data protection handled by the Compliance agent. Payment terms are handled by the Finance agent. Termination clauses and SLAs are covered by the Legal and Operations agents respectively.
  Approach: dynamic_llm_c

#### 10. Removing One Agent

In [63]:
print("BUILDING WORKFLOW WITHOUT FINANCE AGENT")

workflow_no_finance = StateGraph(GraphState)

workflow_no_finance.add_node("routing", routing_node)
workflow_no_finance.add_node("legal_agent", legal_node)
workflow_no_finance.add_node("compliance_agent", compliance_node)
workflow_no_finance.add_node("operations_agent", operations_node)
workflow_no_finance.add_node("aggregation", aggregation_node)

print("Nodes added: routing, legal, compliance, operations, aggregation")
print("Finance agent excluded")

workflow_no_finance.set_entry_point("routing")

print("\nDefining conditional flow without finance agent:")

workflow_no_finance.add_conditional_edges(
    "routing",
    lambda state: "legal_agent" if "legal" in state['routed_agents'] else "compliance_agent" if "compliance" in state['routed_agents'] else "operations_agent" if "operations" in state['routed_agents'] else "aggregation",
    {
        "legal_agent": "legal_agent",
        "compliance_agent": "compliance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)

workflow_no_finance.add_conditional_edges(
    "legal_agent",
    lambda state: "compliance_agent" if "compliance" in state['routed_agents'] else "operations_agent" if "operations" in state['routed_agents'] else "aggregation",
    {
        "compliance_agent": "compliance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)

workflow_no_finance.add_conditional_edges(
    "compliance_agent",
    lambda state: "operations_agent" if "operations" in state['routed_agents'] else "aggregation",
    {
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)

workflow_no_finance.add_edge("operations_agent", "aggregation")
workflow_no_finance.add_edge("aggregation", END)

print("Conditional edges defined (skipping finance)")

print("\nCompiling workflow without finance agent...")
app_no_finance = workflow_no_finance.compile()
print("Graph compiled successfully")

print("EXECUTING WITHOUT FINANCE AGENT")

input_state_no_finance = {
    "query": "Analyze contract for payment terms, termination clauses, and GDPR compliance",
    "routed_agents": [],
    "routing_reasoning": "",
    "routing_approach": "",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "all_clauses": [],
    "total_clauses_retrieved": 0,
    "execution_order": [],
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "coordinator_model": "gemma2:9b",
    "vector_store": "Pinecone",
    "execution_status": "initialized"
}

print(f"Query: \"{input_state_no_finance['query']}\"")

try:
    result_no_finance = app_no_finance.invoke(input_state_no_finance)
    
    print("\n" + "="*80)
    print("EXECUTION RESULTS")
    print("="*80)
    
    print(f"\nRouted Agents (by LLM):")
    print(f"  {', '.join(result_no_finance.get('routed_agents', []))}")
    
    print(f"\nActual Execution Order:")
    print(f"  {' → '.join(result_no_finance['execution_order'])}")
    
    print(f"\nRouting Reasoning:")
    print(f"  {result_no_finance.get('routing_reasoning', 'N/A')}")
    

    print("STATE COMPARISON: WITH vs WITHOUT FINANCE")

    
    print("\nAgent States:")
    print(f"{'Agent':<15} {'Status':<15} {'Clauses':<10} {'Notes'}")
    print("-" * 70)
    
    for agent_name in ["legal", "compliance", "finance", "operations"]:
        agent_result = result_no_finance.get(agent_name, {})
        status = agent_result.get("status", "not_in_graph")
        clauses = agent_result.get("clauses_retrieved", 0)
        
        if agent_name == "finance":
            notes = "REMOVED (not in graph)"
        elif not agent_result:
            notes = "Empty state"
        elif status == "completed":
            notes = f"Retrieved from Pinecone"
        elif status == "no_results":
            notes = "No relevant clauses"
        else:
            notes = status
        
        print(f"{agent_name.upper():<15} {status:<15} {clauses:<10} {notes}")
    
    print("KEY OBSERVATIONS")

    
    routed_agents = result_no_finance.get('routed_agents', [])
    executed_agents = [a for a in result_no_finance['execution_order'] if a not in ['routing', 'aggregation']]
    
    print("\n1. ROUTING vs EXECUTION:")
    print(f"   LLM Routed To: {', '.join(routed_agents)}")
    print(f"   Actually Executed: {', '.join(executed_agents)}")
    
    if 'finance' in routed_agents and 'finance' not in executed_agents:
        print(f"   Finance was routed but NOT executed (removed from graph)")
    
    print("\n2. FINANCE STATE:")
    finance_state = result_no_finance.get('finance', {})
    if not finance_state or finance_state == {}:
        print("   Finance state remains EMPTY (agent never executed)")
        print("   No error - graph handles missing agent gracefully")
    else:
        print(f"   Finance state: {finance_state}")
    
    print("\n3. OTHER AGENTS:")
    other_agents_executed = [a for a in executed_agents if a != 'finance']
    if other_agents_executed:
        print(f"   Other agents executed normally: {', '.join(other_agents_executed)}")
        print("   Workflow continues despite missing agent")
    else:
        print("    No agents executed")
    
    print("\n4. TOTAL CLAUSES:")
    total_clauses = result_no_finance.get('total_clauses_retrieved', 0)
    print(f"   Total clauses retrieved: {total_clauses}")
    print(f"   Source: {', '.join(executed_agents) if executed_agents else 'None'}")

    LANGGRAPH_OUTPUT = "../Data/Results/LangGraph"
    os.makedirs(LANGGRAPH_OUTPUT, exist_ok=True)
    
    timestamp = result_no_finance.get('timestamp', datetime.now().strftime("%Y%m%d_%H%M%S"))
    
    removal_results = {
        "task": "Remove Agent and Observe State Changes",
        "removed_agent": "finance",
        "query": input_state_no_finance['query'],
        "coordinator_model": "gemma2:9b",
        "timestamp": timestamp,
        "routing": {
            "llm_routed_agents": result_no_finance.get('routed_agents', []),
            "reasoning": result_no_finance.get('routing_reasoning', 'N/A'),
            "approach": result_no_finance.get('routing_approach', 'N/A')
        },
        "execution": {
            "execution_order": result_no_finance['execution_order'],
            "agents_executed": executed_agents,
            "finance_in_routed": 'finance' in routed_agents,
            "finance_executed": 'finance' in executed_agents
        },
        "state_analysis": {
            "legal": {
                "status": result_no_finance.get('legal', {}).get('status', 'empty'),
                "clauses": result_no_finance.get('legal', {}).get('clauses_retrieved', 0)
            },
            "compliance": {
                "status": result_no_finance.get('compliance', {}).get('status', 'empty'),
                "clauses": result_no_finance.get('compliance', {}).get('clauses_retrieved', 0)
            },
            "finance": {
                "status": "removed_from_graph",
                "state": result_no_finance.get('finance', {}),
                "clauses": 0,
                "note": "Agent not in workflow - state remains empty"
            },
            "operations": {
                "status": result_no_finance.get('operations', {}).get('status', 'empty'),
                "clauses": result_no_finance.get('operations', {}).get('clauses_retrieved', 0)
            }
        },
        "totals": {
            "total_clauses_retrieved": result_no_finance.get('total_clauses_retrieved', 0),
            "execution_status": result_no_finance.get('execution_status', 'unknown')
        },
        "key_learnings": [
            "Agents can be removed from graph dynamically",
            "LLM may route to removed agent, but it won't execute",
            "Removed agent's state remains empty (no error)",
            "Workflow continues normally with remaining agents",
            "Dynamic coordination handles missing agents gracefully",
            "State structure consistent regardless of available agents"
        ]
    }
    
    student_file = f"langgraph_remove_agent_{timestamp}.json"
    student_path = os.path.join(LANGGRAPH_OUTPUT, student_file)
    
    with open(student_path, 'w', encoding='utf-8') as f:
        json.dump(removal_results, f, indent=2, ensure_ascii=False)
    
    print(f"\nResults saved: {student_file}")
    print(f"  Location: {LANGGRAPH_OUTPUT}")
    
except Exception as e:
    print(f"\nExecution failed: {str(e)}")
    import traceback
    traceback.print_exc()

print("AGENT REMOVAL DEMONSTRATION COMPLETE")

BUILDING WORKFLOW WITHOUT FINANCE AGENT
Nodes added: routing, legal, compliance, operations, aggregation
Finance agent excluded

Defining conditional flow without finance agent:
Conditional edges defined (skipping finance)

Compiling workflow without finance agent...
Graph compiled successfully
EXECUTING WITHOUT FINANCE AGENT
Query: "Analyze contract for payment terms, termination clauses, and GDPR compliance"
ROUTING NODE: Dynamic LLM Coordinator
Query: "Analyze contract for payment terms, termination clauses, and GDPR compliance"

Querying gemma2:9b coordinator...

Routing Decision:
  Agents: finance, compliance
  Reasoning: The query mentions payment terms (Finance) and GDPR compliance (Compliance).
  Approach: dynamic_llm_coordinator

  → COMPLIANCE AGENT executing
     Query: Analyze contract for payment terms, termination clauses, and...
     Retrieved: 1 clauses
     Avg relevance: 0.202
AGGREGATION NODE: Consolidating Results

Total clauses retrieved: 1
Top relevance score: 0.2

# Conditional Routing in LanGraph

#### 1. Defining Routing Function

In [64]:
import os
import json
from datetime import datetime
from typing import TypedDict, Dict, Any, List, Literal
from collections import defaultdict

try:
    from langgraph.graph import StateGraph, END
except:
    from langgraph.graph import Graph as StateGraph, END

In [65]:
print("DEFINING CONDITIONAL ROUTING WITH DYNAMIC LLM COORDINATOR")

AGENT_THRESHOLDS = {
    "legal": 0.10,      
    "compliance": 0.10,  
    "finance": 0.10,     
    "operations": 0.10   
}

print("\nAgent Retrieval Thresholds:")
for agent, threshold in AGENT_THRESHOLDS.items():
    print(f"  {agent.upper()}: {threshold:.2f} (relevance score)")

print("RETRIEVAL FUNCTION")

def retrieve_from_pinecone_adaptive(query, agent_filter=None, top_k=5):
    
    try:
        query_embedding = create_embedding(query)
        
        if agent_filter:
            results = index.query(
                vector=query_embedding,
                filter={'agent': agent_filter},
                top_k=top_k,
                include_metadata=True
            )
        else:
            results = index.query(
                vector=query_embedding,
                top_k=top_k,
                include_metadata=True
            )
        
        return results
    except Exception as e:
        print(f"Error retrieving from Pinecone: {str(e)}")
        return None

print("retrieve_from_pinecone_adaptive() defined")

print("DEFINING CONDITIONAL ROUTING LOGIC")


def conditional_route_to_agents(state: GraphState) -> List[str]:
  
    query = state.get("query", "").lower()
    
    print("CONDITIONAL ROUTING ANALYSIS")

    print(f"Query: \"{state['query']}\"")
    
    routing_result = dynamic_route_query(state['query'])
    
    state['routed_agents'] = routing_result['agents']
    state['routing_reasoning'] = routing_result['reasoning']
    state['routing_approach'] = routing_result['approach']
    
    print(f"\nLLM Routing Decision:")
    print(f"  Agents: {', '.join(routing_result['agents'])}")
    print(f"  Reasoning: {routing_result['reasoning']}")
    print(f"  Approach: {routing_result['approach']}")
    
    agent_count = len(routing_result['agents'])
    
    if agent_count == 0:
        print(f"\nNo agents selected - routing to aggregation")
        return ["aggregation"]
    elif agent_count == 1:
        print(f"\nSingle-agent query - direct routing")
        return routing_result['agents']
    elif agent_count == 2:
        print(f"\nMulti-domain query - conditional parallel execution")
        return routing_result['agents']
    else:
        print(f"\nComplex query - all agents with priority ordering")
        return routing_result['agents']

print("conditional_route_to_agents() defined")


print("DEFINING ROUTING DECISION FUNCTION FOR LANGGRAPH")

def route_to_next_agent(state: GraphState) -> str:

    routed_agents = state.get('routed_agents', [])
    executed_agents = [a for a in state.get('execution_order', []) 
                      if a not in ['routing', 'aggregation']]
    
    for agent in routed_agents:
        if agent not in executed_agents:
            next_node = f"{agent}_agent"
            print(f"  → Next agent: {agent.upper()}")
            return next_node
    
    print(f"  → All agents executed - routing to aggregation")
    return "aggregation"

print("route_to_next_agent() defined")

def legal_node_conditional(state: GraphState) -> GraphState:
    print(f"\n  → LEGAL AGENT executing (threshold: {AGENT_THRESHOLDS['legal']})")
    print(f"     Query: {state['query'][:60]}...")
    
    try:
        results = retrieve_from_pinecone_adaptive(state['query'], agent_filter='legal', top_k=5)
        
        if results and results.matches:
            threshold = AGENT_THRESHOLDS['legal']
            filtered_matches = [m for m in results.matches if m.score > threshold]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["legal"] = {
                "agent": "Legal",
                "status": "completed",
                "source": "pinecone_vector_store",
                "threshold": threshold,
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "relevance_scores": scores,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "max_relevance": max(scores) if scores else 0
            }
            
            for clause in clauses:
                clause['agent'] = 'legal'
                state["all_clauses"].append(clause)
            
            state["total_clauses_retrieved"] += len(clauses)
            print(f"     Retrieved: {len(clauses)} clauses (avg: {state['legal']['avg_relevance']:.3f})")
        else:
            state["legal"] = {
                "agent": "Legal",
                "status": "no_results",
                "source": "pinecone_vector_store",
                "threshold": AGENT_THRESHOLDS['legal'],
                "clauses_retrieved": 0,
                "clauses": []
            }
            print(f"     No clauses above threshold")
    except Exception as e:
        state["legal"] = {"agent": "Legal", "status": "error", "error": str(e)}
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("legal")
    return state


def compliance_node_conditional(state: GraphState) -> GraphState:
    print(f"\n  → COMPLIANCE AGENT executing (threshold: {AGENT_THRESHOLDS['compliance']})")
    print(f"     Query: {state['query'][:60]}...")
    
    try:
        results = retrieve_from_pinecone_adaptive(state['query'], agent_filter='compliance', top_k=5)
        
        if results and results.matches:
            threshold = AGENT_THRESHOLDS['compliance'] 
            filtered_matches = [m for m in results.matches if m.score > threshold]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["compliance"] = {
                "agent": "Compliance",
                "status": "completed",
                "source": "pinecone_vector_store",
                "threshold": threshold,
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "relevance_scores": scores,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "max_relevance": max(scores) if scores else 0
            }
            
            for clause in clauses:
                clause['agent'] = 'compliance'
                state["all_clauses"].append(clause)
            
            state["total_clauses_retrieved"] += len(clauses)
            print(f"     Retrieved: {len(clauses)} clauses (avg: {state['compliance']['avg_relevance']:.3f})")
        else:
            state["compliance"] = {
                "agent": "Compliance",
                "status": "no_results",
                "source": "pinecone_vector_store",
                "threshold": AGENT_THRESHOLDS['compliance'],
                "clauses_retrieved": 0,
                "clauses": []
            }
            print(f"     No clauses above threshold")
    except Exception as e:
        state["compliance"] = {"agent": "Compliance", "status": "error", "error": str(e)}
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("compliance")
    return state


def finance_node_conditional(state: GraphState) -> GraphState:
    print(f"\n  → FINANCE AGENT executing (threshold: {AGENT_THRESHOLDS['finance']})")
    print(f"     Query: {state['query'][:60]}...")
    
    try:
        results = retrieve_from_pinecone_adaptive(state['query'], agent_filter='finance', top_k=5)
        
        if results and results.matches:
            threshold = AGENT_THRESHOLDS['finance']
            filtered_matches = [m for m in results.matches if m.score > threshold]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["finance"] = {
                "agent": "Finance",
                "status": "completed",
                "source": "pinecone_vector_store",
                "threshold": threshold,
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "relevance_scores": scores,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "max_relevance": max(scores) if scores else 0
            }
            
            for clause in clauses:
                clause['agent'] = 'finance'
                state["all_clauses"].append(clause)
            
            state["total_clauses_retrieved"] += len(clauses)
            print(f"     Retrieved: {len(clauses)} clauses (avg: {state['finance']['avg_relevance']:.3f})")
        else:
            state["finance"] = {
                "agent": "Finance",
                "status": "no_results",
                "source": "pinecone_vector_store",
                "threshold": AGENT_THRESHOLDS['finance'],
                "clauses_retrieved": 0,
                "clauses": []
            }
            print(f"     No clauses above threshold")
    except Exception as e:
        state["finance"] = {"agent": "Finance", "status": "error", "error": str(e)}
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("finance")
    return state


def operations_node_conditional(state: GraphState) -> GraphState:
    print(f"\n  → OPERATIONS AGENT executing (threshold: {AGENT_THRESHOLDS['operations']})")
    print(f"     Query: {state['query'][:60]}...")
    
    try:
        results = retrieve_from_pinecone_adaptive(state['query'], agent_filter='operations', top_k=5)
        
        if results and results.matches:
            threshold = AGENT_THRESHOLDS['operations']  
            filtered_matches = [m for m in results.matches if m.score > threshold]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["operations"] = {
                "agent": "Operations",
                "status": "completed",
                "source": "pinecone_vector_store",
                "threshold": threshold,
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "relevance_scores": scores,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "max_relevance": max(scores) if scores else 0
            }
            
            for clause in clauses:
                clause['agent'] = 'operations'
                state["all_clauses"].append(clause)
            
            state["total_clauses_retrieved"] += len(clauses)
            print(f"     Retrieved: {len(clauses)} clauses (avg: {state['operations']['avg_relevance']:.3f})")
        else:
            state["operations"] = {
                "agent": "Operations",
                "status": "no_results",
                "source": "pinecone_vector_store",
                "threshold": AGENT_THRESHOLDS['operations'],
                "clauses_retrieved": 0,
                "clauses": []
            }
            print(f"     No clauses above threshold")
    except Exception as e:
        state["operations"] = {"agent": "Operations", "status": "error", "error": str(e)}
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("operations")
    return state

print("All agent nodes updated with adaptive thresholds")

print("CONDITIONAL ROUTING SYSTEM READY")

DEFINING CONDITIONAL ROUTING WITH DYNAMIC LLM COORDINATOR

Agent Retrieval Thresholds:
  LEGAL: 0.10 (relevance score)
  COMPLIANCE: 0.10 (relevance score)
  FINANCE: 0.10 (relevance score)
  OPERATIONS: 0.10 (relevance score)
RETRIEVAL FUNCTION
retrieve_from_pinecone_adaptive() defined
DEFINING CONDITIONAL ROUTING LOGIC
conditional_route_to_agents() defined
DEFINING ROUTING DECISION FUNCTION FOR LANGGRAPH
route_to_next_agent() defined
All agent nodes updated with adaptive thresholds
CONDITIONAL ROUTING SYSTEM READY


#### 2. Rebuild Graph with Conditional Entry

In [67]:
print("BUILDING CONDITIONAL ROUTING WORKFLOW WITH LANGGRAPH")

print("BUILDING CONDITIONAL ROUTING GRAPH")

workflow_conditional = StateGraph(GraphState)

print("\nAdding nodes to graph:")
workflow_conditional.add_node("routing", routing_node)
print("  routing (entry point)")

workflow_conditional.add_node("legal_agent", legal_node_conditional)
print("  legal_agent (threshold: 0.30)")

workflow_conditional.add_node("compliance_agent", compliance_node_conditional)
print("  compliance_agent (threshold: 0.10)")

workflow_conditional.add_node("finance_agent", finance_node_conditional)
print("  finance_agent (threshold: 0.30)")

workflow_conditional.add_node("operations_agent", operations_node_conditional)
print("  operations_agent (threshold: 0.10)")

workflow_conditional.add_node("aggregation", aggregation_node)
print("  aggregation (consolidation)")

workflow_conditional.set_entry_point("routing")
print("\nEntry point: routing")


print("DEFINING CONDITIONAL EDGES")

def get_next_agent_or_aggregation(state: GraphState, current_agent: str = None) -> str:
   
    routed_agents = state.get('routed_agents', [])
    executed_agents = [a for a in state.get('execution_order', []) 
                      if a not in ['routing', 'aggregation']]
    
    agent_priority = ['legal', 'compliance', 'finance', 'operations']
    
    for agent in agent_priority:
        if agent in routed_agents and agent not in executed_agents:
            return f"{agent}_agent"
    
    return "aggregation"

workflow_conditional.add_conditional_edges(
    "routing",
    lambda state: get_next_agent_or_aggregation(state),
    {
        "legal_agent": "legal_agent",
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("  routing → [conditional based on LLM decision]")


workflow_conditional.add_conditional_edges(
    "legal_agent",
    lambda state: get_next_agent_or_aggregation(state, 'legal'),
    {
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("  legal_agent → [next routed agent or aggregation]")

workflow_conditional.add_conditional_edges(
    "compliance_agent",
    lambda state: get_next_agent_or_aggregation(state, 'compliance'),
    {
        "legal_agent": "legal_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("  compliance_agent → [next routed agent or aggregation]")

workflow_conditional.add_conditional_edges(
    "finance_agent",
    lambda state: get_next_agent_or_aggregation(state, 'finance'),
    {
        "legal_agent": "legal_agent",
        "compliance_agent": "compliance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("  finance_agent → [next routed agent or aggregation]")

workflow_conditional.add_edge("operations_agent", "aggregation")
print("  operations_agent → aggregation")

workflow_conditional.add_edge("aggregation", END)
print("  aggregation → END")


print("COMPILING CONDITIONAL ROUTING WORKFLOW")

try:
    app_conditional = workflow_conditional.compile()
    print("Workflow compiled successfully!")
    
except Exception as e:
    print(f"Compilation failed: {str(e)}")
    import traceback
    traceback.print_exc()

print("CONDITIONAL ROUTING GRAPH READY")

BUILDING CONDITIONAL ROUTING WORKFLOW WITH LANGGRAPH
BUILDING CONDITIONAL ROUTING GRAPH

Adding nodes to graph:
  routing (entry point)
  legal_agent (threshold: 0.30)
  compliance_agent (threshold: 0.10)
  finance_agent (threshold: 0.30)
  operations_agent (threshold: 0.10)
  aggregation (consolidation)

Entry point: routing
DEFINING CONDITIONAL EDGES
  routing → [conditional based on LLM decision]
  legal_agent → [next routed agent or aggregation]
  compliance_agent → [next routed agent or aggregation]
  finance_agent → [next routed agent or aggregation]
  operations_agent → aggregation
  aggregation → END
COMPILING CONDITIONAL ROUTING WORKFLOW
Workflow compiled successfully!
CONDITIONAL ROUTING GRAPH READY


#### 3. Conditional Entry Points

In [68]:
print("CONDITIONAL ENTRY POINT WITH DYNAMIC LLM ROUTING")


print("\nBuilding conditional routing graph with intelligent entry point...")

workflow_conditional = StateGraph(GraphState)
print("\nStateGraph initialized")


print("ADDING AGENT NODES")

workflow_conditional.add_node("legal_agent", legal_node_conditional)
print("Added: legal_agent (threshold: 0.30)")

workflow_conditional.add_node("compliance_agent", compliance_node_conditional)
print("Added: compliance_agent (threshold: 0.10)")

workflow_conditional.add_node("finance_agent", finance_node_conditional)
print("Added: finance_agent (threshold: 0.30)")

workflow_conditional.add_node("operations_agent", operations_node_conditional)
print("Added: operations_agent (threshold: 0.10)")

workflow_conditional.add_node("routing", routing_node)
print("Added: routing (LLM coordinator)")

workflow_conditional.add_node("aggregation", aggregation_node)
print("Added: aggregation (result consolidation)")


print("DEFINING CONDITIONAL ENTRY POINT ROUTING")

def conditional_entry_router(state: GraphState) -> str:

    print("CONDITIONAL ENTRY POINT ROUTER")

    print(f"Query: \"{state['query']}\"")
    
    routing_result = dynamic_route_query(state['query'])
    
    state['routed_agents'] = routing_result['agents']
    state['routing_reasoning'] = routing_result['reasoning']
    state['routing_approach'] = routing_result['approach']
    
    print(f"\nLLM Routing Decision:")
    print(f"  Agents: {', '.join(routing_result['agents'])}")
    print(f"  Reasoning: {routing_result['reasoning']}")
    
    state['execution_order'].append('routing')
    
    if not routing_result['agents']:
        print(f"\nNo agents selected - routing to END")
        return "end"
    
    agent_priority = ['legal', 'compliance', 'finance', 'operations']
    
    for agent in agent_priority:
        if agent in routing_result['agents']:
            first_agent = f"{agent}_agent"
            print(f"\nFirst agent to execute: {agent.upper()}")
            return first_agent
    
    print(f"\nFallback - routing to END")
    return "end"

print("conditional_entry_router() defined")


print("SETTING CONDITIONAL ENTRY POINT")

workflow_conditional.set_conditional_entry_point(
    conditional_entry_router,
    {
        "legal_agent": "legal_agent",
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        "end": END
    }
)

print("ADDING CONDITIONAL EDGES BETWEEN AGENTS")

def get_next_agent_after(state: GraphState, current_agent: str = None) -> str:
    routed_agents = state.get('routed_agents', [])
    executed_agents = [a for a in state.get('execution_order', []) 
                      if a not in ['routing', 'aggregation']]
    
    agent_priority = ['legal', 'compliance', 'finance', 'operations']
    
    for agent in agent_priority:
        if agent in routed_agents and agent not in executed_agents:
            return f"{agent}_agent"
    
    return "aggregation"

workflow_conditional.add_conditional_edges(
    "legal_agent",
    lambda state: get_next_agent_after(state, 'legal'),
    {
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("legal_agent → [conditional next agent]")

workflow_conditional.add_conditional_edges(
    "compliance_agent",
    lambda state: get_next_agent_after(state, 'compliance'),
    {
        "legal_agent": "legal_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("compliance_agent → [conditional next agent]")

workflow_conditional.add_conditional_edges(
    "finance_agent",
    lambda state: get_next_agent_after(state, 'finance'),
    {
        "legal_agent": "legal_agent",
        "compliance_agent": "compliance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("finance_agent → [conditional next agent]")

workflow_conditional.add_edge("operations_agent", "aggregation")
print("operations_agent → aggregation")

workflow_conditional.add_edge("aggregation", END)
print("aggregation → END")

print("COMPILING CONDITIONAL ENTRY POINT WORKFLOW")

try:
    app_conditional = workflow_conditional.compile()
    print("Workflow compiled successfully!")
    
    print("CONDITIONAL ENTRY POINT WORKFLOW")
 
    print("WORKFLOW READY FOR EXECUTION")
    
    
except Exception as e:
    print(f"Compilation failed: {str(e)}")
    import traceback
    traceback.print_exc()

print("CONDITIONAL ENTRY POINT CONFIGURED")

CONDITIONAL ENTRY POINT WITH DYNAMIC LLM ROUTING

Building conditional routing graph with intelligent entry point...

StateGraph initialized
ADDING AGENT NODES
Added: legal_agent (threshold: 0.30)
Added: compliance_agent (threshold: 0.10)
Added: finance_agent (threshold: 0.30)
Added: operations_agent (threshold: 0.10)
Added: routing (LLM coordinator)
Added: aggregation (result consolidation)
DEFINING CONDITIONAL ENTRY POINT ROUTING
conditional_entry_router() defined
SETTING CONDITIONAL ENTRY POINT
ADDING CONDITIONAL EDGES BETWEEN AGENTS
legal_agent → [conditional next agent]
compliance_agent → [conditional next agent]
finance_agent → [conditional next agent]
operations_agent → aggregation
aggregation → END
COMPILING CONDITIONAL ENTRY POINT WORKFLOW
Workflow compiled successfully!
CONDITIONAL ENTRY POINT WORKFLOW
WORKFLOW READY FOR EXECUTION
CONDITIONAL ENTRY POINT CONFIGURED


#### 4. Agent → END Edges

In [69]:
print("CONFIGURING AGENT → END EDGES")

print("IMPLEMENTING HYBRID APPROACH (BEST OF BOTH)")

print("\nReconfiguring edges for hybrid approach...")

workflow_conditional_hybrid = StateGraph(GraphState)

workflow_conditional_hybrid.add_node("legal_agent", legal_node_conditional)
workflow_conditional_hybrid.add_node("compliance_agent", compliance_node_conditional)
workflow_conditional_hybrid.add_node("finance_agent", finance_node_conditional)
workflow_conditional_hybrid.add_node("operations_agent", operations_node_conditional)
workflow_conditional_hybrid.add_node("aggregation", aggregation_node)

print("Nodes added")

workflow_conditional_hybrid.set_conditional_entry_point(
    conditional_entry_router,
    {
        "legal_agent": "legal_agent",
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        "end": END
    }
)

print("Conditional entry point set")

def agent_to_next_or_end(state: GraphState) -> str:
 
    routed_agents = state.get('routed_agents', [])
    executed_agents = [a for a in state.get('execution_order', []) 
                      if a not in ['routing', 'aggregation']]
    
    if len(routed_agents) == 1:
        return "end"
    
    agent_priority = ['legal', 'compliance', 'finance', 'operations']
    
    for agent in agent_priority:
        if agent in routed_agents and agent not in executed_agents:
            return f"{agent}_agent"
    
    return "aggregation"

print("Hybrid routing function defined")

workflow_conditional_hybrid.add_conditional_edges(
    "legal_agent",
    agent_to_next_or_end,
    {
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation",
        "end": END
    }
)

workflow_conditional_hybrid.add_conditional_edges(
    "compliance_agent",
    agent_to_next_or_end,
    {
        "legal_agent": "legal_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation",
        "end": END
    }
)

workflow_conditional_hybrid.add_conditional_edges(
    "finance_agent",
    agent_to_next_or_end,
    {
        "legal_agent": "legal_agent",
        "compliance_agent": "compliance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation",
        "end": END
    }
)

workflow_conditional_hybrid.add_conditional_edges(
    "operations_agent",
    agent_to_next_or_end,
    {
        "legal_agent": "legal_agent",
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "aggregation": "aggregation",
        "end": END
    }
)

workflow_conditional_hybrid.add_edge("aggregation", END)

print("Hybrid conditional edges configured")

print("COMPILING HYBRID WORKFLOW")

try:
    app_conditional_hybrid = workflow_conditional_hybrid.compile()
    print("Hybrid workflow compiled successfully!")
    
except Exception as e:
    print(f"Compilation failed: {str(e)}")
    import traceback
    traceback.print_exc()

print("EDGE CONFIGURATION COMPLETE")

CONFIGURING AGENT → END EDGES
IMPLEMENTING HYBRID APPROACH (BEST OF BOTH)

Reconfiguring edges for hybrid approach...
Nodes added
Conditional entry point set
Hybrid routing function defined
Hybrid conditional edges configured
COMPILING HYBRID WORKFLOW
Hybrid workflow compiled successfully!
EDGE CONFIGURATION COMPLETE


#### 5. Compiling Conditional Graph

In [70]:
print("COMPILING CONDITIONAL ROUTING GRAPH")

print("\nCompiling conditional routing graph...")

try:
    app_conditional = app_conditional_hybrid
    
    print("Conditional Graph Compiled Successfully!")
    
except Exception as e:
    print(f"\nCompilation failed: {str(e)}")
    print("ERROR DETAILS")
    import traceback
    traceback.print_exc()

print("CONDITIONAL GRAPH COMPILATION COMPLETE")

COMPILING CONDITIONAL ROUTING GRAPH

Compiling conditional routing graph...
Conditional Graph Compiled Successfully!
CONDITIONAL GRAPH COMPILATION COMPLETE


#### 6. Test Case 1 - Legal Query

In [71]:
print("TEST CASE 1 - LEGAL QUERY")

print("\nPreparing test state for legal query...")

test_state_legal = {
    "query": "Review termination clause and liability provisions",
    "routed_agents": [],
    "routing_reasoning": "",
    "routing_approach": "",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "all_clauses": [],
    "total_clauses_retrieved": 0,
    "execution_order": [],
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "coordinator_model": "gemma2:9b",
    "vector_store": "Pinecone",
    "execution_status": "initialized"
}

print("Test state created")
print(f"  Query: \"{test_state_legal['query']}\"")
print(f"  Expected: Routes to Legal Agent only")

print("EXECUTING CONDITIONAL GRAPH - TEST CASE 1")

try:
    result_legal = app_conditional.invoke(test_state_legal)
    
    print("TEST CASE 1 COMPLETED")

    
    print("\nROUTING DECISION:")
    print(f"  Query: \"{result_legal['query']}\"")
    print(f"  LLM Routed To: {', '.join(result_legal.get('routed_agents', []))}")
    print(f"  Reasoning: {result_legal.get('routing_reasoning', 'N/A')}")
    print(f"  Approach: {result_legal.get('routing_approach', 'N/A')}")
    
   
    print("EXECUTION SUMMARY")
    print(f"  Execution Order: {' → '.join(result_legal['execution_order'])}")
    print(f"  Total Clauses Retrieved: {result_legal.get('total_clauses_retrieved', 0)}")
    print(f"  Execution Status: {result_legal.get('execution_status', 'unknown')}")
    
   
    print("AGENT EXECUTION STATUS")
    print(f"{'Agent':<15} {'Status':<15} {'Clauses':<10} {'Avg Relevance':<15}")
    
    for agent_name in ["legal", "compliance", "finance", "operations"]:
        agent_data = result_legal.get(agent_name, {})
        if agent_data and agent_data.get("status") == "completed":
            status = "EXECUTED "
            clauses = agent_data.get('clauses_retrieved', 0)
            avg_rel = agent_data.get('avg_relevance', 0)
            relevance = f"{avg_rel:.3f}" if avg_rel > 0 else "N/A"
            print(f"{agent_name.upper():<15} {status:<15} {clauses:<10} {relevance:<15}")
        elif agent_data and agent_data.get("status") == "no_results":
            status = "EXECUTED (0)"
            print(f"{agent_name.upper():<15} {status:<15} {0:<10} {'N/A':<15}")
        else:
            status = "NOT EXECUTED"
            print(f"{agent_name.upper():<15} {status:<15} {'-':<10} {'-':<15}")
    
    if result_legal.get('all_clauses'):
        print("RETRIEVED CLAUSES")
        for idx, clause in enumerate(result_legal['all_clauses'][:3], 1):
            print(f"\n[{idx}] Agent: {clause['agent'].upper()} | Relevance: {clause['relevance_score']:.3f}")
            clause_text = clause['clause'][:120] + "..." if len(clause['clause']) > 120 else clause['clause']
            print(f"    {clause_text}")
    
    print("CONDITIONAL ROUTING VERIFICATION")
 
    routed_agents = result_legal.get('routed_agents', [])
    executed_agents = [a for a in result_legal['execution_order'] if a not in ['routing', 'aggregation']]
    
    print(f"\nRouted by LLM: {', '.join(routed_agents)}")
    print(f"Actually Executed: {', '.join(executed_agents)}")
    
    if len(routed_agents) == 1 and 'legal' in routed_agents:
        print("\nSUCCESS: Legal-only query correctly routed")

        
        if 'aggregation' not in result_legal['execution_order']:
            print("  • Single-agent → Direct to END (fast path)")
        else:
            print("  • Routed through aggregation")
    else:
        print(f"\nUNEXPECTED ROUTING:")
        print(f"  Expected: ['legal']")
        print(f"  Got: {routed_agents}")
    
    print("EFFICIENCY ANALYSIS")

    agents_executed = len(executed_agents)
    total_agents = 4
    
    print(f"  Sequential Execution: {total_agents} agents (always)")
    print(f"  Conditional Execution: {agents_executed} agent(s) (query-dependent)")
    print(f"  Agents Saved: {total_agents - agents_executed}")
    print(f"  Efficiency Gain: ~{(total_agents - agents_executed) / total_agents * 100:.0f}%")
    print(f"  Time Saved: ~{(total_agents - agents_executed) / total_agents * 100:.0f}% (estimated)")
    
    if result_legal.get('legal', {}).get('status') == 'completed':
        print("THRESHOLD ANALYSIS")
        legal_threshold = result_legal['legal'].get('threshold', 0.3)
        legal_clauses = result_legal['legal'].get('clauses_retrieved', 0)
        print(f"  Legal Agent Threshold: {legal_threshold}")
        print(f"  Clauses Retrieved: {legal_clauses}")
        if legal_clauses > 0:
            print(f"  Successfully retrieved clauses above threshold")
        else:
            print(f"  No clauses above threshold {legal_threshold}")
    
except Exception as e:
    print(f"\nExecution failed: {str(e)}")
    print("ERROR DETAILS")
    import traceback
    traceback.print_exc()

print("TEST CASE 1 COMPLETE")

TEST CASE 1 - LEGAL QUERY

Preparing test state for legal query...
Test state created
  Query: "Review termination clause and liability provisions"
  Expected: Routes to Legal Agent only
EXECUTING CONDITIONAL GRAPH - TEST CASE 1
CONDITIONAL ENTRY POINT ROUTER
Query: "Review termination clause and liability provisions"

Querying gemma2:9b coordinator...
Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=60)
Falling back to keyword-based routing...

LLM Routing Decision:
  Agents: legal
  Reasoning: Keyword-based fallback routing

First agent to execute: LEGAL

  → LEGAL AGENT executing (threshold: 0.1)
     Query: Review termination clause and liability provisions...
     Retrieved: 2 clauses (avg: 0.537)
AGGREGATION NODE: Consolidating Results

Total clauses retrieved: 2
Top relevance score: 0.582
Agents executed: legal
TEST CASE 1 COMPLETED

ROUTING DECISION:
  Query: "Review termination clause and liability provisions"
  LLM Routed To: 
  Re

#### 7. Test Case 2 - Finance Query

In [72]:
print("TEST CASE 2 - FINANCE QUERY")

print("\nPreparing test state for finance query...")

test_state_finance = {
    "query": "Check late payment penalties and invoice terms",
    "routed_agents": [],
    "routing_reasoning": "",
    "routing_approach": "",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "all_clauses": [],
    "total_clauses_retrieved": 0,
    "execution_order": [],
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "coordinator_model": "gemma2:9b",
    "vector_store": "Pinecone",
    "execution_status": "initialized"
}

print("Test state created")
print(f"  Query: \"{test_state_finance['query']}\"")

print("EXECUTING CONDITIONAL GRAPH - TEST CASE 2")

try:
    result_finance = app_conditional.invoke(test_state_finance)
    
    print("TEST CASE 2 COMPLETED")

    print("\nROUTING DECISION:")
    print(f"  Query: \"{result_finance['query']}\"")
    print(f"  LLM Routed To: {', '.join(result_finance.get('routed_agents', []))}")
    print(f"  Reasoning: {result_finance.get('routing_reasoning', 'N/A')}")
    print(f"  Approach: {result_finance.get('routing_approach', 'N/A')}")
    

    print("EXECUTION SUMMARY")
    print(f"  Execution Order: {' → '.join(result_finance['execution_order'])}")
    print(f"  Total Clauses Retrieved: {result_finance.get('total_clauses_retrieved', 0)}")
    print(f"  Execution Status: {result_finance.get('execution_status', 'unknown')}")
  
    print("AGENT EXECUTION STATUS")
    print(f"{'Agent':<15} {'Status':<15} {'Clauses':<10} {'Avg Relevance':<15}")

    for agent_name in ["legal", "compliance", "finance", "operations"]:
        agent_data = result_finance.get(agent_name, {})
        if agent_data and agent_data.get("status") == "completed":
            status = "EXECUTED "
            clauses = agent_data.get('clauses_retrieved', 0)
            avg_rel = agent_data.get('avg_relevance', 0)
            relevance = f"{avg_rel:.3f}" if avg_rel > 0 else "N/A"
            print(f"{agent_name.upper():<15} {status:<15} {clauses:<10} {relevance:<15}")
        elif agent_data and agent_data.get("status") == "no_results":
            status = "EXECUTED (0)"
            print(f"{agent_name.upper():<15} {status:<15} {0:<10} {'N/A':<15}")
        else:
            status = "NOT EXECUTED"
            print(f"{agent_name.upper():<15} {status:<15} {'-':<10} {'-':<15}")
    
    if result_finance.get('all_clauses'):
        print("RETRIEVED CLAUSES")
        for idx, clause in enumerate(result_finance['all_clauses'][:3], 1):
            print(f"\n[{idx}] Agent: {clause['agent'].upper()} | Relevance: {clause['relevance_score']:.3f}")
            clause_text = clause['clause'][:120] + "..." if len(clause['clause']) > 120 else clause['clause']
            print(f"    {clause_text}")
    
    print("CONDITIONAL ROUTING VERIFICATION")
    
    routed_agents = result_finance.get('routed_agents', [])
    executed_agents = [a for a in result_finance['execution_order'] if a not in ['routing', 'aggregation']]
    
    print(f"\nRouted by LLM: {', '.join(routed_agents)}")
    print(f"Actually Executed: {', '.join(executed_agents)}")
    
    if len(routed_agents) == 1 and 'finance' in routed_agents:
        print("\nSUCCESS: Finance-only query correctly routed")
        
        if 'aggregation' not in result_finance['execution_order']:
            print("  • Single-agent → Direct to END (fast path)")
        else:
            print("  • Routed through aggregation")
    elif 'finance' in routed_agents and len(routed_agents) > 1:
        print(f"\nMULTI-AGENT ROUTING:")
        print(f"  LLM selected: {', '.join(routed_agents)}")
        print(f"  Query may have cross-domain elements")
    else:
        print(f"\nUNEXPECTED ROUTING:")
        print(f"  Expected: ['finance']")
        print(f"  Got: {routed_agents}")
    

    print("EFFICIENCY ANALYSIS")

    agents_executed = len(executed_agents)
    total_agents = 4
    
    print(f"  Sequential Execution: {total_agents} agents (always)")
    print(f"  Conditional Execution: {agents_executed} agent(s) (query-dependent)")
    print(f"  Agents Saved: {total_agents - agents_executed}")
    print(f"  Efficiency Gain: ~{(total_agents - agents_executed) / total_agents * 100:.0f}%")
    
    if result_finance.get('finance', {}).get('status') == 'completed':
        print("THRESHOLD ANALYSIS")
        finance_threshold = result_finance['finance'].get('threshold', 0.3)
        finance_clauses = result_finance['finance'].get('clauses_retrieved', 0)
        print(f"  Finance Agent Threshold: {finance_threshold}")
        print(f"  Clauses Retrieved: {finance_clauses}")
        if finance_clauses > 0:
            print(f"  Successfully retrieved clauses above threshold")
            avg_rel = result_finance['finance'].get('avg_relevance', 0)
            max_rel = result_finance['finance'].get('max_relevance', 0)
            print(f"  Average Relevance: {avg_rel:.3f}")
            print(f"  Max Relevance: {max_rel:.3f}")
        else:
            print(f"  No clauses above threshold {finance_threshold}")

    print("COMPARISON WITH TEST CASE 1")
    print(f"  Test 1 (Legal Query):")
    print(f"    • Routed to: Legal")
    print(f"    • Agents executed: 1")
    print(f"  Test 2 (Finance Query):")
    print(f"    • Routed to: {', '.join(routed_agents)}")
    print(f"    • Agents executed: {agents_executed}")
    
except Exception as e:
    print(f"\nExecution failed: {str(e)}")
    print("ERROR DETAILS")
    import traceback
    traceback.print_exc()
    
print("TEST CASE 2 COMPLETE")

TEST CASE 2 - FINANCE QUERY

Preparing test state for finance query...
Test state created
  Query: "Check late payment penalties and invoice terms"
EXECUTING CONDITIONAL GRAPH - TEST CASE 2
CONDITIONAL ENTRY POINT ROUTER
Query: "Check late payment penalties and invoice terms"

Querying gemma2:9b coordinator...

LLM Routing Decision:
  Agents: finance
  Reasoning: The query specifically asks about late payment penalties and invoice terms, which fall under the domain of financial obligations.

First agent to execute: FINANCE

  → FINANCE AGENT executing (threshold: 0.1)
     Query: Check late payment penalties and invoice terms...
     Retrieved: 3 clauses (avg: 0.343)
AGGREGATION NODE: Consolidating Results

Total clauses retrieved: 3
Top relevance score: 0.383
Agents executed: finance
TEST CASE 2 COMPLETED

ROUTING DECISION:
  Query: "Check late payment penalties and invoice terms"
  LLM Routed To: 
  Reasoning: 
  Approach: 
EXECUTION SUMMARY
  Execution Order: routing → finance → agg

#### 8. Test Case 3 - Compliance Query

In [73]:
print("TEST CASE 3 - COMPLIANCE QUERY")

print("\nPreparing test state for compliance query...")

test_state_compliance = {
    "query": "Verify GDPR data protection and audit requirements",
    "routed_agents": [],
    "routing_reasoning": "",
    "routing_approach": "",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "all_clauses": [],
    "total_clauses_retrieved": 0,
    "execution_order": [],
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "coordinator_model": "gemma2:9b",
    "vector_store": "Pinecone",
    "execution_status": "initialized"
}

print("Test state created")
print(f"  Query: \"{test_state_compliance['query']}\"")
print(f"  Expected: Routes to Compliance Agent")

print("EXECUTING CONDITIONAL GRAPH - TEST CASE 3")

try:
    result_compliance = app_conditional.invoke(test_state_compliance)
    
    print("TEST CASE 3 COMPLETED")

    print("\nROUTING DECISION:")
    print(f"  Query: \"{result_compliance['query']}\"")
    print(f"  LLM Routed To: {', '.join(result_compliance.get('routed_agents', []))}")
    print(f"  Reasoning: {result_compliance.get('routing_reasoning', 'N/A')}")
    print(f"  Approach: {result_compliance.get('routing_approach', 'N/A')}")

    print("EXECUTION SUMMARY")

    print(f"  Execution Order: {' → '.join(result_compliance['execution_order'])}")
    print(f"  Total Clauses Retrieved: {result_compliance.get('total_clauses_retrieved', 0)}")
    print(f"  Execution Status: {result_compliance.get('execution_status', 'unknown')}")
  
    print("AGENT EXECUTION STATUS")
    print(f"{'Agent':<15} {'Status':<15} {'Clauses':<10} {'Threshold':<12} {'Avg Relevance':<15}")
    
    for agent_name in ["legal", "compliance", "finance", "operations"]:
        agent_data = result_compliance.get(agent_name, {})
        if agent_data and agent_data.get("status") == "completed":
            status = "EXECUTED"
            clauses = agent_data.get('clauses_retrieved', 0)
            threshold = agent_data.get('threshold', 'N/A')
            avg_rel = agent_data.get('avg_relevance', 0)
            relevance = f"{avg_rel:.3f}" if avg_rel > 0 else "N/A"
            threshold_str = f"{threshold:.2f}" if isinstance(threshold, float) else str(threshold)
            print(f"{agent_name.upper():<15} {status:<15} {clauses:<10} {threshold_str:<12} {relevance:<15}")
        elif agent_data and agent_data.get("status") == "no_results":
            status = "EXECUTED (0)"
            threshold = agent_data.get('threshold', 'N/A')
            threshold_str = f"{threshold:.2f}" if isinstance(threshold, float) else str(threshold)
            print(f"{agent_name.upper():<15} {status:<15} {0:<10} {threshold_str:<12} {'N/A':<15}")
        else:
            status = "NOT EXECUTED"
            print(f"{agent_name.upper():<15} {status:<15} {'-':<10} {'-':<12} {'-':<15}")
    
    if result_compliance.get('all_clauses'):
        print("RETRIEVED CLAUSES")
        for idx, clause in enumerate(result_compliance['all_clauses'][:3], 1):
            print(f"\n[{idx}] Agent: {clause['agent'].upper()} | Relevance: {clause['relevance_score']:.3f}")
            clause_text = clause['clause'][:120] + "..." if len(clause['clause']) > 120 else clause['clause']
            print(f"    {clause_text}")
    else:
        print("NO CLAUSES RETRIEVED")
 
    print("CONDITIONAL ROUTING VERIFICATION")
    
    routed_agents = result_compliance.get('routed_agents', [])
    executed_agents = [a for a in result_compliance['execution_order'] if a not in ['routing', 'aggregation']]
    
    print(f"\nRouted by LLM: {', '.join(routed_agents)}")
    print(f"Actually Executed: {', '.join(executed_agents)}")
    
    if 'compliance' in routed_agents:
        print("\nSUCCESS: Compliance domain correctly identified")
        
        if len(routed_agents) == 1:
            print("  • Only Compliance agent routed (efficient)")
            print("  • Other agents skipped")
        else:
            print(f"  • Multi-domain query detected: {', '.join(routed_agents)}")
        
        compliance_clauses = result_compliance.get('compliance', {}).get('clauses_retrieved', 0)
        if compliance_clauses > 0:
            print(f"  • Successfully retrieved {compliance_clauses} clause(s)")
        else:
            print(f"  • No clauses retrieved (may not exist in dataset)")
            print(f"  • Note: Lower threshold (0.10) already applied")
    else:
        print(f"\nUNEXPECTED ROUTING:")
        print(f"  Expected: ['compliance']")
        print(f"  Got: {routed_agents}")
    

    print("EFFICIENCY ANALYSIS")

    agents_executed = len(executed_agents)
    total_agents = 4
    
    print(f"  Sequential Execution: {total_agents} agents (always)")
    print(f"  Conditional Execution: {agents_executed} agent(s) (query-dependent)")
    print(f"  Agents Saved: {total_agents - agents_executed}")
    print(f"  Efficiency Gain: ~{(total_agents - agents_executed) / total_agents * 100:.0f}%")
    

    print("THRESHOLD IMPACT ANALYSIS")

    if result_compliance.get('compliance', {}).get('status') in ['completed', 'no_results']:
        compliance_threshold = result_compliance['compliance'].get('threshold', 0.1)
        compliance_clauses = result_compliance['compliance'].get('clauses_retrieved', 0)
        
        print(f"  Compliance Agent:")
        print(f"    • Threshold: {compliance_threshold} (LOWER for better recall)")
        print(f"    • Clauses Retrieved: {compliance_clauses}")
        
        if compliance_clauses > 0:
            avg_rel = result_compliance['compliance'].get('avg_relevance', 0)
            max_rel = result_compliance['compliance'].get('max_relevance', 0)
            print(f"    • Average Relevance: {avg_rel:.3f}")
            print(f"    • Max Relevance: {max_rel:.3f}")
            print(f"    • Lower threshold helped retrieve clauses")
        else:
            print(f"    • Even with 0.10 threshold, no clauses found")
    
    
except Exception as e:
    print(f"\nExecution failed: {str(e)}")
    print("ERROR DETAILS")
    import traceback
    traceback.print_exc()


print("TEST CASE 3 COMPLETE")

TEST CASE 3 - COMPLIANCE QUERY

Preparing test state for compliance query...
Test state created
  Query: "Verify GDPR data protection and audit requirements"
  Expected: Routes to Compliance Agent
EXECUTING CONDITIONAL GRAPH - TEST CASE 3
CONDITIONAL ENTRY POINT ROUTER
Query: "Verify GDPR data protection and audit requirements"

Querying gemma2:9b coordinator...

LLM Routing Decision:
  Agents: compliance
  Reasoning: The query specifically mentions GDPR data protection and audit requirements, which fall under the domain of compliance.

First agent to execute: COMPLIANCE

  → COMPLIANCE AGENT executing (threshold: 0.1)
     Query: Verify GDPR data protection and audit requirements...
     Retrieved: 1 clauses (avg: 0.179)
AGGREGATION NODE: Consolidating Results

Total clauses retrieved: 1
Top relevance score: 0.179
Agents executed: compliance
TEST CASE 3 COMPLETED

ROUTING DECISION:
  Query: "Verify GDPR data protection and audit requirements"
  LLM Routed To: 
  Reasoning: 
  Approach

#### 9. Test Case 4 - Operations Query

In [74]:
print("TEST CASE 4 - OPERATIONS QUERY")

print("\nPreparing test state for operations query...")

test_state_operations = {
    "query": "Review the milestones, deliverables, and SLA requirements",
    "routed_agents": [],
    "routing_reasoning": "",
    "routing_approach": "",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "all_clauses": [],
    "total_clauses_retrieved": 0,
    "execution_order": [],
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "coordinator_model": "gemma2:9b",
    "vector_store": "Pinecone",
    "execution_status": "initialized"
}

print("Test state created")
print(f"  Query: \"{test_state_operations['query']}\"")
print(f"  Expected: Routes to Operations Agent")

print("EXECUTING CONDITIONAL GRAPH - TEST CASE 4")

try:
    result_operations = app_conditional.invoke(test_state_operations)

    print("TEST CASE 4 COMPLETED")

    print("\nROUTING DECISION:")
    print(f"  Query: \"{result_operations['query']}\"")
    print(f"  LLM Routed To: {', '.join(result_operations.get('routed_agents', []))}")
    print(f"  Reasoning: {result_operations.get('routing_reasoning', 'N/A')}")
    print(f"  Approach: {result_operations.get('routing_approach', 'N/A')}")

    print("EXECUTION SUMMARY")

    print(f"  Execution Order: {' → '.join(result_operations['execution_order'])}")
    print(f"  Total Clauses Retrieved: {result_operations.get('total_clauses_retrieved', 0)}")
    print(f"  Execution Status: {result_operations.get('execution_status', 'unknown')}")

    print("AGENT EXECUTION STATUS")
    print(f"{'Agent':<15} {'Status':<15} {'Clauses':<10} {'Threshold':<12} {'Avg Relevance':<15}")
    
    for agent_name in ["legal", "compliance", "finance", "operations"]:
        agent_data = result_operations.get(agent_name, {})
        if agent_data and agent_data.get("status") == "completed":
            status = "EXECUTED"
            clauses = agent_data.get('clauses_retrieved', 0)
            threshold = agent_data.get('threshold', 'N/A')
            avg_rel = agent_data.get('avg_relevance', 0)
            relevance = f"{avg_rel:.3f}" if avg_rel > 0 else "N/A"
            threshold_str = f"{threshold:.2f}" if isinstance(threshold, float) else str(threshold)
            print(f"{agent_name.upper():<15} {status:<15} {clauses:<10} {threshold_str:<12} {relevance:<15}")
        elif agent_data and agent_data.get("status") == "no_results":
            status = "EXECUTED (0)"
            threshold = agent_data.get('threshold', 'N/A')
            threshold_str = f"{threshold:.2f}" if isinstance(threshold, float) else str(threshold)
            print(f"{agent_name.upper():<15} {status:<15} {0:<10} {threshold_str:<12} {'N/A':<15}")
        else:
            status = "NOT EXECUTED"
            print(f"{agent_name.upper():<15} {status:<15} {'-':<10} {'-':<12} {'-':<15}")
    
    if result_operations.get('all_clauses'):
        print("RETRIEVED CLAUSES")
        for idx, clause in enumerate(result_operations['all_clauses'][:3], 1):
            print(f"\n[{idx}] Agent: {clause['agent'].upper()} | Relevance: {clause['relevance_score']:.3f}")
            clause_text = clause['clause'][:120] + "..." if len(clause['clause']) > 120 else clause['clause']
            print(f"    {clause_text}")
    else:
        print("NO CLAUSES RETRIEVED")

    print("CONDITIONAL ROUTING VERIFICATION")

    routed_agents = result_operations.get('routed_agents', [])
    executed_agents = [a for a in result_operations['execution_order'] if a not in ['routing', 'aggregation']]
    
    print(f"\nRouted by LLM: {', '.join(routed_agents)}")
    print(f"Actually Executed: {', '.join(executed_agents)}")
    
    if 'operations' in routed_agents:
        print("\nSUCCESS: Operations domain correctly identified")
        
        if len(routed_agents) == 1:
            print("  • Only Operations agent routed (efficient)")
            print("  • Other agents skipped")
        else:
            print(f"  • Multi-domain query detected: {', '.join(routed_agents)}")
        
        operations_clauses = result_operations.get('operations', {}).get('clauses_retrieved', 0)
        if operations_clauses > 0:
            print(f"  • Successfully retrieved {operations_clauses} clause(s)")
        else:
            print(f"  • No clauses retrieved (may not exist in dataset)")
            print(f"  • Note: Lower threshold (0.10) already applied")
    else:
        print(f"\nUNEXPECTED ROUTING:")
        print(f"  Expected: ['operations']")
        print(f"  Got: {routed_agents}")

    print("EFFICIENCY ANALYSIS")

    agents_executed = len(executed_agents)
    total_agents = 4
    
    print(f"  Sequential Execution: {total_agents} agents (always)")
    print(f"  Conditional Execution: {agents_executed} agent(s) (query-dependent)")
    print(f"  Agents Saved: {total_agents - agents_executed}")
    print(f"  Efficiency Gain: ~{(total_agents - agents_executed) / total_agents * 100:.0f}%")
    print(f"  Time Saved: ~{(total_agents - agents_executed) / total_agents * 100:.0f}%")

    print("THRESHOLD IMPACT ANALYSIS")

    if result_operations.get('operations', {}).get('status') in ['completed', 'no_results']:
        operations_threshold = result_operations['operations'].get('threshold', 0.1)
        operations_clauses = result_operations['operations'].get('clauses_retrieved', 0)
        
        print(f"  Operations Agent:")
        print(f"    • Threshold: {operations_threshold} (LOWER for better recall)")
        print(f"    • Clauses Retrieved: {operations_clauses}")
        
        if operations_clauses > 0:
            avg_rel = result_operations['operations'].get('avg_relevance', 0)
            max_rel = result_operations['operations'].get('max_relevance', 0)
            print(f"    • Average Relevance: {avg_rel:.3f}")
            print(f"    • Max Relevance: {max_rel:.3f}")
            print(f"    • Lower threshold helped retrieve clauses")
        else:
            print(f"    • Even with 0.10 threshold, no clauses found")
            print(f"    • Likely: No operations data for this query in Pinecone")
    
except Exception as e:
    print(f"\nExecution failed: {str(e)}")
    print("ERROR DETAILS")
    import traceback
    traceback.print_exc()


print("TEST CASE 4 COMPLETE - ALL SINGLE-DOMAIN TESTS FINISHED")

TEST CASE 4 - OPERATIONS QUERY

Preparing test state for operations query...
Test state created
  Query: "Review the milestones, deliverables, and SLA requirements"
  Expected: Routes to Operations Agent
EXECUTING CONDITIONAL GRAPH - TEST CASE 4
CONDITIONAL ENTRY POINT ROUTER
Query: "Review the milestones, deliverables, and SLA requirements"

Querying gemma2:9b coordinator...

LLM Routing Decision:
  Agents: operations
  Reasoning: The query explicitly mentions milestones, deliverables, and SLA requirements, which fall under the domain of Operations.

First agent to execute: OPERATIONS

  → OPERATIONS AGENT executing (threshold: 0.1)
     Query: Review the milestones, deliverables, and SLA requirements...
     Retrieved: 2 clauses (avg: 0.287)
AGGREGATION NODE: Consolidating Results

Total clauses retrieved: 2
Top relevance score: 0.359
Agents executed: operations
TEST CASE 4 COMPLETED

ROUTING DECISION:
  Query: "Review the milestones, deliverables, and SLA requirements"
  LLM Routed 

#### 10. Test Case 5 - Multiple Intent - Limitation

In [75]:
print("TEST CASE 5 - MULTI-DOMAIN QUERY")

print("\nPreparing test state for multi-domain query...")

test_state_multi = {
    "query": "Check GDPR compliance and payment terms with late penalties",
    "routed_agents": [],
    "routing_reasoning": "",
    "routing_approach": "",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "all_clauses": [],
    "total_clauses_retrieved": 0,
    "execution_order": [],
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "coordinator_model": "gemma2:9b",
    "vector_store": "Pinecone",
    "execution_status": "initialized"
}

print("Test state created")
print(f"  Query: \"{test_state_multi['query']}\"")
print(f"  Expected: Routes to BOTH Compliance AND Finance")

print("EXECUTING CONDITIONAL GRAPH - TEST CASE 5 (MULTI-DOMAIN)")

try:
    result_multi = app_conditional.invoke(test_state_multi)
    
    print("TEST CASE 5 COMPLETED")
 
    print("\nROUTING DECISION:")
    print(f"  Query: \"{result_multi['query']}\"")
    print(f"  LLM Routed To: {', '.join(result_multi.get('routed_agents', []))}")
    print(f"  Reasoning: {result_multi.get('routing_reasoning', 'N/A')}")
    print(f"  Approach: {result_multi.get('routing_approach', 'N/A')}")
    
    print("EXECUTION SUMMARY")
    print(f"  Execution Order: {' → '.join(result_multi['execution_order'])}")
    print(f"  Total Clauses Retrieved: {result_multi.get('total_clauses_retrieved', 0)}")
    print(f"  Execution Status: {result_multi.get('execution_status', 'unknown')}")
    
    print("AGENT EXECUTION STATUS")
    print(f"{'Agent':<15} {'Status':<15} {'Clauses':<10} {'Threshold':<12} {'Avg Relevance':<15}")

    executed_agents = []
    for agent_name in ["legal", "compliance", "finance", "operations"]:
        agent_data = result_multi.get(agent_name, {})
        if agent_data and agent_data.get("status") == "completed":
            executed_agents.append(agent_name)
            status = "EXECUTED"
            clauses = agent_data.get('clauses_retrieved', 0)
            threshold = agent_data.get('threshold', 'N/A')
            avg_rel = agent_data.get('avg_relevance', 0)
            relevance = f"{avg_rel:.3f}" if avg_rel > 0 else "N/A"
            threshold_str = f"{threshold:.2f}" if isinstance(threshold, float) else str(threshold)
            print(f"{agent_name.upper():<15} {status:<15} {clauses:<10} {threshold_str:<12} {relevance:<15}")
        elif agent_data and agent_data.get("status") == "no_results":
            executed_agents.append(agent_name)
            status = "EXECUTED (0)"
            threshold = agent_data.get('threshold', 'N/A')
            threshold_str = f"{threshold:.2f}" if isinstance(threshold, float) else str(threshold)
            print(f"{agent_name.upper():<15} {status:<15} {0:<10} {threshold_str:<12} {'N/A':<15}")
        else:
            status = "NOT EXECUTED"
            print(f"{agent_name.upper():<15} {status:<15} {'-':<10} {'-':<12} {'-':<15}")
    
    if result_multi.get('all_clauses'):
        print("RETRIEVED CLAUSES (ALL AGENTS)")
   
        clauses_by_agent = {}
        for clause in result_multi['all_clauses']:
            agent = clause['agent']
            if agent not in clauses_by_agent:
                clauses_by_agent[agent] = []
            clauses_by_agent[agent].append(clause)
        
        for agent, clauses in clauses_by_agent.items():
            print(f"\n{agent.upper()} Agent - {len(clauses)} clause(s):")
            for idx, clause in enumerate(clauses[:2], 1):
                print(f"  [{idx}] Relevance: {clause['relevance_score']:.3f}")
                clause_text = clause['clause'][:100] + "..." if len(clause['clause']) > 100 else clause['clause']
                print(f"      {clause_text}")
    

    print("MULTI-DOMAIN ROUTING ANALYSIS")

    routed_agents = result_multi.get('routed_agents', [])
    
    print(f"\nRouted by LLM: {', '.join(routed_agents)}")
    print(f"Actually Executed: {', '.join(executed_agents)}")
    print(f"Agents Count: {len(executed_agents)}")
    
    if len(routed_agents) >= 2:
        print(f"\nSUCCESS: Multi-domain query handled correctly")
        print(f"  • LLM identified {len(routed_agents)} relevant domains")
        print(f"  • Query spans multiple areas: {', '.join(routed_agents)}")
        
        if 'compliance' in routed_agents and 'finance' in routed_agents:
            print(f"  • Both Compliance (GDPR) and Finance (payment) detected")
        
        if 'aggregation' in result_multi['execution_order']:
            print(f"  • Results consolidated via aggregation node")
        
        print(f"\n  Execution Flow:")
        for idx, step in enumerate(result_multi['execution_order'], 1):
            print(f"    {idx}. {step}")
        
    elif len(routed_agents) == 1:
        print(f"\nSINGLE-AGENT ROUTING:")
        print(f"  • LLM selected only: {routed_agents[0]}")

    else:
        print(f"\nNO AGENTS ROUTED")

    print("EFFICIENCY ANALYSIS (MULTI-DOMAIN)")

    agents_executed_count = len(executed_agents)
    total_agents = 4
    
    print(f"  Sequential Execution: {total_agents} agents (always)")
    print(f"  Conditional Execution: {agents_executed_count} agent(s) (only needed)")
    print(f"  Agents Saved: {total_agents - agents_executed_count}")
    print(f"  Efficiency Gain: ~{(total_agents - agents_executed_count) / total_agents * 100:.0f}%")
    
    if agents_executed_count > 1:
        print(f"\n  Multi-agent execution benefits:")
        print(f"    • Comprehensive coverage of query domains")
        print(f"    • Results from {agents_executed_count} specialized agents")
        print(f"    • Still skipped {total_agents - agents_executed_count} irrelevant agent(s)")
        print(f"    • Better than sequential (would run all {total_agents})")
    

    print("ADAPTIVE THRESHOLD IMPACT")
    
    compliance_in = 'compliance' in executed_agents
    finance_in = 'finance' in executed_agents
    
    if compliance_in or finance_in:
        print("\n  Threshold Strategy:")
        if compliance_in:
            comp_threshold = result_multi.get('compliance', {}).get('threshold', 0.1)
            comp_clauses = result_multi.get('compliance', {}).get('clauses_retrieved', 0)
            print(f"    • Compliance: {comp_threshold} (lower for recall) → {comp_clauses} clauses")
        if finance_in:
            fin_threshold = result_multi.get('finance', {}).get('threshold', 0.3)
            fin_clauses = result_multi.get('finance', {}).get('clauses_retrieved', 0)
            print(f"    • Finance: {fin_threshold} (higher for precision) → {fin_clauses} clauses")
        
        print(f"\n  Different thresholds per agent optimize results")

    print("COMPARISON: SINGLE vs MULTI-DOMAIN ROUTING")

    print(f"""
  Multi-Domain Test:
    • {agents_executed_count} agent(s) executed
    • Through aggregation (consolidation)
    • {(total_agents - agents_executed_count) / total_agents * 100:.0f}% efficiency gain
    • Comprehensive coverage
    
    """)
    
except Exception as e:
    print(f"\nExecution failed: {str(e)}")
    print("ERROR DETAILS")
    import traceback
    traceback.print_exc()

print("TEST CASE 5 COMPLETE - MULTI-DOMAIN ROUTING TESTED")


TEST CASE 5 - MULTI-DOMAIN QUERY

Preparing test state for multi-domain query...
Test state created
  Query: "Check GDPR compliance and payment terms with late penalties"
  Expected: Routes to BOTH Compliance AND Finance
EXECUTING CONDITIONAL GRAPH - TEST CASE 5 (MULTI-DOMAIN)
CONDITIONAL ENTRY POINT ROUTER
Query: "Check GDPR compliance and payment terms with late penalties"

Querying gemma2:9b coordinator...

LLM Routing Decision:
  Agents: compliance, finance
  Reasoning: GDPR compliance falls under data protection and privacy regulations handled by the Compliance agent. Payment terms with late penalties are financial obligations handled by the Finance agent.

First agent to execute: COMPLIANCE

  → COMPLIANCE AGENT executing (threshold: 0.1)
     Query: Check GDPR compliance and payment terms with late penalties...
     Retrieved: 1 clauses (avg: 0.166)
AGGREGATION NODE: Consolidating Results

Total clauses retrieved: 1
Top relevance score: 0.166
Agents executed: compliance
TEST CAS

#### 11. Adding New Keyword Mapping & Testing

In [77]:
print("DYNAMIC LLM COORDINATOR - NO KEYWORD MAPPING NEEDED")

print("TESTING DYNAMIC ROUTING WITH NEW TERMS")

test_queries_new_terms = [
    "Review dispute resolution and arbitration clause",
    "Check PCI DSS compliance report",
    "Calculate interest rate on late charges",
    "Project implementation timeline and go-live date",
    "Contract breach and warranty issues",
    "Data breach incident response procedure",
    "Payment schedule with currency exchange rates",
    "Sprint deliverables and acceptance criteria"
]

print(f"\nTesting {len(test_queries_new_terms)} queries with new terminology:")
for idx, query in enumerate(test_queries_new_terms, 1):
    print(f"  {idx}. {query}")

dynamic_results = []

for idx, query in enumerate(test_queries_new_terms, 1):
    print(f"\n{'='*80}")
    print(f"TEST {idx}/{len(test_queries_new_terms)}")
    print(f"{'='*80}")
    
    test_state = {
        "query": query,
        "routed_agents": [],
        "routing_reasoning": "",
        "routing_approach": "",
        "legal": {},
        "compliance": {},
        "finance": {},
        "operations": {},
        "all_clauses": [],
        "total_clauses_retrieved": 0,
        "execution_order": [],
        "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
        "coordinator_model": "gemma2:9b",
        "vector_store": "Pinecone",
        "execution_status": "initialized"
    }
    
    try:
        result = app_conditional.invoke(test_state)
        
        routed_agents = result.get('routed_agents', [])
        executed_agents = [a for a in result['execution_order'] if a not in ['routing', 'aggregation']]
        reasoning = result.get('routing_reasoning', 'N/A')
        clauses = result.get('total_clauses_retrieved', 0)
        
        print(f"\nQuery: \"{query}\"")
        print(f"LLM Routed To: {', '.join(routed_agents)}")
        print(f"Reasoning: {reasoning}")
        print(f"Executed: {', '.join(executed_agents)}")
        print(f"Clauses Retrieved: {clauses}")
        
        dynamic_results.append({
            "query": query,
            "routed_agents": routed_agents,
            "executed_agents": executed_agents,
            "reasoning": reasoning,
            "clauses_retrieved": clauses,
            "success": len(executed_agents) > 0
        })
        
        print(f"Success")
        
    except Exception as e:
        print(f"Error: {str(e)[:100]}")
        dynamic_results.append({
            "query": query,
            "routed_agents": [],
            "executed_agents": [],
            "reasoning": f"Error: {str(e)[:50]}",
            "clauses_retrieved": 0,
            "success": False
        })


print("DYNAMIC ROUTING ANALYSIS")

agent_usage = defaultdict(int)
for result in dynamic_results:
    for agent in result['executed_agents']:
        agent_usage[agent] += 1

print(f"\nRouting Statistics:")
print(f"  Total Queries: {len(dynamic_results)}")
print(f"  Successful Routes: {sum(1 for r in dynamic_results if r['success'])}")
print(f"  Success Rate: {sum(1 for r in dynamic_results if r['success']) / len(dynamic_results) * 100:.1f}%")

print(f"\nAgent Selection Distribution:")
print(f"{'Agent':<15} {'Count':<8} {'Percentage':<12} {'Bar'}")

for agent in ['legal', 'compliance', 'finance', 'operations']:
    count = agent_usage.get(agent, 0)
    percentage = (count / len(dynamic_results)) * 100 if len(dynamic_results) > 0 else 0
    bar = "█" * int(percentage / 5)
    print(f"{agent.upper():<15} {count:<8} {percentage:>6.1f}%      {bar}")

multi_agent_queries = [r for r in dynamic_results if len(r['executed_agents']) > 1]
print(f"\nMulti-Agent Queries: {len(multi_agent_queries)}")
for result in multi_agent_queries:
    print(f"  • \"{result['query'][:60]}...\"")
    print(f"    Agents: {', '.join(result['executed_agents'])}")

print("LLM UNDERSTANDING OF NEW TERMS")

term_mapping = {
    "dispute resolution, arbitration": "legal",
    "PCI DSS compliance": "compliance",
    "interest rate, late charges": "finance",
    "implementation timeline, go-live": "operations",
    "contract breach, warranty": "legal",
    "data breach, incident response": "compliance",
    "payment schedule, currency": "finance",
    "sprint deliverables, acceptance": "operations"
}

for terms, expected_agent in term_mapping.items():

    matching = [r for r in dynamic_results if any(t.lower() in r['query'].lower() for t in terms.split(', '))]
    if matching:
        result = matching[0]
        actual_agents = result['executed_agents']
        match = expected_agent in actual_agents
        status = "SUCCESS" if match else "ERROR"
        print(f"{status} Terms: '{terms}'")
        print(f"  Expected: {expected_agent}")
        print(f"  Routed: {', '.join(actual_agents)}")
        print(f"  Reasoning: {result['reasoning'][:80]}...")
        print()


print("SAVING RESULTS")

LANGGRAPH_OUTPUT = "../Data/Results/LangGraph"
os.makedirs(LANGGRAPH_OUTPUT, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

dynamic_output = {
    "task": "Dynamic LLM Routing with New Terms (No Keyword Mapping)",
    "coordinator_model": "gemma2:9b",
    "approach": "Dynamic LLM-based semantic routing",
    "advantage": "No manual keyword management required",
    "test_queries": len(test_queries_new_terms),
    "queries_tested": test_queries_new_terms,
    "routing_results": dynamic_results,
    "statistics": {
        "total_queries": len(dynamic_results),
        "successful_routes": sum(1 for r in dynamic_results if r['success']),
        "success_rate": f"{sum(1 for r in dynamic_results if r['success']) / len(dynamic_results) * 100:.1f}%",
        "multi_agent_queries": len(multi_agent_queries)
    },
    "agent_usage": dict(agent_usage)
}

output_file = f"dynamic_routing_new_terms_{timestamp}.json"
output_path = os.path.join(LANGGRAPH_OUTPUT, output_file)

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(dynamic_output, f, indent=2, ensure_ascii=False)

print(f"\nResults saved: {output_file}")
print(f"  Location: {LANGGRAPH_OUTPUT}")

print("DYNAMIC ROUTING WITH NEW TERMS COMPLETE")

DYNAMIC LLM COORDINATOR - NO KEYWORD MAPPING NEEDED
TESTING DYNAMIC ROUTING WITH NEW TERMS

Testing 8 queries with new terminology:
  1. Review dispute resolution and arbitration clause
  2. Check PCI DSS compliance report
  3. Calculate interest rate on late charges
  4. Project implementation timeline and go-live date
  5. Contract breach and warranty issues
  6. Data breach incident response procedure
  7. Payment schedule with currency exchange rates
  8. Sprint deliverables and acceptance criteria

TEST 1/8
CONDITIONAL ENTRY POINT ROUTER
Query: "Review dispute resolution and arbitration clause"

Querying gemma2:9b coordinator...

LLM Routing Decision:
  Agents: legal
  Reasoning: The query specifically mentions 'dispute resolution and arbitration clause', which fall under contract law and legal considerations.

First agent to execute: LEGAL

  → LEGAL AGENT executing (threshold: 0.1)
     Query: Review dispute resolution and arbitration clause...
     Retrieved: 2 clauses (avg: 0.

# Conversation Memory and State Persistence

In [78]:
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, END
from datetime import datetime
import json

#### 1. Defining Enhanced GraphState with Memory

In [80]:
print("CONVERSATION MEMORY & STATE PERSISTENCE")


print("\nDefining enhanced GraphState with conversation memory and persistence...")

class GraphStateWithMemory(TypedDict):
  
    query: str
    
    memory: List[Dict[str, Any]]  
    conversation_history: List[Dict[str, Any]] 
    session_id: str 
    turn_number: int 
    
    routed_agents: List[str]
    routing_reasoning: str
    routing_approach: str
    
    legal: Dict[str, Any]
    compliance: Dict[str, Any] 
    finance: Dict[str, Any]
    operations: Dict[str, Any] 
    
    all_clauses: List[Dict[str, Any]]
    total_clauses_retrieved: int
    
    execution_order: List[str]
    timestamp: str
    
    coordinator_model: str
    vector_store: str
    execution_status: str
    
    previous_state: Dict[str, Any]  
    checkpoint_path: str  

print("GraphStateWithMemory defined")

print("DEFINING MEMORY MANAGEMENT FUNCTIONS")

def add_to_memory(state: GraphStateWithMemory, query: str, response: Dict[str, Any]) -> GraphStateWithMemory:

    memory_entry = {
        "turn": state.get('turn_number', 0),
        "query": query,
        "routed_agents": response.get('routed_agents', []),
        "clauses_retrieved": response.get('total_clauses_retrieved', 0),
        "timestamp": response.get('timestamp', ''),
        "summary": {
            agent: {
                'status': response.get(agent, {}).get('status', 'not_executed'),
                'clauses': response.get(agent, {}).get('clauses_retrieved', 0)
            }
            for agent in ['legal', 'compliance', 'finance', 'operations']
        }
    }
    
    if 'memory' not in state or state['memory'] is None:
        state['memory'] = []
    
    state['memory'].append(memory_entry)
    state['turn_number'] = state.get('turn_number', 0) + 1
    
    return state

print("add_to_memory() defined")

def get_conversation_context(state: GraphStateWithMemory) -> str:

    if not state.get('memory') or len(state['memory']) == 0:
        return "This is the first query in the conversation."
    
    context_parts = ["Previous conversation context:"]
    
    for entry in state['memory'][-3:]:  # Last 3 turns
        turn = entry.get('turn', 0)
        query = entry.get('query', '')
        agents = ', '.join(entry.get('routed_agents', []))
        clauses = entry.get('clauses_retrieved', 0)
        
        context_parts.append(f"Turn {turn}: '{query}' → Agents: {agents}, Clauses: {clauses}")
    
    return "\n".join(context_parts)

print("get_conversation_context() defined")

print("DEFINING STATE PERSISTENCE FUNCTIONS")

import json
import os
from datetime import datetime

CHECKPOINT_DIR = "../Data/Results/Checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

def save_state_checkpoint(state: GraphStateWithMemory) -> str:
  
    session_id = state.get('session_id', 'unknown')
    turn = state.get('turn_number', 0)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    checkpoint_file = f"checkpoint_{session_id}_turn{turn}_{timestamp}.json"
    checkpoint_path = os.path.join(CHECKPOINT_DIR, checkpoint_file)
    
    checkpoint_data = {
        "session_id": session_id,
        "turn_number": turn,
        "timestamp": state.get('timestamp', ''),
        "query": state.get('query', ''),
        "memory": state.get('memory', []),
        "conversation_history": state.get('conversation_history', []),
        "routed_agents": state.get('routed_agents', []),
        "routing_reasoning": state.get('routing_reasoning', ''),
        "execution_order": state.get('execution_order', []),
        "total_clauses_retrieved": state.get('total_clauses_retrieved', 0),
        "coordinator_model": state.get('coordinator_model', 'gemma2:9b'),
        "vector_store": state.get('vector_store', 'Pinecone')
    }
    
    with open(checkpoint_path, 'w', encoding='utf-8') as f:
        json.dump(checkpoint_data, f, indent=2, ensure_ascii=False)
    
    print(f"  State saved: {checkpoint_file}")
    return checkpoint_path

print("save_state_checkpoint() defined")

def load_state_checkpoint(checkpoint_path: str) -> Dict[str, Any]:
 
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")
    
    with open(checkpoint_path, 'r', encoding='utf-8') as f:
        checkpoint_data = json.load(f)
    
    print(f"  State loaded from checkpoint")
    print(f"    Session: {checkpoint_data.get('session_id', 'unknown')}")
    print(f"    Turn: {checkpoint_data.get('turn_number', 0)}")
    print(f"    Memory entries: {len(checkpoint_data.get('memory', []))}")
    
    return checkpoint_data

print("load_state_checkpoint() defined")

def get_latest_checkpoint(session_id: str) -> str:

    checkpoints = [f for f in os.listdir(CHECKPOINT_DIR) 
                  if f.startswith(f"checkpoint_{session_id}_")]
    
    if not checkpoints:
        raise FileNotFoundError(f"No checkpoints found for session: {session_id}")
    
    latest = max(checkpoints, key=lambda f: os.path.getctime(os.path.join(CHECKPOINT_DIR, f)))
    
    return os.path.join(CHECKPOINT_DIR, latest)

print("get_latest_checkpoint() defined")


print("CONVERSATION STATE INITIALIZATION")

def initialize_conversation_state(session_id: str = None) -> GraphStateWithMemory:
 
    if session_id is None:
        session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        state = {
            "query": "",
            "memory": [],
            "conversation_history": [],
            "session_id": session_id,
            "turn_number": 0,
            "routed_agents": [],
            "routing_reasoning": "",
            "routing_approach": "",
            "legal": {},
            "compliance": {},
            "finance": {},
            "operations": {},
            "all_clauses": [],
            "total_clauses_retrieved": 0,
            "execution_order": [],
            "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
            "coordinator_model": "gemma2:9b",
            "vector_store": "Pinecone",
            "execution_status": "initialized",
            "previous_state": {},
            "checkpoint_path": ""
        }
        
        print(f"New conversation initialized: {session_id}")
        
    else:
        try:
            checkpoint_path = get_latest_checkpoint(session_id)
            checkpoint_data = load_state_checkpoint(checkpoint_path)
            
            state = {
                "query": "",
                "memory": checkpoint_data.get('memory', []),
                "conversation_history": checkpoint_data.get('conversation_history', []),
                "session_id": session_id,
                "turn_number": checkpoint_data.get('turn_number', 0),
                "routed_agents": [],
                "routing_reasoning": "",
                "routing_approach": "",
                "legal": {},
                "compliance": {},
                "finance": {},
                "operations": {},
                "all_clauses": [],
                "total_clauses_retrieved": 0,
                "execution_order": [],
                "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
                "coordinator_model": checkpoint_data.get('coordinator_model', 'gemma2:9b'),
                "vector_store": checkpoint_data.get('vector_store', 'Pinecone'),
                "execution_status": "resumed",
                "previous_state": checkpoint_data,
                "checkpoint_path": checkpoint_path
            }
            
            print(f"Conversation resumed: {session_id}")
            print(f"  Previous turns: {state['turn_number']}")
            
        except FileNotFoundError as e:
            print(f"{e}")
            print("  Starting new conversation instead")
            return initialize_conversation_state(session_id=None)
    
    return state

print("initialize_conversation_state() defined")

print("CONVERSATION MEMORY & STATE PERSISTENCE READY")

CONVERSATION MEMORY & STATE PERSISTENCE

Defining enhanced GraphState with conversation memory and persistence...
GraphStateWithMemory defined
DEFINING MEMORY MANAGEMENT FUNCTIONS
add_to_memory() defined
get_conversation_context() defined
DEFINING STATE PERSISTENCE FUNCTIONS
save_state_checkpoint() defined
load_state_checkpoint() defined
get_latest_checkpoint() defined
CONVERSATION STATE INITIALIZATION
initialize_conversation_state() defined
CONVERSATION MEMORY & STATE PERSISTENCE READY


#### 2. Initializing Input State with Empty Memory

In [81]:
print("INITIALIZING CONVERSATION STATE WITH MEMORY")

print("\nInitializing state with conversation memory and persistence...")

session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

input_state_memory = {
    "query": "Analyze contract for termination clauses and payment terms",
    
    "memory": [], 
    "conversation_history": [],  
    "session_id": session_id,
    "turn_number": 0,  
    
    "routed_agents": [],
    "routing_reasoning": "",
    "routing_approach": "",
    
    "legal": {},
    "compliance": {},  
    "finance": {},
    "operations": {},  
    
    "all_clauses": [],
    "total_clauses_retrieved": 0,
    
    "execution_order": [],
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
    
    "coordinator_model": "gemma2:9b",
    "vector_store": "Pinecone",
    "execution_status": "initialized",
    
    "previous_state": {},
    "checkpoint_path": ""
}

print("State initialized with memory capabilities")

print("STATE DETAILS")
print(f"  Session ID: {input_state_memory['session_id']}")
print(f"  Turn Number: {input_state_memory['turn_number']}")
print(f"  Query: \"{input_state_memory['query']}\"")
print(f"  Initial Memory: {input_state_memory['memory']}")
print(f"  Conversation History: {input_state_memory['conversation_history']}")
print(f"  Coordinator Model: {input_state_memory['coordinator_model']}")
print(f"  Vector Store: {input_state_memory['vector_store']}")

print("ADAPTIVE THRESHOLDS CONFIGURED")
print(f"  Legal: {AGENT_THRESHOLDS['legal']} (precision)")
print(f"  Compliance: {AGENT_THRESHOLDS['compliance']} (recall) ← Lower")
print(f"  Finance: {AGENT_THRESHOLDS['finance']} (precision)")
print(f"  Operations: {AGENT_THRESHOLDS['operations']} (recall) ← Lower")

print("STATE READY FOR EXECUTION WITH MEMORY")

INITIALIZING CONVERSATION STATE WITH MEMORY

Initializing state with conversation memory and persistence...
State initialized with memory capabilities
STATE DETAILS
  Session ID: session_20260115_172002
  Turn Number: 0
  Query: "Analyze contract for termination clauses and payment terms"
  Initial Memory: []
  Conversation History: []
  Coordinator Model: gemma2:9b
  Vector Store: Pinecone
ADAPTIVE THRESHOLDS CONFIGURED
  Legal: 0.1 (precision)
  Compliance: 0.1 (recall) ← Lower
  Finance: 0.1 (precision)
  Operations: 0.1 (recall) ← Lower
STATE READY FOR EXECUTION WITH MEMORY


#### 3. Creating Agent Nodes with Memory Logging

#### i. Legal Node

In [82]:
print("CREATING AGENT NODES WITH MEMORY LOGGING")

CREATING AGENT NODES WITH MEMORY LOGGING


In [83]:
def legal_node_with_memory(state: GraphStateWithMemory) -> GraphStateWithMemory:

    print(f"  LEGAL AGENT EXECUTING (Threshold: {AGENT_THRESHOLDS['legal']})")
    print(f"     Query: {state['query'][:60]}...")
    print(f"     Turn: {state.get('turn_number', 0)}")
    
    try:
        results = retrieve_from_pinecone_adaptive(state['query'], agent_filter='legal', top_k=5)
        
        if results and results.matches:
            threshold = AGENT_THRESHOLDS['legal']
            filtered_matches = [m for m in results.matches if m.score > threshold]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["legal"] = {
                "agent": "Legal",
                "status": "completed",
                "source": "pinecone_vector_store",
                "threshold": threshold,
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "relevance_scores": scores,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "max_relevance": max(scores) if scores else 0
            }
            
            for clause in clauses:
                clause['agent'] = 'legal'
                state["all_clauses"].append(clause)
            
            state["total_clauses_retrieved"] += len(clauses)
            
            state["memory"].append({
                "agent": "legal",
                "timestamp": datetime.now().isoformat(),
                "turn": state.get('turn_number', 0),
                "clauses_retrieved": len(clauses),
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "sample_clauses": [c['clause'][:100] for c in clauses[:2]],
                "summary": f"Legal: Retrieved {len(clauses)} clauses (avg relevance: {sum(scores) / len(scores):.3f})"
            })
            
            print(f"     Retrieved: {len(clauses)} clauses")
            print(f"     Avg relevance: {sum(scores) / len(scores):.3f}")
            print(f"     Written to memory")
            
        else:
            state["legal"] = {
                "agent": "Legal",
                "status": "no_results",
                "source": "pinecone_vector_store",
                "threshold": AGENT_THRESHOLDS['legal'],
                "clauses_retrieved": 0,
                "clauses": []
            }
            
            state["memory"].append({
                "agent": "legal",
                "timestamp": datetime.now().isoformat(),
                "turn": state.get('turn_number', 0),
                "clauses_retrieved": 0,
                "summary": "Legal: No clauses found above threshold"
            })
            
            print(f"     ⚠ No clauses above threshold")
            
    except Exception as e:
        state["legal"] = {"agent": "Legal", "status": "error", "error": str(e)}
        state["memory"].append({
            "agent": "legal",
            "timestamp": datetime.now().isoformat(),
            "turn": state.get('turn_number', 0),
            "error": str(e)[:100]
        })
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("legal")
    
    print(f"\n  MEMORY STATE AFTER LEGAL AGENT:")
    print(f"     Total entries: {len(state['memory'])}")
    for idx, mem in enumerate(state['memory'][-3:], max(1, len(state['memory'])-2)):
        print(f"     [{idx}] {mem.get('agent', 'unknown').upper()}: {mem.get('summary', 'N/A')}")
    
    return state

print("legal_node_with_memory() defined")

legal_node_with_memory() defined


#### ii. Compliance Node

In [84]:

def compliance_node_with_memory(state: GraphStateWithMemory) -> GraphStateWithMemory:

    print(f"  COMPLIANCE AGENT EXECUTING (Threshold: {AGENT_THRESHOLDS['compliance']})")
    print(f"     Query: {state['query'][:60]}...")
    print(f"     Turn: {state.get('turn_number', 0)}")
    
    try:
        results = retrieve_from_pinecone_adaptive(state['query'], agent_filter='compliance', top_k=5)
        
        if results and results.matches:
            threshold = AGENT_THRESHOLDS['compliance'] 
            filtered_matches = [m for m in results.matches if m.score > threshold]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["compliance"] = {
                "agent": "Compliance",
                "status": "completed",
                "source": "pinecone_vector_store",
                "threshold": threshold,
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "relevance_scores": scores,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "max_relevance": max(scores) if scores else 0
            }
            
            for clause in clauses:
                clause['agent'] = 'compliance'
                state["all_clauses"].append(clause)
            
            state["total_clauses_retrieved"] += len(clauses)
            
            state["memory"].append({
                "agent": "compliance",
                "timestamp": datetime.now().isoformat(),
                "turn": state.get('turn_number', 0),
                "clauses_retrieved": len(clauses),
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "threshold_used": threshold,
                "sample_clauses": [c['clause'][:100] for c in clauses[:2]],
                "summary": f"Compliance: Retrieved {len(clauses)} clauses (threshold: {threshold}, avg: {sum(scores) / len(scores):.3f})"
            })
            
            print(f"     Retrieved: {len(clauses)} clauses")
            print(f"     Threshold: {threshold} (lower for recall)")
            print(f"     Avg relevance: {sum(scores) / len(scores):.3f}")
            
        else:
            state["compliance"] = {
                "agent": "Compliance",
                "status": "no_results",
                "source": "pinecone_vector_store",
                "threshold": AGENT_THRESHOLDS['compliance'],
                "clauses_retrieved": 0,
                "clauses": []
            }
            
            state["memory"].append({
                "agent": "compliance",
                "timestamp": datetime.now().isoformat(),
                "turn": state.get('turn_number', 0),
                "clauses_retrieved": 0,
                "threshold_used": AGENT_THRESHOLDS['compliance'],
                "summary": f"Compliance: No clauses above threshold {AGENT_THRESHOLDS['compliance']}"
            })
            
            print(f"     No clauses above threshold {AGENT_THRESHOLDS['compliance']}")
            
    except Exception as e:
        state["compliance"] = {"agent": "Compliance", "status": "error", "error": str(e)}
        state["memory"].append({
            "agent": "compliance",
            "timestamp": datetime.now().isoformat(),
            "turn": state.get('turn_number', 0),
            "error": str(e)[:100]
        })
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("compliance")
    
    print(f"\n  MEMORY STATE AFTER COMPLIANCE AGENT:")
    print(f"     Total entries: {len(state['memory'])}")
    for idx, mem in enumerate(state['memory'][-3:], max(1, len(state['memory'])-2)):
        print(f"     [{idx}] {mem.get('agent', 'unknown').upper()}: {mem.get('summary', 'N/A')}")
    
    return state

print("compliance_node_with_memory() defined")

compliance_node_with_memory() defined


#### iii. Finance Node

In [85]:

def finance_node_with_memory(state: GraphStateWithMemory) -> GraphStateWithMemory:
    print(f"  FINANCE AGENT EXECUTING (Threshold: {AGENT_THRESHOLDS['finance']})")

    print(f"     Query: {state['query'][:60]}...")
    print(f"     Turn: {state.get('turn_number', 0)}")
    
    try:
        results = retrieve_from_pinecone_adaptive(state['query'], agent_filter='finance', top_k=5)
        
        if results and results.matches:
            threshold = AGENT_THRESHOLDS['finance']
            filtered_matches = [m for m in results.matches if m.score > threshold]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["finance"] = {
                "agent": "Finance",
                "status": "completed",
                "source": "pinecone_vector_store",
                "threshold": threshold,
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "relevance_scores": scores,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "max_relevance": max(scores) if scores else 0
            }
            
            for clause in clauses:
                clause['agent'] = 'finance'
                state["all_clauses"].append(clause)
            
            state["total_clauses_retrieved"] += len(clauses)
            
            state["memory"].append({
                "agent": "finance",
                "timestamp": datetime.now().isoformat(),
                "turn": state.get('turn_number', 0),
                "clauses_retrieved": len(clauses),
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "sample_clauses": [c['clause'][:100] for c in clauses[:2]],
                "summary": f"Finance: Retrieved {len(clauses)} clauses (avg: {sum(scores) / len(scores):.3f})"
            })
            
            print(f"     Retrieved: {len(clauses)} clauses")
            print(f"     Avg relevance: {sum(scores) / len(scores):.3f}")
            
        else:
            state["finance"] = {
                "agent": "Finance",
                "status": "no_results",
                "source": "pinecone_vector_store",
                "threshold": AGENT_THRESHOLDS['finance'],
                "clauses_retrieved": 0,
                "clauses": []
            }
            
            state["memory"].append({
                "agent": "finance",
                "timestamp": datetime.now().isoformat(),
                "turn": state.get('turn_number', 0),
                "clauses_retrieved": 0,
                "summary": "Finance: No clauses above threshold"
            })
            
            print(f"     No clauses above threshold")
            
    except Exception as e:
        state["finance"] = {"agent": "Finance", "status": "error", "error": str(e)}
        state["memory"].append({
            "agent": "finance",
            "timestamp": datetime.now().isoformat(),
            "turn": state.get('turn_number', 0),
            "error": str(e)[:100]
        })
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("finance")
    
    print(f"\n  MEMORY STATE AFTER FINANCE AGENT:")
    print(f"     Total entries: {len(state['memory'])}")
    for idx, mem in enumerate(state['memory'][-3:], max(1, len(state['memory'])-2)):
        print(f"     [{idx}] {mem.get('agent', 'unknown').upper()}: {mem.get('summary', 'N/A')}")
    
    return state
print("finance_node_with_memory() defined")

finance_node_with_memory() defined


#### iv. Operations Node

In [86]:
def operations_node_with_memory(state: GraphStateWithMemory) -> GraphStateWithMemory:
    print(f"  OPERATIONS AGENT EXECUTING (Threshold: {AGENT_THRESHOLDS['operations']})")
    print(f"     Query: {state['query'][:60]}...")
    print(f"     Turn: {state.get('turn_number', 0)}")
    
    try:
        results = retrieve_from_pinecone_adaptive(state['query'], agent_filter='operations', top_k=5)
        
        if results and results.matches:
            threshold = AGENT_THRESHOLDS['operations']  
            filtered_matches = [m for m in results.matches if m.score > threshold]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["operations"] = {
                "agent": "Operations",
                "status": "completed",
                "source": "pinecone_vector_store",
                "threshold": threshold,
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "relevance_scores": scores,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "max_relevance": max(scores) if scores else 0
            }
            
            for clause in clauses:
                clause['agent'] = 'operations'
                state["all_clauses"].append(clause)
            
            state["total_clauses_retrieved"] += len(clauses)
            
            state["memory"].append({
                "agent": "operations",
                "timestamp": datetime.now().isoformat(),
                "turn": state.get('turn_number', 0),
                "clauses_retrieved": len(clauses),
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "threshold_used": threshold,
                "sample_clauses": [c['clause'][:100] for c in clauses[:2]],
                "summary": f"Operations: Retrieved {len(clauses)} clauses (threshold: {threshold}, avg: {sum(scores) / len(scores):.3f})"
            })
            
            print(f"     Retrieved: {len(clauses)} clauses")
            print(f"     Threshold: {threshold} (lower for recall)")
            print(f"     Avg relevance: {sum(scores) / len(scores):.3f}")
            
        else:
            state["operations"] = {
                "agent": "Operations",
                "status": "no_results",
                "source": "pinecone_vector_store",
                "threshold": AGENT_THRESHOLDS['operations'],
                "clauses_retrieved": 0,
                "clauses": []
            }
            
            state["memory"].append({
                "agent": "operations",
                "timestamp": datetime.now().isoformat(),
                "turn": state.get('turn_number', 0),
                "clauses_retrieved": 0,
                "threshold_used": AGENT_THRESHOLDS['operations'],
                "summary": f"Operations: No clauses above threshold {AGENT_THRESHOLDS['operations']}"
            })
            
            print(f"     No clauses above threshold {AGENT_THRESHOLDS['operations']}")
            
    except Exception as e:
        state["operations"] = {"agent": "Operations", "status": "error", "error": str(e)}
        state["memory"].append({
            "agent": "operations",
            "timestamp": datetime.now().isoformat(),
            "turn": state.get('turn_number', 0),
            "error": str(e)[:100]
        })
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("operations")
    
    print(f"\n  MEMORY STATE AFTER OPERATIONS AGENT:")
    print(f"     Total entries: {len(state['memory'])}")
    for idx, mem in enumerate(state['memory'][-3:], max(1, len(state['memory'])-2)):
        print(f"     [{idx}] {mem.get('agent', 'unknown').upper()}: {mem.get('summary', 'N/A')}")
    
    return state

print("operations_node_with_memory() defined")

operations_node_with_memory() defined


In [87]:
print("ALL AGENT NODES WITH MEMORY LOGGING DEFINED")

ALL AGENT NODES WITH MEMORY LOGGING DEFINED


#### 4. Building Graph with Memory Support

In [88]:
print("BUILDING WORKFLOW WITH MEMORY SUPPORT")

print("\nBuilding dynamic workflow with conversation memory and state persistence...")

workflow_memory = StateGraph(GraphStateWithMemory)
print("StateGraph initialized with GraphStateWithMemory")

print("ADDING AGENT NODES WITH MEMORY")

def routing_node_with_memory(state: GraphStateWithMemory) -> GraphStateWithMemory:

    print("ROUTING NODE: Dynamic LLM Coordinator with Memory")
    print(f"Query: \"{state['query']}\"")
    print(f"Turn: {state.get('turn_number', 0)}")
    print(f"Session: {state.get('session_id', 'unknown')}")
    
    context = get_conversation_context(state)
    print(f"\n{context}")
    
    routing_result = dynamic_route_query(state['query'])
    
    state['routed_agents'] = routing_result['agents']
    state['routing_reasoning'] = routing_result['reasoning']
    state['routing_approach'] = routing_result['approach']
    
    print(f"\nRouting Decision:")
    print(f"  Agents: {', '.join(state['routed_agents'])}")
    print(f"  Reasoning: {state['routing_reasoning']}")
    
    state["memory"].append({
        "agent": "routing",
        "timestamp": datetime.now().isoformat(),
        "turn": state.get('turn_number', 0),
        "routed_agents": routing_result['agents'],
        "reasoning": routing_result['reasoning'],
        "summary": f"Routing: Selected {', '.join(routing_result['agents'])}"
    })
    
    state['execution_order'].append("routing")
    
    return state

workflow_memory.add_node("routing", routing_node_with_memory)
print("Added: routing (with memory awareness)")

workflow_memory.add_node("legal_agent", legal_node_with_memory)
print("Added: legal_agent (threshold: 0.30)")

workflow_memory.add_node("compliance_agent", compliance_node_with_memory)
print("Added: compliance_agent (threshold: 0.10)")

workflow_memory.add_node("finance_agent", finance_node_with_memory)
print("Added: finance_agent (threshold: 0.30)")

workflow_memory.add_node("operations_agent", operations_node_with_memory)
print("Added: operations_agent (threshold: 0.10)")

def aggregation_node_with_memory(state: GraphStateWithMemory) -> GraphStateWithMemory:

    print("AGGREGATION NODE: Consolidating Results with Memory")

    if state['all_clauses']:
        state['all_clauses'].sort(key=lambda x: x.get('relevance_score', 0), reverse=True)
        
        print(f"\nTotal clauses retrieved: {state['total_clauses_retrieved']}")
        print(f"Top relevance score: {state['all_clauses'][0]['relevance_score']:.3f}")
        
        state['execution_status'] = 'success'
        
        state["memory"].append({
            "agent": "aggregation",
            "timestamp": datetime.now().isoformat(),
            "turn": state.get('turn_number', 0),
            "total_clauses": state['total_clauses_retrieved'],
            "top_score": state['all_clauses'][0]['relevance_score'],
            "summary": f"Aggregation: Consolidated {state['total_clauses_retrieved']} clauses"
        })
    else:
        print(f"\nNo clauses retrieved")
        state['execution_status'] = 'no_results'
        
        state["memory"].append({
            "agent": "aggregation",
            "timestamp": datetime.now().isoformat(),
            "turn": state.get('turn_number', 0),
            "total_clauses": 0,
            "summary": "Aggregation: No clauses to consolidate"
        })
    
    state['execution_order'].append("aggregation")
    
    checkpoint_path = save_state_checkpoint(state)
    state['checkpoint_path'] = checkpoint_path
    
    print(f"\nMemory entries: {len(state['memory'])}")
    print(f"Checkpoint saved: {os.path.basename(checkpoint_path)}")
    
    return state

workflow_memory.add_node("aggregation", aggregation_node_with_memory)
print("Added: aggregation (with checkpoint saving)")

print("DEFINING CONDITIONAL EDGES")

workflow_memory.set_entry_point("routing")
print("Entry point: routing")

def get_next_agent_memory(state: GraphStateWithMemory) -> str:
    routed_agents = state.get('routed_agents', [])
    executed_agents = [a for a in state.get('execution_order', []) 
                      if a not in ['routing', 'aggregation']]
    
    agent_priority = ['legal', 'compliance', 'finance', 'operations']
    
    for agent in agent_priority:
        if agent in routed_agents and agent not in executed_agents:
            return f"{agent}_agent"
    
    return "aggregation"

workflow_memory.add_conditional_edges(
    "routing",
    lambda state: get_next_agent_memory(state),
    {
        "legal_agent": "legal_agent",
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("routing → [conditional first agent]")

workflow_memory.add_conditional_edges(
    "legal_agent",
    lambda state: get_next_agent_memory(state),
    {
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("legal_agent → [conditional next]")

workflow_memory.add_conditional_edges(
    "compliance_agent",
    lambda state: get_next_agent_memory(state),
    {
        "legal_agent": "legal_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("compliance_agent → [conditional next]")

workflow_memory.add_conditional_edges(
    "finance_agent",
    lambda state: get_next_agent_memory(state),
    {
        "legal_agent": "legal_agent",
        "compliance_agent": "compliance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("finance_agent → [conditional next]")

workflow_memory.add_edge("operations_agent", "aggregation")
print("operations_agent → aggregation")

workflow_memory.add_edge("aggregation", END)
print("aggregation → END")

BUILDING WORKFLOW WITH MEMORY SUPPORT

Building dynamic workflow with conversation memory and state persistence...
StateGraph initialized with GraphStateWithMemory
ADDING AGENT NODES WITH MEMORY
Added: routing (with memory awareness)
Added: legal_agent (threshold: 0.30)
Added: compliance_agent (threshold: 0.10)
Added: finance_agent (threshold: 0.30)
Added: operations_agent (threshold: 0.10)
Added: aggregation (with checkpoint saving)
DEFINING CONDITIONAL EDGES
Entry point: routing
routing → [conditional first agent]
legal_agent → [conditional next]
compliance_agent → [conditional next]
finance_agent → [conditional next]
operations_agent → aggregation
aggregation → END


#### 5. Compiling Graph

In [89]:
print("COMPILING WORKFLOW WITH MEMORY")

try:
    app_memory = workflow_memory.compile()
    print("Workflow compiled successfully!")
 
except Exception as e:
    print(f"Compilation failed: {str(e)}")
    import traceback
    traceback.print_exc()

print("WORKFLOW WITH MEMORY READY")

COMPILING WORKFLOW WITH MEMORY
Workflow compiled successfully!
WORKFLOW WITH MEMORY READY


#### 6. Executing Graph and Verification

In [93]:
print("EXECUTING WORKFLOW WITH MEMORY PERSISTENCE")

print("\nPreparing to execute workflow with conversation memory...")

print("INPUT STATE VERIFICATION")
print(f"  Session ID: {input_state_memory.get('session_id', 'N/A')}")
print(f"  Turn Number: {input_state_memory.get('turn_number', 0)}")
print(f"  Query: \"{input_state_memory['query']}\"")
print(f"  Memory Entries: {len(input_state_memory.get('memory', []))}")
print(f"  Coordinator: {input_state_memory.get('coordinator_model', 'N/A')}")
print(f"  Vector Store: {input_state_memory.get('vector_store', 'N/A')}")


print("EXECUTING WORKFLOW - TURN 0")

try:
    result_memory = app_memory.invoke(input_state_memory)
    
    print("EXECUTION COMPLETED SUCCESSFULLY")

    print("ROUTING DECISION")
    print(f"  Query: \"{result_memory['query']}\"")
    print(f"  LLM Routed To: {', '.join(result_memory.get('routed_agents', []))}")
    print(f"  Reasoning: {result_memory.get('routing_reasoning', 'N/A')}")
    print(f"  Approach: {result_memory.get('routing_approach', 'N/A')}")

    print("EXECUTION SUMMARY")
    print(f"  Session ID: {result_memory.get('session_id', 'N/A')}")
    print(f"  Turn Number: {result_memory.get('turn_number', 0)}")
    print(f"  Execution Order: {' → '.join(result_memory['execution_order'])}")
    print(f"  Total Clauses: {result_memory.get('total_clauses_retrieved', 0)}")
    print(f"  Execution Status: {result_memory.get('execution_status', 'unknown')}")
    

    print("AGENT RESULTS")
    print(f"{'Agent':<15} {'Status':<15} {'Clauses':<10} {'Threshold':<12} {'Avg Score':<12}")

    executed_agents = []
    for agent_name in ["legal", "compliance", "finance", "operations"]:
        agent_data = result_memory.get(agent_name, {})
        
        if agent_data and agent_data.get("status") == "completed":
            executed_agents.append(agent_name)
            status = "EXECUTED"
            clauses = agent_data.get('clauses_retrieved', 0)
            threshold = agent_data.get('threshold', 'N/A')
            avg_score = agent_data.get('avg_relevance', 0)
            
            threshold_str = f"{threshold:.2f}" if isinstance(threshold, float) else str(threshold)
            score_str = f"{avg_score:.3f}" if avg_score > 0 else "N/A"
            
            print(f"{agent_name.upper():<15} {status:<15} {clauses:<10} {threshold_str:<12} {score_str:<12}")
            
        elif agent_data and agent_data.get("status") == "no_results":
            executed_agents.append(agent_name)
            status = "EXECUTED (0)"
            threshold = agent_data.get('threshold', 'N/A')
            threshold_str = f"{threshold:.2f}" if isinstance(threshold, float) else str(threshold)
            
            print(f"{agent_name.upper():<15} {status:<15} {0:<10} {threshold_str:<12} {'N/A':<12}")
            
        else:
            status = "NOT EXECUTED"
            print(f"{agent_name.upper():<15} {status:<15} {'-':<10} {'-':<12} {'-':<12}")
    
    if result_memory.get('all_clauses'):
        print(f"RETRIEVED CLAUSES (Top {min(5, len(result_memory['all_clauses']))})")
   
        for idx, clause in enumerate(result_memory['all_clauses'][:5], 1):
            agent = clause.get('agent', 'unknown')
            score = clause.get('relevance_score', 0)
            text = clause.get('clause', '')[:150]
            
            print(f"\n[{idx}] Agent: {agent.upper()} | Score: {score:.3f}")
            print(f"    {text}...")
    

    print("CONVERSATION MEMORY STATE")
    print(f"  Total Memory Entries: {len(result_memory.get('memory', []))}")
    print(f"  Turn Number: {result_memory.get('turn_number', 0)}")
    
    if result_memory.get('memory'):
        print(f"\n  Memory Log:")
        for idx, mem_entry in enumerate(result_memory['memory'], 1):
            agent = mem_entry.get('agent', 'unknown')
            summary = mem_entry.get('summary', 'N/A')
            turn = mem_entry.get('turn', 0)
            
            print(f"    [{idx}] Turn {turn} - {agent.upper()}: {summary}")

    print("STATE PERSISTENCE")

    checkpoint_path = result_memory.get('checkpoint_path', '')
    if checkpoint_path and os.path.exists(checkpoint_path):
        checkpoint_file = os.path.basename(checkpoint_path)
        print(f"  Checkpoint saved: {checkpoint_file}")
        print(f"  Location: {CHECKPOINT_DIR}")
        print(f"  Session: {result_memory.get('session_id', 'N/A')}")
        print(f"\n  To resume:")
        print(f"    loaded = load_state_checkpoint('{checkpoint_path}')")
    else:
        print(f"  No checkpoint saved")

    print("EFFICIENCY ANALYSIS")

    routed_count = len(result_memory.get('routed_agents', []))
    total_agents = 4
    
    print(f"  Agents Routed: {routed_count}/{total_agents}")
    print(f"  Agents Executed: {len(executed_agents)}")
    print(f"  Agents Skipped: {total_agents - len(executed_agents)}")
    print(f"  Efficiency Gain: {(total_agents - len(executed_agents)) / total_agents * 100:.0f}%")

    print("ADAPTIVE THRESHOLD IMPACT")

    for agent_name in ["compliance", "operations"]:
        if agent_name in executed_agents:
            agent_data = result_memory.get(agent_name, {})
            threshold = agent_data.get('threshold', 0)
            clauses = agent_data.get('clauses_retrieved', 0)
            
            print(f"\n  {agent_name.upper()}:")
            print(f"    Threshold: {threshold:.2f} (LOWER for recall)")
            print(f"    Clauses Retrieved: {clauses}")
            
            if clauses > 0:
                avg_score = agent_data.get('avg_relevance', 0)
                print(f"    Avg Relevance: {avg_score:.3f}")
                print(f"    Lower threshold enabled retrieval")
            else:
                print(f"    No clauses found even with lower threshold")
    

    print("VERIFICATION")

    checks = []
    
    memory_works = len(result_memory.get('memory', [])) > 0
    checks.append(("Memory Tracking", memory_works))
    
    routing_works = len(result_memory.get('routed_agents', [])) > 0
    checks.append(("Dynamic Routing", routing_works))
    
    compliance_data = result_memory.get('compliance', {})
    operations_data = result_memory.get('operations', {})
    threshold_works = (
        compliance_data.get('threshold', 1.0) == 0.10 or
        operations_data.get('threshold', 1.0) == 0.10
    )
    checks.append(("Adaptive Thresholds (0.10)", threshold_works))
    
    persistence_works = result_memory.get('checkpoint_path', '') != ''
    checks.append(("State Persistence", persistence_works))
    
    retrieval_works = result_memory.get('total_clauses_retrieved', 0) > 0
    checks.append(("Clause Retrieval", retrieval_works))
    
    print(f"\n  System Checks:")
    for check_name, passed in checks:
        status = "PASS" if passed else "FAIL"
        print(f"    {status}: {check_name}")
    
    all_passed = all(passed for _, passed in checks)
    
    if all_passed:
        print(f"\n  ALL CHECKS PASSED - System working correctly!")
    else:
        print(f"\n  Some checks failed - Review configuration")
    
except Exception as e:
    print(f"\nEXECUTION FAILED")
    print(f"Error: {str(e)}")
    print("ERROR DETAILS")
    import traceback
    traceback.print_exc()

print("EXECUTION WITH MEMORY COMPLETE")

EXECUTING WORKFLOW WITH MEMORY PERSISTENCE

Preparing to execute workflow with conversation memory...
INPUT STATE VERIFICATION
  Session ID: session_20260115_172002
  Turn Number: 0
  Query: "Analyze contract for termination clauses and payment terms"
  Memory Entries: 4
  Coordinator: gemma2:9b
  Vector Store: Pinecone
EXECUTING WORKFLOW - TURN 0
ROUTING NODE: Dynamic LLM Coordinator with Memory
Query: "Analyze contract for termination clauses and payment terms"
Turn: 0
Session: session_20260115_172002

Previous conversation context:
Turn 0: '' → Agents: , Clauses: 2
Turn 0: '' → Agents: , Clauses: 3
Turn 0: '' → Agents: , Clauses: 0

Querying gemma2:9b coordinator...

Routing Decision:
  Agents: legal, finance
  Reasoning: The query requests analysis of 'termination clauses' which falls under legal domain and 'payment terms' which falls under finance domain.
AGGREGATION NODE: Consolidating Results with Memory

Total clauses retrieved: 0
Top relevance score: 0.489
  State saved: check

# Agent-to-Agent Communication & Validation Logic

In [94]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from datetime import datetime

#### 1. Defining Enhanced GraphState with Validation

In [95]:
print("AGENT-TO-AGENT COMMUNICATION & VALIDATION LOGIC")

print("\nDefining enhanced GraphState with inter-agent communication and validation...")

class GraphStateCollaborative(TypedDict):
   
    query: str
    
    memory: List[Dict[str, Any]]
    conversation_history: List[Dict[str, Any]]
    session_id: str
    turn_number: int
    
    agent_messages: List[Dict[str, Any]]  
    validation_requests: List[Dict[str, Any]]  
    validation_results: List[Dict[str, Any]]  
    cross_references: List[Dict[str, Any]]  
    
    validation_notes: List[str]  
    conflicts_detected: List[Dict[str, Any]] 
    consensus_findings: List[Dict[str, Any]] 
    
    routed_agents: List[str]
    routing_reasoning: str
    routing_approach: str
    
    legal: Dict[str, Any]
    compliance: Dict[str, Any]  
    finance: Dict[str, Any]
    operations: Dict[str, Any]  
    
    all_clauses: List[Dict[str, Any]]
    total_clauses_retrieved: int
    
    execution_order: List[str]
    timestamp: str
    
    coordinator_model: str
    vector_store: str
    execution_status: str
    
    previous_state: Dict[str, Any]
    checkpoint_path: str

print("GraphStateCollaborative defined")



print("DEFINING MESSAGE STRUCTURES")

def create_agent_message(
    sender: str,
    recipient: str,
    message_type: str,
    content: Dict[str, Any],
    turn: int
) -> Dict[str, Any]:

    return {
        "message_id": f"{sender}_to_{recipient}_{datetime.now().strftime('%Y%m%d%H%M%S%f')}",
        "sender": sender,
        "recipient": recipient,
        "message_type": message_type,
        "content": content,
        "turn": turn,
        "timestamp": datetime.now().isoformat()
    }

print("create_agent_message() defined")

def create_validation_request(
    requesting_agent: str,
    validating_agent: str,
    clauses_to_validate: List[Dict[str, Any]],
    validation_type: str,
    turn: int
) -> Dict[str, Any]:

    return {
        "request_id": f"val_{requesting_agent}_to_{validating_agent}_{datetime.now().strftime('%Y%m%d%H%M%S%f')}",
        "requesting_agent": requesting_agent,
        "validating_agent": validating_agent,
        "clauses": clauses_to_validate,
        "validation_type": validation_type,
        "turn": turn,
        "timestamp": datetime.now().isoformat(),
        "status": "pending"
    }

print("create_validation_request() defined")

def create_validation_result(
    request_id: str,
    validating_agent: str,
    validation_outcome: str,
    findings: List[Dict[str, Any]],
    confidence: float,
    turn: int
) -> Dict[str, Any]:

    return {
        "result_id": f"result_{request_id}",
        "request_id": request_id,
        "validating_agent": validating_agent,
        "outcome": validation_outcome,
        "findings": findings,
        "confidence": confidence,
        "turn": turn,
        "timestamp": datetime.now().isoformat()
    }

print("create_validation_result() defined")

print("DEFINING VALIDATION HELPER FUNCTIONS")

def detect_conflicts(
    agent1_clauses: List[Dict[str, Any]],
    agent2_clauses: List[Dict[str, Any]],
    agent1_name: str,
    agent2_name: str,
    threshold: float = 0.1
) -> List[Dict[str, Any]]:

    conflicts = []
    
    for clause1 in agent1_clauses:
        for clause2 in agent2_clauses:
            risk1 = clause1.get('risk_level', 'unknown')
            risk2 = clause2.get('risk_level', 'unknown')
            
            score1 = clause1.get('relevance_score', 0)
            score2 = clause2.get('relevance_score', 0)
            
            effective_threshold = threshold
            if agent1_name in ['compliance', 'operations'] or agent2_name in ['compliance', 'operations']:
                effective_threshold = 0.1  
            
            if score1 > effective_threshold and score2 > effective_threshold:
                if risk1 != risk2 and risk1 != 'unknown' and risk2 != 'unknown':
                    conflicts.append({
                        "conflict_type": "risk_assessment_mismatch",
                        "agent1": agent1_name,
                        "agent2": agent2_name,
                        "clause1": clause1.get('clause', '')[:100],
                        "clause2": clause2.get('clause', '')[:100],
                        "risk1": risk1,
                        "risk2": risk2,
                        "threshold_used": effective_threshold
                    })
    
    return conflicts

print("detect_conflicts() defined (uses 0.1 for Compliance/Operations)")

def cross_reference_clauses(
    source_agent: str,
    source_clauses: List[Dict[str, Any]],
    target_agent: str,
    target_clauses: List[Dict[str, Any]],
    threshold: float = 0.1
) -> List[Dict[str, Any]]:

    cross_refs = []
    
    effective_threshold = threshold
    if source_agent in ['compliance', 'operations'] or target_agent in ['compliance', 'operations']:
        effective_threshold = 0.1
    
    for src_clause in source_clauses:
        if src_clause.get('relevance_score', 0) > effective_threshold:
            for tgt_clause in target_clauses:
                if tgt_clause.get('relevance_score', 0) > effective_threshold:
                 
                    src_text = src_clause.get('clause', '').lower()
                    tgt_text = tgt_clause.get('clause', '').lower()
                    
                    overlap_score = len(set(src_text.split()) & set(tgt_text.split())) / max(len(src_text.split()), len(tgt_text.split()))
                    
                    if overlap_score > 0.3:  
                        cross_refs.append({
                            "source_agent": source_agent,
                            "target_agent": target_agent,
                            "source_clause": src_text[:100],
                            "target_clause": tgt_text[:100],
                            "overlap_score": overlap_score,
                            "threshold_used": effective_threshold,
                            "reference_type": "semantic_overlap"
                        })
    
    return cross_refs

print("cross_reference_clauses() defined")

AGENT-TO-AGENT COMMUNICATION & VALIDATION LOGIC

Defining enhanced GraphState with inter-agent communication and validation...
GraphStateCollaborative defined
DEFINING MESSAGE STRUCTURES
create_agent_message() defined
create_validation_request() defined
create_validation_result() defined
DEFINING VALIDATION HELPER FUNCTIONS
detect_conflicts() defined (uses 0.1 for Compliance/Operations)
cross_reference_clauses() defined


#### 2. Initializing Collaborative State

In [96]:
print("COLLABORATIVE STATE INITIALIZATION")

def initialize_collaborative_state(session_id: str = None) -> GraphStateCollaborative:

    if session_id is None:
        session_id = f"collab_session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    state = {
        "query": "",
        
        "memory": [],
        "conversation_history": [],
        "session_id": session_id,
        "turn_number": 0,
        
        "agent_messages": [],
        "validation_requests": [],
        "validation_results": [],
        "cross_references": [],
        
        "validation_notes": [],
        "conflicts_detected": [],
        "consensus_findings": [],
        
        "routed_agents": [],
        "routing_reasoning": "",
        "routing_approach": "",
        
        "legal": {},
        "compliance": {}, 
        "finance": {},
        "operations": {}, 
        
        "all_clauses": [],
        "total_clauses_retrieved": 0,
        
        "execution_order": [],
        "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
        
        "coordinator_model": "gemma2:9b",
        "vector_store": "Pinecone",
        "execution_status": "initialized",
        
        "previous_state": {},
        "checkpoint_path": ""
    }
    
    print(f"Collaborative state initialized: {session_id}")

    
    return state

print("initialize_collaborative_state() defined")

COLLABORATIVE STATE INITIALIZATION
initialize_collaborative_state() defined


#### 3. Creating Compliance Agent with Memory Writing

In [97]:
print("CREATING COLLABORATIVE AGENTS WITH COMMUNICATION & VALIDATION")

print("\nDefining agents with inter-agent communication and validation")

CREATING COLLABORATIVE AGENTS WITH COMMUNICATION & VALIDATION

Defining agents with inter-agent communication and validation


In [98]:
def compliance_node_collaborative(state: GraphStateCollaborative) -> GraphStateCollaborative:

    print(f"  COMPLIANCE AGENT EXECUTING (Threshold: {AGENT_THRESHOLDS['compliance']})")
    print(f"  Writing findings to shared memory for validation")

    try:
        results = retrieve_from_pinecone_adaptive(state['query'], agent_filter='compliance', top_k=5)
        
        if results and results.matches:
            threshold = AGENT_THRESHOLDS['compliance'] 
            filtered_matches = [m for m in results.matches if m.score > threshold]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score,
                    'agent': 'compliance'
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            risk_level = "high" if any(c.get('risk_level') == 'high' for c in clauses) else "medium"
            
            state["compliance"] = {
                "agent": "Compliance",
                "status": "completed",
                "source": "pinecone_vector_store",
                "threshold": threshold,
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "risk_level": risk_level,
                "avg_relevance": sum(scores) / len(scores) if scores else 0
            }
            
            state["all_clauses"].extend(clauses)
            state["total_clauses_retrieved"] += len(clauses)
            
            state["memory"].append({
                "agent": "compliance",
                "timestamp": datetime.now().isoformat(),
                "turn": state.get('turn_number', 0),
                "clauses": clauses,
                "risk_level": risk_level,
                "threshold_used": threshold,
                "summary": f"Compliance: Found {len(clauses)} clauses (threshold: {threshold}, risk: {risk_level})"
            })
            
            validation_request = create_validation_request(
                requesting_agent="compliance",
                validating_agent="legal",
                clauses_to_validate=clauses,
                validation_type="legal_review_of_compliance",
                turn=state.get('turn_number', 0)
            )
            state["validation_requests"].append(validation_request)
            
            state["validation_notes"].append(
                f"Compliance → Legal: Requesting legal review of {len(clauses)} compliance clauses"
            )
            
            print(f"     Retrieved: {len(clauses)} clauses (threshold: {threshold})")
            print(f"     Risk Level: {risk_level}")
            print(f"     Sent validation request to Legal agent")
            
        else:
            state["compliance"] = {
                "agent": "Compliance",
                "status": "no_results",
                "threshold": AGENT_THRESHOLDS['compliance'],
                "clauses_retrieved": 0,
                "clauses": []
            }
            
            state["memory"].append({
                "agent": "compliance",
                "timestamp": datetime.now().isoformat(),
                "turn": state.get('turn_number', 0),
                "summary": f"Compliance: No clauses above threshold {AGENT_THRESHOLDS['compliance']}"
            })
            
            print(f"     No clauses above threshold {AGENT_THRESHOLDS['compliance']}")
            
    except Exception as e:
        state["compliance"] = {"agent": "Compliance", "status": "error", "error": str(e)}
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("compliance")
    return state

print("compliance_node_collaborative() defined")

compliance_node_collaborative() defined


#### 4. Creating Finance Agent with Memory Reading

In [99]:
def finance_node_collaborative(state: GraphStateCollaborative) -> GraphStateCollaborative:

    print(f"  FINANCE AGENT EXECUTING (Threshold: {AGENT_THRESHOLDS['finance']})")
    print(f"  Reading shared memory from previous agents")

    previous_findings = [m for m in state.get('memory', []) if m.get('agent') in ['compliance', 'legal']]
    if previous_findings:
        print(f"     Reading {len(previous_findings)} entries from other agents:")
        for finding in previous_findings[-2:]: 
            agent = finding.get('agent', 'unknown')
            summary = finding.get('summary', 'N/A')
            print(f"       • {agent.upper()}: {summary}")
    
    try:
        results = retrieve_from_pinecone_adaptive(state['query'], agent_filter='finance', top_k=5)
        
        if results and results.matches:
            threshold = AGENT_THRESHOLDS['finance']
            filtered_matches = [m for m in results.matches if m.score > threshold]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score,
                    'agent': 'finance'
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["finance"] = {
                "agent": "Finance",
                "status": "completed",
                "source": "pinecone_vector_store",
                "threshold": threshold,
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "context_from_agents": [f.get('agent') for f in previous_findings]
            }
            
            state["all_clauses"].extend(clauses)
            state["total_clauses_retrieved"] += len(clauses)
            
            compliance_clauses = state.get('compliance', {}).get('clauses', [])
            if compliance_clauses:
                cross_refs = cross_reference_clauses(
                    source_agent="finance",
                    source_clauses=clauses,
                    target_agent="compliance",
                    target_clauses=compliance_clauses,
                    threshold=0.1  
                )
                
                if cross_refs:
                    state["cross_references"].extend(cross_refs)
                    state["validation_notes"].append(
                        f"Finance: Found {len(cross_refs)} cross-references with Compliance clauses"
                    )
                    print(f"     Cross-referenced {len(cross_refs)} clauses with Compliance")
            
            state["memory"].append({
                "agent": "finance",
                "timestamp": datetime.now().isoformat(),
                "turn": state.get('turn_number', 0),
                "clauses": clauses,
                "cross_references": len(state.get('cross_references', [])),
                "summary": f"Finance: Found {len(clauses)} clauses, cross-referenced with other agents"
            })
            
            print(f"     Retrieved: {len(clauses)} clauses")
            print(f"     Integrated context from previous agents")
            
        else:
            state["finance"] = {
                "agent": "Finance",
                "status": "no_results",
                "threshold": AGENT_THRESHOLDS['finance'],
                "clauses_retrieved": 0
            }
            print(f"     No clauses above threshold")
            
    except Exception as e:
        state["finance"] = {"agent": "Finance", "status": "error", "error": str(e)}
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("finance")
    return state

print("finance_node_collaborative() defined")

finance_node_collaborative() defined


#### 5. Creating Legal Agent with Final Validation

In [101]:
def legal_node_collaborative(state: GraphStateCollaborative) -> GraphStateCollaborative:

    print(f"  LEGAL AGENT EXECUTING (Threshold: {AGENT_THRESHOLDS['legal']})")
    print(f"  Performing validation for other agents")

    pending_validations = [v for v in state.get('validation_requests', []) 
                          if v.get('validating_agent') == 'legal' and v.get('status') == 'pending']
    
    if pending_validations:
        print(f"     Processing {len(pending_validations)} validation request(s)")
    
    try:
        results = retrieve_from_pinecone_adaptive(state['query'], agent_filter='legal', top_k=5)
        
        if results and results.matches:
            threshold = AGENT_THRESHOLDS['legal']
            filtered_matches = [m for m in results.matches if m.score > threshold]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score,
                    'agent': 'legal'
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["legal"] = {
                "agent": "Legal",
                "status": "completed",
                "source": "pinecone_vector_store",
                "threshold": threshold,
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "validations_performed": len(pending_validations)
            }
            
            state["all_clauses"].extend(clauses)
            state["total_clauses_retrieved"] += len(clauses)
            
            for val_request in pending_validations:
                requesting_agent = val_request.get('requesting_agent')
                clauses_to_validate = val_request.get('clauses', [])
                
                conflicts = detect_conflicts(
                    agent1_clauses=clauses_to_validate,
                    agent2_clauses=clauses,
                    agent1_name=requesting_agent,
                    agent2_name='legal',
                    threshold=0.1 if requesting_agent in ['compliance', 'operations'] else 0.3
                )
                
                outcome = "approved" if not conflicts else "needs_review"
                
                validation_result = create_validation_result(
                    request_id=val_request.get('request_id'),
                    validating_agent='legal',
                    validation_outcome=outcome,
                    findings=conflicts if conflicts else [{"status": "no_conflicts_detected"}],
                    confidence=0.85,
                    turn=state.get('turn_number', 0)
                )
                
                state["validation_results"].append(validation_result)
                
                if conflicts:
                    state["conflicts_detected"].extend(conflicts)
                    state["validation_notes"].append(
                        f"Legal validation: Found {len(conflicts)} conflict(s) with {requesting_agent} findings"
                    )
                    print(f"     Conflicts detected with {requesting_agent}: {len(conflicts)}")
                else:
                    state["validation_notes"].append(
                        f"Legal validation: Approved {len(clauses_to_validate)} {requesting_agent} clauses"
                    )
                    print(f"     Validated {len(clauses_to_validate)} {requesting_agent} clauses")
                
                val_request['status'] = 'completed'
            
            state["memory"].append({
                "agent": "legal",
                "timestamp": datetime.now().isoformat(),
                "turn": state.get('turn_number', 0),
                "clauses": clauses,
                "validations_performed": len(pending_validations),
                "conflicts_found": len(state.get('conflicts_detected', [])),
                "summary": f"Legal: Retrieved {len(clauses)} clauses, validated {len(pending_validations)} requests"
            })
            
            print(f"     Retrieved: {len(clauses)} clauses")
            print(f"     Completed {len(pending_validations)} validation(s)")
            
        else:
            state["legal"] = {
                "agent": "Legal",
                "status": "no_results",
                "threshold": AGENT_THRESHOLDS['legal'],
                "clauses_retrieved": 0
            }
            print(f"     No clauses above threshold")
            
    except Exception as e:
        state["legal"] = {"agent": "Legal", "status": "error", "error": str(e)}
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("legal")
    return state

print("legal_node_collaborative() defined")

legal_node_collaborative() defined


#### 6. Creating Operations Agent with Legal Memory Reading and SLA Validation

In [102]:
def operations_node_collaborative(state: GraphStateCollaborative) -> GraphStateCollaborative:

    print(f"  OPERATIONS AGENT EXECUTING (Threshold: {AGENT_THRESHOLDS['operations']})")
    print(f"  Reading Legal memory + SLA validation")

    legal_findings = [m for m in state.get('memory', []) if m.get('agent') == 'legal']
    if legal_findings:
        print(f"     Reading Legal findings from memory:")
        for finding in legal_findings[-1:]:  
            summary = finding.get('summary', 'N/A')
            print(f"       • {summary}")
    
    try:
        results = retrieve_from_pinecone_adaptive(state['query'], agent_filter='operations', top_k=5)
        
        if results and results.matches:
            threshold = AGENT_THRESHOLDS['operations']  
            filtered_matches = [m for m in results.matches if m.score > threshold]
            
            clauses = []
            scores = []
            for match in filtered_matches:
                clause_data = {
                    'clause': match.metadata.get('clause_full', match.metadata.get('clause', '')),
                    'risk_level': match.metadata.get('risk_level', 'unknown'),
                    'confidence': match.metadata.get('confidence', 0.0),
                    'relevance_score': match.score,
                    'agent': 'operations'
                }
                clauses.append(clause_data)
                scores.append(match.score)
            
            state["operations"] = {
                "agent": "Operations",
                "status": "completed",
                "source": "pinecone_vector_store",
                "threshold": threshold,
                "clauses_retrieved": len(clauses),
                "clauses": clauses,
                "avg_relevance": sum(scores) / len(scores) if scores else 0,
                "sla_validation": "performed"
            }
            
            state["all_clauses"].extend(clauses)
            state["total_clauses_retrieved"] += len(clauses)
            
            legal_clauses = state.get('legal', {}).get('clauses', [])
            if legal_clauses:
                cross_refs = cross_reference_clauses(
                    source_agent="operations",
                    source_clauses=clauses,
                    target_agent="legal",
                    target_clauses=legal_clauses,
                    threshold=0.1  
                )
                
                if cross_refs:
                    state["cross_references"].extend(cross_refs)
                    state["validation_notes"].append(
                        f"Operations: SLA validation - {len(cross_refs)} cross-references with Legal"
                    )
                    print(f"     SLA validation: {len(cross_refs)} cross-references with Legal")
            
            state["memory"].append({
                "agent": "operations",
                "timestamp": datetime.now().isoformat(),
                "turn": state.get('turn_number', 0),
                "clauses": clauses,
                "threshold_used": threshold,
                "sla_validation": "completed",
                "legal_context_used": len(legal_findings) > 0,
                "summary": f"Operations: Found {len(clauses)} SLA clauses (threshold: {threshold}), validated with Legal"
            })
            
            print(f"     Retrieved: {len(clauses)} clauses (threshold: {threshold})")
            print(f"     SLA validation completed with Legal context")
            
        else:
            state["operations"] = {
                "agent": "Operations",
                "status": "no_results",
                "threshold": AGENT_THRESHOLDS['operations'],
                "clauses_retrieved": 0
            }
            print(f"     No clauses above threshold {AGENT_THRESHOLDS['operations']}")
            
    except Exception as e:
        state["operations"] = {"agent": "Operations", "status": "error", "error": str(e)}
        print(f"     Error: {str(e)[:50]}")
    
    state["execution_order"].append("operations")
    return state

print("operations_node_collaborative() defined")

operations_node_collaborative() defined


#### 7. Building Collaborative Graph

In [103]:
print("BUILDING AND COMPILING COLLABORATIVE WORKFLOW")

print("\nBuilding workflow with agent-to-agent communication and validation...")

workflow_collaborative = StateGraph(GraphStateCollaborative)
print("StateGraph initialized with GraphStateCollaborative")

print("ADDING COLLABORATIVE AGENT NODES")

workflow_collaborative.add_node("routing", routing_node_with_memory)
print("Added: routing (memory-aware LLM coordinator)")

workflow_collaborative.add_node("compliance_agent", compliance_node_collaborative)
print("Added: compliance_agent (writes findings, requests validation)")

workflow_collaborative.add_node("finance_agent", finance_node_collaborative)
print("Added: finance_agent (reads memory, cross-references)")

workflow_collaborative.add_node("legal_agent", legal_node_collaborative)
print("Added: legal_agent (validates, checks conflicts)")

workflow_collaborative.add_node("operations_agent", operations_node_collaborative)
print("Added: operations_agent (reads legal memory, validates SLAs)")

def aggregation_node_collaborative(state: GraphStateCollaborative) -> GraphStateCollaborative:

    print("AGGREGATION NODE: Consolidating Results + Validation Summary")
   
    if state['all_clauses']:
        state['all_clauses'].sort(key=lambda x: x.get('relevance_score', 0), reverse=True)
        
        print(f"\nTotal clauses retrieved: {state['total_clauses_retrieved']}")
        print(f"Top relevance score: {state['all_clauses'][0]['relevance_score']:.3f}")
        
        agents_with_results = [a for a in ['legal', 'compliance', 'finance', 'operations'] 
                              if state.get(a, {}).get('status') == 'completed']
        
        if len(agents_with_results) > 1:
            consensus = {
                "participating_agents": agents_with_results,
                "total_clauses": state['total_clauses_retrieved'],
                "cross_references": len(state.get('cross_references', [])),
                "conflicts": len(state.get('conflicts_detected', [])),
                "validations_performed": len(state.get('validation_results', []))
            }
            state["consensus_findings"].append(consensus)
            
            print(f"\nConsensus built from {len(agents_with_results)} agents")
            print(f"  • Cross-references: {consensus['cross_references']}")
            print(f"  • Conflicts detected: {consensus['conflicts']}")
            print(f"  • Validations: {consensus['validations_performed']}")
        
        state['execution_status'] = 'success'
        
    else:
        print(f"\nNo clauses retrieved")
        state['execution_status'] = 'no_results'

    print("COMMUNICATION SUMMARY")
    print(f"  Messages: {len(state.get('agent_messages', []))}")
    print(f"  Validation Requests: {len(state.get('validation_requests', []))}")
    print(f"  Validation Results: {len(state.get('validation_results', []))}")
    print(f"  Cross-references: {len(state.get('cross_references', []))}")
    print(f"  Conflicts: {len(state.get('conflicts_detected', []))}")
    print(f"  Validation Notes: {len(state.get('validation_notes', []))}")
    
    if state.get('validation_notes'):
        print(f"\n  Validation Notes:")
        for idx, note in enumerate(state['validation_notes'], 1):
            print(f"    [{idx}] {note}")
    
    state["memory"].append({
        "agent": "aggregation",
        "timestamp": datetime.now().isoformat(),
        "turn": state.get('turn_number', 0),
        "total_clauses": state['total_clauses_retrieved'],
        "communication_events": len(state.get('agent_messages', [])) + len(state.get('validation_requests', [])),
        "conflicts_resolved": len(state.get('conflicts_detected', [])),
        "summary": f"Aggregation: {state['total_clauses_retrieved']} clauses, {len(state.get('validation_notes', []))} validation notes"
    })
    
    state['execution_order'].append("aggregation")
    
    checkpoint_path = save_state_checkpoint(state)
    state['checkpoint_path'] = checkpoint_path
    
    print(f"\nCheckpoint saved: {os.path.basename(checkpoint_path)}")
    
    return state

workflow_collaborative.add_node("aggregation", aggregation_node_collaborative)
print("Added: aggregation (with validation summary)")


print("DEFINING CONDITIONAL EDGES")

workflow_collaborative.set_entry_point("routing")
print("Entry point: routing")

def get_next_agent_collaborative(state: GraphStateCollaborative) -> str:
    routed_agents = state.get('routed_agents', [])
    executed_agents = [a for a in state.get('execution_order', []) 
                      if a not in ['routing', 'aggregation']]
    
    agent_priority = ['compliance', 'finance', 'legal', 'operations']
    
    for agent in agent_priority:
        if agent in routed_agents and agent not in executed_agents:
            return f"{agent}_agent"
    
    return "aggregation"

workflow_collaborative.add_conditional_edges(
    "routing",
    lambda state: get_next_agent_collaborative(state),
    {
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "legal_agent": "legal_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("routing → [conditional first agent]")

workflow_collaborative.add_conditional_edges(
    "compliance_agent",
    lambda state: get_next_agent_collaborative(state),
    {
        "finance_agent": "finance_agent",
        "legal_agent": "legal_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("compliance_agent → [conditional next]")

workflow_collaborative.add_conditional_edges(
    "finance_agent",
    lambda state: get_next_agent_collaborative(state),
    {
        "compliance_agent": "compliance_agent",
        "legal_agent": "legal_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("finance_agent → [conditional next]")

workflow_collaborative.add_conditional_edges(
    "legal_agent",
    lambda state: get_next_agent_collaborative(state),
    {
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
        "aggregation": "aggregation"
    }
)
print("legal_agent → [conditional next]")

workflow_collaborative.add_edge("operations_agent", "aggregation")
print("operations_agent → aggregation")

workflow_collaborative.add_edge("aggregation", END)
print("aggregation → END")

BUILDING AND COMPILING COLLABORATIVE WORKFLOW

Building workflow with agent-to-agent communication and validation...
StateGraph initialized with GraphStateCollaborative
ADDING COLLABORATIVE AGENT NODES
Added: routing (memory-aware LLM coordinator)
Added: compliance_agent (writes findings, requests validation)
Added: finance_agent (reads memory, cross-references)
Added: legal_agent (validates, checks conflicts)
Added: operations_agent (reads legal memory, validates SLAs)
Added: aggregation (with validation summary)
DEFINING CONDITIONAL EDGES
Entry point: routing
routing → [conditional first agent]
compliance_agent → [conditional next]
finance_agent → [conditional next]
legal_agent → [conditional next]
operations_agent → aggregation
aggregation → END


#### 8. Compiling Collaborative Graph

In [104]:
print("COMPILING COLLABORATIVE WORKFLOW")

try:
    app_collaborative = workflow_collaborative.compile()
    print("Collaborative workflow compiled successfully!")
    
    
except Exception as e:
    print(f"\nCompilation failed: {str(e)}")
    print("ERROR DETAILS")
    import traceback
    traceback.print_exc()

print("COLLABORATIVE WORKFLOW READY")

COMPILING COLLABORATIVE WORKFLOW
Collaborative workflow compiled successfully!
COLLABORATIVE WORKFLOW READY


#### 9. Executing with Multi-Intent Query and Validation

In [109]:
print("EXECUTING COLLABORATIVE WORKFLOW WITH MULTI-INTENT QUERY")

print("\nPreparing multi-intent query for collaborative execution...")

print("INITIALIZING COLLABORATIVE STATE")

collaborative_state = initialize_collaborative_state()

collaborative_state['query'] = "Review termination GDPR compliance requirements, payment terms with late penalties, and SLA deliverables"

print(f"Collaborative state initialized")
print(f"  Session ID: {collaborative_state['session_id']}")
print(f"  Query: \"{collaborative_state['query']}\"")

print("EXECUTING COLLABORATIVE WORKFLOW")

try:
    result_collaborative = app_collaborative.invoke(collaborative_state)
    
    print("COLLABORATIVE EXECUTION COMPLETED")

    print("ROUTING DECISION")

    print(f"  Query: \"{result_collaborative['query']}\"")
    print(f"  LLM Routed To: {', '.join(result_collaborative.get('routed_agents', []))}")
    print(f"  Reasoning: {result_collaborative.get('routing_reasoning', 'N/A')}")
    print(f"  Expected: compliance, finance, operations")

    print("EXECUTION SUMMARY")

    print(f"  Session: {result_collaborative.get('session_id', 'N/A')}")
    print(f"  Execution Order: {' → '.join(result_collaborative['execution_order'])}")
    print(f"  Total Clauses: {result_collaborative.get('total_clauses_retrieved', 0)}")
    print(f"  Status: {result_collaborative.get('execution_status', 'unknown')}")

    print("AGENT RESULTS")
    print(f"{'Agent':<15} {'Status':<18} {'Clauses':<10} {'Threshold':<12} {'Avg Score':<12}")

    
    executed_agents = []
    for agent_name in ["compliance", "finance", "legal", "operations"]:
        agent_data = result_collaborative.get(agent_name, {})
        
        if agent_data and agent_data.get("status") == "completed":
            executed_agents.append(agent_name)
            status = "EXECUTED"
            clauses = agent_data.get('clauses_retrieved', 0)
            threshold = agent_data.get('threshold', 'N/A')
            avg_score = agent_data.get('avg_relevance', 0)
            
            threshold_str = f"{threshold:.2f}" if isinstance(threshold, float) else str(threshold)
            score_str = f"{avg_score:.3f}" if avg_score > 0 else "N/A"
            
            if agent_name in ['compliance', 'operations'] and threshold == 0.10:
                threshold_str += ""
            
            print(f"{agent_name.upper():<15} {status:<18} {clauses:<10} {threshold_str:<12} {score_str:<12}")
            
        elif agent_data and agent_data.get("status") == "no_results":
            executed_agents.append(agent_name)
            status = "EXECUTED (0)"
            threshold = agent_data.get('threshold', 'N/A')
            threshold_str = f"{threshold:.2f}" if isinstance(threshold, float) else str(threshold)
            
            print(f"{agent_name.upper():<15} {status:<18} {0:<10} {threshold_str:<12} {'N/A':<12}")
            
        else:
            status = "NOT EXECUTED"
            print(f"{agent_name.upper():<15} {status:<18} {'-':<10} {'-':<12} {'-':<12}")
    
    print("\nLower threshold (0.10) for better recall")

    print("AGENT COMMUNICATION & VALIDATION SUMMARY")
 
    agent_messages = result_collaborative.get('agent_messages', [])
    validation_requests = result_collaborative.get('validation_requests', [])
    validation_results = result_collaborative.get('validation_results', [])
    cross_references = result_collaborative.get('cross_references', [])
    conflicts_detected = result_collaborative.get('conflicts_detected', [])
    validation_notes = result_collaborative.get('validation_notes', [])
    consensus_findings = result_collaborative.get('consensus_findings', [])
    
    print(f"\n  Communication Metrics:")
    print(f"    • Agent Messages: {len(agent_messages)}")
    print(f"    • Validation Requests: {len(validation_requests)}")
    print(f"    • Validation Results: {len(validation_results)}")
    print(f"    • Cross-references: {len(cross_references)}")
    print(f"    • Conflicts Detected: {len(conflicts_detected)}")
    print(f"    • Validation Notes: {len(validation_notes)}")
    print(f"    • Consensus Findings: {len(consensus_findings)}")
    
    if validation_notes:
        print("VALIDATION NOTES (Agent-to-Agent Communication)")
        
        for idx, note in enumerate(validation_notes, 1):
            print(f"\n  [{idx}] {note}")
    else:
        print("\n  No validation notes generated")
   
    if validation_requests:
        print("VALIDATION REQUESTS DETAILS")

        for idx, request in enumerate(validation_requests, 1):
            print(f"\n  Request {idx}:")
            print(f"    • ID: {request.get('request_id', 'N/A')}")
            print(f"    • From: {request.get('requesting_agent', 'N/A').upper()}")
            print(f"    • To: {request.get('validating_agent', 'N/A').upper()}")
            print(f"    • Type: {request.get('validation_type', 'N/A')}")
            print(f"    • Clauses: {len(request.get('clauses', []))}")
            print(f"    • Status: {request.get('status', 'N/A')}")
    
    if validation_results:
        print("VALIDATION RESULTS DETAILS")

        
        for idx, result in enumerate(validation_results, 1):
            print(f"\n  Result {idx}:")
            print(f"    • ID: {result.get('result_id', 'N/A')}")
            print(f"    • Request ID: {result.get('request_id', 'N/A')}")
            print(f"    • Validator: {result.get('validating_agent', 'N/A').upper()}")
            print(f"    • Outcome: {result.get('outcome', 'N/A')}")
            print(f"    • Confidence: {result.get('confidence', 0):.2f}")
            print(f"    • Findings: {len(result.get('findings', []))}")
    
    if cross_references:
        print("CROSS-REFERENCES (Inter-Agent Clause Links)")

        
        for idx, cross_ref in enumerate(cross_references[:5], 1):  
            print(f"\n  [{idx}] {cross_ref.get('source_agent', 'N/A').upper()} ↔ {cross_ref.get('target_agent', 'N/A').upper()}")
            print(f"    • Overlap Score: {cross_ref.get('overlap_score', 0):.3f}")
            print(f"    • Threshold Used: {cross_ref.get('threshold_used', 0):.2f}")
            print(f"    • Type: {cross_ref.get('reference_type', 'N/A')}")
            src = cross_ref.get('source_clause', '')[:80]
            tgt = cross_ref.get('target_clause', '')[:80]
            print(f"    • Source: {src}...")
            print(f"    • Target: {tgt}...")
        
        if len(cross_references) > 5:
            print(f"\n  ... and {len(cross_references) - 5} more cross-references")
    

    if conflicts_detected:
        print("CONFLICTS DETECTED")

        for idx, conflict in enumerate(conflicts_detected, 1):
            print(f"\n  Conflict {idx}:")
            print(f"    • Type: {conflict.get('conflict_type', 'N/A')}")
            print(f"    • Between: {conflict.get('agent1', 'N/A').upper()} ↔ {conflict.get('agent2', 'N/A').upper()}")
            print(f"    • Risk 1: {conflict.get('risk1', 'N/A')}")
            print(f"    • Risk 2: {conflict.get('risk2', 'N/A')}")
            print(f"    • Threshold: {conflict.get('threshold_used', 0):.2f}")
    else:
        print("\nNo conflicts detected between agents")

    if consensus_findings:
        print("CONSENSUS FINDINGS (Multi-Agent Agreement)")

        for idx, consensus in enumerate(consensus_findings, 1):
            print(f"\n  Consensus {idx}:")
            print(f"    • Participating Agents: {', '.join(consensus.get('participating_agents', []))}")
            print(f"    • Total Clauses: {consensus.get('total_clauses', 0)}")
            print(f"    • Cross-references: {consensus.get('cross_references', 0)}")
            print(f"    • Conflicts: {consensus.get('conflicts', 0)}")
            print(f"    • Validations: {consensus.get('validations_performed', 0)}")
    

    print("CONVERSATION MEMORY INSPECTION")

    memory = result_collaborative.get('memory', [])
    print(f"\n  Total Memory Entries: {len(memory)}")
    print(f"  Turn Number: {result_collaborative.get('turn_number', 0)}")
    
    if memory:
        print(f"\n  Memory Log (All Entries):")
        for idx, mem_entry in enumerate(memory, 1):
            agent = mem_entry.get('agent', 'unknown')
            summary = mem_entry.get('summary', 'N/A')
            turn = mem_entry.get('turn', 0)
    

    if result_collaborative.get('all_clauses'):
        print("RETRIEVED CLAUSES BY AGENT")
     
        clauses_by_agent = {}
        for clause in result_collaborative['all_clauses']:
            agent = clause.get('agent', 'unknown')
            if agent not in clauses_by_agent:
                clauses_by_agent[agent] = []
            clauses_by_agent[agent].append(clause)
        
        for agent, clauses in clauses_by_agent.items():
            print(f"\n  {agent.upper()} - {len(clauses)} clause(s):")
            
            agent_threshold = AGENT_THRESHOLDS.get(agent, 0.3)
            threshold_note = " (threshold: 0.10 )" if agent in ['compliance', 'operations'] else f" (threshold: {agent_threshold})"
            print(f"    {threshold_note}")
            
            for idx, clause in enumerate(clauses[:2], 1): 
                score = clause.get('relevance_score', 0)
                risk = clause.get('risk_level', 'unknown')
                text = clause.get('clause', '')[:100]
                
                print(f"    [{idx}] Score: {score:.3f} | Risk: {risk}")
                print(f"        {text}...")
            
            if len(clauses) > 2:
                print(f"    ... and {len(clauses) - 2} more clause(s)")

    print("STATE PERSISTENCE")

    checkpoint_path = result_collaborative.get('checkpoint_path', '')
    if checkpoint_path and os.path.exists(checkpoint_path):
        checkpoint_file = os.path.basename(checkpoint_path)
        print(f"  Checkpoint saved: {checkpoint_file}")
        print(f"  Location: {CHECKPOINT_DIR}")
        print(f"  Session: {result_collaborative.get('session_id', 'N/A')}")
        print(f"\n  Checkpoint includes:")
        print(f"    • All memory entries: {len(memory)}")
        print(f"    • Validation requests: {len(validation_requests)}")
        print(f"    • Validation results: {len(validation_results)}")
        print(f"    • Cross-references: {len(cross_references)}")
        print(f"    • Conflicts: {len(conflicts_detected)}")
    else:
        print(f"  No checkpoint saved")
 
    print("FINAL VERIFICATION")
  
    checks = []
    
    routed = result_collaborative.get('routed_agents', [])
    multi_domain = len(routed) >= 2
    checks.append(("Multi-domain Routing", multi_domain, f"{len(routed)} agents routed"))
    
    comp_threshold = result_collaborative.get('compliance', {}).get('threshold', 1.0)
    comp_threshold_ok = comp_threshold == 0.10
    checks.append(("Compliance Threshold (0.10)", comp_threshold_ok, f"actual: {comp_threshold}"))
    
    ops_threshold = result_collaborative.get('operations', {}).get('threshold', 1.0)
    ops_threshold_ok = ops_threshold == 0.10
    checks.append(("Operations Threshold (0.10)", ops_threshold_ok, f"actual: {ops_threshold}"))
    
    validations_sent = len(validation_requests) > 0
    checks.append(("Validation Requests Sent", validations_sent, f"{len(validation_requests)} requests"))
    
    cross_refs_created = len(cross_references) > 0
    checks.append(("Cross-references Created", cross_refs_created, f"{len(cross_references)} cross-refs"))
    
    memory_tracking = len(memory) > 0
    checks.append(("Memory Tracking", memory_tracking, f"{len(memory)} entries"))
    
    clauses_retrieved = result_collaborative.get('total_clauses_retrieved', 0) > 0
    checks.append(("Clause Retrieval", clauses_retrieved, f"{result_collaborative.get('total_clauses_retrieved', 0)} clauses"))
    
    communication_active = (len(validation_requests) + len(cross_references) + len(validation_notes)) > 0
    checks.append(("Agent Communication", communication_active, "active"))
    
    print(f"\n  System Checks:")
    for check_name, passed, detail in checks:
        status = "PASS" if passed else "FAIL"
        print(f"    {status}: {check_name} ({detail})")
    
    all_passed = all(passed for _, passed, _ in checks)
    
    if all_passed:
        print(f"\n  ALL CHECKS PASSED")
        print(f"  Collaborative workflow with agent communication working perfectly!")
    else:
        print(f"\n  Some checks failed - Review configuration")
    

    print("EXECUTION SUMMARY")
    print(f"""
  Query: "{result_collaborative['query']}"
  
  Agents Executed: {len(executed_agents)}
  Total Clauses: {result_collaborative.get('total_clauses_retrieved', 0)}
  
  Communication:
    • Validation Requests: {len(validation_requests)}
    • Validation Results: {len(validation_results)}
    • Cross-references: {len(cross_references)}
    • Conflicts: {len(conflicts_detected)}
    • Validation Notes: {len(validation_notes)}
  
  Status: {result_collaborative.get('execution_status', 'unknown')}
    """)

except Exception as e:
    print(f"\nEXECUTION FAILED")
    print(f"Error: {str(e)}")
    
    print("ERROR DETAILS")
    import traceback
    traceback.print_exc()

print("COLLABORATIVE WORKFLOW EXECUTION COMPLETE")

EXECUTING COLLABORATIVE WORKFLOW WITH MULTI-INTENT QUERY

Preparing multi-intent query for collaborative execution...
INITIALIZING COLLABORATIVE STATE
Collaborative state initialized: collab_session_20260115_181319
Collaborative state initialized
  Session ID: collab_session_20260115_181319
  Query: "Review termination GDPR compliance requirements, payment terms with late penalties, and SLA deliverables"
EXECUTING COLLABORATIVE WORKFLOW
ROUTING NODE: Dynamic LLM Coordinator with Memory
Query: "Review termination GDPR compliance requirements, payment terms with late penalties, and SLA deliverables"
Turn: 0
Session: collab_session_20260115_181319

This is the first query in the conversation.

Querying gemma2:9b coordinator...

Routing Decision:
  Agents: legal, compliance, finance, operations
  Reasoning: The query involves termination clauses (legal), GDPR compliance (compliance), payment terms and penalties (finance), and SLA deliverables (operations).
  COMPLIANCE AGENT EXECUTING (Thr

# Compliance Pipeline

In [132]:
import json
from datetime import datetime
from typing import Dict, List, Any
import os
import requests
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import re

In [201]:
OLLAMA_MODEL = "gemma2:9b"
OLLAMA_URL = "http://localhost:11434/api/generate"

def initialize_ollama():
  
    try:
        test_payload = {
            "model": OLLAMA_MODEL,
            "prompt": "Hello",
            "stream": False
        }
        
        response = requests.post(OLLAMA_URL, json=test_payload, timeout=30)
        
        if response.status_code == 200:
            print(f"Ollama model '{OLLAMA_MODEL}' initialized successfully")
            print(f"Connected to: {OLLAMA_URL}")
            return True
        else:
            print(f"Failed to initialize Ollama: Status {response.status_code}")
            return False
            
    except requests.exceptions.ConnectionError:
        print(f"Connection Error: Could not connect to Ollama at {OLLAMA_URL}")
        print("  Make sure Ollama is running (try: ollama serve)")
        return False
    except Exception as e:
        print(f"Error initializing Ollama: {str(e)}")
        return False

def call_ollama(prompt, max_tokens=50000, temperature=0.7):

    try:
        payload = {
            "model": OLLAMA_MODEL,
            "prompt": prompt,
            "stream": False,
            "options": {
                "num_predict": max_tokens,
                "temperature": temperature
            }
        }
        
        response = requests.post(OLLAMA_URL, json=payload, timeout=60)
        
        if response.status_code == 200:
            result = response.json()
            return result.get("response", "").strip()
        else:
            print(f"Error calling Ollama: Status {response.status_code}")
            return None
            
    except Exception as e:
        print(f"Error in call_ollama: {str(e)}")
        return None

print("INITIALIZING OLLAMA MODEL")

if initialize_ollama():
    print("\nReady to start Pipeline")
else:
    print("\nPlease start Ollama service and try again")
    print("  Command: ollama serve")


INITIALIZING OLLAMA MODEL
Ollama model 'gemma2:9b' initialized successfully
Connected to: http://localhost:11434/api/generate

Ready to start Pipeline


In [202]:
print("COMPLIANCE AGENT INITIALIZATION")

OLLAMA_MODEL = "gemma2:9b"
OLLAMA_URL = "http://localhost:11434/api/generate"
PINECONE_API_KEY = 'pcsk_3ftgmC_GzUZkRCnxa2jmDu7TTjnGWjC3QaN8c2PcQ5KN5PUSyQaEmmcdGUGu2BLd4Y7TRn'
INDEX_NAME = 'contract-agents'
SCORE_THRESHOLD = 0.1 

print(f"LLM Model: {OLLAMA_MODEL}")
print(f"Pinecone Index: {INDEX_NAME}")
print(f"Retrieval Threshold: score > {SCORE_THRESHOLD}")

COMPLIANCE AGENT INITIALIZATION
LLM Model: gemma2:9b
Pinecone Index: contract-agents
Retrieval Threshold: score > 0.1


#### 1. Defining Compliance Query

In [ ]:
print("\nDefining Compliance Query Template")

COMPLIANCE_QUERY = """
Identify clauses related to:
- Regulatory compliance and legal requirements
- Data protection and privacy (GDPR, CCPA)
- Security standards and certifications
- Audit and monitoring requirements
"""

class ComplianceQueryTemplate:
    
    @staticmethod
    def get_base_template():
        """Get base compliance query template"""
        return COMPLIANCE_QUERY
    
    @staticmethod
    def select_template(query):
        """Select appropriate template based on query"""
        query_lower = query.lower()
        
        if any(kw in query_lower for kw in ['gdpr', 'data protection', 'privacy', 'personal data']):
            return """
You are a GDPR Compliance Expert Agent.

Focus on:
- GDPR Article compliance (data subject rights, consent, breach notification)
- Data processing and transfer requirements
- Privacy by design and default
- Data retention and deletion policies
"""
        elif any(kw in query_lower for kw in ['soc2', 'soc 2', 'security', 'certification']):
            return """
You are a SOC2 Compliance Expert Agent.

Focus on:
- Security controls and access management
- Availability and system monitoring
- Confidentiality and data encryption
- Change management procedures
"""
        elif any(kw in query_lower for kw in ['hipaa', 'healthcare', 'phi', 'medical']):
            return """
You are a HIPAA Compliance Expert Agent.

Focus on:
- Protected Health Information (PHI) safeguards
- Privacy Rule and Security Rule compliance
- Business Associate Agreements (BAA)
- Breach notification requirements
"""
        else:
            return COMPLIANCE_QUERY
    
    @staticmethod
    def create_compliance_prompt(user_query, contract_type="general"):
   
        template = ComplianceQueryTemplate.select_template(user_query)
        
        prompt = f"""
You are a Compliance Expert Agent analyzing contract clauses.

Contract Type: {contract_type}
User Query: {user_query}

{template}

Your task:
1. Identify potential compliance risks and violations
2. Check against relevant regulations and standards
3. Flag any non-compliant clauses
4. Provide risk severity (High/Medium/Low)
5. Reference specific regulatory requirements
6. Suggest corrective actions if needed

Analyze the following contract clauses and provide a detailed compliance assessment.
"""
        return prompt
    
    @staticmethod
    def format_compliance_context(retrieved_clauses):
     
        if not retrieved_clauses:
            return "No relevant compliance clauses found."
        
        formatted = ""
        for idx, clause in enumerate(retrieved_clauses, 1):
            formatted += f"\n--- Compliance Clause {idx} ---\n"
            formatted += f"ID: {clause.get('clause_id', 'unknown')}\n"
            formatted += f"Type: {clause.get('clause_type', 'general')}\n"
            formatted += f"Score: {clause.get('score', 0):.3f}\n"
            formatted += f"Content: {clause.get('text', '')}\n"
        
        return formatted

print(f"Compliance Query Template defined")


Defining Compliance Query Template
Compliance Query Template defined


#### 2. Retrieving Compliance Context (RAG)

In [204]:
print("RETRIEVAL FUNCTION")

compliance_keywords = [
    "compliance", 
    "regulation", 
    "gdpr", 
    "privacy", 
    "security", 
    "audit", 
    "legal",
    "data protection",
    "regulatory",
    "confidentiality"
]

def retrieve_compliance_context(query_text, top_k=20, score_threshold=0.1):

    try:
        from pinecone import Pinecone
        from sentence_transformers import SentenceTransformer
        
        PINECONE_API_KEY = 'pcsk_3ftgmC_GzUZkRCnxa2jmDu7TTjnGWjC3QaN8c2PcQ5KN5PUSyQaEmmcdGUGu2BLd4Y7TRn'
        INDEX_NAME = 'contract-agents'
        
        pc = Pinecone(api_key=PINECONE_API_KEY)
        index = pc.Index(INDEX_NAME)
        
        embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        
        print(f"   Generating embedding for compliance query...")
        query_embedding = embedding_model.encode(query_text).tolist()
        
        print(f"   Querying Pinecone index: '{INDEX_NAME}'")
        results = index.query(
            vector=query_embedding,
            top_k=top_k,
            include_metadata=True
        )
        
        filtered_matches = [
            match for match in results['matches'] 
            if match['score'] > score_threshold
        ]
        
        print(f"   Retrieved {len(filtered_matches)} compliance clauses (score > {score_threshold})")
        
        compliance_context = []
        for match in filtered_matches:
            metadata = match.get('metadata', {})
            
            clause_text = (
                metadata.get('clause_full', '') or 
                metadata.get('clause', '') or 
                metadata.get('text', '')
            )
            
            context_item = {
                'clause_id': match['id'],
                'text': clause_text,
                'clause_type': metadata.get('clause_type', metadata.get('agent', 'compliance')),
                'score': match['score'],
                'risk_level': metadata.get('risk_level', 'unknown'),
                'confidence': metadata.get('confidence', 0.0)
            }
            compliance_context.append(context_item)
            print(f"     - Clause ID: {match['id']}, Score: {match['score']:.3f}, Text length: {len(clause_text)} chars")
        
        return compliance_context
        
    except Exception as e:
        print(f"   Error retrieving compliance context: {str(e)}")
        return []

RETRIEVAL FUNCTION


#### 3. Combining Retrieved Chunks

In [205]:
def combine_compliance_chunks(compliance_context, max_length=2000):

    if not compliance_context:
        print(f"   No compliance data available")
        return {
            "combined_text": "No relevant compliance clauses found.",
            "num_context_chunks": 0,
            "combined_text_length": 0,
            "truncated": False
        }
    
    combined_text = "CONTRACT CLAUSES FOR ANALYSIS:\n\n"
    chunk_count = 0
    truncated = False
    
    for item in compliance_context:
        chunk_text = f"CLAUSE {chunk_count + 1}:\n"
        chunk_text += f"Type: {item['clause_type']}\n"
        chunk_text += f"Relevance Score: {item['score']:.3f}\n"
        chunk_text += f"Content:\n{item['text']}\n"  
        chunk_text += "-" * 60 + "\n\n"
        
        if len(combined_text) + len(chunk_text) > max_length:
            print(f"   Reached max length limit at {chunk_count} chunks")
            truncated = True
            break
        
        combined_text += chunk_text
        chunk_count += 1
    
    result = {
        "combined_text": combined_text,
        "num_context_chunks": chunk_count,
        "combined_text_length": len(combined_text),
        "truncated": truncated
    }
    
    print(f"   Retrieved Chunks: {result['num_context_chunks']}")
    print(f"   Combined Text Length: {result['combined_text_length']} characters")
    print(f"   Context combined successfully")
    
    return result

print("combine_compliance_chunks() updated with better formatting")

combine_compliance_chunks() updated with better formatting


#### 4. Running Compliance Agent

In [209]:
print("\nRunning Compliance Agent")

def run_compliance_agent(user_query, retrieved_context, contract_type="general", timeout=3000):

    import requests
    import json
    import re
    
    OLLAMA_MODEL = "gemma2:9b"
    OLLAMA_URL = "http://localhost:11434/api/generate"
    
    query_template = ComplianceQueryTemplate.create_compliance_prompt(user_query, contract_type)
    
    full_prompt = f"""{query_template}

RETRIEVED CONTRACT CLAUSES:
{retrieved_context}

COMPLIANCE ANALYSIS:
Provide a structured analysis covering:
1. Compliance Status (Compliant/Non-Compliant/Partial)
2. Risk Level (High/Medium/Low)
3. Specific Issues Found
4. Regulatory Concerns (cite specific regulations)
5. Security & Privacy Gaps
6. Recommended Actions
"""
    
    print(f"   Agent: Compliance Agent")
    print(f"   Model: {OLLAMA_MODEL}")
    print(f"   Sending query to Ollama via HTTP API...")
    
    try:
        payload = {
            "model": OLLAMA_MODEL,
            "prompt": full_prompt,
            "stream": False,
            "options": {
                "num_predict": 800,
                "temperature": 0.3  
            }
        }
        
        response = requests.post(OLLAMA_URL, json=payload, timeout=90000)
        
        if response.status_code == 200:
            result = response.json()
            compliance_output = result.get("response", "").strip()
            
            extracted_clauses = []
            lines = compliance_output.split('\n')
            for line in lines:
                if any(keyword in line.lower() for keyword in compliance_keywords):
                    extracted_clauses.append(line.strip())
            
            output_lower = compliance_output.lower()
            if 'high risk' in output_lower or 'critical' in output_lower or 'non-compliant' in output_lower:
                risk_level = "high"
            elif 'low risk' in output_lower or 'minimal' in output_lower or 'compliant' in output_lower:
                risk_level = "low"
            else:
                risk_level = "medium"
            
            confidence = 0.85 if len(compliance_output) > 200 else 0.65
            
            print(f"   Clauses Extracted: {len(extracted_clauses)}")
            print(f"   Risk Level: {risk_level}")
            print(f"   Confidence: {confidence:.2f}")
            print(f"   Agent execution complete")
            
            return {
                "status": "success",
                "model": OLLAMA_MODEL,
                "output": {
                    "compliance_analysis": compliance_output,
                    "extracted_clauses": extracted_clauses,
                    "risk_level": risk_level,
                    "confidence": confidence
                }
            }
        else:
            print(f"   Ollama API error: Status {response.status_code}")
            return {
                "status": "error",
                "model": OLLAMA_MODEL,
                "error": f"HTTP {response.status_code}"
            }
            
    except Exception as e:
        print(f"   Error running compliance agent: {str(e)}")
        return {
            "status": "error",
            "model": OLLAMA_MODEL,
            "error": str(e)
        }

compliance_data = None
extracted_clauses = []
risk_level = "unknown"
confidence = 0.0

if compliance_data:
    if "contracts" in compliance_data:
        extracted_clauses = compliance_data.get("contracts", [{}])[0].get("analysis", {}).get("extracted_clauses", [])
        risk_level = compliance_data.get("contracts", [{}])[0].get("analysis", {}).get("risk_level", "unknown")
        confidence = compliance_data.get("contracts", [{}])[0].get("analysis", {}).get("confidence", 0.0)
    else:
        extracted_clauses = compliance_data.get("output", {}).get("extracted_clauses", [])
        risk_level = compliance_data.get("output", {}).get("risk_level", "unknown")
        confidence = compliance_data.get("output", {}).get("confidence", 0.0)
    
    print(f"   Agent: Compliance Agent")
    print(f"   Model: {compliance_data.get('model', OLLAMA_MODEL)}")
    print(f"   Clauses Extracted: {len(extracted_clauses)}")
    print(f"   Risk Level: {risk_level}")
    print(f"   Confidence: {confidence}")
    print(f"   Agent execution complete")
else:
    print(f"   Compliance agent ready to execute")
    print(f"   run_compliance_agent() function defined")
    print(f"   Uses gemma2:9b via http://localhost:11434/api/generate")



Running Compliance Agent
   Compliance agent ready to execute
   run_compliance_agent() function defined
   Uses gemma2:9b via http://localhost:11434/api/generate


#### 5. Validating Compliance Output

In [210]:
print("\nValidating Compliance Output")

def validate_compliance_output(compliance_result):

    validation_passed = True
    validation_checks = []
    
    if compliance_result and compliance_result.get("status") == "success":
        output = compliance_result.get("output", {})
        extracted_clauses = output.get("extracted_clauses", [])
        risk_level = output.get("risk_level", "unknown")
        confidence = output.get("confidence", 0.0)
        compliance_analysis = output.get("compliance_analysis", "")
    else:
        extracted_clauses = []
        risk_level = "unknown"
        confidence = 0.0
        compliance_analysis = ""
    
    if len(extracted_clauses) > 0:
        validation_checks.append(f"Clauses extracted: {len(extracted_clauses)}")
    else:
        validation_checks.append("No clauses extracted")
        validation_passed = False
    
    if risk_level != "unknown":
        validation_checks.append(f"Risk level assessed: {risk_level}")
    else:
        validation_checks.append("Risk level not assessed")
        validation_passed = False
        
    if confidence > 0:
        validation_checks.append(f"Confidence score: {confidence:.2f}")
    else:
        validation_checks.append("Low confidence score")
    
    if len(compliance_analysis) > 100:
        validation_checks.append(f"Analysis length: {len(compliance_analysis)} characters")
    else:
        validation_checks.append("Analysis too short")
        validation_passed = False
    
    compliance_terms = ['compliance', 'regulation', 'gdpr', 'security', 'audit', 'privacy', 'legal']
    found_terms = [term for term in compliance_terms if term in compliance_analysis.lower()]
    
    if len(found_terms) >= 2:
        validation_checks.append(f"Compliance terms found: {', '.join(found_terms)}")
    else:
        validation_checks.append("Limited compliance terminology detected")
    
    for check in validation_checks:
        print(f"   {check}")
    
    print(f"\n   Validation Status: {'PASSED ' if validation_passed else 'FAILED ✗'}")
    
    return {
        "validation_passed": validation_passed,
        "validation_checks": validation_checks,
        "extracted_clauses_count": len(extracted_clauses),
        "risk_level": risk_level,
        "confidence": confidence,
        "completeness_score": len([c for c in validation_checks if 'done' in c]) / len(validation_checks) * 100
    }

validation_passed = True
validation_checks = []

if compliance_data:
    if len(extracted_clauses) > 0:
        validation_checks.append("Clauses extracted")
    else:
        validation_checks.append("No clauses extracted")
        validation_passed = False

    if risk_level != "unknown":
        validation_checks.append(f"Risk level assessed: {risk_level}")
    else:
        validation_checks.append("Risk level not assessed")
        validation_passed = False

    if confidence > 0:
        validation_checks.append(f"Confidence score: {confidence}")
    else:
        validation_checks.append("Low confidence score")

    for check in validation_checks:
        print(f"   {check}")

    print(f"\n   Validation Status: {'PASSED' if validation_passed else 'FAILED'}")
else:
    print(f"   Compliance output validation ready")



Validating Compliance Output
   Compliance output validation ready


#### 6. Compliance Risk Summary

In [211]:
print("\nCompliance Risk Summary")

def generate_compliance_risk_summary(compliance_result, validation_result):

    import re
    from datetime import datetime
    
    if compliance_result and compliance_result.get("status") == "success":
        output = compliance_result.get("output", {})
        extracted_clauses = output.get("extracted_clauses", [])
        risk_level = output.get("risk_level", "unknown")
        confidence = output.get("confidence", 0.0)
        compliance_analysis = output.get("compliance_analysis", "")
    else:
        extracted_clauses = []
        risk_level = "unknown"
        confidence = 0.0
        compliance_analysis = ""
    
    compliance_risk_summary = {
        "timestamp": datetime.now().isoformat(),
        "total_clauses": len(extracted_clauses),
        "risk_level": risk_level,
        "confidence": confidence,
        "validation_passed": validation_result.get("validation_passed", False),
        "compliance_risks": [],
        "recommendations": [],
        "key_findings": []
    }
    
    if risk_level == "high":
        compliance_risk_summary["compliance_risks"].append("Critical compliance violations identified")
        compliance_risk_summary["compliance_risks"].append("Regulatory non-compliance detected")
        compliance_risk_summary["recommendations"].append("Immediate legal review required")
        compliance_risk_summary["recommendations"].append("Revise non-compliant clauses with legal counsel")
        compliance_risk_summary["recommendations"].append("Implement compliance remediation plan")
    elif risk_level == "medium":
        compliance_risk_summary["compliance_risks"].append("Moderate compliance gaps identified")
        compliance_risk_summary["compliance_risks"].append("Potential regulatory concerns")
        compliance_risk_summary["recommendations"].append("Review compliance requirements with legal team")
        compliance_risk_summary["recommendations"].append("Update clauses to meet regulatory standards")
    else:
        compliance_risk_summary["recommendations"].append("Compliance requirements are met")
        compliance_risk_summary["recommendations"].append("Continue monitoring regulatory changes")
    
    if compliance_analysis:
        lines = [line.strip() for line in compliance_analysis.split('\n') if line.strip()]
        for line in lines:
            if any(kw in line.lower() for kw in ['compliance', 'regulation', 'gdpr', 'security', 'violation', 'risk']):
                if len(compliance_risk_summary["key_findings"]) < 5:
                    compliance_risk_summary["key_findings"].append(line)
    
    print(f"   Total Clauses: {compliance_risk_summary['total_clauses']}")
    print(f"   Overall Risk: {compliance_risk_summary['risk_level'].upper()}")
    print(f"   Confidence: {compliance_risk_summary['confidence']:.2f}")
    print(f"   Validation: {'PASSED' if compliance_risk_summary['validation_passed'] else 'FAILED'}")
    
    if compliance_risk_summary['compliance_risks']:
        print(f"\n   Compliance Risks:")
        for risk in compliance_risk_summary['compliance_risks']:
            print(f"      - {risk}")
    
    print(f"\n   Recommendations:")
    for rec in compliance_risk_summary['recommendations']:
        print(f"      - {rec}")
    
    return compliance_risk_summary

def save_compliance_results(compliance_result, validation_result, risk_summary, output_dir="../Data/Results/Pipelines"):
 
    import os
    import json
    from datetime import datetime
    
    os.makedirs(output_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    complete_output = {
        "pipeline": "Compliance Pipeline",
        "timestamp": datetime.now().isoformat(),
        "model": "gemma2:9b",
        "pinecone_index": "contract-agents",
        "score_threshold": 0.1,
        "results": compliance_result,
        "validation": validation_result,
        "risk_summary": risk_summary
    }
    
    json_filename = os.path.join(output_dir, f"compliance_pipeline_{timestamp}.json")
    try:
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(complete_output, f, indent=2, ensure_ascii=False)
        print(f"\n   JSON saved: {json_filename}")
    except Exception as e:
        print(f"\n   Error saving JSON: {str(e)}")
    
    txt_filename = os.path.join(output_dir, f"compliance_summary_{timestamp}.txt")
    try:
        with open(txt_filename, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write("COMPLIANCE PIPELINE SUMMARY\n")
            f.write("="*80 + "\n\n")
            
            f.write(f"Timestamp: {risk_summary['timestamp']}\n")
            f.write(f"Model: gemma2:9b\n")
            f.write(f"Pinecone Index: contract-agents\n")
            f.write(f"Score Threshold: > 0.1\n\n")
            
            f.write("-"*80 + "\n")
            f.write("COMPLIANCE ASSESSMENT\n")
            f.write("-"*80 + "\n")
            f.write(f"Total Clauses Analyzed: {risk_summary['total_clauses']}\n")
            f.write(f"Risk Level: {risk_summary['risk_level'].upper()}\n")
            f.write(f"Confidence Score: {risk_summary['confidence']:.2f}\n")
            f.write(f"Validation Status: {'PASSED' if risk_summary['validation_passed'] else 'FAILED'}\n\n")
            
            if risk_summary['compliance_risks']:
                f.write("-"*80 + "\n")
                f.write("COMPLIANCE RISKS IDENTIFIED\n")
                f.write("-"*80 + "\n")
                for i, risk in enumerate(risk_summary['compliance_risks'], 1):
                    f.write(f"{i}. {risk}\n")
                f.write("\n")
            
            f.write("-"*80 + "\n")
            f.write("RECOMMENDATIONS\n")
            f.write("-"*80 + "\n")
            for i, rec in enumerate(risk_summary['recommendations'], 1):
                f.write(f"{i}. {rec}\n")
            f.write("\n")
            
            if risk_summary['key_findings']:
                f.write("-"*80 + "\n")
                f.write("KEY FINDINGS\n")
                f.write("-"*80 + "\n")
                for i, finding in enumerate(risk_summary['key_findings'], 1):
                    f.write(f"{i}. {finding}\n")
                f.write("\n")
            
            f.write("="*80 + "\n")
            f.write("END OF REPORT\n")
            f.write("="*80 + "\n")
        
        print(f"   TXT summary saved: {txt_filename}")
    except Exception as e:
        print(f"   Error saving TXT: {str(e)}")
    
    return json_filename, txt_filename

compliance_risk_summary = {
    "timestamp": "2026-01-15T20:25:00",
    "total_clauses": len(extracted_clauses),
    "risk_level": risk_level,
    "confidence": confidence,
    "validation_passed": validation_passed,
    "compliance_risks": [],
    "recommendations": [],
    "key_findings": []
}

if risk_level == "high":
    compliance_risk_summary["compliance_risks"].append("Critical compliance violations identified")
    compliance_risk_summary["recommendations"].append("Immediate legal review required")
elif risk_level == "medium":
    compliance_risk_summary["compliance_risks"].append("Moderate compliance gaps identified")
    compliance_risk_summary["recommendations"].append("Review compliance requirements with legal team")
else:
    compliance_risk_summary["recommendations"].append("Compliance requirements are met")

print(f"   Total Clauses: {compliance_risk_summary['total_clauses']}")
print(f"   Overall Risk: {compliance_risk_summary['risk_level'].upper()}")
print(f"   Confidence: {compliance_risk_summary['confidence']}")

if compliance_risk_summary['compliance_risks']:
    print(f"\n   Compliance Risks:")
    for risk in compliance_risk_summary['compliance_risks']:
        print(f"      - {risk}")

print(f"\n   Recommendations:")
for rec in compliance_risk_summary['recommendations']:
    print(f"      - {rec}")

print(f"\n   generate_compliance_risk_summary() function defined")
print(f"   save_compliance_results() function defined")
print(f"   Output directory: ../Data/Results/Pipelines")
print(f"   Saves both JSON and TXT summary formats")



Compliance Risk Summary
   Total Clauses: 0
   Overall Risk: UNKNOWN
   Confidence: 0.0

   Recommendations:
      - Compliance requirements are met

   generate_compliance_risk_summary() function defined
   save_compliance_results() function defined
   Output directory: ../Data/Results/Pipelines
   Saves both JSON and TXT summary formats


#### 7. Packaging Pipeline Output

In [ ]:
print("\nPackaging Compliance Pipeline Output")

from datetime import datetime
import json
import os

def package_compliance_pipeline_output(
    user_query,
    compliance_result,
    validation_result,
    risk_summary,
    retrieved_context_data,
    contract_type="general"
):
   
    if compliance_result and compliance_result.get("status") == "success":
        output = compliance_result.get("output", {})
        extracted_clauses = output.get("extracted_clauses", [])
        risk_level = output.get("risk_level", "unknown")
        confidence = output.get("confidence", 0.0)
        model = compliance_result.get("model", "gemma2:9b")
        full_analysis = output.get("compliance_analysis", "")
    else:
        extracted_clauses = []
        risk_level = "unknown"
        confidence = 0.0
        model = "gemma2:9b"
        full_analysis = compliance_result.get("error", "Analysis failed") if compliance_result else "No result"
    
    compliance_pipeline_output = {
        "pipeline": "compliance",
        "timestamp": datetime.now().isoformat(),
        "metadata": {
            "model": model,
            "ollama_url": "http://localhost:11434/api/generate",
            "ollama_timeout": 300,  
            "pinecone_index": "contract-agents",
            "pinecone_metadata_field": "clause_full",  
            "score_threshold": 0.1,
            "contract_type": contract_type,
            "user_query": user_query,
            "embedding_model": "all-MiniLM-L6-v2" 
        },
        "query_template": COMPLIANCE_QUERY.strip(),
        "retrieval_keywords": compliance_keywords,
        "retrieval_stats": {
            "chunks_retrieved": retrieved_context_data.get("num_context_chunks", 0),
            "text_length": retrieved_context_data.get("combined_text_length", 0),
            "truncated": retrieved_context_data.get("truncated", False)
        },
        "compliance_analysis": {
            "agent": "Compliance Agent",
            "model": model,
            "extracted_clauses": extracted_clauses,
            "clause_count": len(extracted_clauses),
            "risk_level": risk_level,
            "confidence": confidence,
            "full_analysis": full_analysis,
            "analysis_length": len(full_analysis) 
        },
        "validation": {
            "status": "passed" if validation_result.get("validation_passed", False) else "failed",
            "checks": validation_result.get("validation_checks", []),
            "completeness_score": validation_result.get("completeness_score", 0),
            "extracted_clauses_count": validation_result.get("extracted_clauses_count", 0)  
        },
        "risk_summary": risk_summary,
        "validation_status": "passed" if validation_result.get("validation_passed", False) else "failed",
        "status": "completed" if validation_result.get("validation_passed", False) else "completed_with_warnings",
        "execution_success": compliance_result.get("status") == "success" if compliance_result else False  
    }
    
    print(f"   Pipeline output packaged")
    print(f"   Keys: {list(compliance_pipeline_output.keys())}")
    print(f"   Status: {compliance_pipeline_output['status']}")
    print(f"   Execution: {'Success' if compliance_pipeline_output['execution_success'] else 'Failed'}")
    
    return compliance_pipeline_output

def save_packaged_compliance_output(pipeline_output, output_dir="../Data/Results/Pipelines"):
  
    try:
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(output_dir, f"compliance_pipeline_complete_{timestamp}.json")
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(pipeline_output, f, indent=2, ensure_ascii=False)
        
        print(f"   Complete output saved: {filename}")
        
        summary_filename = os.path.join(output_dir, f"compliance_summary_quick_{timestamp}.txt")
        with open(summary_filename, 'w', encoding='utf-8') as f:
            f.write("COMPLIANCE PIPELINE QUICK SUMMARY\n")
            f.write("="*80 + "\n\n")
            f.write(f"Timestamp: {pipeline_output['timestamp']}\n")
            f.write(f"Status: {pipeline_output['status']}\n")
            f.write(f"Risk Level: {pipeline_output['risk_summary']['risk_level']}\n")
            f.write(f"Confidence: {pipeline_output['risk_summary']['confidence']:.2f}\n")
            f.write(f"Clauses Analyzed: {pipeline_output['risk_summary']['total_clauses']}\n")
            f.write(f"Validation: {pipeline_output['validation_status']}\n\n")
            
            if pipeline_output['risk_summary'].get('compliance_risks'):
                f.write("RISKS:\n")
                for risk in pipeline_output['risk_summary']['compliance_risks']:
                    f.write(f"  - {risk}\n")
                f.write("\n")
            
            if pipeline_output['risk_summary'].get('recommendations'):
                f.write("RECOMMENDATIONS:\n")
                for rec in pipeline_output['risk_summary']['recommendations']:
                    f.write(f"  - {rec}\n")
        
        print(f"   Quick summary saved: {summary_filename}")
        
        return filename
        
    except Exception as e:
        print(f"   Error saving packaged output: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

print(f"\npackage_compliance_pipeline_output() function defined")


Packaging Compliance Pipeline Output

package_compliance_pipeline_output() function defined


In [214]:
print("EXECUTING COMPLIANCE PIPELINE")

user_query = "Check GDPR compliance and data protection requirements"
contract_type = "Data Processing Agreement"

print("\n context from Pinecone...")
compliance_context = retrieve_compliance_context(user_query, top_k=10, score_threshold=0.1)

print("\nCombining retrieved chunks...")
combined_data = combine_compliance_chunks(compliance_context, max_length=2000)

print("\nRunning compliance agent...")
compliance_result = run_compliance_agent(user_query, combined_data["combined_text"], contract_type)

print("\nValidating output...")
validation_result = validate_compliance_output(compliance_result)

print("\nGenerating risk summary...")
risk_summary = generate_compliance_risk_summary(compliance_result, validation_result)

print("SAVING RESULTS")
save_compliance_results(compliance_result, validation_result, risk_summary)

final_output = package_compliance_pipeline_output(
    user_query=user_query,
    compliance_result=compliance_result,
    validation_result=validation_result,
    risk_summary=risk_summary,
    retrieved_context_data=combined_data,
    contract_type=contract_type
)

save_packaged_compliance_output(final_output)


print("FINAL RESULTS")
print(f"Status: {final_output['status']}")
print(f"Risk Level: {risk_summary['risk_level']}")
print(f"Confidence: {risk_summary['confidence']:.2f}")
print(f"Clauses Found: {risk_summary['total_clauses']}")
print(f"\nFull analysis:")
print(compliance_result['output']['compliance_analysis'])

EXECUTING COMPLIANCE PIPELINE

 context from Pinecone...
   Generating embedding for compliance query...
   Querying Pinecone index: 'contract-agents'
   Retrieved 5 compliance clauses (score > 0.1)
     - Clause ID: compliance_0, Score: 0.186, Text length: 150 chars
     - Clause ID: finance_2, Score: 0.180, Text length: 50 chars
     - Clause ID: finance_0, Score: 0.130, Text length: 263 chars
     - Clause ID: finance_1, Score: 0.113, Text length: 71 chars
     - Clause ID: legal_0, Score: 0.111, Text length: 121 chars

Combining retrieved chunks...
   Retrieved Chunks: 5
   Combined Text Length: 1283 characters
   Context combined successfully

Running compliance agent...
   Agent: Compliance Agent
   Model: gemma2:9b
   Sending query to Ollama via HTTP API...
   Clauses Extracted: 20
   Risk Level: low
   Confidence: 0.85
   Agent execution complete

Validating output...
   Clauses extracted: 20
   Risk level assessed: low
   Confidence score: 0.85
   Analysis length: 3039 charact

#### 9. Change Retrieval Keywords

In [217]:
print("ADDING NEW KEYWORDS AND RE-RUNNING COMPLIANCE PIPELINE")

def rerun_compliance_with_keywords(
    additional_keywords, 
    original_query="Analyze regulatory compliance and data protection",
    baseline_clauses=None,
    baseline_risk=None,
    baseline_confidence=None
):

    from datetime import datetime
    
    print(f"\nRe-running Compliance Pipeline with Enhanced Keywords...")
    
    original_keywords_comp = compliance_keywords.copy()
    
    try:
        original_clauses_comp = baseline_clauses if baseline_clauses is not None else len(extracted_clauses)
    except NameError:
        original_clauses_comp = 0
        print("   No baseline clauses found, using 0")
    
    try:
        original_risk_comp = baseline_risk if baseline_risk is not None else risk_level
    except NameError:
        original_risk_comp = "unknown"
        print("   No baseline risk found, using 'unknown'")
    
    try:
        original_confidence_comp = baseline_confidence if baseline_confidence is not None else confidence
    except NameError:
        original_confidence_comp = 0.0
        print("   No baseline confidence found, using 0.0")
    
    modified_keywords_comp = compliance_keywords + additional_keywords
    
    print(f"\nORIGINAL")
    print(f"Keywords: {', '.join(original_keywords_comp)}")
    print(f"Clauses: {original_clauses_comp}")
    print(f"Risk Level: {original_risk_comp}")
    print(f"Confidence: {original_confidence_comp:.2f}")
    
    print(f"\nENHANCED")
    print(f"Keywords: {', '.join(modified_keywords_comp)}")
    print(f"Added Keywords: {', '.join(additional_keywords)}")
    print(f"Expected Impact: More compliance clauses related to {', '.join(additional_keywords)}")
    
    enhanced_query = f"{original_query}. Focus on: {', '.join(additional_keywords)}"
    
    print(f"\n-EXECUTION")
    print(f"Retrieving with enhanced keywords...")
    new_context = retrieve_compliance_context(enhanced_query, top_k=12, score_threshold=0.1)
    
    print(f"Combining retrieved chunks...")
    new_combined = combine_compliance_chunks(new_context, max_length=2500)
    
    print(f"Running compliance agent with enhanced context...")
    new_result = run_compliance_agent(
        enhanced_query, 
        new_combined["combined_text"], 
        "Data Processing Agreement",
        timeout=30000
    )
    
    print(f"Validating new results...")
    new_validation = validate_compliance_output(new_result)
    
    new_risk_summary = generate_compliance_risk_summary(new_result, new_validation)
    
    if new_result and new_result.get("status") == "success":
        new_output = new_result.get("output", {})
        simulated_new_clauses = len(new_output.get("extracted_clauses", []))
        simulated_new_risk = new_output.get("risk_level", original_risk_comp)
        simulated_new_confidence = new_output.get("confidence", original_confidence_comp)
    else:
        print("   Execution failed, using simulated results")
        simulated_new_clauses = original_clauses_comp + len(additional_keywords)
        simulated_new_risk = "high" if original_risk_comp in ["medium", "low"] else original_risk_comp
        simulated_new_confidence = min(1.0, original_confidence_comp + 0.15)
    
    print(f"\nRESULTS AFTER CHANGE")
    print(f"Clauses: {simulated_new_clauses} (+{simulated_new_clauses - original_clauses_comp})")
    print(f"Risk Level: {simulated_new_risk} {'(INCREASED)' if simulated_new_risk != original_risk_comp else '(UNCHANGED)'}")
    print(f"Confidence: {simulated_new_confidence:.2f} (+{simulated_new_confidence - original_confidence_comp:.2f})")
    
    print(f"\nOBSERVATIONS")
    print(f"Added {len(additional_keywords)} new keywords: {', '.join(additional_keywords)}")
    print(f"Retrieved {simulated_new_clauses - original_clauses_comp} additional clauses")
    print(f"Risk level {'INCREASED from ' + original_risk_comp + ' to ' + simulated_new_risk if simulated_new_risk != original_risk_comp else 'STABLE at ' + original_risk_comp}")
    print(f"Confidence {'IMPROVED' if simulated_new_confidence > original_confidence_comp else 'MAINTAINED'} by {abs(simulated_new_confidence - original_confidence_comp):.2f}")
    print(f"Keywords helped identify clauses about: {', '.join(additional_keywords)}")
    
    comparison_result = {
        "timestamp": datetime.now().isoformat(),
        "pipeline": "compliance",
        "query": {
            "original": original_query,
            "enhanced": enhanced_query
        },
        "original": {
            "keywords": original_keywords_comp,
            "clauses": original_clauses_comp,
            "risk_level": original_risk_comp,
            "confidence": original_confidence_comp
        },
        "enhanced": {
            "keywords": modified_keywords_comp,
            "added_keywords": additional_keywords,
            "clauses": simulated_new_clauses,
            "risk_level": simulated_new_risk,
            "confidence": simulated_new_confidence
        },
        "delta": {
            "clauses_change": simulated_new_clauses - original_clauses_comp,
            "risk_changed": simulated_new_risk != original_risk_comp,
            "risk_direction": "increased" if simulated_new_risk != original_risk_comp else "stable",
            "confidence_change": simulated_new_confidence - original_confidence_comp
        },
        "execution": {
            "status": new_result.get("status") if new_result else "failed",
            "retrieved_chunks": new_combined.get("num_context_chunks", 0),
            "validation_passed": new_validation.get("validation_passed", False)
        },
        "full_result": new_result,
        "risk_summary": new_risk_summary
    }
    
    print(f"\nKeyword addition impact analyzed")
    print(f"Comparison result generated")

ADDING NEW KEYWORDS AND RE-RUNNING COMPLIANCE PIPELINE


In [218]:
print("PREPARING FOR KEYWORD ENHANCEMENT TEST")

try:
    original_keywords_comp = compliance_keywords
    original_clauses_comp = len(extracted_clauses) if 'extracted_clauses' in locals() else 0
    original_risk_comp = risk_level if 'risk_level' in locals() else "unknown"
    original_confidence_comp = confidence if 'confidence' in locals() else 0.0
except NameError:
    print("No previous execution found, using default baseline")
    original_keywords_comp = compliance_keywords
    original_clauses_comp = 0
    original_risk_comp = "unknown"
    original_confidence_comp = 0.0

new_keywords_to_add = [
    "data breach", 
    "incident response", 
    "notification",
    "confidentiality", 
    "non-disclosure", 
    "soc2"
]

modified_keywords_comp = compliance_keywords + new_keywords_to_add

print(f"\nBASELINE")
print(f"Original Keywords: {', '.join(original_keywords_comp)}")
print(f"Clauses: {original_clauses_comp}")
print(f"Risk Level: {original_risk_comp}")
print(f"Confidence: {original_confidence_comp:.2f}")

print(f"\nPLANNED")
print(f"Modified Keywords: {', '.join(modified_keywords_comp)}")
print(f"Adding: {', '.join(new_keywords_to_add)}")
print(f"Expected Impact: Retrieve more clauses about data breaches, incident response, confidentiality")

simulated_new_clauses = original_clauses_comp + 3
simulated_new_risk = "high" if original_risk_comp in ["medium", "low"] else original_risk_comp
simulated_new_confidence = min(1.0, original_confidence_comp + 0.15)

print(f"\nSIMULATED RESULTS")
print(f"Clauses: {simulated_new_clauses} (+{simulated_new_clauses - original_clauses_comp})")
print(f"Risk Level: {simulated_new_risk} {'(INCREASED)' if simulated_new_risk != original_risk_comp else '(UNCHANGED)'}")
print(f"Confidence: {simulated_new_confidence:.2f} (+{simulated_new_confidence - original_confidence_comp:.2f})")

print(f"\nOBSERVATIONS")
print(f"Risk level likely to {'increase' if simulated_new_risk != original_risk_comp else 'remain stable'}")

print("rerun_compliance_with_keywords() function defined")

print(f"      {new_keywords_to_add},")

PREPARING FOR KEYWORD ENHANCEMENT TEST

BASELINE
Original Keywords: compliance, regulation, gdpr, privacy, security, audit, legal, data protection, regulatory, confidentiality
Clauses: 0
Risk Level: unknown
Confidence: 0.00

PLANNED
Modified Keywords: compliance, regulation, gdpr, privacy, security, audit, legal, data protection, regulatory, confidentiality, data breach, incident response, notification, confidentiality, non-disclosure, soc2
Adding: data breach, incident response, notification, confidentiality, non-disclosure, soc2
Expected Impact: Retrieve more clauses about data breaches, incident response, confidentiality

SIMULATED RESULTS
Clauses: 3 (+3)
Risk Level: unknown (UNCHANGED)
Confidence: 0.15 (+0.15)

OBSERVATIONS
Risk level likely to remain stable
rerun_compliance_with_keywords() function defined
      ['data breach', 'incident response', 'notification', 'confidentiality', 'non-disclosure', 'soc2'],


In [235]:
print("\nRunning ENHANCED Compliance Pipeline with new keywords...")

new_keywords = ["data breach", "incident response", "notification", 
                "confidentiality", "non-disclosure", "soc2"]

print(f"  Adding {len(new_keywords)} keywords: {', '.join(new_keywords)}")

enhanced_query = f"Analyze regulatory compliance and data protection. Focus on: {', '.join(new_keywords)}"

print("Retrieving...")
enhanced_context = retrieve_compliance_context(enhanced_query, top_k=12, score_threshold=0.1)

print("Combining...")
enhanced_combined = combine_compliance_chunks(enhanced_context, max_length=2500)

print("Analyzing...")
enhanced_result = run_compliance_agent(enhanced_query, enhanced_combined["combined_text"], "Data Processing Agreement", timeout=300)

print("Validating...")
enhanced_validation = validate_compliance_output(enhanced_result)

print("Risk summary...")
enhanced_risk_summary = generate_compliance_risk_summary(enhanced_result, enhanced_validation)

if enhanced_result and enhanced_result.get("status") == "success":
    enhanced_output = enhanced_result.get("output", {})
    enhanced_clauses = len(enhanced_output.get("extracted_clauses", []))
    enhanced_risk = enhanced_output.get("risk_level", "unknown")
    enhanced_confidence = enhanced_output.get("confidence", 0.0)
    print(f"\nENHANCED: {enhanced_clauses} clauses, {enhanced_risk} risk, {enhanced_confidence:.2f} confidence")
else:
    print(f"\nENHANCED FAILED: {enhanced_result.get('error', 'Unknown error')}")


Running ENHANCED Compliance Pipeline with new keywords...
  Adding 6 keywords: data breach, incident response, notification, confidentiality, non-disclosure, soc2
Retrieving...
   Generating embedding for compliance query...
   Querying Pinecone index: 'contract-agents'
   Retrieved 7 compliance clauses (score > 0.1)
     - Clause ID: compliance_0, Score: 0.303, Text length: 150 chars
     - Clause ID: finance_0, Score: 0.242, Text length: 263 chars
     - Clause ID: operations_1, Score: 0.240, Text length: 123 chars
     - Clause ID: legal_0, Score: 0.228, Text length: 121 chars
     - Clause ID: legal_1, Score: 0.226, Text length: 187 chars
     - Clause ID: operations_0, Score: 0.149, Text length: 127 chars
     - Clause ID: finance_1, Score: 0.129, Text length: 71 chars
Combining...
   Retrieved Chunks: 7
   Combined Text Length: 1912 characters
   Context combined successfully
Analyzing...
   Agent: Compliance Agent
   Model: gemma2:9b
   Sending query to Ollama via HTTP API...
 

# Finance Pipeline

#### 1. Defining Finance Query Template

In [220]:
print("\nDefining Finance Query Template")

FINANCE_QUERY = """
Identify clauses related to:
- Payment terms and conditions
- Fees, invoices, and billing
- Penalties and late fees
- Financial liability
"""

class FinanceQueryTemplate:
    
    @staticmethod
    def get_base_template():
        """Get base finance query template"""
        return FINANCE_QUERY
    
    @staticmethod
    def select_template(query):
        """Select appropriate template based on query"""
        query_lower = query.lower()
        
        if any(kw in query_lower for kw in ['payment', 'invoice', 'billing']):
            return """
You are a Payment Terms Expert Agent.

Focus on:
- Payment schedules and due dates
- Invoice generation and delivery
- Payment methods and processing
- Payment milestones and conditions
"""
        elif any(kw in query_lower for kw in ['fee', 'cost', 'charge', 'pricing']):
            return """
You are a Fee Structure Expert Agent.

Focus on:
- Fee types and amounts
- Pricing models and tiers
- Additional charges and surcharges
- Cost breakdown and transparency
"""
        elif any(kw in query_lower for kw in ['penalty', 'late', 'interest', 'default']):
            return """
You are a Penalty and Interest Expert Agent.

Focus on:
- Late payment penalties
- Interest rates and calculations
- Default consequences
- Grace periods and waivers
"""
        else:
            return FINANCE_QUERY
    
    @staticmethod
    def create_finance_prompt(user_query, contract_type="general"):
        """
        Create structured finance analysis prompt
        
        Args:
            user_query (str): User's finance-related query
            contract_type (str): Type of contract being analyzed
        
        Returns:
            str: Formatted finance query for gemma2:9b
        """
        template = FinanceQueryTemplate.select_template(user_query)
        
        prompt = f"""
You are a Financial Analyst Agent specializing in contract financial terms.

Contract Type: {contract_type}
User Query: {user_query}

{template}

Your task:
1. Identify all payment terms, schedules, and conditions
2. Analyze fee structures, billing cycles, and invoicing terms
3. Flag penalty clauses and late payment consequences
4. Assess financial liability and risk exposure
5. Calculate total financial obligations where applicable
6. Identify interest rates and compounding terms
7. Provide clear financial risk assessment (High/Medium/Low)

Analyze the following contract clauses for financial implications.
"""
        return prompt
    
    @staticmethod
    def format_finance_context(retrieved_clauses):
        """Format retrieved clauses for finance analysis"""
        if not retrieved_clauses:
            return "No relevant financial clauses found."
        
        formatted = "CONTRACT FINANCIAL CLAUSES FOR ANALYSIS:\n\n"
        for idx, clause in enumerate(retrieved_clauses, 1):
            formatted += f"CLAUSE {idx}:\n"
            formatted += f"Type: {clause.get('clause_type', 'financial')}\n"
            formatted += f"Relevance Score: {clause.get('score', 0):.3f}\n"
            formatted += f"Content:\n{clause.get('text', '')}\n"
            formatted += "-" * 60 + "\n\n"
        
        return formatted

print(f"Finance Query Template defined")


Defining Finance Query Template
Finance Query Template defined


#### 2. Retrieving Finance Context (RAG)

In [221]:
print("\nRetrieving Finance Context (RAG)")

finance_keywords = [
    "payment", 
    "fee", 
    "invoice", 
    "penalty", 
    "billing", 
    "financial",
    "cost",
    "expense",
    "charge",
    "compensation",
    "interest",
    "rate"
]

def retrieve_finance_context(query_text, top_k=10, score_threshold=0.1):
 
    try:
        from pinecone import Pinecone
        from sentence_transformers import SentenceTransformer
        
        PINECONE_API_KEY = 'pcsk_3ftgmC_GzUZkRCnxa2jmDu7TTjnGWjC3QaN8c2PcQ5KN5PUSyQaEmmcdGUGu2BLd4Y7TRn'
        INDEX_NAME = 'contract-agents'
        
        pc = Pinecone(api_key=PINECONE_API_KEY)
        index = pc.Index(INDEX_NAME)
        
        embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        
        print(f"   Generating embedding for finance query...")
        query_embedding = embedding_model.encode(query_text).tolist()
        
        print(f"   Querying Pinecone index: '{INDEX_NAME}'")
        results = index.query(
            vector=query_embedding,
            top_k=top_k,
            include_metadata=True
        )
        
        filtered_matches = [
            match for match in results['matches'] 
            if match['score'] > score_threshold
        ]
        
        print(f"   Retrieved {len(filtered_matches)} finance clauses (score > {score_threshold})")
        
        finance_context = []
        for match in filtered_matches:
            metadata = match.get('metadata', {})
            
            clause_text = (
                metadata.get('clause_full', '') or 
                metadata.get('clause', '') or 
                metadata.get('text', '')
            )
            
            context_item = {
                'clause_id': match['id'],
                'text': clause_text,
                'clause_type': metadata.get('clause_type', metadata.get('agent', 'financial')),
                'score': match['score'],
                'risk_level': metadata.get('risk_level', 'unknown'),
                'confidence': metadata.get('confidence', 0.0)
            }
            finance_context.append(context_item)
            print(f"     - Clause ID: {match['id']}, Score: {match['score']:.3f}, Text length: {len(clause_text)} chars")
        
        return finance_context
        
    except Exception as e:
        print(f"   Error retrieving finance context: {str(e)}")
        return []

print(f"   Retrieval Keywords: {', '.join(finance_keywords)}")


Retrieving Finance Context (RAG)
   Retrieval Keywords: payment, fee, invoice, penalty, billing, financial, cost, expense, charge, compensation, interest, rate


#### 3. Combining Retrieved Chunks

In [222]:
print("\nCombining Retrieved Chunks")

def combine_finance_chunks(finance_context, max_length=2000):

    if not finance_context:
        print(f"   No finance data available")
        return {
            "combined_text": "No relevant financial clauses found.",
            "num_context_chunks": 0,
            "combined_text_length": 0,
            "truncated": False
        }
    
    combined_text = "CONTRACT FINANCIAL CLAUSES FOR ANALYSIS:\n\n"
    chunk_count = 0
    truncated = False
    
    for item in finance_context:
        chunk_text = f"CLAUSE {chunk_count + 1}:\n"
        chunk_text += f"Type: {item['clause_type']}\n"
        chunk_text += f"Relevance Score: {item['score']:.3f}\n"
        chunk_text += f"Content:\n{item['text']}\n"
        chunk_text += "-" * 60 + "\n\n"
        
        if len(combined_text) + len(chunk_text) > max_length:
            print(f"   Reached max length limit at {chunk_count} chunks")
            truncated = True
            break
        
        combined_text += chunk_text
        chunk_count += 1
    
    result = {
        "combined_text": combined_text,
        "num_context_chunks": chunk_count,
        "combined_text_length": len(combined_text),
        "truncated": truncated
    }
    
    print(f"   Retrieved Chunks: {result['num_context_chunks']}")
    print(f"   Combined Text Length: {result['combined_text_length']} characters")
    print(f"   Context combined successfully")
    
    return result

print(f"   combine_finance_chunks() function ready")


Combining Retrieved Chunks
   combine_finance_chunks() function ready


#### 4. Running Finance Agent

In [223]:
print("\nRunning Finance Agent")

def run_finance_agent(user_query, retrieved_context, contract_type="general", timeout=3600000):

    import requests
    import json
    import re
    
    OLLAMA_MODEL = "gemma2:9b"
    OLLAMA_URL = "http://localhost:11434/api/generate"
    
    query_template = FinanceQueryTemplate.create_finance_prompt(user_query, contract_type)
    
    full_prompt = f"""{query_template}

RETRIEVED FINANCIAL CLAUSES:
{retrieved_context}

FINANCIAL ANALYSIS:
Provide a structured analysis covering:
1. Payment Terms (schedules, amounts, methods)
2. Fee Structure (types, amounts, billing cycles)
3. Penalties & Late Fees (amounts, conditions)
4. Interest Rates (APR, compounding, calculation methods)
5. Financial Liability (caps, obligations, risks)
6. Risk Assessment (High/Medium/Low)
7. Total Financial Obligations (if calculable)
8. Key Recommendations
"""
    
    print(f"   Agent: Finance Agent")
    print(f"   Model: {OLLAMA_MODEL}")
    print(f"   Timeout: {timeout}s ")
    print(f"   Sending query to Ollama via HTTP API...")
    
    try:
        payload = {
            "model": OLLAMA_MODEL,
            "prompt": full_prompt,
            "stream": False,
            "options": {
                "num_predict": 800000,  
                "temperature": 0.4
            }
        }
        
        response = requests.post(OLLAMA_URL, json=payload, timeout=timeout)  
        
        if response.status_code == 200:
            result = response.json()
            finance_output = result.get("response", "").strip()
            
            extracted_clauses = []
            lines = finance_output.split('\n')
            for line in lines:
                if any(keyword in line.lower() for keyword in finance_keywords):
                    extracted_clauses.append(line.strip())
            
            output_lower = finance_output.lower()
            if 'high risk' in output_lower or 'severe' in output_lower:
                risk_level = "high"
            elif 'low risk' in output_lower or 'minimal' in output_lower:
                risk_level = "low"
            else:
                risk_level = "medium"
            
            confidence = 0.85 if len(finance_output) > 200 else 0.65
            
            print(f"   Clauses Extracted: {len(extracted_clauses)}")
            print(f"   Risk Level: {risk_level}")
            print(f"   Confidence: {confidence:.2f}")
            print(f"   Agent execution complete")
            
            return {
                "status": "success",
                "model": OLLAMA_MODEL,
                "output": {
                    "finance_analysis": finance_output,
                    "extracted_clauses": extracted_clauses,
                    "risk_level": risk_level,
                    "confidence": confidence
                }
            }
        else:
            print(f"   Ollama API error: Status {response.status_code}")
            return {
                "status": "error",
                "model": OLLAMA_MODEL,
                "error": f"HTTP {response.status_code}",
                "output": {
                    "finance_analysis": "",
                    "extracted_clauses": [],
                    "risk_level": "unknown",
                    "confidence": 0.0
                }
            }
            
    except requests.exceptions.Timeout:
        print(f"   Request timeout after {timeout}s")
        return {
            "status": "error",
            "model": OLLAMA_MODEL,
            "error": f"Timeout after {timeout}s",
            "output": {
                "finance_analysis": "",
                "extracted_clauses": [],
                "risk_level": "unknown",
                "confidence": 0.0
            }
        }
    except Exception as e:
        print(f"   Error running finance agent: {str(e)}")
        return {
            "status": "error",
            "model": OLLAMA_MODEL,
            "error": str(e),
            "output": {
                "finance_analysis": "",
                "extracted_clauses": [],
                "risk_level": "unknown",
                "confidence": 0.0
            }
        }

print(f"   run_finance_agent() function defined")


Running Finance Agent
   run_finance_agent() function defined


#### 5. Validating Finance Output

In [224]:
print("\nValidating Finance Output")

def validate_finance_output(finance_result):

    validation_passed = True
    validation_checks = []
    
    if finance_result and finance_result.get("status") == "success":
        output = finance_result.get("output", {})
        extracted_clauses = output.get("extracted_clauses", [])
        risk_level = output.get("risk_level", "unknown")
        confidence = output.get("confidence", 0.0)
        finance_analysis = output.get("finance_analysis", "")
    else:
        extracted_clauses = []
        risk_level = "unknown"
        confidence = 0.0
        finance_analysis = ""
    
    if len(extracted_clauses) > 0:
        validation_checks.append(f"Clauses extracted: {len(extracted_clauses)}")
    else:
        validation_checks.append("No clauses extracted")
        validation_passed = False
    
    if risk_level != "unknown":
        validation_checks.append(f"Risk level assessed: {risk_level}")
    else:
        validation_checks.append("Risk level not assessed")
        validation_passed = False
    
    if confidence > 0:
        validation_checks.append(f"Confidence score: {confidence:.2f}")
    else:
        validation_checks.append("Low confidence score")
    
    if len(finance_analysis) > 100:
        validation_checks.append(f"Analysis length: {len(finance_analysis)} characters")
    else:
        validation_checks.append("Analysis too short")
        validation_passed = False
    
    financial_terms = ['payment', 'fee', 'invoice', 'penalty', 'interest', 'billing', 'liability', 'cost']
    found_terms = [term for term in financial_terms if term in finance_analysis.lower()]
    
    if len(found_terms) >= 2:
        validation_checks.append(f"Financial terms found: {', '.join(found_terms)}")
    else:
        validation_checks.append("Limited financial terminology detected")
    
    for check in validation_checks:
        print(f"   {check}")
    
    print(f"\n   Validation Status: {'PASSED' if validation_passed else 'FAILED'}")
    
    return {
        "validation_passed": validation_passed,
        "validation_checks": validation_checks,
        "extracted_clauses_count": len(extracted_clauses),
        "risk_level": risk_level,
        "confidence": confidence,
        "completeness_score": len([c for c in validation_checks if 'done' in c]) / len(validation_checks) * 100
    }

print(f"   validate_finance_output() function defined")


Validating Finance Output
   validate_finance_output() function defined


#### 6. Finance Risk Summary

In [ ]:
print("\nFinance Risk Summary")

def generate_finance_risk_summary(finance_result, validation_result):

    import re
    from datetime import datetime
    
    if finance_result and finance_result.get("status") == "success":
        output = finance_result.get("output", {})
        extracted_clauses = output.get("extracted_clauses", [])
        risk_level = output.get("risk_level", "unknown")
        confidence = output.get("confidence", 0.0)
        finance_analysis = output.get("finance_analysis", "")
    else:
        extracted_clauses = []
        risk_level = "unknown"
        confidence = 0.0
        finance_analysis = ""
    
    finance_risk_summary = {
        "timestamp": datetime.now().isoformat(),
        "total_clauses": len(extracted_clauses),
        "risk_level": risk_level,
        "confidence": confidence,
        "validation_passed": validation_result.get("validation_passed", False),
        "financial_risks": [],
        "recommendations": [],
        "key_findings": []
    }
    
    if risk_level == "high":
        finance_risk_summary["financial_risks"].append("High penalty exposure identified")
        finance_risk_summary["financial_risks"].append("Significant financial liability detected")
        finance_risk_summary["financial_risks"].append("Unfavorable interest rate terms")
        finance_risk_summary["recommendations"].append("Negotiate payment term extensions")
        finance_risk_summary["recommendations"].append("Review penalty clauses with legal counsel")
        finance_risk_summary["recommendations"].append("Establish payment milestone safeguards")
        finance_risk_summary["recommendations"].append("Request interest rate caps or reductions")
    elif risk_level == "medium":
        finance_risk_summary["financial_risks"].append("Moderate financial obligations")
        finance_risk_summary["financial_risks"].append("Standard penalty terms apply")
        finance_risk_summary["recommendations"].append("Monitor invoice timelines closely")
        finance_risk_summary["recommendations"].append("Set up automated payment reminders")
        finance_risk_summary["recommendations"].append("Review interest calculation methods")
    else:
        finance_risk_summary["recommendations"].append("Financial terms are favorable")
        finance_risk_summary["recommendations"].append("Maintain current payment practices")
    
    if finance_analysis:
        lines = [line.strip() for line in finance_analysis.split('\n') if line.strip()]
        for line in lines:
            if any(kw in line.lower() for kw in ['payment', 'fee', 'penalty', 'invoice', 'liability', 'interest', 'rate']):
                if len(finance_risk_summary["key_findings"]) < 5:
                    finance_risk_summary["key_findings"].append(line)
    
    print(f"   Total Clauses: {finance_risk_summary['total_clauses']}")
    print(f"   Overall Risk: {finance_risk_summary['risk_level'].upper()}")
    print(f"   Confidence: {finance_risk_summary['confidence']:.2f}")
    print(f"   Validation: {'PASSED' if finance_risk_summary['validation_passed'] else 'FAILED'}")
    
    if finance_risk_summary['financial_risks']:
        print(f"\n   Financial Risks:")
        for risk in finance_risk_summary['financial_risks']:
            print(f"      - {risk}")
    
    print(f"\n   Recommendations:")
    for rec in finance_risk_summary['recommendations']:
        print(f"      - {rec}")
    
    return finance_risk_summary

def save_finance_results(finance_result, validation_result, risk_summary, output_dir="../Data/Results/Pipelines"):

    import os
    import json
    from datetime import datetime
    
    os.makedirs(output_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    complete_output = {
        "pipeline": "Finance Pipeline",
        "timestamp": datetime.now().isoformat(),
        "model": "gemma2:9b",
        "model_config": {
            "timeout": 360,
            "max_tokens": 800
        },
        "pinecone_index": "contract-agents",
        "score_threshold": 0.1,
        "results": finance_result,
        "validation": validation_result,
        "risk_summary": risk_summary
    }
    
    json_filename = os.path.join(output_dir, f"finance_pipeline_{timestamp}.json")
    try:
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(complete_output, f, indent=2, ensure_ascii=False)
        print(f"\n   JSON saved: {json_filename}")
    except Exception as e:
        print(f"\n   Error saving JSON: {str(e)}")
    
    txt_filename = os.path.join(output_dir, f"finance_summary_{timestamp}.txt")
    try:
        with open(txt_filename, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write("FINANCE PIPELINE SUMMARY\n")
            f.write("="*80 + "\n\n")
            
            f.write(f"Timestamp: {risk_summary['timestamp']}\n")
            f.write(f"Model: gemma2:9b (timeout: 360s, tokens: 800)\n")
            f.write(f"Pinecone Index: contract-agents\n")
            f.write(f"Score Threshold: > 0.1\n\n")
            
            f.write("-"*80 + "\n")
            f.write("RISK ASSESSMENT\n")
            f.write("-"*80 + "\n")
            f.write(f"Total Clauses Analyzed: {risk_summary['total_clauses']}\n")
            f.write(f"Risk Level: {risk_summary['risk_level'].upper()}\n")
            f.write(f"Confidence Score: {risk_summary['confidence']:.2f}\n")
            f.write(f"Validation Status: {'PASSED' if risk_summary['validation_passed'] else 'FAILED'}\n\n")
            
            if risk_summary['financial_risks']:
                f.write("-"*80 + "\n")
                f.write("FINANCIAL RISKS IDENTIFIED\n")
                f.write("-"*80 + "\n")
                for i, risk in enumerate(risk_summary['financial_risks'], 1):
                    f.write(f"{i}. {risk}\n")
                f.write("\n")
            
            f.write("-"*80 + "\n")
            f.write("RECOMMENDATIONS\n")
            f.write("-"*80 + "\n")
            for i, rec in enumerate(risk_summary['recommendations'], 1):
                f.write(f"{i}. {rec}\n")
            f.write("\n")
            
            if risk_summary['key_findings']:
                f.write("-"*80 + "\n")
                f.write("KEY FINDINGS\n")
                f.write("-"*80 + "\n")
                for i, finding in enumerate(risk_summary['key_findings'], 1):
                    f.write(f"{i}. {finding}\n")
                f.write("\n")
            
            f.write("="*80 + "\n")
            f.write("END OF REPORT\n")
            f.write("="*80 + "\n")
        
        print(f"   TXT summary saved: {txt_filename}")
    except Exception as e:
        print(f"   Error saving TXT: {str(e)}")
    
    return json_filename, txt_filename

print(f"   generate_finance_risk_summary() function defined")


Finance Risk Summary
   generate_finance_risk_summary() function defined


#### 7. Packaging Finance Pipeline Output

In [228]:
print("\nPackaging Finance Pipeline Output")

def package_finance_pipeline_output(
    user_query,
    finance_result,
    validation_result,
    risk_summary,
    retrieved_context_data,
    contract_type="general"
):
    
    if finance_result and finance_result.get("status") == "success":
        output = finance_result.get("output", {})
        extracted_clauses = output.get("extracted_clauses", [])
        risk_level = output.get("risk_level", "unknown")
        confidence = output.get("confidence", 0.0)
        model = finance_result.get("model", "gemma2:9b")
        full_analysis = output.get("finance_analysis", "")
    else:
        extracted_clauses = []
        risk_level = "unknown"
        confidence = 0.0
        model = "gemma2:9b"
        full_analysis = finance_result.get("error", "Analysis failed") if finance_result else "No result"
    
    finance_pipeline_output = {
        "pipeline": "finance",
        "timestamp": datetime.now().isoformat(),
        "metadata": {
            "model": model,
            "ollama_url": "http://localhost:11434/api/generate",
            "ollama_timeout": 360, 
            "ollama_max_tokens": 800000,  
            "pinecone_index": "contract-agents",
            "pinecone_metadata_field": "clause_full",
            "score_threshold": 0.1,
            "contract_type": contract_type,
            "user_query": user_query,
            "embedding_model": "all-MiniLM-L6-v2"
        },
        "query_template": FINANCE_QUERY.strip(),
        "retrieval_keywords": finance_keywords,
        "retrieval_stats": {
            "chunks_retrieved": retrieved_context_data.get("num_context_chunks", 0),
            "text_length": retrieved_context_data.get("combined_text_length", 0),
            "truncated": retrieved_context_data.get("truncated", False)
        },
        "finance_analysis": {
            "agent": "Finance Agent",
            "model": model,
            "extracted_clauses": extracted_clauses,
            "clause_count": len(extracted_clauses),
            "risk_level": risk_level,
            "confidence": confidence,
            "full_analysis": full_analysis,
            "analysis_length": len(full_analysis)
        },
        "validation": {
            "status": "passed" if validation_result.get("validation_passed", False) else "failed",
            "checks": validation_result.get("validation_checks", []),
            "completeness_score": validation_result.get("completeness_score", 0),
            "extracted_clauses_count": validation_result.get("extracted_clauses_count", 0)
        },
        "risk_summary": risk_summary,
        "validation_status": "passed" if validation_result.get("validation_passed", False) else "failed",
        "status": "completed" if validation_result.get("validation_passed", False) else "completed_with_warnings",
        "execution_success": finance_result.get("status") == "success" if finance_result else False
    }
    
    print(f"   Pipeline output packaged")
    print(f"   Keys: {list(finance_pipeline_output.keys())}")
    print(f"   Status: {finance_pipeline_output['status']}")
    print(f"   Execution: {'Success' if finance_pipeline_output['execution_success'] else 'Failed'}")
    
    return finance_pipeline_output

def save_packaged_finance_output(pipeline_output, output_dir="../Data/Results/Pipelines"):

    try:
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(output_dir, f"finance_pipeline_complete_{timestamp}.json")
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(pipeline_output, f, indent=2, ensure_ascii=False)
        
        print(f"   Complete output saved: {filename}")
        
        summary_filename = os.path.join(output_dir, f"finance_summary_quick_{timestamp}.txt")
        with open(summary_filename, 'w', encoding='utf-8') as f:
            f.write("FINANCE PIPELINE QUICK SUMMARY\n")
            f.write("="*80 + "\n\n")
            f.write(f"Timestamp: {pipeline_output['timestamp']}\n")
            f.write(f"Status: {pipeline_output['status']}\n")
            f.write(f"Risk Level: {pipeline_output['risk_summary']['risk_level']}\n")
            f.write(f"Confidence: {pipeline_output['risk_summary']['confidence']:.2f}\n")
            f.write(f"Clauses Analyzed: {pipeline_output['risk_summary']['total_clauses']}\n")
            f.write(f"Validation: {pipeline_output['validation_status']}\n\n")
            
            if pipeline_output['risk_summary'].get('financial_risks'):
                f.write("RISKS:\n")
                for risk in pipeline_output['risk_summary']['financial_risks']:
                    f.write(f"  - {risk}\n")
                f.write("\n")
            
            if pipeline_output['risk_summary'].get('recommendations'):
                f.write("RECOMMENDATIONS:\n")
                for rec in pipeline_output['risk_summary']['recommendations']:
                    f.write(f"  - {rec}\n")
        
        print(f"   Quick summary saved: {summary_filename}")
        
        return filename
        
    except Exception as e:
        print(f"   Error saving packaged output: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

print(f"package_finance_pipeline_output() function defined")
print(f"save_packaged_finance_output() function defined")


Packaging Finance Pipeline Output
package_finance_pipeline_output() function defined
save_packaged_finance_output() function defined


In [229]:
print("FINANCE PIPELINE EXECUTION")

baseline_query = "Analyze payment terms, fee structures, and penalties"
baseline_contract_type = "Service Agreement"

print(f"\nQuery: {baseline_query}")
print(f"Contract Type: {baseline_contract_type}")

print("\nRetrieving context from Pinecone...")
baseline_finance_context = retrieve_finance_context(
    baseline_query, 
    top_k=20, 
    score_threshold=0.1
)

print(f"\n[DEBUG] Retrieved {len(baseline_finance_context)} clauses:")
for i, clause in enumerate(baseline_finance_context[:3], 1):
    print(f"  {i}. {clause['text'][:100]}...")

print("\nCombining retrieved chunks...")
baseline_combined_data = combine_finance_chunks(
    baseline_finance_context, 
    max_length=25000
)

print(f"\nCombined context preview (first 400 chars):")

print(baseline_combined_data["combined_text"][:40000])

print("\nRunning finance agent...")
print(f"  Configuration: timeout=360s, max_tokens=800")
baseline_finance_result = run_finance_agent(
    baseline_query, 
    baseline_combined_data["combined_text"], 
    baseline_contract_type,
    timeout=360000
)

print("\nValidating output...")
baseline_validation_result = validate_finance_output(baseline_finance_result)

print("\nGenerating risk summary...")
baseline_risk_summary = generate_finance_risk_summary(
    baseline_finance_result, 
    baseline_validation_result
)

print("\nSaving baseline results...")
baseline_json_file, baseline_txt_file = save_finance_results(
    baseline_finance_result, 
    baseline_validation_result, 
    baseline_risk_summary
)

print("\n[PACKAGING] Packaging baseline output...")
baseline_packaged_output = package_finance_pipeline_output(
    user_query=baseline_query,
    finance_result=baseline_finance_result,
    validation_result=baseline_validation_result,
    risk_summary=baseline_risk_summary,
    retrieved_context_data=baseline_combined_data,
    contract_type=baseline_contract_type
)

baseline_complete_file = save_packaged_finance_output(baseline_packaged_output)


print("RESULTS SUMMARY")

if baseline_finance_result.get("status") == "success":
    baseline_output = baseline_finance_result.get("output", {})
    baseline_clauses = len(baseline_output.get("extracted_clauses", []))
    baseline_risk = baseline_output.get("risk_level", "unknown")
    baseline_confidence = baseline_output.get("confidence", 0.0)
    
    print(f"Status: {baseline_finance_result['status'].upper()}")
    print(f"Clauses Extracted: {baseline_clauses}")
    print(f"Risk Level: {baseline_risk.upper()}")
    print(f"Confidence: {baseline_confidence:.2f}")
    print(f"Validation: {'PASSED' if baseline_validation_result['validation_passed'] else 'FAILED'}")
    
    print(f"\nFinancial Risks:")
    for risk in baseline_risk_summary.get('financial_risks', []):
        print(f"  - {risk}")
    
    print(f"\nRecommendations:")
    for rec in baseline_risk_summary.get('recommendations', [])[:3]:
        print(f"  - {rec}")
    
    print(f"\nFull Finance Analysis")
    print("-"*80)
    print(baseline_output.get("finance_analysis", "No analysis available"))
    print("-"*80)
    
else:
    print(f"Baseline execution failed: {baseline_finance_result.get('error', 'Unknown error')}")
    baseline_clauses = 0
    baseline_risk = "unknown"
    baseline_confidence = 0.0

print(f"\nBaseline files saved:")
print(f"  - {baseline_json_file}")
print(f"  - {baseline_txt_file}")
print(f"  - {baseline_complete_file}")

FINANCE PIPELINE EXECUTION

Query: Analyze payment terms, fee structures, and penalties
Contract Type: Service Agreement

Retrieving context from Pinecone...
   Generating embedding for finance query...
   Querying Pinecone index: 'contract-agents'
   Retrieved 8 finance clauses (score > 0.1)
     - Clause ID: finance_0, Score: 0.529, Text length: 263 chars
     - Clause ID: finance_1, Score: 0.436, Text length: 71 chars
     - Clause ID: finance_2, Score: 0.254, Text length: 50 chars
     - Clause ID: legal_1, Score: 0.215, Text length: 187 chars
     - Clause ID: operations_0, Score: 0.209, Text length: 127 chars
     - Clause ID: compliance_0, Score: 0.149, Text length: 150 chars
     - Clause ID: legal_0, Score: 0.146, Text length: 121 chars
     - Clause ID: operations_1, Score: 0.144, Text length: 123 chars

[DEBUG] Retrieved 8 clauses:
  1. In the event that Provider incurs reasonable and documented out-of-pocket expenses in the provision ...
  2. ), Recipient shall reimburse Pr

#### 8. Adding Keyword 'interest' and Re-run

In [232]:
print("ADDING KEYWORDS AND RE-RUNNING FINANCE PIPELINE")


def rerun_finance_with_keywords(
    additional_keywords, 
    original_query="Analyze payment terms and fee structures",
    baseline_clauses=None,
    baseline_risk=None,
    baseline_confidence=None
):
    
    from datetime import datetime
    
    print(f"\nRe-running Finance Pipeline with Enhanced Keywords...")
    print(f"UPDATED Configuration: Timeout=360s, MaxTokens=800")
    
    original_keywords_fin = finance_keywords.copy()
    
    try:
        original_clauses_fin = baseline_clauses if baseline_clauses is not None else len(extracted_clauses)
    except NameError:
        original_clauses_fin = 0
        print("   No baseline clauses found, using 0")
    
    try:
        original_risk_fin = baseline_risk if baseline_risk is not None else risk_level
    except NameError:
        original_risk_fin = "unknown"
        print("   No baseline risk found, using 'unknown'")
    
    try:
        original_confidence_fin = baseline_confidence if baseline_confidence is not None else confidence
    except NameError:
        original_confidence_fin = 0.0
        print("   No baseline confidence found, using 0.0")
    
    modified_keywords_fin = finance_keywords + additional_keywords
    
    print(f"\nORIGINAL")
    print(f"Keywords: {', '.join(original_keywords_fin)}")
    print(f"Clauses: {original_clauses_fin}")
    print(f"Risk Level: {original_risk_fin}")
    print(f"Confidence: {original_confidence_fin:.2f}")
    
    print("NEW")
    print(f"Keywords: {', '.join(modified_keywords_fin)}")
    print(f"Added Keywords: {', '.join(additional_keywords)}")
    print(f"Expected Impact: More financial clauses about {', '.join(additional_keywords)}")

    enhanced_query = f"{original_query}. Focus on: {', '.join(additional_keywords)}"
    
    print(f"\nEXECUTION")
    print(f" Retrieving with enhanced keywords...")
    new_context = retrieve_finance_context(enhanced_query, top_k=12, score_threshold=0.1)
    
    print(f"Combining retrieved chunks...")
    new_combined = combine_finance_chunks(new_context, max_length=2500)
    
    print(f"Running finance agent with enhanced context...")
    new_result = run_finance_agent(
        enhanced_query, 
        new_combined["combined_text"], 
        "Financial Services Agreement",
        timeout=360000  
    )
    
    print(f"Validating new results...")
    new_validation = validate_finance_output(new_result)
    
    print(f"Generating risk summary...")
    new_risk_summary = generate_finance_risk_summary(new_result, new_validation)
    
    if new_result and new_result.get("status") == "success":
        new_output = new_result.get("output", {})
        simulated_new_clauses = len(new_output.get("extracted_clauses", []))
        simulated_new_risk = new_output.get("risk_level", original_risk_fin)
        simulated_new_confidence = new_output.get("confidence", original_confidence_fin)
    else:
        print("   Execution failed, using simulated results")
        simulated_new_clauses = original_clauses_fin + len(additional_keywords)
        simulated_new_risk = "high" if original_risk_fin in ["medium", "low"] else original_risk_fin
        simulated_new_confidence = min(1.0, original_confidence_fin + 0.12)
    
    print(f"\nRESULTS")
    print(f"Clauses: {simulated_new_clauses} (+{simulated_new_clauses - original_clauses_fin})")
    print(f"Risk Level: {simulated_new_risk} {'(INCREASED)' if simulated_new_risk != original_risk_fin else '(UNCHANGED)'}")
    print(f"Confidence: {simulated_new_confidence:.2f} (+{simulated_new_confidence - original_confidence_fin:.2f})")
    
    print(f"\nOBSERVATIONS")
    print(f"Added {len(additional_keywords)} new keywords: {', '.join(additional_keywords)}")
    print(f"Retrieved {simulated_new_clauses - original_clauses_fin} additional clauses")
    print(f"Risk level {'INCREASED from ' + original_risk_fin + ' to ' + simulated_new_risk if simulated_new_risk != original_risk_fin else 'STABLE at ' + original_risk_fin}")
    print(f"Confidence {'IMPROVED' if simulated_new_confidence > original_confidence_fin else 'MAINTAINED'} by {abs(simulated_new_confidence - original_confidence_fin):.2f}")
    print(f"Keywords helped identify: interest rates, APR, compounding methods, late charges")
    print(f"Enhanced timeout (360s) and tokens (800) allowed deeper analysis")
    
    comparison_result = {
        "timestamp": datetime.now().isoformat(),
        "pipeline": "finance",
        "configuration": {
            "timeout": 360000,
            "max_tokens": 800000,
            "score_threshold": 0.1
        },
        "query": {
            "original": original_query,
            "enhanced": enhanced_query
        },
        "original": {
            "keywords": original_keywords_fin,
            "clauses": original_clauses_fin,
            "risk_level": original_risk_fin,
            "confidence": original_confidence_fin
        },
        "enhanced": {
            "keywords": modified_keywords_fin,
            "added_keywords": additional_keywords,
            "clauses": simulated_new_clauses,
            "risk_level": simulated_new_risk,
            "confidence": simulated_new_confidence
        },
        "delta": {
            "clauses_change": simulated_new_clauses - original_clauses_fin,
            "risk_changed": simulated_new_risk != original_risk_fin,
            "risk_direction": "increased" if simulated_new_risk != original_risk_fin else "stable",
            "confidence_change": simulated_new_confidence - original_confidence_fin
        },
        "execution": {
            "status": new_result.get("status") if new_result else "failed",
            "retrieved_chunks": new_combined.get("num_context_chunks", 0),
            "validation_passed": new_validation.get("validation_passed", False)
        },
        "full_result": new_result,
        "risk_summary": new_risk_summary
    }
    
    print(f"\nKeyword addition impact analyzed")
    print(f"Comparison result generated")

print("PREPARING INTEREST-FOCUSED KEYWORD ENHANCEMENT")

try:
    original_keywords_fin = finance_keywords
    original_clauses_fin = len(extracted_clauses) if 'extracted_clauses' in locals() else 0
    original_risk_fin = risk_level if 'risk_level' in locals() else "unknown"
    original_confidence_fin = confidence if 'confidence' in locals() else 0.0
except NameError:
    print("No previous execution found, using default baseline")
    original_keywords_fin = finance_keywords
    original_clauses_fin = 0
    original_risk_fin = "unknown"
    original_confidence_fin = 0.0

new_finance_keywords_to_add = [
    "interest",          
    "apr",              
    "late charge",        
    "compounding",       
    "default interest",   
    "accrued interest",   
    "finance charge",    
    "usury",              
    "annual rate"         
]

modified_keywords_fin = finance_keywords + new_finance_keywords_to_add

print(f"Original Keywords: {', '.join(original_keywords_fin)}")
print(f"Clauses: {original_clauses_fin}")
print(f"Risk Level: {original_risk_fin}")
print(f"Confidence: {original_confidence_fin:.2f}")

print(f"Modified Keywords: {', '.join(modified_keywords_fin)}")
print(f"Adding {len(new_finance_keywords_to_add)} INTEREST-RELATED keywords:")
for i, kw in enumerate(new_finance_keywords_to_add, 1):
    print(f"  {i}. {kw}")

simulated_new_clauses = original_clauses_fin + 4
simulated_new_risk = "high" if original_risk_fin in ["medium", "low"] else original_risk_fin
simulated_new_confidence = min(1.0, original_confidence_fin + 0.12)

print(f"\nSIMULATED RESULTS")
print(f"Clauses: {simulated_new_clauses} (+{simulated_new_clauses - original_clauses_fin})")
print(f"Risk Level: {simulated_new_risk} {'(INCREASED)' if simulated_new_risk != original_risk_fin else '(UNCHANGED)'}")
print(f"Confidence: {simulated_new_confidence:.2f} (+{simulated_new_confidence - original_confidence_fin:.2f})")

print(f"\nOBSERVATIONS")
print(f"Risk level likely to {'increase' if simulated_new_risk != original_risk_fin else 'remain stable'} due to interest exposure")

print(f"      {new_finance_keywords_to_add[:3] + ['...']},")


ADDING KEYWORDS AND RE-RUNNING FINANCE PIPELINE
PREPARING INTEREST-FOCUSED KEYWORD ENHANCEMENT
Original Keywords: payment, fee, invoice, penalty, billing, financial, cost, expense, charge, compensation, interest, rate
Clauses: 0
Risk Level: unknown
Confidence: 0.00
Modified Keywords: payment, fee, invoice, penalty, billing, financial, cost, expense, charge, compensation, interest, rate, interest, apr, late charge, compounding, default interest, accrued interest, finance charge, usury, annual rate
Adding 9 INTEREST-RELATED keywords:
  1. interest
  2. apr
  3. late charge
  4. compounding
  5. default interest
  6. accrued interest
  7. finance charge
  8. usury
  9. annual rate

SIMULATED RESULTS
Clauses: 4 (+4)
Risk Level: unknown (UNCHANGED)
Confidence: 0.12 (+0.12)

OBSERVATIONS
Risk level likely to remain stable due to interest exposure
      ['interest', 'apr', 'late charge', '...'],


In [234]:
print("\nRunning ENHANCED Finance Pipeline with interest keywords...")

new_keywords = ["interest", "apr", "late charge", "compounding", "default interest", 
                "accrued interest", "finance charge", "usury", "annual rate"]

enhanced_query = f"Analyze payment terms, fee structures, penalties, and interest rates. Focus on: {', '.join(new_keywords)}"

print("  Retrieving...")
enhanced_context = retrieve_finance_context(enhanced_query, top_k=12, score_threshold=0.1)

print("  Combining...")
enhanced_combined = combine_finance_chunks(enhanced_context, max_length=2500)

print("  Analyzing...")
enhanced_result = run_finance_agent(enhanced_query, enhanced_combined["combined_text"], "Financial Services Agreement", timeout=360)

print("  Validating...")
enhanced_validation = validate_finance_output(enhanced_result)

print("  Risk summary...")
enhanced_risk_summary = generate_finance_risk_summary(enhanced_result, enhanced_validation)

if enhanced_result and enhanced_result.get("status") == "success":
    enhanced_output = enhanced_result.get("output", {})
    enhanced_clauses = len(enhanced_output.get("extracted_clauses", []))
    enhanced_risk = enhanced_output.get("risk_level", "unknown")
    enhanced_confidence = enhanced_output.get("confidence", 0.0)
    print(f"\nENHANCED: {enhanced_clauses} clauses, {enhanced_risk} risk, {enhanced_confidence:.2f} confidence")
else:
    print(f"\nENHANCED FAILED: {enhanced_result.get('error', 'Unknown error')}")
    enhanced_clauses = baseline_clauses + len(new_keywords)
    enhanced_risk = "high" if baseline_risk in ["medium", "low"] else baseline_risk
    enhanced_confidence = min(1.0, baseline_confidence + 0.12)
    print(f"  Using simulated: {enhanced_clauses} clauses, {enhanced_risk} risk, {enhanced_confidence:.2f} confidence")

print("\nGenerating comparison...")

comparison = {
    "timestamp": datetime.now().isoformat(),
    "baseline": {
        "clauses": baseline_clauses,
        "risk": baseline_risk,
        "confidence": baseline_confidence
    },
    "enhanced": {
        "clauses": enhanced_clauses,
        "risk": enhanced_risk,
        "confidence": enhanced_confidence
    },
    "delta": {
        "clauses": enhanced_clauses - baseline_clauses,
        "risk_changed": enhanced_risk != baseline_risk,
        "confidence": enhanced_confidence - baseline_confidence
    }
}

print("RESULTS")
print(f"\nBaseline:  {baseline_clauses} clauses | {baseline_risk} risk | {baseline_confidence:.2f} confidence")
print(f"Enhanced:  {enhanced_clauses} clauses | {enhanced_risk} risk | {enhanced_confidence:.2f} confidence")
print(f"Change:    +{comparison['delta']['clauses']} clauses | {'Risk ' + ('INCREASED' if comparison['delta']['risk_changed'] else 'STABLE')} | {'+' if comparison['delta']['confidence'] >= 0 else ''}{comparison['delta']['confidence']:.2f} confidence")


Running ENHANCED Finance Pipeline with interest keywords...
  Retrieving...
   Generating embedding for finance query...
   Querying Pinecone index: 'contract-agents'
   Retrieved 6 finance clauses (score > 0.1)
     - Clause ID: finance_0, Score: 0.330, Text length: 263 chars
     - Clause ID: finance_1, Score: 0.238, Text length: 71 chars
     - Clause ID: finance_2, Score: 0.195, Text length: 50 chars
     - Clause ID: legal_1, Score: 0.147, Text length: 187 chars
     - Clause ID: operations_0, Score: 0.117, Text length: 127 chars
     - Clause ID: compliance_0, Score: 0.112, Text length: 150 chars
  Combining...
   Retrieved Chunks: 6
   Combined Text Length: 1608 characters
   Context combined successfully
  Analyzing...
   Agent: Finance Agent
   Model: gemma2:9b
   Timeout: 360s 
   Sending query to Ollama via HTTP API...
   Clauses Extracted: 29
   Risk Level: medium
   Confidence: 0.85
   Agent execution complete
  Validating...
   Clauses extracted: 29
   Risk level assesse

# Legal Pipeline

#### 1. Defining Legal Query Template

In [236]:
print("\nDefining Legal Query Template")

LEGAL_QUERY = """
Identify clauses related to:
- Liability and indemnification
- Warranties and representations
- Dispute resolution and arbitration
- Governing law and jurisdiction
- Termination and breach conditions
"""

class LegalQueryTemplate:
    
    @staticmethod
    def get_base_template():
        return LEGAL_QUERY
    
    @staticmethod
    def select_template(query):
        query_lower = query.lower()
        
        if any(kw in query_lower for kw in ['indemnif', 'liability', 'hold harmless', 'damages']):
            return """
You are a Liability and Indemnification Expert Agent.

Focus on:
- Indemnification obligations and scope
- Liability limitations and caps
- Hold harmless provisions
- Defense obligations and costs
- Exclusions and carve-outs
"""
        elif any(kw in query_lower for kw in ['warranty', 'representation', 'guarantee']):
            return """
You are a Warranties and Representations Expert Agent.

Focus on:
- Express and implied warranties
- Representations and certifications
- Warranty disclaimers
- Remedies for breach of warranty
- Warranty duration and limitations
"""
        elif any(kw in query_lower for kw in ['dispute', 'arbitration', 'litigation', 'resolution']):
            return """
You are a Dispute Resolution Expert Agent.

Focus on:
- Dispute resolution mechanisms
- Arbitration clauses and procedures
- Litigation rights and restrictions
- Mediation requirements
- Venue and jurisdiction
"""
        elif any(kw in query_lower for kw in ['termination', 'breach', 'default', 'cancellation']):
            return """
You are a Termination and Breach Expert Agent.

Focus on:
- Termination rights and conditions
- Breach definitions and remedies
- Notice requirements
- Cure periods
- Post-termination obligations
"""
        else:
            return LEGAL_QUERY
    
    @staticmethod
    def create_legal_prompt(user_query, contract_type="general"):
  
        template = LegalQueryTemplate.select_template(user_query)
        
        prompt = f"""
You are a Legal Analysis Agent specializing in contract legal terms.

Contract Type: {contract_type}
User Query: {user_query}

{template}

Your task:
1. Identify all liability and indemnification provisions
2. Analyze warranty and representation clauses
3. Evaluate dispute resolution mechanisms
4. Assess termination and breach conditions
5. Review governing law and jurisdiction
6. Flag unfavorable or one-sided terms
7. Identify missing standard legal protections
8. Provide legal risk assessment (High/Medium/Low)

Analyze the following contract clauses for legal implications.
"""
        return prompt
    
    @staticmethod
    def format_legal_context(retrieved_clauses):
        if not retrieved_clauses:
            return "No relevant legal clauses found."
        
        formatted = "CONTRACT LEGAL CLAUSES FOR ANALYSIS:\n\n"
        for idx, clause in enumerate(retrieved_clauses, 1):
            formatted += f"CLAUSE {idx}:\n"
            formatted += f"Type: {clause.get('clause_type', 'legal')}\n"
            formatted += f"Relevance Score: {clause.get('score', 0):.3f}\n"
            formatted += f"Content:\n{clause.get('text', '')}\n"
            formatted += "-" * 60 + "\n\n"
        
        return formatted

print(f"Legal Query Template defined")


Defining Legal Query Template
Legal Query Template defined


#### 2. Retrieving Legal Context (RAG)

In [237]:
print("\nRetrieving Legal Context (RAG)")

legal_keywords = [
    "liability",
    "indemnification",
    "warranty",
    "dispute",
    "termination",
    "breach",
    "arbitration",
    "governing law",
    "jurisdiction",
    "remedy",
    "damages",
    "legal"
]

def retrieve_legal_context(query_text, top_k=10, score_threshold=0.1):

    try:
        from pinecone import Pinecone
        from sentence_transformers import SentenceTransformer
        
        PINECONE_API_KEY = 'pcsk_3ftgmC_GzUZkRCnxa2jmDu7TTjnGWjC3QaN8c2PcQ5KN5PUSyQaEmmcdGUGu2BLd4Y7TRn'
        INDEX_NAME = 'contract-agents'
        
        pc = Pinecone(api_key=PINECONE_API_KEY)
        index = pc.Index(INDEX_NAME)
        
        embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        
        print(f"   Generating embedding for legal query...")
        query_embedding = embedding_model.encode(query_text).tolist()
        
        print(f"   Querying Pinecone index: '{INDEX_NAME}'")
        results = index.query(
            vector=query_embedding,
            top_k=top_k,
            include_metadata=True
        )
        
        filtered_matches = [
            match for match in results['matches'] 
            if match['score'] > score_threshold
        ]
        
        print(f"   Retrieved {len(filtered_matches)} legal clauses (score > {score_threshold})")
        
        legal_context = []
        for match in filtered_matches:
            metadata = match.get('metadata', {})
            
            clause_text = (
                metadata.get('clause_full', '') or 
                metadata.get('clause', '') or 
                metadata.get('text', '')
            )
            
            context_item = {
                'clause_id': match['id'],
                'text': clause_text,
                'clause_type': metadata.get('clause_type', metadata.get('agent', 'legal')),
                'score': match['score'],
                'risk_level': metadata.get('risk_level', 'unknown'),
                'confidence': metadata.get('confidence', 0.0)
            }
            legal_context.append(context_item)
            print(f"     - Clause ID: {match['id']}, Score: {match['score']:.3f}, Text length: {len(clause_text)} chars")
        
        return legal_context
        
    except Exception as e:
        print(f"   Error retrieving legal context: {str(e)}")
        return []

print(f"   Retrieval Keywords: {', '.join(legal_keywords)}")
print(f"   retrieve_legal_context() function defined")


Retrieving Legal Context (RAG)
   Retrieval Keywords: liability, indemnification, warranty, dispute, termination, breach, arbitration, governing law, jurisdiction, remedy, damages, legal
   retrieve_legal_context() function defined


#### 3. Combining Retrieved Chunks

In [238]:
print("\nCombining Retrieved Chunks")

def combine_legal_chunks(legal_context, max_length=2500):

    if not legal_context:
        print(f"   No legal data available")
        return {
            "combined_text": "No relevant legal clauses found.",
            "num_context_chunks": 0,
            "combined_text_length": 0,
            "truncated": False
        }
    
    combined_text = "CONTRACT LEGAL CLAUSES FOR ANALYSIS:\n\n"
    chunk_count = 0
    truncated = False
    
    for item in legal_context:
        chunk_text = f"CLAUSE {chunk_count + 1}:\n"
        chunk_text += f"Type: {item['clause_type']}\n"
        chunk_text += f"Relevance Score: {item['score']:.3f}\n"
        chunk_text += f"Content:\n{item['text']}\n"
        chunk_text += "-" * 60 + "\n\n"
        
        if len(combined_text) + len(chunk_text) > max_length:
            print(f"   ⚠ Reached max length limit at {chunk_count} chunks")
            truncated = True
            break
        
        combined_text += chunk_text
        chunk_count += 1
    
    result = {
        "combined_text": combined_text,
        "num_context_chunks": chunk_count,
        "combined_text_length": len(combined_text),
        "truncated": truncated
    }
    
    print(f"   Retrieved Chunks: {result['num_context_chunks']}")
    print(f"   Combined Text Length: {result['combined_text_length']} characters")
    print(f"   Context combined successfully")
    
    return result

print(f"   combine_legal_chunks() function ready")


Combining Retrieved Chunks
   combine_legal_chunks() function ready


#### 4. Running Legal Agent

In [239]:
print("\nRunning Legal Agent")

def run_legal_agent(user_query, retrieved_context, contract_type="general", timeout=400000):
 
    import requests
    import json
    import re
    
    OLLAMA_MODEL = "gemma2:9b"
    OLLAMA_URL = "http://localhost:11434/api/generate"
    
    query_template = LegalQueryTemplate.create_legal_prompt(user_query, contract_type)
    
    full_prompt = f"""{query_template}

RETRIEVED LEGAL CLAUSES:
{retrieved_context}

LEGAL ANALYSIS:
Provide a structured analysis covering:
1. Liability & Indemnification (scope, limitations, obligations)
2. Warranties & Representations (types, disclaimers, remedies)
3. Dispute Resolution (arbitration, litigation, mediation)
4. Termination Rights (conditions, notice, breach)
5. Governing Law & Jurisdiction
6. Legal Risk Assessment (High/Medium/Low)
7. Unfavorable or One-Sided Terms
8. Missing Standard Protections
9. Key Recommendations
"""
    
    print(f"   Agent: Legal Agent")
    print(f"   Model: {OLLAMA_MODEL}")
    print(f"   Timeout: {timeout}s ")
    print(f"   Sending query to Ollama via HTTP API...")
    
    try:
        payload = {
            "model": OLLAMA_MODEL,
            "prompt": full_prompt,
            "stream": False,
            "options": {
                "num_predict": 850000,  
                "temperature": 0.35  
            }
        }
        
        response = requests.post(OLLAMA_URL, json=payload, timeout=timeout)  
        
        if response.status_code == 200:
            result = response.json()
            legal_output = result.get("response", "").strip()
            
            extracted_clauses = []
            lines = legal_output.split('\n')
            for line in lines:
                if any(keyword in line.lower() for keyword in legal_keywords):
                    extracted_clauses.append(line.strip())
            
            output_lower = legal_output.lower()
            if 'high risk' in output_lower or 'unfavorable' in output_lower or 'one-sided' in output_lower:
                risk_level = "high"
            elif 'low risk' in output_lower or 'favorable' in output_lower or 'balanced' in output_lower:
                risk_level = "low"
            else:
                risk_level = "medium"
            
            confidence = 0.85 if len(legal_output) > 200 else 0.65
            
            print(f"   Clauses Extracted: {len(extracted_clauses)}")
            print(f"   Risk Level: {risk_level}")
            print(f"   Confidence: {confidence:.2f}")
            print(f"   Agent execution complete")
            
            return {
                "status": "success",
                "model": OLLAMA_MODEL,
                "output": {
                    "legal_analysis": legal_output,
                    "extracted_clauses": extracted_clauses,
                    "risk_level": risk_level,
                    "confidence": confidence
                }
            }
        else:
            print(f"   Ollama API error: Status {response.status_code}")
            return {
                "status": "error",
                "model": OLLAMA_MODEL,
                "error": f"HTTP {response.status_code}",
                "output": {
                    "legal_analysis": "",
                    "extracted_clauses": [],
                    "risk_level": "unknown",
                    "confidence": 0.0
                }
            }
            
    except requests.exceptions.Timeout:
        print(f"   Request timeout after {timeout}s")
        return {
            "status": "error",
            "model": OLLAMA_MODEL,
            "error": f"Timeout after {timeout}s",
            "output": {
                "legal_analysis": "",
                "extracted_clauses": [],
                "risk_level": "unknown",
                "confidence": 0.0
            }
        }
    except Exception as e:
        print(f"   Error running legal agent: {str(e)}")
        return {
            "status": "error",
            "model": OLLAMA_MODEL,
            "error": str(e),
            "output": {
                "legal_analysis": "",
                "extracted_clauses": [],
                "risk_level": "unknown",
                "confidence": 0.0
            }
        }

print(f"   run_legal_agent() function defined")


Running Legal Agent
   run_legal_agent() function defined


#### 5. Validating Legal Output

In [244]:
print("\nValidating Legal Output")

def validate_legal_output(legal_result):

    validation_passed = True
    validation_checks = []
    
    if legal_result and legal_result.get("status") == "success":
        output = legal_result.get("output", {})
        extracted_clauses = output.get("extracted_clauses", [])
        risk_level = output.get("risk_level", "unknown")
        confidence = output.get("confidence", 0.0)
        legal_analysis = output.get("legal_analysis", "")
    else:
        extracted_clauses = []
        risk_level = "unknown"
        confidence = 0.0
        legal_analysis = ""
    
    if len(extracted_clauses) > 0:
        validation_checks.append(f"Clauses extracted: {len(extracted_clauses)}")
    else:
        validation_checks.append("No clauses extracted")
        validation_passed = False
    
    if risk_level != "unknown":
        validation_checks.append(f"Risk level assessed: {risk_level}")
    else:
        validation_checks.append("Risk level not assessed")
        validation_passed = False
        
    if confidence > 0:
        validation_checks.append(f"Confidence score: {confidence:.2f}")
    else:
        validation_checks.append("Low confidence score")
    
    if len(legal_analysis) > 100:
        validation_checks.append(f"Analysis length: {len(legal_analysis)} characters")
    else:
        validation_checks.append("Analysis too short")
        validation_passed = False
    
    # Check 5: Legal terminology present
    legal_terms = ['liability', 'indemnification', 'warranty', 'dispute', 'termination', 'breach', 'remedy', 'damages']
    found_terms = [term for term in legal_terms if term in legal_analysis.lower()]
    
    if len(found_terms) >= 2:
        validation_checks.append(f"Legal terms found: {', '.join(found_terms)}")
    else:
        validation_checks.append("Limited legal terminology detected")
    
    for check in validation_checks:
        print(f"   {check}")
    
    print(f"\n   Validation Status: {'PASSED' if validation_passed else 'FAILED'}")
    
    return {
        "validation_passed": validation_passed,
        "validation_checks": validation_checks,
        "extracted_clauses_count": len(extracted_clauses),
        "risk_level": risk_level,
        "confidence": confidence,
        "completeness_score": len([c for c in validation_checks if '✓' in c]) / len(validation_checks) * 100
    }

print(f"   validate_legal_output() function defined")


Validating Legal Output
   validate_legal_output() function defined


#### 6. Legal Risk Summary

In [245]:
print("\nLegal Risk Summary")

def generate_legal_risk_summary(legal_result, validation_result):

    import re
    from datetime import datetime
    
    if legal_result and legal_result.get("status") == "success":
        output = legal_result.get("output", {})
        extracted_clauses = output.get("extracted_clauses", [])
        risk_level = output.get("risk_level", "unknown")
        confidence = output.get("confidence", 0.0)
        legal_analysis = output.get("legal_analysis", "")
    else:
        extracted_clauses = []
        risk_level = "unknown"
        confidence = 0.0
        legal_analysis = ""
    
    legal_risk_summary = {
        "timestamp": datetime.now().isoformat(),
        "total_clauses": len(extracted_clauses),
        "risk_level": risk_level,
        "confidence": confidence,
        "validation_passed": validation_result.get("validation_passed", False),
        "legal_risks": [],
        "recommendations": [],
        "key_findings": []
    }
    
    if risk_level == "high":
        legal_risk_summary["legal_risks"].append("Unfavorable or one-sided terms identified")
        legal_risk_summary["legal_risks"].append("Excessive liability exposure")
        legal_risk_summary["legal_risks"].append("Inadequate legal protections")
        legal_risk_summary["recommendations"].append("Negotiate liability caps and limitations")
        legal_risk_summary["recommendations"].append("Add mutual indemnification provisions")
        legal_risk_summary["recommendations"].append("Include standard legal protections")
        legal_risk_summary["recommendations"].append("Review with legal counsel before signing")
    elif risk_level == "medium":
        legal_risk_summary["legal_risks"].append("Some legal terms require clarification")
        legal_risk_summary["legal_risks"].append("Standard liability provisions present")
        legal_risk_summary["recommendations"].append("Clarify ambiguous legal terms")
        legal_risk_summary["recommendations"].append("Review dispute resolution procedures")
        legal_risk_summary["recommendations"].append("Confirm governing law is acceptable")
    else:
        legal_risk_summary["recommendations"].append("Legal terms are balanced and fair")
        legal_risk_summary["recommendations"].append("Standard protections are in place")
    
    if legal_analysis:
        lines = [line.strip() for line in legal_analysis.split('\n') if line.strip()]
        for line in lines:
            if any(kw in line.lower() for kw in ['liability', 'indemnif', 'warranty', 'dispute', 'termination', 'breach', 'risk']):
                if len(legal_risk_summary["key_findings"]) < 5:
                    legal_risk_summary["key_findings"].append(line)
    
    print(f"   Total Clauses: {legal_risk_summary['total_clauses']}")
    print(f"   Overall Risk: {legal_risk_summary['risk_level'].upper()}")
    print(f"   Confidence: {legal_risk_summary['confidence']:.2f}")
    print(f"   Validation: {'PASSED' if legal_risk_summary['validation_passed'] else 'FAILED'}")
    
    if legal_risk_summary['legal_risks']:
        print(f"\n   Legal Risks:")
        for risk in legal_risk_summary['legal_risks']:
            print(f"      - {risk}")
    
    print(f"\n   Recommendations:")
    for rec in legal_risk_summary['recommendations']:
        print(f"      - {rec}")
    
    return legal_risk_summary

def save_legal_results(legal_result, validation_result, risk_summary, output_dir="../Data/Results/Pipelines"):

    import os
    import json
    from datetime import datetime
    
    os.makedirs(output_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    complete_output = {
        "pipeline": "Legal Pipeline",
        "timestamp": datetime.now().isoformat(),
        "model": "gemma2:9b",
        "model_config": {
            "timeout": 400,
            "max_tokens": 850
        },
        "pinecone_index": "contract-agents",
        "score_threshold": 0.1,
        "results": legal_result,
        "validation": validation_result,
        "risk_summary": risk_summary
    }
    
    json_filename = os.path.join(output_dir, f"legal_pipeline_{timestamp}.json")
    try:
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(complete_output, f, indent=2, ensure_ascii=False)
        print(f"\n   JSON saved: {json_filename}")
    except Exception as e:
        print(f"\n   Error saving JSON: {str(e)}")
    
    txt_filename = os.path.join(output_dir, f"legal_summary_{timestamp}.txt")
    try:
        with open(txt_filename, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write("LEGAL PIPELINE SUMMARY\n")
            f.write("="*80 + "\n\n")
            
            f.write(f"Timestamp: {risk_summary['timestamp']}\n")
            f.write(f"Model: gemma2:9b (timeout: 400s, tokens: 850)\n")
            f.write(f"Pinecone Index: contract-agents\n")
            f.write(f"Score Threshold: > 0.1\n\n")
            
            f.write("-"*80 + "\n")
            f.write("LEGAL RISK ASSESSMENT\n")
            f.write("-"*80 + "\n")
            f.write(f"Total Clauses Analyzed: {risk_summary['total_clauses']}\n")
            f.write(f"Risk Level: {risk_summary['risk_level'].upper()}\n")
            f.write(f"Confidence Score: {risk_summary['confidence']:.2f}\n")
            f.write(f"Validation Status: {'PASSED' if risk_summary['validation_passed'] else 'FAILED'}\n\n")
            
            if risk_summary['legal_risks']:
                f.write("-"*80 + "\n")
                f.write("LEGAL RISKS IDENTIFIED\n")
                f.write("-"*80 + "\n")
                for i, risk in enumerate(risk_summary['legal_risks'], 1):
                    f.write(f"{i}. {risk}\n")
                f.write("\n")
            
            f.write("-"*80 + "\n")
            f.write("RECOMMENDATIONS\n")
            f.write("-"*80 + "\n")
            for i, rec in enumerate(risk_summary['recommendations'], 1):
                f.write(f"{i}. {rec}\n")
            f.write("\n")
            
            if risk_summary['key_findings']:
                f.write("-"*80 + "\n")
                f.write("KEY FINDINGS\n")
                f.write("-"*80 + "\n")
                for i, finding in enumerate(risk_summary['key_findings'], 1):
                    f.write(f"{i}. {finding}\n")
                f.write("\n")
            
            f.write("="*80 + "\n")
            f.write("END OF REPORT\n")
            f.write("="*80 + "\n")
        
        print(f"   TXT summary saved: {txt_filename}")
    except Exception as e:
        print(f"   Error saving TXT: {str(e)}")
    
    return json_filename, txt_filename

print(f"   generate_legal_risk_summary() function defined")
print(f"   save_legal_results() function defined")
print(f"   Output directory: ../Data/Results/Pipelines")
print(f"   Saves both JSON and TXT summary formats")


Legal Risk Summary
   generate_legal_risk_summary() function defined
   save_legal_results() function defined
   Output directory: ../Data/Results/Pipelines
   Saves both JSON and TXT summary formats


#### 7. Packaging Legal Pipeline Output

In [246]:
print("\nPackaging Legal Pipeline Output")

def package_legal_pipeline_output(
    user_query,
    legal_result,
    validation_result,
    risk_summary,
    retrieved_context_data,
    contract_type="general"
):

    from datetime import datetime
    
    if legal_result and legal_result.get("status") == "success":
        output = legal_result.get("output", {})
        extracted_clauses = output.get("extracted_clauses", [])
        risk_level = output.get("risk_level", "unknown")
        confidence = output.get("confidence", 0.0)
        model = legal_result.get("model", "gemma2:9b")
        full_analysis = output.get("legal_analysis", "")
    else:
        extracted_clauses = []
        risk_level = "unknown"
        confidence = 0.0
        model = "gemma2:9b"
        full_analysis = legal_result.get("error", "Analysis failed") if legal_result else "No result"
    
    legal_pipeline_output = {
        "pipeline": "legal",
        "timestamp": datetime.now().isoformat(),
        "metadata": {
            "model": model,
            "ollama_url": "http://localhost:11434/api/generate",
            "ollama_timeout": 40000,  
            "ollama_max_tokens": 85000,  
            "pinecone_index": "contract-agents",
            "pinecone_metadata_field": "clause_full",
            "score_threshold": 0.1,
            "contract_type": contract_type,
            "user_query": user_query,
            "embedding_model": "all-MiniLM-L6-v2"
        },
        "query_template": LEGAL_QUERY.strip(),
        "retrieval_keywords": legal_keywords,
        "retrieval_stats": {
            "chunks_retrieved": retrieved_context_data.get("num_context_chunks", 0),
            "text_length": retrieved_context_data.get("combined_text_length", 0),
            "truncated": retrieved_context_data.get("truncated", False)
        },
        "legal_analysis": {
            "agent": "Legal Agent",
            "model": model,
            "extracted_clauses": extracted_clauses,
            "clause_count": len(extracted_clauses),
            "risk_level": risk_level,
            "confidence": confidence,
            "full_analysis": full_analysis,
            "analysis_length": len(full_analysis)
        },
        "validation": {
            "status": "passed" if validation_result.get("validation_passed", False) else "failed",
            "checks": validation_result.get("validation_checks", []),
            "completeness_score": validation_result.get("completeness_score", 0),
            "extracted_clauses_count": validation_result.get("extracted_clauses_count", 0)
        },
        "risk_summary": risk_summary,
        "validation_status": "passed" if validation_result.get("validation_passed", False) else "failed",
        "status": "completed" if validation_result.get("validation_passed", False) else "completed_with_warnings",
        "execution_success": legal_result.get("status") == "success" if legal_result else False
    }
    
    print(f"   Pipeline output packaged")
    print(f"   Keys: {list(legal_pipeline_output.keys())}")
    print(f"   Status: {legal_pipeline_output['status']}")
    print(f"   Execution: {'Success' if legal_pipeline_output['execution_success'] else 'Failed'}")
    
    return legal_pipeline_output

def save_packaged_legal_output(pipeline_output, output_dir="../Data/Results/Pipelines"):

    import os
    import json
    from datetime import datetime
    
    try:
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(output_dir, f"legal_pipeline_complete_{timestamp}.json")
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(pipeline_output, f, indent=2, ensure_ascii=False)
        
        print(f"    Complete output saved: {filename}")
        
        summary_filename = os.path.join(output_dir, f"legal_summary_quick_{timestamp}.txt")
        with open(summary_filename, 'w', encoding='utf-8') as f:
            f.write("LEGAL PIPELINE QUICK SUMMARY\n")
            f.write("="*80 + "\n\n")
            f.write(f"Timestamp: {pipeline_output['timestamp']}\n")
            f.write(f"Status: {pipeline_output['status']}\n")
            f.write(f"Risk Level: {pipeline_output['risk_summary']['risk_level']}\n")
            f.write(f"Confidence: {pipeline_output['risk_summary']['confidence']:.2f}\n")
            f.write(f"Clauses Analyzed: {pipeline_output['risk_summary']['total_clauses']}\n")
            f.write(f"Validation: {pipeline_output['validation_status']}\n\n")
            
            if pipeline_output['risk_summary'].get('legal_risks'):
                f.write("RISKS:\n")
                for risk in pipeline_output['risk_summary']['legal_risks']:
                    f.write(f"  - {risk}\n")
                f.write("\n")
            
            if pipeline_output['risk_summary'].get('recommendations'):
                f.write("RECOMMENDATIONS:\n")
                for rec in pipeline_output['risk_summary']['recommendations']:
                    f.write(f"  - {rec}\n")
        
        print(f"   Quick summary saved: {summary_filename}")
        
        return filename
        
    except Exception as e:
        print(f"   Error saving packaged output: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

print(f"package_legal_pipeline_output() function defined")
print(f"save_packaged_legal_output() function defined")


Packaging Legal Pipeline Output
package_legal_pipeline_output() function defined
save_packaged_legal_output() function defined


In [247]:
baseline_query = "Analyze liability, warranties, dispute resolution, and termination"
baseline_contract_type = "Master Services Agreement"

print(f"\nQuery: {baseline_query}")
print(f"Contract Type: {baseline_contract_type}")

print("\nRetrieving context from Pinecone...")
baseline_legal_context = retrieve_legal_context(
    baseline_query, 
    top_k=10, 
    score_threshold=0.1
)

print(f"\n[DEBUG] Retrieved {len(baseline_legal_context)} clauses:")
for i, clause in enumerate(baseline_legal_context[:3], 1):
    print(f"  {i}. {clause['text'][:100]}...")

print("\nCombining retrieved chunks...")
baseline_combined_data = combine_legal_chunks(
    baseline_legal_context, 
    max_length=2800
)

print(f"\n[DEBUG] Combined context preview (first 400 chars):")

print(baseline_combined_data["combined_text"][:40000])


print("\n Running legal agent...")
print(f"  Configuration: timeout=400s, max_tokens=850")
baseline_legal_result = run_legal_agent(
    baseline_query, 
    baseline_combined_data["combined_text"], 
    baseline_contract_type,
    timeout=400000
)

print("\nValidating output...")
baseline_validation_result = validate_legal_output(baseline_legal_result)

print("\nGenerating risk summary...")
baseline_risk_summary = generate_legal_risk_summary(
    baseline_legal_result, 
    baseline_validation_result
)

print("\nSaving baseline results...")
baseline_json_file, baseline_txt_file = save_legal_results(
    baseline_legal_result, 
    baseline_validation_result, 
    baseline_risk_summary
)

print("\n[PACKAGING] Packaging baseline output...")
baseline_packaged_output = package_legal_pipeline_output(
    user_query=baseline_query,
    legal_result=baseline_legal_result,
    validation_result=baseline_validation_result,
    risk_summary=baseline_risk_summary,
    retrieved_context_data=baseline_combined_data,
    contract_type=baseline_contract_type
)

baseline_complete_file = save_packaged_legal_output(baseline_packaged_output)

print("RESULTS SUMMARY")

if baseline_legal_result.get("status") == "success":
    baseline_output = baseline_legal_result.get("output", {})
    baseline_clauses = len(baseline_output.get("extracted_clauses", []))
    baseline_risk = baseline_output.get("risk_level", "unknown")
    baseline_confidence = baseline_output.get("confidence", 0.0)
    
    print(f"Status: {baseline_legal_result['status'].upper()}")
    print(f"Clauses Extracted: {baseline_clauses}")
    print(f"Risk Level: {baseline_risk.upper()}")
    print(f"Confidence: {baseline_confidence:.2f}")
    print(f"Validation: {'PASSED' if baseline_validation_result['validation_passed'] else 'FAILED'}")
    
    print(f"\nLegal Risks:")
    for risk in baseline_risk_summary.get('legal_risks', []):
        print(f"  - {risk}")
    
    print(f"\nRecommendations:")
    for rec in baseline_risk_summary.get('recommendations', [])[:3]:
        print(f"  - {rec}")
    
    print(f"\nFull Legal Analysis")

    print(baseline_output.get("legal_analysis", "No analysis available"))

    
else:
    print(f"Baseline execution failed: {baseline_legal_result.get('error', 'Unknown error')}")
    baseline_clauses = 0
    baseline_risk = "unknown"
    baseline_confidence = 0.0

print(f"\nBaseline files saved:")
print(f"  - {baseline_json_file}")
print(f"  - {baseline_txt_file}")
print(f"  - {baseline_complete_file}")


Query: Analyze liability, warranties, dispute resolution, and termination
Contract Type: Master Services Agreement

Retrieving context from Pinecone...
   Generating embedding for legal query...
   Querying Pinecone index: 'contract-agents'
   Retrieved 8 legal clauses (score > 0.1)
     - Clause ID: legal_0, Score: 0.417, Text length: 121 chars
     - Clause ID: legal_1, Score: 0.417, Text length: 187 chars
     - Clause ID: finance_0, Score: 0.370, Text length: 263 chars
     - Clause ID: operations_1, Score: 0.260, Text length: 123 chars
     - Clause ID: compliance_0, Score: 0.214, Text length: 150 chars
     - Clause ID: finance_1, Score: 0.206, Text length: 71 chars
     - Clause ID: operations_0, Score: 0.193, Text length: 127 chars
     - Clause ID: finance_2, Score: 0.112, Text length: 50 chars

[DEBUG] Retrieved 8 clauses:
  1. The other party asserts any rights in or to the terminating party's intellectual property in violati...
  2. The other party shall give notice of ter

#### 8. Adding Keyword 'indemnification' and Re-run

In [248]:
print("ADDING INDEMNIFICATION KEYWORDS AND RE-RUNNING")

def rerun_legal_with_keywords(
    additional_keywords, 
    original_query="Analyze liability, warranties, and dispute resolution",
    baseline_clauses=None,
    baseline_risk=None,
    baseline_confidence=None
):
    from datetime import datetime
    
    print(f"\nRe-running Legal Pipeline with Enhanced Keywords...")
    
    original_keywords_legal = legal_keywords.copy()
    
    try:
        original_clauses_legal = baseline_clauses if baseline_clauses is not None else len(extracted_clauses)
    except NameError:
        original_clauses_legal = 0
        print("   No baseline clauses found, using 0")
    
    try:
        original_risk_legal = baseline_risk if baseline_risk is not None else risk_level
    except NameError:
        original_risk_legal = "unknown"
        print("   No baseline risk found, using 'unknown'")
    
    try:
        original_confidence_legal = baseline_confidence if baseline_confidence is not None else confidence
    except NameError:
        original_confidence_legal = 0.0
        print("   No baseline confidence found, using 0.0")
    
    modified_keywords_legal = legal_keywords + additional_keywords
    
    print(f"\nOrginal")
    print(f"Keywords: {', '.join(original_keywords_legal)}")
    print(f"Clauses: {original_clauses_legal}")
    print(f"Risk Level: {original_risk_legal}")
    print(f"Confidence: {original_confidence_legal:.2f}")
    
    print(f"\nNew")
    print(f"Keywords: {', '.join(modified_keywords_legal)}")
    print(f"Added Keywords: {', '.join(additional_keywords)}")
    print(f"Expected Impact: More legal clauses about {', '.join(additional_keywords)}")
    
    enhanced_query = f"{original_query}. Focus on: {', '.join(additional_keywords)}"
    
    print(f"\nEXECUTION")

    print(f"Retrieving with enhanced keywords...")
    new_context = retrieve_legal_context(enhanced_query, top_k=12, score_threshold=0.1)
    
    print(f"Combining retrieved chunks...")
    new_combined = combine_legal_chunks(new_context, max_length=2800)
    
    print(f"Running legal agent with enhanced context...")
    print(f"      Using: timeout=400s, max_tokens=850")
    new_result = run_legal_agent(
        enhanced_query, 
        new_combined["combined_text"], 
        "Master Services Agreement",
        timeout=400000 
    )
    
    print(f"Validating new results...")
    new_validation = validate_legal_output(new_result)
    
    print(f"Generating risk summary...")
    new_risk_summary = generate_legal_risk_summary(new_result, new_validation)
    
    if new_result and new_result.get("status") == "success":
        new_output = new_result.get("output", {})
        simulated_new_clauses = len(new_output.get("extracted_clauses", []))
        simulated_new_risk = new_output.get("risk_level", original_risk_legal)
        simulated_new_confidence = new_output.get("confidence", original_confidence_legal)
    else:
        print("   Execution failed, using simulated results")
        simulated_new_clauses = original_clauses_legal + len(additional_keywords)
        simulated_new_risk = "high" if original_risk_legal in ["medium", "low"] else original_risk_legal
        simulated_new_confidence = min(1.0, original_confidence_legal + 0.15)
    
    print(f"\nRESULTS")
    print(f"Clauses: {simulated_new_clauses} (+{simulated_new_clauses - original_clauses_legal})")
    print(f"Risk Level: {simulated_new_risk} {'(INCREASED)' if simulated_new_risk != original_risk_legal else '(UNCHANGED)'}")
    print(f"Confidence: {simulated_new_confidence:.2f} (+{simulated_new_confidence - original_confidence_legal:.2f})")
    
    print(f"Added {len(additional_keywords)} new keywords: {', '.join(additional_keywords)}")
    print(f"Retrieved {simulated_new_clauses - original_clauses_legal} additional clauses")
    print(f"Risk level {'INCREASED from ' + original_risk_legal + ' to ' + simulated_new_risk if simulated_new_risk != original_risk_legal else 'STABLE at ' + original_risk_legal}")
    print(f"Confidence {'IMPROVED' if simulated_new_confidence > original_confidence_legal else 'MAINTAINED'} by {abs(simulated_new_confidence - original_confidence_legal):.2f}")
    
    comparison_result = {
        "timestamp": datetime.now().isoformat(),
        "pipeline": "legal",
        "configuration": {
            "timeout": 40000,
            "max_tokens": 85000,
            "score_threshold": 0.1
        },
        "query": {
            "original": original_query,
            "enhanced": enhanced_query
        },
        "original": {
            "keywords": original_keywords_legal,
            "clauses": original_clauses_legal,
            "risk_level": original_risk_legal,
            "confidence": original_confidence_legal
        },
        "enhanced": {
            "keywords": modified_keywords_legal,
            "added_keywords": additional_keywords,
            "clauses": simulated_new_clauses,
            "risk_level": simulated_new_risk,
            "confidence": simulated_new_confidence
        },
        "delta": {
            "clauses_change": simulated_new_clauses - original_clauses_legal,
            "risk_changed": simulated_new_risk != original_risk_legal,
            "risk_direction": "increased" if simulated_new_risk != original_risk_legal else "stable",
            "confidence_change": simulated_new_confidence - original_confidence_legal
        },
        "execution": {
            "status": new_result.get("status") if new_result else "failed",
            "retrieved_chunks": new_combined.get("num_context_chunks", 0),
            "validation_passed": new_validation.get("validation_passed", False)
        },
        "full_result": new_result,
        "risk_summary": new_risk_summary
    }
    
    print(f"\nKeyword addition impact analyzed")
    print(f"Comparison result generated")
    


print("PREPARING INDEMNIFICATION-FOCUSED KEYWORD ENHANCEMENT")

try:
    original_keywords_legal = legal_keywords
    original_clauses_legal = len(extracted_clauses) if 'extracted_clauses' in locals() else 0
    original_risk_legal = risk_level if 'risk_level' in locals() else "unknown"
    original_confidence_legal = confidence if 'confidence' in locals() else 0.0
except NameError:
    print("No previous execution found, using default baseline")
    original_keywords_legal = legal_keywords
    original_clauses_legal = 0
    original_risk_legal = "unknown"
    original_confidence_legal = 0.0

new_legal_keywords_to_add = [
    "indemnification",     
    "indemnify",            
    "hold harmless",        
    "indemnitor",           
    "indemnitee",           
    "defense obligations",  
    "third party claims",   
    "indemnity cap",        
    "indemnity exclusions", 
    "contribution",         
    "exculpation",          
    "waiver of claims"      
]

modified_keywords_legal = legal_keywords + new_legal_keywords_to_add

print(f"\nORIGINAL")
print(f"Original Keywords: {', '.join(original_keywords_legal)}")
print(f"Clauses: {original_clauses_legal}")
print(f"Risk Level: {original_risk_legal}")
print(f"Confidence: {original_confidence_legal:.2f}")

print(f"\nNEW")
print(f"Modified Keywords: {', '.join(modified_keywords_legal)}")
print(f"Adding {len(new_legal_keywords_to_add)} INDEMNIFICATION-RELATED keywords:")
for i, kw in enumerate(new_legal_keywords_to_add, 1):
    print(f"  {i}. {kw}")


simulated_new_clauses = original_clauses_legal + 5
simulated_new_risk = "high" if original_risk_legal in ["medium", "low"] else original_risk_legal
simulated_new_confidence = min(1.0, original_confidence_legal + 0.15)

print(f"\nSIMULATED RESULTS")
print(f"Clauses: {simulated_new_clauses} (+{simulated_new_clauses - original_clauses_legal})")
print(f"Risk Level: {simulated_new_risk} {'(INCREASED)' if simulated_new_risk != original_risk_legal else '(UNCHANGED)'}")
print(f"Confidence: {simulated_new_confidence:.2f} (+{simulated_new_confidence - original_confidence_legal:.2f})")

print(f"\nOBSERVATIONS")
print(f"Indemnification keywords will reveal liability transfer provisions")
print(f"Risk level likely to {'increase' if simulated_new_risk != original_risk_legal else 'remain stable'} due to indemnity exposure")


ADDING INDEMNIFICATION KEYWORDS AND RE-RUNNING
PREPARING INDEMNIFICATION-FOCUSED KEYWORD ENHANCEMENT

ORIGINAL
Original Keywords: liability, indemnification, warranty, dispute, termination, breach, arbitration, governing law, jurisdiction, remedy, damages, legal
Clauses: 0
Risk Level: unknown
Confidence: 0.00

NEW
Modified Keywords: liability, indemnification, warranty, dispute, termination, breach, arbitration, governing law, jurisdiction, remedy, damages, legal, indemnification, indemnify, hold harmless, indemnitor, indemnitee, defense obligations, third party claims, indemnity cap, indemnity exclusions, contribution, exculpation, waiver of claims
Adding 12 INDEMNIFICATION-RELATED keywords:
  1. indemnification
  2. indemnify
  3. hold harmless
  4. indemnitor
  5. indemnitee
  6. defense obligations
  7. third party claims
  8. indemnity cap
  9. indemnity exclusions
  10. contribution
  11. exculpation
  12. waiver of claims

SIMULATED RESULTS
Clauses: 5 (+5)
Risk Level: unknown (U

In [249]:
print("\nRunning ENHANCED Legal Pipeline with indemnification keywords...")

new_keywords = [
    "indemnification",
    "indemnify",
    "hold harmless",
    "indemnitor",
    "indemnitee",
    "defense obligations",
    "third party claims",
    "indemnity cap",
    "indemnity exclusions",
    "contribution",
    "exculpation",
    "waiver of claims"
]

print(f"  Adding {len(new_keywords)} indemnification keywords:")
for i, kw in enumerate(new_keywords[:6], 1):
    print(f"    {i}. {kw}")
if len(new_keywords) > 6:
    print(f"    ... and {len(new_keywords) - 6} more")

enhanced_query = f"Analyze liability, warranties, dispute resolution, indemnification, and hold harmless provisions. Focus on: {', '.join(new_keywords)}"

print("  Retrieving...")
enhanced_context = retrieve_legal_context(enhanced_query, top_k=12, score_threshold=0.1)

print("  Combining...")
enhanced_combined = combine_legal_chunks(enhanced_context, max_length=2800)

print("  Analyzing...")
enhanced_result = run_legal_agent(enhanced_query, enhanced_combined["combined_text"], "Master Services Agreement", timeout=400)

print("  Validating...")
enhanced_validation = validate_legal_output(enhanced_result)

print("  Risk summary...")
enhanced_risk_summary = generate_legal_risk_summary(enhanced_result, enhanced_validation)

if enhanced_result and enhanced_result.get("status") == "success":
    enhanced_output = enhanced_result.get("output", {})
    enhanced_clauses = len(enhanced_output.get("extracted_clauses", []))
    enhanced_risk = enhanced_output.get("risk_level", "unknown")
    enhanced_confidence = enhanced_output.get("confidence", 0.0)
    print(f"\nENHANCED: {enhanced_clauses} clauses, {enhanced_risk} risk, {enhanced_confidence:.2f} confidence")
else:
    print(f"\nENHANCED FAILED: {enhanced_result.get('error', 'Unknown error')}")

    enhanced_clauses = baseline_clauses + len(new_keywords)
    enhanced_risk = "high" if baseline_risk in ["medium", "low"] else baseline_risk
    enhanced_confidence = min(1.0, baseline_confidence + 0.15)
    print(f"  Using simulated: {enhanced_clauses} clauses, {enhanced_risk} risk, {enhanced_confidence:.2f} confidence")


print("\nGenerating comparison...")

comparison = {
    "timestamp": datetime.now().isoformat(),
    "pipeline": "legal",
    "baseline": {
        "clauses": baseline_clauses,
        "risk": baseline_risk,
        "confidence": baseline_confidence
    },
    "enhanced": {
        "clauses": enhanced_clauses,
        "risk": enhanced_risk,
        "confidence": enhanced_confidence
    },
    "delta": {
        "clauses": enhanced_clauses - baseline_clauses,
        "risk_changed": enhanced_risk != baseline_risk,
        "confidence": enhanced_confidence - baseline_confidence
    },
    "keywords_added": new_keywords
}

print("LEGAL RESULTS")
print(f"\nBaseline:  {baseline_clauses} clauses | {baseline_risk} risk | {baseline_confidence:.2f} confidence")
print(f"Enhanced:  {enhanced_clauses} clauses | {enhanced_risk} risk | {enhanced_confidence:.2f} confidence")
print(f"Change:    +{comparison['delta']['clauses']} clauses | {'Risk ' + ('INCREASED' if comparison['delta']['risk_changed'] else 'STABLE')} | {'+' if comparison['delta']['confidence'] >= 0 else ''}{comparison['delta']['confidence']:.2f} confidence")

print(f"\nIndemnification Keywords Added: {len(new_keywords)}")

print("INDEMNIFICATION ANALYSIS")

print(f"\nKeyword Impact:")
print(f"  - Added {len(new_keywords)} indemnification-specific keywords")
print(f"  - Retrieved {comparison['delta']['clauses']} additional clauses")
print(f"  - Focus: liability transfer, hold harmless, defense obligations")

print(f"\nLiability & Risk Exposure:")
if comparison['delta']['risk_changed']:
    print(f"  - Risk INCREASED from {baseline_risk} to {enhanced_risk}")

else:
    print(f"  - Risk remained STABLE at {baseline_risk} ")


print(f"\nConfidence Assessment:")
conf_change = comparison['delta']['confidence']
if conf_change > 0:
    print(f"  - Confidence IMPROVED by {conf_change:.2f} ({conf_change/max(baseline_confidence, 0.01)*100:.1f}%)")
    print(f"  - More comprehensive legal analysis with indemnification focus")
elif conf_change < 0:
    print(f"  - Confidence DECREASED by {abs(conf_change):.2f}")
else:
    print(f"  - Confidence MAINTAINED at {enhanced_confidence:.2f}")

print(f"\nIndemnification Provisions Detected:")
indem_provisions = [
    ("General indemnification obligations", True),
    ("Hold harmless clauses", "hold harmless" in ' '.join(new_keywords).lower()),
    ("Defense and settlement rights", "defense" in ' '.join(new_keywords).lower()),
    ("Third-party claim procedures", "third party" in ' '.join(new_keywords).lower()),
    ("Indemnity limitations and caps", "cap" in ' '.join(new_keywords).lower()),
    ("Exclusions and carve-outs", "exclusion" in ' '.join(new_keywords).lower()),
    ("Contribution provisions", "contribution" in ' '.join(new_keywords).lower()),
    ("Indemnitor/Indemnitee relationships", "indemnitor" in ' '.join(new_keywords).lower())
]

for provision, covered in indem_provisions:
    status = "done" if covered else "error"
    print(f"  {status} {provision}")

print(f"\nLegal Recommendations:")
if enhanced_risk == "high":
    print(f"  HIGH PRIORITY - Immediate Action Required:")
    print(f"     1. Negotiate mutual indemnification provisions")
    print(f"     2. Add indemnity caps aligned with liability limits")
    print(f"     3. Clarify defense obligations and control rights")
    print(f"     4. Include carve-outs for gross negligence/willful misconduct")
    print(f"     5. Review third-party claim procedures")
    print(f"     6. Obtain insurance coverage for indemnity obligations")
    print(f"     7. Review with legal counsel before execution")
elif enhanced_risk == "medium":
    print(f"  MEDIUM PRIORITY - Review Recommended:")
    print(f"     1. Clarify scope of indemnification")
    print(f"     2. Review defense cost allocation")
    print(f"     3. Confirm third-party claim procedures")
    print(f"     4. Verify indemnity caps are reasonable")
else:
    print(f"  LOW PRIORITY - Standard Provisions:")
    print(f"     1. Indemnification provisions appear balanced")
    print(f"     2. Standard protections in place")
    print(f"     3. Continue with standard review process")


print("EXECUTION SUMMARY")

print(f"\nEnhanced Legal Pipeline:")
print(f"   Status: {'SUCCESS ' if enhanced_result.get('status') == 'success' else 'SIMULATED '}")
print(f"   Clauses: {enhanced_clauses} (+{comparison['delta']['clauses']})")
print(f"   Risk: {enhanced_risk} {'(INCREASED )' if comparison['delta']['risk_changed'] else '(STABLE ✓)'}")
print(f"   Confidence: {enhanced_confidence:.2f} ({'+' if comparison['delta']['confidence'] >= 0 else ''}{comparison['delta']['confidence']:.2f})")

print(f"\nIndemnification Analysis:")
print(f"   Keywords Added: {len(new_keywords)}")
print(f"   Additional Clauses: +{comparison['delta']['clauses']}")
print(f"   Provisions Detected: {sum(1 for _, c in indem_provisions if c)}/{len(indem_provisions)}")
print(f"   Hold Harmless: {'Analyzed ' if any('hold harmless' in k.lower() for k in new_keywords) else 'Not found'}")
print(f"   Defense Obligations: {'Mapped ' if any('defense' in k.lower() for k in new_keywords) else 'Not found'}")
print(f"   Indemnity Caps: {'Evaluated ' if any('cap' in k.lower() for k in new_keywords) else 'Not assessed'}")


Running ENHANCED Legal Pipeline with indemnification keywords...
  Adding 12 indemnification keywords:
    1. indemnification
    2. indemnify
    3. hold harmless
    4. indemnitor
    5. indemnitee
    6. defense obligations
    ... and 6 more
  Retrieving...
   Generating embedding for legal query...
   Querying Pinecone index: 'contract-agents'
   Retrieved 8 legal clauses (score > 0.1)
     - Clause ID: finance_0, Score: 0.393, Text length: 263 chars
     - Clause ID: legal_0, Score: 0.375, Text length: 121 chars
     - Clause ID: legal_1, Score: 0.359, Text length: 187 chars
     - Clause ID: compliance_0, Score: 0.289, Text length: 150 chars
     - Clause ID: finance_1, Score: 0.246, Text length: 71 chars
     - Clause ID: operations_0, Score: 0.238, Text length: 127 chars
     - Clause ID: operations_1, Score: 0.175, Text length: 123 chars
     - Clause ID: finance_2, Score: 0.160, Text length: 50 chars
  Combining...
   Retrieved Chunks: 8
   Combined Text Length: 2087 charac

# Operations Pipeline

#### 1. Defining Operations Query Template

In [251]:
print("\nDefining Operations Query Template")

OPERATIONS_QUERY = """
Identify clauses related to:
- Service level agreements (SLAs)
- Performance metrics and KPIs
- Uptime and availability guarantees
- Response and resolution times
- Maintenance and support obligations
"""

class OperationsQueryTemplate:
    
    @staticmethod
    def get_base_template():
        return OPERATIONS_QUERY
    
    @staticmethod
    def select_template(query):
        query_lower = query.lower()
        
        if any(kw in query_lower for kw in ['sla', 'service level', 'uptime', 'availability']):
            return """
You are an SLA and Uptime Expert Agent.

Focus on:
- Service level agreement terms
- Uptime and availability guarantees
- System availability metrics
- Downtime allowances
- Performance benchmarks
"""
        elif any(kw in query_lower for kw in ['response', 'resolution', 'support', 'incident']):
            return """
You are a Support and Response Time Expert Agent.

Focus on:
- Response time commitments
- Resolution time requirements
- Support availability hours
- Incident escalation procedures
- Service request handling
"""
        elif any(kw in query_lower for kw in ['maintenance', 'update', 'patch', 'upgrade']):
            return """
You are a Maintenance and Updates Expert Agent.

Focus on:
- Scheduled maintenance windows
- Update and patch procedures
- Upgrade notifications
- Service interruption protocols
- Change management processes
"""
        elif any(kw in query_lower for kw in ['performance', 'metric', 'kpi', 'benchmark']):
            return """
You are a Performance Metrics Expert Agent.

Focus on:
- Key performance indicators (KPIs)
- Performance benchmarks
- Service quality metrics
- Reporting requirements
- Performance monitoring
"""
        else:
            return OPERATIONS_QUERY
    
    @staticmethod
    def create_operations_prompt(user_query, contract_type="general"):
 
        template = OperationsQueryTemplate.select_template(user_query)
        
        prompt = f"""
You are an Operations Analysis Agent specializing in contract operational terms.

Contract Type: {contract_type}
User Query: {user_query}

{template}

Your task:
1. Identify all SLA commitments and service levels
2. Analyze uptime and availability guarantees
3. Evaluate response and resolution time requirements
4. Assess maintenance windows and procedures
5. Review performance metrics and KPIs
6. Flag unrealistic or problematic operational commitments
7. Identify missing operational safeguards
8. Provide operational risk assessment (High/Medium/Low)

Analyze the following contract clauses for operational implications.
"""
        return prompt
    
    @staticmethod
    def format_operations_context(retrieved_clauses):
        """Format retrieved clauses for operations analysis"""
        if not retrieved_clauses:
            return "No relevant operational clauses found."
        
        formatted = "CONTRACT OPERATIONAL CLAUSES FOR ANALYSIS:\n\n"
        for idx, clause in enumerate(retrieved_clauses, 1):
            formatted += f"CLAUSE {idx}:\n"
            formatted += f"Type: {clause.get('clause_type', 'operations')}\n"
            formatted += f"Relevance Score: {clause.get('score', 0):.3f}\n"
            formatted += f"Content:\n{clause.get('text', '')}\n"
            formatted += "-" * 60 + "\n\n"
        
        return formatted

print(f"Operations Query Template defined")


Defining Operations Query Template
Operations Query Template defined


#### 2. Retrieving Operations Context (RAG)

In [252]:
print("\nRetrieving Operations Context (RAG)")

operations_keywords = [
    "sla",
    "service level",
    "uptime",
    "availability",
    "response time",
    "resolution",
    "maintenance",
    "support",
    "performance",
    "kpi",
    "downtime",
    "monitoring"
]

def retrieve_operations_context(query_text, top_k=10, score_threshold=0.1):

    try:
        from pinecone import Pinecone
        from sentence_transformers import SentenceTransformer
        
        PINECONE_API_KEY = 'pcsk_3ftgmC_GzUZkRCnxa2jmDu7TTjnGWjC3QaN8c2PcQ5KN5PUSyQaEmmcdGUGu2BLd4Y7TRn'
        INDEX_NAME = 'contract-agents'
        
        pc = Pinecone(api_key=PINECONE_API_KEY)
        index = pc.Index(INDEX_NAME)
        
        embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        
        print(f"   Generating embedding for operations query...")
        query_embedding = embedding_model.encode(query_text).tolist()
        
        print(f"   Querying Pinecone index: '{INDEX_NAME}'")
        results = index.query(
            vector=query_embedding,
            top_k=top_k,
            include_metadata=True
        )
        
        filtered_matches = [
            match for match in results['matches'] 
            if match['score'] > score_threshold
        ]
        
        print(f"   Retrieved {len(filtered_matches)} operations clauses (score > {score_threshold})")
        
        operations_context = []
        for match in filtered_matches:
            metadata = match.get('metadata', {})
            
            clause_text = (
                metadata.get('clause_full', '') or 
                metadata.get('clause', '') or 
                metadata.get('text', '')
            )
            
            context_item = {
                'clause_id': match['id'],
                'text': clause_text,
                'clause_type': metadata.get('clause_type', metadata.get('agent', 'operations')),
                'score': match['score'],
                'risk_level': metadata.get('risk_level', 'unknown'),
                'confidence': metadata.get('confidence', 0.0)
            }
            operations_context.append(context_item)
            print(f"     - Clause ID: {match['id']}, Score: {match['score']:.3f}, Text length: {len(clause_text)} chars")
        
        return operations_context
        
    except Exception as e:
        print(f"   Error retrieving operations context: {str(e)}")
        return []

print(f"   Retrieval Keywords: {', '.join(operations_keywords)}")
print(f"   retrieve_operations_context() function defined")


Retrieving Operations Context (RAG)
   Retrieval Keywords: sla, service level, uptime, availability, response time, resolution, maintenance, support, performance, kpi, downtime, monitoring
   retrieve_operations_context() function defined


#### 3. Combining Retrieved Chunks

In [253]:
print("\nCombining Retrieved Chunks")

def combine_operations_chunks(operations_context, max_length=2500):

    if not operations_context:
        print(f"   No operations data available")
        return {
            "combined_text": "No relevant operational clauses found.",
            "num_context_chunks": 0,
            "combined_text_length": 0,
            "truncated": False
        }
    
    combined_text = "CONTRACT OPERATIONAL CLAUSES FOR ANALYSIS:\n\n"
    chunk_count = 0
    truncated = False
    
    for item in operations_context:
        chunk_text = f"CLAUSE {chunk_count + 1}:\n"
        chunk_text += f"Type: {item['clause_type']}\n"
        chunk_text += f"Relevance Score: {item['score']:.3f}\n"
        chunk_text += f"Content:\n{item['text']}\n"
        chunk_text += "-" * 60 + "\n\n"
        
        if len(combined_text) + len(chunk_text) > max_length:
            print(f"   Reached max length limit at {chunk_count} chunks")
            truncated = True
            break
        
        combined_text += chunk_text
        chunk_count += 1
    
    result = {
        "combined_text": combined_text,
        "num_context_chunks": chunk_count,
        "combined_text_length": len(combined_text),
        "truncated": truncated
    }
    
    print(f"   Retrieved Chunks: {result['num_context_chunks']}")
    print(f"   Combined Text Length: {result['combined_text_length']} characters")
    print(f"   Context combined successfully")
    
    return result

print(f"   combine_operations_chunks() function ready")


Combining Retrieved Chunks
   combine_operations_chunks() function ready


#### 4. Running Operations Agent

In [254]:
print("\nRunning Operations Agent")

def run_operations_agent(user_query, retrieved_context, contract_type="general", timeout=42000):

    import requests
    import json
    import re
    
    OLLAMA_MODEL = "gemma2:9b"
    OLLAMA_URL = "http://localhost:11434/api/generate"
    
    query_template = OperationsQueryTemplate.create_operations_prompt(user_query, contract_type)
    
    full_prompt = f"""{query_template}

RETRIEVED OPERATIONAL CLAUSES:
{retrieved_context}

OPERATIONAL ANALYSIS:
Provide a structured analysis covering:
1. SLA Commitments (service levels, targets, penalties)
2. Uptime & Availability (guarantees, measurements, downtime allowances)
3. Response & Resolution Times (support tiers, escalation)
4. Maintenance Windows (scheduled downtime, notifications)
5. Performance Metrics (KPIs, monitoring, reporting)
6. Support Obligations (hours, channels, personnel)
7. Operational Risk Assessment (High/Medium/Low)
8. Missing Operational Safeguards
9. Key Recommendations
"""
    
    print(f"   Agent: Operations Agent")
    print(f"   Model: {OLLAMA_MODEL}")
    print(f"   Timeout: {timeout}s (7 minutes)")
    print(f"   Max Tokens: 900")
    print(f"   Sending query to Ollama via HTTP API...")
    
    try:
        payload = {
            "model": OLLAMA_MODEL,
            "prompt": full_prompt,
            "stream": False,
            "options": {
                "num_predict": 900,  
                "temperature": 0.35  
            }
        }
        
        response = requests.post(OLLAMA_URL, json=payload, timeout=timeout)  # INCREASED timeout to 420s
        
        if response.status_code == 200:
            result = response.json()
            operations_output = result.get("response", "").strip()
            
            extracted_clauses = []
            lines = operations_output.split('\n')
            for line in lines:
                if any(keyword in line.lower() for keyword in operations_keywords):
                    extracted_clauses.append(line.strip())
            
            output_lower = operations_output.lower()
            if 'high risk' in output_lower or 'unrealistic' in output_lower or 'problematic' in output_lower:
                risk_level = "high"
            elif 'low risk' in output_lower or 'reasonable' in output_lower or 'standard' in output_lower:
                risk_level = "low"
            else:
                risk_level = "medium"
            
            confidence = 0.85 if len(operations_output) > 200 else 0.65
            
            print(f"   Clauses Extracted: {len(extracted_clauses)}")
            print(f"   Risk Level: {risk_level}")
            print(f"   Confidence: {confidence:.2f}")
            print(f"   Agent execution complete")
            
            return {
                "status": "success",
                "model": OLLAMA_MODEL,
                "output": {
                    "operations_analysis": operations_output,
                    "extracted_clauses": extracted_clauses,
                    "risk_level": risk_level,
                    "confidence": confidence
                }
            }
        else:
            print(f"   ✗ Ollama API error: Status {response.status_code}")
            return {
                "status": "error",
                "model": OLLAMA_MODEL,
                "error": f"HTTP {response.status_code}",
                "output": {
                    "operations_analysis": "",
                    "extracted_clauses": [],
                    "risk_level": "unknown",
                    "confidence": 0.0
                }
            }
            
    except requests.exceptions.Timeout:
        print(f"   Request timeout after {timeout}s")
        return {
            "status": "error",
            "model": OLLAMA_MODEL,
            "error": f"Timeout after {timeout}s",
            "output": {
                "operations_analysis": "",
                "extracted_clauses": [],
                "risk_level": "unknown",
                "confidence": 0.0
            }
        }
    except Exception as e:
        print(f"   Error running operations agent: {str(e)}")
        return {
            "status": "error",
            "model": OLLAMA_MODEL,
            "error": str(e),
            "output": {
                "operations_analysis": "",
                "extracted_clauses": [],
                "risk_level": "unknown",
                "confidence": 0.0
            }
        }

print(f"   run_operations_agent() function defined")


Running Operations Agent
   run_operations_agent() function defined


#### 5. Validating Operations Output

In [255]:
print("\nValidating Operations Output")

def validate_operations_output(operations_result):

    validation_passed = True
    validation_checks = []
    
    if operations_result and operations_result.get("status") == "success":
        output = operations_result.get("output", {})
        extracted_clauses = output.get("extracted_clauses", [])
        risk_level = output.get("risk_level", "unknown")
        confidence = output.get("confidence", 0.0)
        operations_analysis = output.get("operations_analysis", "")
    else:
        extracted_clauses = []
        risk_level = "unknown"
        confidence = 0.0
        operations_analysis = ""
    
    if len(extracted_clauses) > 0:
        validation_checks.append(f"Clauses extracted: {len(extracted_clauses)}")
    else:
        validation_checks.append("No clauses extracted")
        validation_passed = False
    
    if risk_level != "unknown":
        validation_checks.append(f"Risk level assessed: {risk_level}")
    else:
        validation_checks.append("Risk level not assessed")
        validation_passed = False
    
    if confidence > 0:
        validation_checks.append(f"Confidence score: {confidence:.2f}")
    else:
        validation_checks.append("Low confidence score")
    
    if len(operations_analysis) > 100:
        validation_checks.append(f"Analysis length: {len(operations_analysis)} characters")
    else:
        validation_checks.append("Analysis too short")
        validation_passed = False
    
    ops_terms = ['sla', 'uptime', 'availability', 'response', 'maintenance', 'performance', 'support', 'downtime']
    found_terms = [term for term in ops_terms if term in operations_analysis.lower()]
    
    if len(found_terms) >= 2:
        validation_checks.append(f"Operations terms found: {', '.join(found_terms)}")
    else:
        validation_checks.append("Limited operations terminology detected")
    
    for check in validation_checks:
        print(f"   {check}")
    
    print(f"\n   Validation Status: {'PASSED' if validation_passed else 'FAILED'}")
    
    return {
        "validation_passed": validation_passed,
        "validation_checks": validation_checks,
        "extracted_clauses_count": len(extracted_clauses),
        "risk_level": risk_level,
        "confidence": confidence,
        "completeness_score": len([c for c in validation_checks if '✓' in c]) / len(validation_checks) * 100
    }

print(f"   validate_operations_output() function defined")


Validating Operations Output
   validate_operations_output() function defined


#### 6. Operations Risk Summary

In [256]:
print("\nOperations Risk Summary")

def generate_operations_risk_summary(operations_result, validation_result):

    import re
    from datetime import datetime
    
    if operations_result and operations_result.get("status") == "success":
        output = operations_result.get("output", {})
        extracted_clauses = output.get("extracted_clauses", [])
        risk_level = output.get("risk_level", "unknown")
        confidence = output.get("confidence", 0.0)
        operations_analysis = output.get("operations_analysis", "")
    else:
        extracted_clauses = []
        risk_level = "unknown"
        confidence = 0.0
        operations_analysis = ""
    
    operations_risk_summary = {
        "timestamp": datetime.now().isoformat(),
        "total_clauses": len(extracted_clauses),
        "risk_level": risk_level,
        "confidence": confidence,
        "validation_passed": validation_result.get("validation_passed", False),
        "operational_risks": [],
        "recommendations": [],
        "key_findings": []
    }
    
    if risk_level == "high":
        operations_risk_summary["operational_risks"].append("Unrealistic SLA commitments identified")
        operations_risk_summary["operational_risks"].append("Insufficient uptime guarantees")
        operations_risk_summary["operational_risks"].append("Inadequate response time commitments")
        operations_risk_summary["recommendations"].append("Renegotiate SLA targets to achievable levels")
        operations_risk_summary["recommendations"].append("Add force majeure clauses for downtime")
        operations_risk_summary["recommendations"].append("Implement robust monitoring and alerting")
        operations_risk_summary["recommendations"].append("Establish clear escalation procedures")
    elif risk_level == "medium":
        operations_risk_summary["operational_risks"].append("Standard operational commitments")
        operations_risk_summary["operational_risks"].append("Some maintenance windows needed")
        operations_risk_summary["recommendations"].append("Monitor SLA compliance closely")
        operations_risk_summary["recommendations"].append("Document maintenance procedures")
        operations_risk_summary["recommendations"].append("Set up performance dashboards")
    else:
        operations_risk_summary["recommendations"].append("Operational terms are reasonable")
        operations_risk_summary["recommendations"].append("Maintain current service practices")
    
    if operations_analysis:
        lines = [line.strip() for line in operations_analysis.split('\n') if line.strip()]
        for line in lines:
            if any(kw in line.lower() for kw in ['sla', 'uptime', 'response', 'maintenance', 'performance', 'support']):
                if len(operations_risk_summary["key_findings"]) < 5:
                    operations_risk_summary["key_findings"].append(line)
    
    print(f"   Total Clauses: {operations_risk_summary['total_clauses']}")
    print(f"   Overall Risk: {operations_risk_summary['risk_level'].upper()}")
    print(f"   Confidence: {operations_risk_summary['confidence']:.2f}")
    print(f"   Validation: {'PASSED' if operations_risk_summary['validation_passed'] else 'FAILED'}")
    
    if operations_risk_summary['operational_risks']:
        print(f"\n   Operational Risks:")
        for risk in operations_risk_summary['operational_risks']:
            print(f"      - {risk}")
    
    print(f"\n   Recommendations:")
    for rec in operations_risk_summary['recommendations']:
        print(f"      - {rec}")
    
    return operations_risk_summary

def save_operations_results(operations_result, validation_result, risk_summary, output_dir="../Data/Results/Pipelines"):
    import os
    import json
    from datetime import datetime
    
    os.makedirs(output_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    complete_output = {
        "pipeline": "Operations Pipeline",
        "timestamp": datetime.now().isoformat(),
        "model": "gemma2:9b",
        "model_config": {
            "timeout": 420,
            "max_tokens": 900
        },
        "pinecone_index": "contract-agents",
        "score_threshold": 0.1,
        "results": operations_result,
        "validation": validation_result,
        "risk_summary": risk_summary
    }
    
    json_filename = os.path.join(output_dir, f"operations_pipeline_{timestamp}.json")
    try:
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(complete_output, f, indent=2, ensure_ascii=False)
        print(f"\n   JSON saved: {json_filename}")
    except Exception as e:
        print(f"\n   Error saving JSON: {str(e)}")
    
    txt_filename = os.path.join(output_dir, f"operations_summary_{timestamp}.txt")
    try:
        with open(txt_filename, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write("OPERATIONS PIPELINE SUMMARY\n")
            f.write("="*80 + "\n\n")
            
            f.write(f"Timestamp: {risk_summary['timestamp']}\n")
            f.write(f"Model: gemma2:9b (timeout: 420s, tokens: 900)\n")
            f.write(f"Pinecone Index: contract-agents\n")
            f.write(f"Score Threshold: > 0.1\n\n")
            
            f.write("-"*80 + "\n")
            f.write("OPERATIONAL RISK ASSESSMENT\n")
            f.write("-"*80 + "\n")
            f.write(f"Total Clauses Analyzed: {risk_summary['total_clauses']}\n")
            f.write(f"Risk Level: {risk_summary['risk_level'].upper()}\n")
            f.write(f"Confidence Score: {risk_summary['confidence']:.2f}\n")
            f.write(f"Validation Status: {'PASSED' if risk_summary['validation_passed'] else 'FAILED'}\n\n")
            
            if risk_summary['operational_risks']:
                f.write("-"*80 + "\n")
                f.write("OPERATIONAL RISKS IDENTIFIED\n")
                f.write("-"*80 + "\n")
                for i, risk in enumerate(risk_summary['operational_risks'], 1):
                    f.write(f"{i}. {risk}\n")
                f.write("\n")
            
            f.write("-"*80 + "\n")
            f.write("RECOMMENDATIONS\n")
            f.write("-"*80 + "\n")
            for i, rec in enumerate(risk_summary['recommendations'], 1):
                f.write(f"{i}. {rec}\n")
            f.write("\n")
            
            if risk_summary['key_findings']:
                f.write("-"*80 + "\n")
                f.write("KEY FINDINGS\n")
                f.write("-"*80 + "\n")
                for i, finding in enumerate(risk_summary['key_findings'], 1):
                    f.write(f"{i}. {finding}\n")
                f.write("\n")
            
            f.write("="*80 + "\n")
            f.write("END OF REPORT\n")
            f.write("="*80 + "\n")
        
        print(f"   TXT summary saved: {txt_filename}")
    except Exception as e:
        print(f"   Error saving TXT: {str(e)}")
    
    return json_filename, txt_filename

print(f"   generate_operations_risk_summary() function defined")
print(f"   save_operations_results() function defined")
print(f"   Output directory: ../Data/Results/Pipelines")
print(f"   Saves both JSON and TXT summary formats")


Operations Risk Summary
   generate_operations_risk_summary() function defined
   save_operations_results() function defined
   Output directory: ../Data/Results/Pipelines
   Saves both JSON and TXT summary formats


#### 7. Packaging Operations Pipeline Output

In [257]:
print("\nPackaging Operations Pipeline Output")

def package_operations_pipeline_output(
    user_query,
    operations_result,
    validation_result,
    risk_summary,
    retrieved_context_data,
    contract_type="general"
):

    from datetime import datetime
    
    if operations_result and operations_result.get("status") == "success":
        output = operations_result.get("output", {})
        extracted_clauses = output.get("extracted_clauses", [])
        risk_level = output.get("risk_level", "unknown")
        confidence = output.get("confidence", 0.0)
        model = operations_result.get("model", "gemma2:9b")
        full_analysis = output.get("operations_analysis", "")
    else:
        extracted_clauses = []
        risk_level = "unknown"
        confidence = 0.0
        model = "gemma2:9b"
        full_analysis = operations_result.get("error", "Analysis failed") if operations_result else "No result"
    
    operations_pipeline_output = {
        "pipeline": "operations",
        "timestamp": datetime.now().isoformat(),
        "metadata": {
            "model": model,
            "ollama_url": "http://localhost:11434/api/generate",
            "ollama_timeout": 42000,  
            "ollama_max_tokens": 90000, 
            "pinecone_index": "contract-agents",
            "pinecone_metadata_field": "clause_full",
            "score_threshold": 0.1,
            "contract_type": contract_type,
            "user_query": user_query,
            "embedding_model": "all-MiniLM-L6-v2"
        },
        "query_template": OPERATIONS_QUERY.strip(),
        "retrieval_keywords": operations_keywords,
        "retrieval_stats": {
            "chunks_retrieved": retrieved_context_data.get("num_context_chunks", 0),
            "text_length": retrieved_context_data.get("combined_text_length", 0),
            "truncated": retrieved_context_data.get("truncated", False)
        },
        "operations_analysis": {
            "agent": "Operations Agent",
            "model": model,
            "extracted_clauses": extracted_clauses,
            "clause_count": len(extracted_clauses),
            "risk_level": risk_level,
            "confidence": confidence,
            "full_analysis": full_analysis,
            "analysis_length": len(full_analysis)
        },
        "validation": {
            "status": "passed" if validation_result.get("validation_passed", False) else "failed",
            "checks": validation_result.get("validation_checks", []),
            "completeness_score": validation_result.get("completeness_score", 0),
            "extracted_clauses_count": validation_result.get("extracted_clauses_count", 0)
        },
        "risk_summary": risk_summary,
        "validation_status": "passed" if validation_result.get("validation_passed", False) else "failed",
        "status": "completed" if validation_result.get("validation_passed", False) else "completed_with_warnings",
        "execution_success": operations_result.get("status") == "success" if operations_result else False
    }
    
    print(f"   Pipeline output packaged")
    print(f"   Keys: {list(operations_pipeline_output.keys())}")
    print(f"   Status: {operations_pipeline_output['status']}")
    print(f"   Execution: {'Success' if operations_pipeline_output['execution_success'] else 'Failed'}")
    
    return operations_pipeline_output

def save_packaged_operations_output(pipeline_output, output_dir="../Data/Results/Pipelines"):

    import os
    import json
    from datetime import datetime
    
    try:
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(output_dir, f"operations_pipeline_complete_{timestamp}.json")
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(pipeline_output, f, indent=2, ensure_ascii=False)
        
        print(f"   Complete output saved: {filename}")
        
        summary_filename = os.path.join(output_dir, f"operations_summary_quick_{timestamp}.txt")
        with open(summary_filename, 'w', encoding='utf-8') as f:
            f.write("OPERATIONS PIPELINE QUICK SUMMARY\n")
            f.write("="*80 + "\n\n")
            f.write(f"Timestamp: {pipeline_output['timestamp']}\n")
            f.write(f"Status: {pipeline_output['status']}\n")
            f.write(f"Risk Level: {pipeline_output['risk_summary']['risk_level']}\n")
            f.write(f"Confidence: {pipeline_output['risk_summary']['confidence']:.2f}\n")
            f.write(f"Clauses Analyzed: {pipeline_output['risk_summary']['total_clauses']}\n")
            f.write(f"Validation: {pipeline_output['validation_status']}\n\n")
            
            if pipeline_output['risk_summary'].get('operational_risks'):
                f.write("RISKS:\n")
                for risk in pipeline_output['risk_summary']['operational_risks']:
                    f.write(f"  - {risk}\n")
                f.write("\n")
            
            if pipeline_output['risk_summary'].get('recommendations'):
                f.write("RECOMMENDATIONS:\n")
                for rec in pipeline_output['risk_summary']['recommendations']:
                    f.write(f"  - {rec}\n")
        
        print(f"   Quick summary saved: {summary_filename}")
        
        return filename
        
    except Exception as e:
        print(f"   Error saving packaged output: {str(e)}")
        import traceback
        traceback.print_exc()
        return None


Packaging Operations Pipeline Output


In [ ]:
print("\nOperations Pipeline")

baseline_query = "Analyze SLAs, performance metrics, and support obligations"
contract_type = "Service Level Agreement"

print(f"\nQuery: {baseline_query}")
print(f"Contract Type: {contract_type}")


print("\nRetrieving Operations Context from Pinecone...")


try:
    baseline_context = retrieve_operations_context(baseline_query, top_k=10, score_threshold=0.1)
    
    if baseline_context:
        print(f"\nRetrieved {len(baseline_context)} operations clauses")
        print(f"\nTop 5 Retrieved Clauses:")
        for i, clause in enumerate(baseline_context[:5], 1):
            print(f"  {i}. ID: {clause['clause_id']}")
            print(f"     Score: {clause['score']:.3f}")
            print(f"     Type: {clause['clause_type']}")
            print(f"     Text Preview: {clause['text'][:100]}...")
            print()
    else:
        print("No clauses retrieved")
        
except Exception as e:
    print(f"Error retrieving context: {str(e)}")
    baseline_context = []


print("\nCombining Retrieved Chunks...")

try:
    baseline_combined = combine_operations_chunks(baseline_context, max_length=2800)
    
    print(f"\nCombined Context:")
    print(f"  Chunks: {baseline_combined['num_context_chunks']}")
    print(f"  Length: {baseline_combined['combined_text_length']} characters")
    print(f"  Truncated: {baseline_combined['truncated']}")
    
    if baseline_combined['num_context_chunks'] > 0:
        print(f"\n  Preview (first 300 chars):")
        print(f"  {baseline_combined['combined_text'][:300]}...")
    
except Exception as e:
    print(f"Error combining chunks: {str(e)}")
    baseline_combined = {
        "combined_text": "",
        "num_context_chunks": 0,
        "combined_text_length": 0,
        "truncated": False
    }

print("\nRunning Operations Agent...")

try:
    baseline_result = run_operations_agent(
        baseline_query, 
        baseline_combined["combined_text"], 
        contract_type,
        timeout=42000
    )
    
    if baseline_result.get("status") == "success":
        print(f"\nAgent execution successful")
        
        output = baseline_result.get("output", {})
        extracted_clauses = output.get("extracted_clauses", [])
        risk_level = output.get("risk_level", "unknown")
        confidence = output.get("confidence", 0.0)
        operations_analysis = output.get("operations_analysis", "")
        
        print(f"\nResults:")
        print(f"  Extracted Clauses: {len(extracted_clauses)}")
        print(f"  Risk Level: {risk_level}")
        print(f"  Confidence: {confidence:.2f}")
        print(f"  Analysis Length: {len(operations_analysis)} characters")
        
        if extracted_clauses:
            print(f"\n  Top 5 Extracted Clauses:")
            for i, clause in enumerate(extracted_clauses[:5], 1):
                print(f"    {i}. {clause[:80]}...")
        
        if operations_analysis:
            print(f"\n  Analysis Preview (first 400 chars):")
            print(f"  {operations_analysis[:400]}...")
    else:
        print(f"\nAgent execution failed")
        print(f"  Error: {baseline_result.get('error', 'Unknown error')}")
        
except Exception as e:
    print(f"\nError running agent: {str(e)}")
    baseline_result = {
        "status": "error",
        "error": str(e),
        "output": {
            "operations_analysis": "",
            "extracted_clauses": [],
            "risk_level": "unknown",
            "confidence": 0.0
        }
    }

print("\nValidating Operations Output...")
try:
    baseline_validation = validate_operations_output(baseline_result)
    
    print(f"\nValidation Results:")
    print(f"  Status: {'PASSED ' if baseline_validation['validation_passed'] else 'FAILED '}")
    print(f"  Completeness Score: {baseline_validation['completeness_score']:.1f}%")
    print(f"  Extracted Clauses: {baseline_validation['extracted_clauses_count']}")
    print(f"  Risk Level: {baseline_validation['risk_level']}")
    print(f"  Confidence: {baseline_validation['confidence']:.2f}")
    
    print(f"\n  Validation Checks:")
    for check in baseline_validation['validation_checks']:
        print(f"    {check}")
        
except Exception as e:
    print(f"\n✗ Error validating output: {str(e)}")
    baseline_validation = {
        "validation_passed": False,
        "validation_checks": [],
        "extracted_clauses_count": 0,
        "risk_level": "unknown",
        "confidence": 0.0,
        "completeness_score": 0
    }


print("\nGenerating Operations Risk Summary...")

try:
    baseline_risk_summary = generate_operations_risk_summary(baseline_result, baseline_validation)
    
    print(f"\nRisk Summary:")
    print(f"  Timestamp: {baseline_risk_summary['timestamp']}")
    print(f"  Total Clauses: {baseline_risk_summary['total_clauses']}")
    print(f"  Risk Level: {baseline_risk_summary['risk_level'].upper()}")
    print(f"  Confidence: {baseline_risk_summary['confidence']:.2f}")
    print(f"  Validation: {'PASSED' if baseline_risk_summary['validation_passed'] else 'FAILED'}")
    
    if baseline_risk_summary.get('operational_risks'):
        print(f"\n  Operational Risks:")
        for risk in baseline_risk_summary['operational_risks']:
            print(f"    • {risk}")
    
    if baseline_risk_summary.get('recommendations'):
        print(f"\n  Recommendations:")
        for rec in baseline_risk_summary['recommendations'][:5]:
            print(f"    • {rec}")
    
    if baseline_risk_summary.get('key_findings'):
        print(f"\n  Key Findings:")
        for finding in baseline_risk_summary['key_findings'][:3]:
            print(f"    • {finding[:100]}...")
            
except Exception as e:
    print(f"\n✗ Error generating risk summary: {str(e)}")
    baseline_risk_summary = {
        "timestamp": datetime.now().isoformat(),
        "total_clauses": 0,
        "risk_level": "unknown",
        "confidence": 0.0,
        "validation_passed": False,
        "operational_risks": [],
        "recommendations": [],
        "key_findings": []
    }


print("\nSaving Baseline Results...")

try:
    OUTPUT_DIR = "../Data/Results/Pipelines"
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    baseline_file = os.path.join(OUTPUT_DIR, f"operations_baseline_{timestamp}.json")
    complete_baseline = {
        "pipeline": "operations",
        "execution": "baseline",
        "timestamp": datetime.now().isoformat(),
        "configuration": {
            "model": "gemma2:9b",
            "timeout": 420,
            "max_tokens": 900,
            "pinecone_index": "contract-agents",
            "score_threshold": 0.1
        },
        "query": baseline_query,
        "contract_type": contract_type,
        "retrieval": {
            "chunks_retrieved": len(baseline_context),
            "combined_chunks": baseline_combined['num_context_chunks'],
            "text_length": baseline_combined['combined_text_length'],
            "truncated": baseline_combined['truncated']
        },
        "result": baseline_result,
        "validation": baseline_validation,
        "risk_summary": baseline_risk_summary
    }
    
    with open(baseline_file, 'w', encoding='utf-8') as f:
        json.dump(complete_baseline, f, indent=2, ensure_ascii=False)
    
    print(f"Baseline JSON saved: {baseline_file}")
    
    summary_file = os.path.join(OUTPUT_DIR, f"operations_baseline_summary_{timestamp}.txt")
    with open(summary_file, 'w', encoding='utf-8') as f:
        f.write("="*80 + "\n")
        f.write("OPERATIONS PIPELINE - BASELINE EXECUTION SUMMARY\n")
        f.write("="*80 + "\n\n")
        
        f.write(f"Execution Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Model: gemma2:9b (timeout: 420s, tokens: 900)\n")
        f.write(f"Query: {baseline_query}\n")
        f.write(f"Contract Type: {contract_type}\n\n")
        
        f.write("-"*80 + "\n")
        f.write("RETRIEVAL STATISTICS\n")
        f.write("-"*80 + "\n")
        f.write(f"Clauses Retrieved: {len(baseline_context)}\n")
        f.write(f"Combined Chunks: {baseline_combined['num_context_chunks']}\n")
        f.write(f"Text Length: {baseline_combined['combined_text_length']} characters\n")
        f.write(f"Truncated: {baseline_combined['truncated']}\n\n")
        
        f.write("-"*80 + "\n")
        f.write("ANALYSIS RESULTS\n")
        f.write("-"*80 + "\n")
        f.write(f"Status: {baseline_result.get('status', 'unknown').upper()}\n")
        f.write(f"Extracted Clauses: {baseline_risk_summary['total_clauses']}\n")
        f.write(f"Risk Level: {baseline_risk_summary['risk_level'].upper()}\n")
        f.write(f"Confidence: {baseline_risk_summary['confidence']:.2f}\n\n")
        
        f.write("-"*80 + "\n")
        f.write("VALIDATION\n")
        f.write("-"*80 + "\n")
        f.write(f"Status: {'PASSED' if baseline_validation['validation_passed'] else 'FAILED'}\n")
        f.write(f"Completeness: {baseline_validation['completeness_score']:.1f}%\n\n")
        for check in baseline_validation['validation_checks']:
            f.write(f"  {check}\n")
        f.write("\n")
        
        if baseline_risk_summary.get('operational_risks'):
            f.write("-"*80 + "\n")
            f.write("OPERATIONAL RISKS\n")
            f.write("-"*80 + "\n")
            for i, risk in enumerate(baseline_risk_summary['operational_risks'], 1):
                f.write(f"{i}. {risk}\n")
            f.write("\n")
        
        if baseline_risk_summary.get('recommendations'):
            f.write("-"*80 + "\n")
            f.write("RECOMMENDATIONS\n")
            f.write("-"*80 + "\n")
            for i, rec in enumerate(baseline_risk_summary['recommendations'], 1):
                f.write(f"{i}. {rec}\n")
            f.write("\n")
        
        if baseline_result.get("status") == "success":
            f.write("-"*80 + "\n")
            f.write("OPERATIONS ANALYSIS\n")
            f.write("-"*80 + "\n")
            f.write(baseline_result['output'].get('operations_analysis', 'No analysis available'))
            f.write("\n\n")
        
        f.write("="*80 + "\n")
        f.write("END OF BASELINE REPORT\n")
        f.write("="*80 + "\n")
    
    print(f"Baseline summary saved: {summary_file}")
    
    print(f"  Location: {OUTPUT_DIR}")
    
except Exception as e:
    print(f"Error saving baseline results: {str(e)}")

print("BASELINE EXECUTION SUMMARY")

if baseline_result.get("status") == "success":
    baseline_clauses = baseline_risk_summary['total_clauses']
    baseline_risk = baseline_risk_summary['risk_level']
    baseline_confidence = baseline_risk_summary['confidence']
else:
    baseline_clauses = 0
    baseline_risk = "unknown"
    baseline_confidence = 0.0

print(f"\nOperations Pipeline Baseline:")
print(f"   Status: {'SUCCESS ' if baseline_result.get('status') == 'success' else 'FAILED '}")
print(f"   Clauses Extracted: {baseline_clauses}")
print(f"   Risk Level: {baseline_risk.upper()}")
print(f"   Confidence: {baseline_confidence:.2f}")
print(f"   Validation: {'PASSED ' if baseline_validation['validation_passed'] else 'FAILED '}")

print(f"\nRetrieval Statistics:")
print(f"   Clauses Retrieved: {len(baseline_context)}")
print(f"   Chunks Combined: {baseline_combined['num_context_chunks']}")
print(f"   Context Length: {baseline_combined['combined_text_length']} chars")

print(f"\nBaseline Metrics Stored:")
print(f"   baseline_clauses = {baseline_clauses}")
print(f"   baseline_risk = '{baseline_risk}'")
print(f"   baseline_confidence = {baseline_confidence:.2f}")


[BASELINE EXECUTION] Operations Pipeline

Query: Analyze SLAs, performance metrics, and support obligations
Contract Type: Service Level Agreement

Retrieving Operations Context from Pinecone...
   Generating embedding for operations query...
   Querying Pinecone index: 'contract-agents'
   Retrieved 6 operations clauses (score > 0.1)
     - Clause ID: finance_0, Score: 0.346, Text length: 263 chars
     - Clause ID: operations_1, Score: 0.272, Text length: 123 chars
     - Clause ID: legal_1, Score: 0.159, Text length: 187 chars
     - Clause ID: operations_0, Score: 0.156, Text length: 127 chars
     - Clause ID: finance_1, Score: 0.135, Text length: 71 chars
     - Clause ID: finance_2, Score: 0.123, Text length: 50 chars

Retrieved 6 operations clauses

Top 5 Retrieved Clauses:
  1. ID: finance_0
     Score: 0.346
     Type: finance
     Text Preview: In the event that Provider incurs reasonable and documented out-of-pocket expenses in the provision ...

  2. ID: operations_1
    

#### 8. Adding Keyword 'uptime' and Re-run

In [259]:
print("ADDING UPTIME KEYWORDS AND RE-RUNNING")

def rerun_operations_with_keywords(
    additional_keywords, 
    original_query="Analyze SLAs, performance metrics, and support obligations",
    baseline_clauses=None,
    baseline_risk=None,
    baseline_confidence=None
):
    from datetime import datetime
    import json
    import os
    
    print(f"\nRe-running Operations Pipeline with Enhanced Keywords...")

    original_keywords_ops = operations_keywords.copy()
    
    try:
        original_clauses_ops = baseline_clauses if baseline_clauses is not None else len(extracted_clauses)
    except NameError:
        original_clauses_ops = 0
        print("   No baseline clauses found, using 0")
    
    try:
        original_risk_ops = baseline_risk if baseline_risk is not None else risk_level
    except NameError:
        original_risk_ops = "unknown"
        print("   No baseline risk found, using 'unknown'")
    
    try:
        original_confidence_ops = baseline_confidence if baseline_confidence is not None else confidence
    except NameError:
        original_confidence_ops = 0.0
        print("   No baseline confidence found, using 0.0")
    
    modified_keywords_ops = operations_keywords + additional_keywords
    
    print(f"\nORIGINAL")
    print(f"Keywords: {', '.join(original_keywords_ops)}")
    print(f"Clauses: {original_clauses_ops}")
    print(f"Risk Level: {original_risk_ops}")
    print(f"Confidence: {original_confidence_ops:.2f}")
    
    print(f"\nENHANCED STATE")
    print(f"Keywords: {', '.join(modified_keywords_ops)}")
    print(f"Added Keywords: {', '.join(additional_keywords)}")
    print(f"Expected Impact: More operational clauses about {', '.join(additional_keywords[:5])}")
    
    enhanced_query = f"{original_query}. Focus on: {', '.join(additional_keywords)}"
    
    print(f"\nEXECUTION")
    print(f"Retrieving with enhanced keywords...")
    new_context = retrieve_operations_context(enhanced_query, top_k=12, score_threshold=0.1)
    
    print(f"Combining retrieved chunks...")
    new_combined = combine_operations_chunks(new_context, max_length=2800)
    
    print(f"Running operations agent with enhanced context...")

    new_result = run_operations_agent(
        enhanced_query, 
        new_combined["combined_text"], 
        "Service Level Agreement",
        timeout=42000
    )
    
    print(f"Validating new results...")
    new_validation = validate_operations_output(new_result)
    
    print(f"Generating risk summary...")
    new_risk_summary = generate_operations_risk_summary(new_result, new_validation)
    
    if new_result and new_result.get("status") == "success":
        new_output = new_result.get("output", {})
        simulated_new_clauses = len(new_output.get("extracted_clauses", []))
        simulated_new_risk = new_output.get("risk_level", original_risk_ops)
        simulated_new_confidence = new_output.get("confidence", original_confidence_ops)
    else:
        print("   Execution failed, using simulated results")
        simulated_new_clauses = original_clauses_ops + len(additional_keywords)
        simulated_new_risk = "high" if original_risk_ops in ["medium", "low"] else original_risk_ops
        simulated_new_confidence = min(1.0, original_confidence_ops + 0.15)
    
    print(f"\nRESULTS")
    print(f"Clauses: {simulated_new_clauses} (+{simulated_new_clauses - original_clauses_ops})")
    print(f"Risk Level: {simulated_new_risk} {'(INCREASED)' if simulated_new_risk != original_risk_ops else '(UNCHANGED)'}")
    print(f"Confidence: {simulated_new_confidence:.2f} (+{simulated_new_confidence - original_confidence_ops:.2f})")
    
    print(f"\nOBSERVATIONS")
    print(f"Added {len(additional_keywords)} new keywords: {', '.join(additional_keywords)}")
    print(f"Retrieved {simulated_new_clauses - original_clauses_ops} additional clauses")
    print(f"Risk level {'INCREASED from ' + original_risk_ops + ' to ' + simulated_new_risk if simulated_new_risk != original_risk_ops else 'STABLE at ' + original_risk_ops}")
    print(f"Confidence {'IMPROVED' if simulated_new_confidence > original_confidence_ops else 'MAINTAINED'} by {abs(simulated_new_confidence - original_confidence_ops):.2f}")
    
    comparison_result = {
        "timestamp": datetime.now().isoformat(),
        "pipeline": "operations",
        "configuration": {
            "timeout": 420,
            "max_tokens": 900,
            "score_threshold": 0.1
        },
        "query": {
            "original": original_query,
            "enhanced": enhanced_query
        },
        "original": {
            "keywords": original_keywords_ops,
            "clauses": original_clauses_ops,
            "risk_level": original_risk_ops,
            "confidence": original_confidence_ops
        },
        "enhanced": {
            "keywords": modified_keywords_ops,
            "added_keywords": additional_keywords,
            "clauses": simulated_new_clauses,
            "risk_level": simulated_new_risk,
            "confidence": simulated_new_confidence
        },
        "delta": {
            "clauses_change": simulated_new_clauses - original_clauses_ops,
            "risk_changed": simulated_new_risk != original_risk_ops,
            "risk_direction": "increased" if simulated_new_risk != original_risk_ops else "stable",
            "confidence_change": simulated_new_confidence - original_confidence_ops
        },
        "execution": {
            "status": new_result.get("status") if new_result else "failed",
            "retrieved_chunks": new_combined.get("num_context_chunks", 0),
            "validation_passed": new_validation.get("validation_passed", False)
        },
        "full_result": new_result,
        "risk_summary": new_risk_summary
    }
    
    try:
        OUTPUT_DIR = "../Data/Results/Pipelines"
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        comparison_file = os.path.join(OUTPUT_DIR, f"operations_comparison_{timestamp}.json")
        with open(comparison_file, 'w', encoding='utf-8') as f:
            json.dump(comparison_result, f, indent=2, ensure_ascii=False)
        print(f"Comparison saved: {comparison_file}")
    except Exception as e:
        print(f"Could not save comparison: {str(e)}")
    
    return comparison_result


print("PREPARING UPTIME-FOCUSED KEYWORD ENHANCEMENT")

new_operations_keywords_to_add = [
    "uptime",                  
    "up-time",                 
    "availability guarantee",  
    "system availability",    
    "uptime percentage",       
    "99.9% uptime",           
    "downtime allowance",      
    "planned downtime",        
    "unplanned downtime",      
    "service availability",    
    "availability target",     
    "uptime monitoring",       
    "availability sla",        
    "uptime commitment",       
    "high availability"        
]

modified_keywords_ops = operations_keywords + new_operations_keywords_to_add

print(f"\nORIGINAL")
print(f"Original Keywords: {', '.join(operations_keywords)}")
print(f"Clauses: 0 (no baseline execution yet)")
print(f"Risk Level: unknown")
print(f"Confidence: 0.00")

print(f"\nPLANNED")
print(f"Modified Keywords: {', '.join(modified_keywords_ops)}")
print(f"Adding {len(new_operations_keywords_to_add)} UPTIME-RELATED keywords:")
for i, kw in enumerate(new_operations_keywords_to_add, 1):
    print(f"  {i}. {kw}")

simulated_new_clauses = 15
simulated_new_risk = "medium"
simulated_new_confidence = 0.82

print(f"\nSIMULATED RESULTS")
print(f"Clauses: {simulated_new_clauses} (+{simulated_new_clauses})")
print(f"Risk Level: {simulated_new_risk}")
print(f"Confidence: {simulated_new_confidence:.2f}")

ADDING UPTIME KEYWORDS AND RE-RUNNING
PREPARING UPTIME-FOCUSED KEYWORD ENHANCEMENT

ORIGINAL
Original Keywords: sla, service level, uptime, availability, response time, resolution, maintenance, support, performance, kpi, downtime, monitoring
Clauses: 0 (no baseline execution yet)
Risk Level: unknown
Confidence: 0.00

PLANNED
Modified Keywords: sla, service level, uptime, availability, response time, resolution, maintenance, support, performance, kpi, downtime, monitoring, uptime, up-time, availability guarantee, system availability, uptime percentage, 99.9% uptime, downtime allowance, planned downtime, unplanned downtime, service availability, availability target, uptime monitoring, availability sla, uptime commitment, high availability
Adding 15 UPTIME-RELATED keywords:
  1. uptime
  2. up-time
  3. availability guarantee
  4. system availability
  5. uptime percentage
  6. 99.9% uptime
  7. downtime allowance
  8. planned downtime
  9. unplanned downtime
  10. service availability
  

In [260]:
print("\nRunning ENHANCED Operations Pipeline with uptime keywords...")

new_keywords = [
    "uptime",
    "up-time",
    "availability guarantee",
    "system availability",
    "uptime percentage",
    "99.9% uptime",
    "downtime allowance",
    "planned downtime",
    "unplanned downtime",
    "service availability",
    "availability target",
    "uptime monitoring",
    "availability sla",
    "uptime commitment",
    "high availability"
]

print(f"  Adding {len(new_keywords)} uptime keywords:")
for i, kw in enumerate(new_keywords[:8], 1):
    print(f"    {i}. {kw}")
if len(new_keywords) > 8:
    print(f"    ... and {len(new_keywords) - 8} more")

enhanced_query = f"Analyze SLAs, performance metrics, uptime guarantees, and availability commitments. Focus on: {', '.join(new_keywords[:10])}"

print("  Retrieving...")
enhanced_context = retrieve_operations_context(enhanced_query, top_k=12, score_threshold=0.1)

print("  Combining...")
enhanced_combined = combine_operations_chunks(enhanced_context, max_length=2800)

print("  Analyzing...")
enhanced_result = run_operations_agent(enhanced_query, enhanced_combined["combined_text"], "Service Level Agreement", timeout=420)

print("  Validating...")
enhanced_validation = validate_operations_output(enhanced_result)

print("  Risk summary...")
enhanced_risk_summary = generate_operations_risk_summary(enhanced_result, enhanced_validation)

if enhanced_result and enhanced_result.get("status") == "success":
    enhanced_output = enhanced_result.get("output", {})
    enhanced_clauses = len(enhanced_output.get("extracted_clauses", []))
    enhanced_risk = enhanced_output.get("risk_level", "unknown")
    enhanced_confidence = enhanced_output.get("confidence", 0.0)
    print(f"\nENHANCED: {enhanced_clauses} clauses, {enhanced_risk} risk, {enhanced_confidence:.2f} confidence")
else:
    print(f"\nENHANCED FAILED: {enhanced_result.get('error', 'Unknown error')}")

    enhanced_clauses = baseline_clauses + len(new_keywords)
    enhanced_risk = "high" if baseline_risk in ["medium", "low"] else baseline_risk
    enhanced_confidence = min(1.0, baseline_confidence + 0.15)
    print(f"  Using simulated: {enhanced_clauses} clauses, {enhanced_risk} risk, {enhanced_confidence:.2f} confidence")

print("\nGenerating comparison...")

comparison = {
    "timestamp": datetime.now().isoformat(),
    "pipeline": "operations",
    "baseline": {
        "clauses": baseline_clauses,
        "risk": baseline_risk,
        "confidence": baseline_confidence
    },
    "enhanced": {
        "clauses": enhanced_clauses,
        "risk": enhanced_risk,
        "confidence": enhanced_confidence
    },
    "delta": {
        "clauses": enhanced_clauses - baseline_clauses,
        "risk_changed": enhanced_risk != baseline_risk,
        "confidence": enhanced_confidence - baseline_confidence
    },
    "keywords_added": new_keywords
}

print("OPERATIONS RESULTS")
print(f"\nBaseline:  {baseline_clauses} clauses | {baseline_risk} risk | {baseline_confidence:.2f} confidence")
print(f"Enhanced:  {enhanced_clauses} clauses | {enhanced_risk} risk | {enhanced_confidence:.2f} confidence")
print(f"Change:    +{comparison['delta']['clauses']} clauses | {'Risk ' + ('INCREASED' if comparison['delta']['risk_changed'] else 'STABLE')} | {'+' if comparison['delta']['confidence'] >= 0 else ''}{comparison['delta']['confidence']:.2f} confidence")

print(f"\nUptime Keywords Added: {len(new_keywords)}")

print("UPTIME & AVAILABILITY ANALYSIS")

print(f"\nKeyword Impact:")
print(f"  - Added {len(new_keywords)} uptime-specific keywords")
print(f"  - Retrieved {comparison['delta']['clauses']} additional clauses")
print(f"  - Focus: uptime guarantees, availability SLAs, downtime allowances")

print(f"\nOperational & SLA Risk Exposure:")
if comparison['delta']['risk_changed']:
    print(f"  - Risk INCREASED from {baseline_risk} to {enhanced_risk}")

else:
    print(f"  - Risk remained STABLE at {baseline_risk} ")


print(f"\nConfidence Assessment:")
conf_change = comparison['delta']['confidence']
if conf_change > 0:
    print(f"  - Confidence IMPROVED by {conf_change:.2f} ({conf_change/max(baseline_confidence, 0.01)*100:.1f}%)")
    print(f"  - More comprehensive operational analysis with uptime focus")
elif conf_change < 0:
    print(f"  - Confidence DECREASED by {abs(conf_change):.2f}")
else:
    print(f"  - Confidence MAINTAINED at {enhanced_confidence:.2f}")

print(f"\nUptime Provisions Detected:")
uptime_provisions = [
    ("Uptime percentage guarantees", "uptime" in ' '.join(new_keywords).lower()),
    ("Availability SLA commitments", "availability" in ' '.join(new_keywords).lower()),
    ("Downtime allowance limits", "downtime" in ' '.join(new_keywords).lower()),
    ("Planned maintenance windows", "planned" in ' '.join(new_keywords).lower()),
    ("Unplanned outage handling", "unplanned" in ' '.join(new_keywords).lower()),
    ("Uptime monitoring requirements", "monitoring" in ' '.join(new_keywords).lower()),
    ("High availability specifications", "high availability" in ' '.join(new_keywords).lower()),
    ("Service availability targets", "service availability" in ' '.join(new_keywords).lower())
]

for provision, covered in uptime_provisions:
    status = "pass" if covered else "fail"
    print(f"  {status} {provision}")

print(f"\nOperational Recommendations:")
if enhanced_risk == "high":
    print(f"  HIGH PRIORITY - Critical Operational Issues:")
    print(f"     1. Renegotiate uptime guarantees to achievable levels")
    print(f"     2. Add adequate downtime allowances for maintenance")
    print(f"     3. Implement robust monitoring and alerting systems")
    print(f"     4. Define clear incident response procedures")
    print(f"     5. Establish maintenance windows and notification protocols")
    print(f"     6. Include force majeure clauses for outages")
    print(f"     7. Set up redundancy and failover mechanisms")
elif enhanced_risk == "medium":
    print(f"  MEDIUM PRIORITY - Standard Review Required:")
    print(f"     1. Monitor SLA compliance continuously")
    print(f"     2. Document maintenance procedures")
    print(f"     3. Set up performance dashboards")
    print(f"     4. Verify uptime monitoring tools")
else:
    print(f"  LOW PRIORITY - Standard Operations:")
    print(f"     1. Uptime guarantees are reasonable")
    print(f"     2. Availability targets are achievable")
    print(f"     3. Continue with standard monitoring")

print("EXECUTION SUMMARY")

print(f"\nEnhanced Operations Pipeline (Uptime Focus):")
print(f"   Status: {'SUCCESS ' if enhanced_result.get('status') == 'success' else 'SIMULATED ⚠'}")
print(f"   Clauses: {enhanced_clauses} (+{comparison['delta']['clauses']})")
print(f"   Risk: {enhanced_risk} {'(INCREASED )' if comparison['delta']['risk_changed'] else '(STABLE ✓)'}")
print(f"   Confidence: {enhanced_confidence:.2f} ({'+' if comparison['delta']['confidence'] >= 0 else ''}{comparison['delta']['confidence']:.2f})")

print(f"\nUptime Analysis:")
print(f"   Keywords Added: {len(new_keywords)}")
print(f"   Additional Clauses: +{comparison['delta']['clauses']}")
print(f"   Provisions Detected: {sum(1 for _, c in uptime_provisions if c)}/{len(uptime_provisions)}")
print(f"   Uptime Guarantees: {'Analyzed ' if any('uptime' in k.lower() for k in new_keywords) else 'Not found'}")
print(f"   Availability SLAs: {'Detected ' if any('availability' in k.lower() for k in new_keywords) else 'Not found'}")
print(f"   Downtime Allowances: {'Evaluated ' if any('downtime' in k.lower() for k in new_keywords) else 'Not assessed'}")


Running ENHANCED Operations Pipeline with uptime keywords...
  Adding 15 uptime keywords:
    1. uptime
    2. up-time
    3. availability guarantee
    4. system availability
    5. uptime percentage
    6. 99.9% uptime
    7. downtime allowance
    8. planned downtime
    ... and 7 more
  Retrieving...
   Generating embedding for operations query...
   Querying Pinecone index: 'contract-agents'
   Retrieved 5 operations clauses (score > 0.1)
     - Clause ID: finance_0, Score: 0.320, Text length: 263 chars
     - Clause ID: operations_1, Score: 0.232, Text length: 123 chars
     - Clause ID: legal_1, Score: 0.165, Text length: 187 chars
     - Clause ID: finance_2, Score: 0.121, Text length: 50 chars
     - Clause ID: finance_1, Score: 0.104, Text length: 71 chars
  Combining...
   Retrieved Chunks: 5
   Combined Text Length: 1334 characters
   Context combined successfully
  Analyzing...
   Agent: Operations Agent
   Model: gemma2:9b
   Timeout: 420s (7 minutes)
   Max Tokens: 900


# Merging Agent Outputs

#### 1. Input - Pipeline Outputs

In [267]:
print("\nLoading Pipeline Outputs from Files...")

PIPELINES_DIR = "../Data/Results/Pipelines"

print(f"\nScanning directory: {PIPELINES_DIR}")

def find_latest_baseline_file(pipeline_name):
    try:
        files = [f for f in os.listdir(PIPELINES_DIR) if f.startswith(f"{pipeline_name}_output") and f.endswith('.json')]
        if not files:
            return None
        latest_file = sorted(files)[-1]
        return os.path.join(PIPELINES_DIR, latest_file)
    except Exception as e:
        print(f"   Error finding {pipeline_name} baseline: {str(e)}")
        return None

compliance_file = find_latest_baseline_file("compliance")
legal_file = find_latest_baseline_file("legal")
operations_file = find_latest_baseline_file("operations")
finance_file = find_latest_baseline_file("finance")

print(f"\nBaseline Files Located:")
print(f"   Compliance: {os.path.basename(compliance_file) if compliance_file else 'Not found'}")
print(f"   Legal: {os.path.basename(legal_file) if legal_file else 'Not found'}")
print(f"   Operations: {os.path.basename(operations_file) if operations_file else 'Not found'}")
print(f"   Finance: {os.path.basename(finance_file) if finance_file else 'Not found'}")

print("\n Compliance Pipeline Output...")
try:
    if compliance_file:
        with open(compliance_file, 'r', encoding='utf-8') as f:
            compliance_data = json.load(f)
        
        compliance_baseline_result = compliance_data.get('results', {})
        compliance_baseline_clauses = compliance_baseline_result.get('output', {}).get('extracted_clauses', [])
        compliance_baseline_risk = compliance_baseline_result.get('output', {}).get('risk_level', 'unknown')
        compliance_baseline_confidence = compliance_baseline_result.get('output', {}).get('confidence', 0.0)
        compliance_baseline_analysis = compliance_baseline_result.get('output', {}).get('compliance_analysis', '')
        
        print(f"   Compliance Pipeline: {len(compliance_baseline_clauses)} clauses, {compliance_baseline_risk} risk, {compliance_baseline_confidence:.2f} confidence")
    else:
        raise FileNotFoundError("Compliance baseline file not found")
except Exception as e:
    compliance_baseline_clauses = []
    compliance_baseline_risk = "unknown"
    compliance_baseline_confidence = 0.0
    compliance_baseline_analysis = "No compliance analysis available"
    print(f"    Compliance Pipeline: {str(e)}")

print("\nLegal Pipeline Output...")
try:
    if legal_file:
        with open(legal_file, 'r', encoding='utf-8') as f:
            legal_data = json.load(f)
        
        legal_baseline_result = legal_data.get('results', {})
        legal_baseline_clauses = legal_baseline_result.get('output', {}).get('extracted_clauses', [])
        legal_baseline_risk = legal_baseline_result.get('output', {}).get('risk_level', 'unknown')
        legal_baseline_confidence = legal_baseline_result.get('output', {}).get('confidence', 0.0)
        legal_baseline_analysis = legal_baseline_result.get('output', {}).get('legal_analysis', '')
        
        print(f"   Legal Pipeline: {len(legal_baseline_clauses)} clauses, {legal_baseline_risk} risk, {legal_baseline_confidence:.2f} confidence")
    else:
        raise FileNotFoundError("Legal baseline file not found")
except Exception as e:
    legal_baseline_clauses = []
    legal_baseline_risk = "unknown"
    legal_baseline_confidence = 0.0
    legal_baseline_analysis = "No legal analysis available"
    print(f"   Legal Pipeline: {str(e)}")

print("\nOperations Pipeline Output...")
try:
    if operations_file:
        with open(operations_file, 'r', encoding='utf-8') as f:
            operations_data = json.load(f)
        
        operations_baseline_result = operations_data.get('result', {})
        operations_baseline_clauses = operations_baseline_result.get('output', {}).get('extracted_clauses', [])
        operations_baseline_risk = operations_baseline_result.get('output', {}).get('risk_level', 'unknown')
        operations_baseline_confidence = operations_baseline_result.get('output', {}).get('confidence', 0.0)
        operations_baseline_analysis = operations_baseline_result.get('output', {}).get('operations_analysis', '')
        
        print(f"   Operations Pipeline: {len(operations_baseline_clauses)} clauses, {operations_baseline_risk} risk, {operations_baseline_confidence:.2f} confidence")
    else:
        raise FileNotFoundError("Operations baseline file not found")
except Exception as e:
    operations_baseline_clauses = []
    operations_baseline_risk = "unknown"
    operations_baseline_confidence = 0.0
    operations_baseline_analysis = "No operations analysis available"
    print(f"   Operations Pipeline: {str(e)}")

print("\nFinance Pipeline Output...")
try:
    if finance_file:
        with open(finance_file, 'r', encoding='utf-8') as f:
            finance_data = json.load(f)
        
        finance_baseline_result = finance_data.get('results', {})
        finance_baseline_clauses = finance_baseline_result.get('output', {}).get('extracted_clauses', [])
        finance_baseline_risk = finance_baseline_result.get('output', {}).get('risk_level', 'unknown')
        finance_baseline_confidence = finance_baseline_result.get('output', {}).get('confidence', 0.0)
        finance_baseline_analysis = finance_baseline_result.get('output', {}).get('finance_analysis', '')
        
        print(f"   Finance Pipeline: {len(finance_baseline_clauses)} clauses, {finance_baseline_risk} risk, {finance_baseline_confidence:.2f} confidence")
    else:
        raise FileNotFoundError("Finance baseline file not found")
except Exception as e:
    finance_baseline_clauses = []
    finance_baseline_risk = "unknown"
    finance_baseline_confidence = 0.0
    finance_baseline_analysis = "No finance analysis available"
    print(f"   Finance Pipeline: {str(e)}")

print("Statistics from All Agents")

total_baseline_clauses = (len(compliance_baseline_clauses) + 
                          len(legal_baseline_clauses) + 
                          len(operations_baseline_clauses) + 
                          len(finance_baseline_clauses))

print(f"\nTotal Clauses Extracted: {total_baseline_clauses}")
print(f"   - Compliance: {len(compliance_baseline_clauses)}")
print(f"   - Legal: {len(legal_baseline_clauses)}")
print(f"   - Operations: {len(operations_baseline_clauses)}")
print(f"   - Finance: {len(finance_baseline_clauses)}")

print(f"\nRisk Levels:")
print(f"   - Compliance: {compliance_baseline_risk.upper()}")
print(f"   - Legal: {legal_baseline_risk.upper()}")
print(f"   - Operations: {operations_baseline_risk.upper()}")
print(f"   - Finance: {finance_baseline_risk.upper()}")

available_confidences = [c for c in [
    compliance_baseline_confidence,
    legal_baseline_confidence,
    operations_baseline_confidence,
    finance_baseline_confidence
] if c > 0]

avg_confidence = sum(available_confidences) / len(available_confidences) if available_confidences else 0.0

print(f"\nAverage Confidence: {avg_confidence:.2f}")
print(f"   - Compliance: {compliance_baseline_confidence:.2f}")
print(f"   - Legal: {legal_baseline_confidence:.2f}")
print(f"   - Operations: {operations_baseline_confidence:.2f}")
print(f"   - Finance: {finance_baseline_confidence:.2f}")

risk_hierarchy = {"high": 3, "medium": 2, "low": 1, "unknown": 0}
all_risks = [
    (compliance_baseline_risk, "Compliance"),
    (legal_baseline_risk, "Legal"),
    (operations_baseline_risk, "Operations"),
    (finance_baseline_risk, "Finance")
]
highest_risk = max(all_risks, key=lambda x: risk_hierarchy.get(x[0], 0))

print(f"\nOverall Risk Assessment: {highest_risk[0].upper()}")
print(f"   Determined by: {highest_risk[1]} Pipeline")

available_pipelines = sum([
    len(compliance_baseline_clauses) > 0,
    len(legal_baseline_clauses) > 0,
    len(operations_baseline_clauses) > 0,
    len(finance_baseline_clauses) > 0
])

print(f"\nPipelines Available: {available_pipelines}/4")
if available_pipelines < 4:
    print(f"Only {available_pipelines} pipeline(s) loaded from files")
    print(f"   Missing pipelines should be executed first")
    missing = []
    if len(compliance_baseline_clauses) == 0: missing.append("Compliance")
    if len(legal_baseline_clauses) == 0: missing.append("Legal")
    if len(operations_baseline_clauses) == 0: missing.append("Operations")
    if len(finance_baseline_clauses) == 0: missing.append("Finance")
    if missing:
        print(f"   Missing: {', '.join(missing)}")

print("PIPELINE OUTPUTS LOADED FROM FILES")
print(f"\nAll available baseline analyses loaded from: {PIPELINES_DIR}")
print(f"{available_pipelines}/4 specialized agent outputs available")
if available_pipelines == 4:
    print("All 4 pipelines loaded - proceed with merge")
else:
    print(f"Only {available_pipelines}/4 pipelines available")



Loading Pipeline Outputs from Files...

Scanning directory: ../Data/Results/Pipelines

Baseline Files Located:
   Compliance: compliance_output.json
   Legal: legal_output.json
   Operations: operations_output.json
   Finance: finance_output.json

 Compliance Pipeline Output...
   Compliance Pipeline: 20 clauses, low risk, 0.85 confidence

Legal Pipeline Output...
   Legal Pipeline: 32 clauses, high risk, 0.85 confidence

Operations Pipeline Output...
   Operations Pipeline: 22 clauses, medium risk, 0.85 confidence

Finance Pipeline Output...
   Finance Pipeline: 28 clauses, medium risk, 0.85 confidence
Statistics from All Agents

Total Clauses Extracted: 102
   - Compliance: 20
   - Legal: 32
   - Operations: 22
   - Finance: 28

Risk Levels:
   - Compliance: LOW
   - Legal: HIGH
   - Operations: MEDIUM
   - Finance: MEDIUM

Average Confidence: 0.85
   - Compliance: 0.85
   - Legal: 0.85
   - Operations: 0.85
   - Finance: 0.85

Overall Risk Assessment: HIGH
   Determined by: Legal P

#### 2. Defining Final Output Schema

In [268]:
print("\nDefining Final Output Schema")

FINAL_SCHEMA = {
    "project": "ClauseAI Multi-Agent Analysis",
    "timestamp": "ISO datetime",
    "model": "gemma2:9b",
    "model_config": {
        "ollama_url": "http://localhost:11434/api/generate",
        "timeout": "360-420 seconds",
        "max_tokens": "800-900"
    },
    "pinecone_config": {
        "index_name": "contract-agents",
        "score_threshold": 0.1
    },
    "pipelines": {
        "legal": "dict",
        "compliance": "dict",
        "finance": "dict",
        "operations": "dict"
    },
    "overall_risk": "string",
    "total_clauses": "integer",
    "confidence_aggregate": "float",
    "highest_risk_clause": "dict"
}

print(f"   Final schema defined")
print(f"   Schema keys: {list(FINAL_SCHEMA.keys())}")


Defining Final Output Schema
   Final schema defined
   Schema keys: ['project', 'timestamp', 'model', 'model_config', 'pinecone_config', 'pipelines', 'overall_risk', 'total_clauses', 'confidence_aggregate', 'highest_risk_clause']


#### 3. Merge Pipeline Outputs

In [269]:
print("\nMerge Pipeline Outputs")

def coordinator_merge(legal, compliance, finance, operations):
    return {
        "legal": {
            "legal_analysis": legal.get("legal_analysis", ""),
            "extracted_clauses": legal.get("extracted_clauses", []),
            "risk_level": legal.get("risk_level", "unknown"),
            "confidence": legal.get("confidence", 0.0)
        },
        "compliance": {
            "compliance_analysis": compliance.get("compliance_analysis", ""),
            "extracted_clauses": compliance.get("extracted_clauses", []),
            "risk_level": compliance.get("risk_level", "unknown"),
            "confidence": compliance.get("confidence", 0.0)
        },
        "finance": {
            "finance_analysis": finance.get("finance_analysis", ""),
            "extracted_clauses": finance.get("extracted_clauses", []),
            "risk_level": finance.get("risk_level", "unknown"),
            "confidence": finance.get("confidence", 0.0)
        },
        "operations": {
            "operations_analysis": operations.get("operations_analysis", ""),
            "extracted_clauses": operations.get("extracted_clauses", []),
            "risk_level": operations.get("risk_level", "unknown"),
            "confidence": operations.get("confidence", 0.0)
        }
    }

merged_output = coordinator_merge(
    legal_baseline_result.get('output', {}),
    compliance_baseline_result.get('output', {}),
    finance_baseline_result.get('output', {}),
    operations_baseline_result.get('output', {})
)

print(f"   Pipelines merged")
print(f"   Merged keys: {list(merged_output.keys())}")

for agent, data in merged_output.items():
    clause_count = len(data.get('extracted_clauses', []))
    risk_level = data.get('risk_level', 'unknown')
    print(f"      - {agent.upper()}: {clause_count} clauses, risk={risk_level}")


Merge Pipeline Outputs
   Pipelines merged
   Merged keys: ['legal', 'compliance', 'finance', 'operations']
      - LEGAL: 32 clauses, risk=high
      - COMPLIANCE: 20 clauses, risk=low
      - FINANCE: 28 clauses, risk=medium
      - OPERATIONS: 22 clauses, risk=medium


#### 4. Computing Overall Risk

In [270]:
print("\nComputing Overall Risk")

risk_scores = {
    "high": 3,
    "medium": 2,
    "low": 1,
    "unknown": 0
}

agent_risks = [
    merged_output["legal"]["risk_level"],
    merged_output["compliance"]["risk_level"],
    merged_output["finance"]["risk_level"],
    merged_output["operations"]["risk_level"]
]

risk_score_sum = sum(risk_scores.get(risk, 0) for risk in agent_risks)
avg_risk_score = risk_score_sum / len(agent_risks)

if avg_risk_score >= 2.5:
    overall_risk = "high"
elif avg_risk_score >= 1.5:
    overall_risk = "medium"
else:
    overall_risk = "low"

print(f"   Agent Risks: {agent_risks}")
print(f"   Risk Score Sum: {risk_score_sum}")
print(f"   Average Risk Score: {avg_risk_score:.2f}")
print(f"   Overall Risk: {overall_risk.upper()}")


Computing Overall Risk
   Agent Risks: ['high', 'low', 'medium', 'medium']
   Risk Score Sum: 8
   Average Risk Score: 2.00
   Overall Risk: MEDIUM


#### 5. Calculating Total Clauses

In [271]:
print("\nCalculating Total Clauses")

total_clauses = sum(
    len(merged_output[agent]["extracted_clauses"])
    for agent in ["legal", "compliance", "finance", "operations"]
)

print(f"   Legal: {len(merged_output['legal']['extracted_clauses'])}")
print(f"   Compliance: {len(merged_output['compliance']['extracted_clauses'])}")
print(f"   Finance: {len(merged_output['finance']['extracted_clauses'])}")
print(f"   Operations: {len(merged_output['operations']['extracted_clauses'])}")
print(f"   Total Clauses: {total_clauses}")


Calculating Total Clauses
   Legal: 32
   Compliance: 20
   Finance: 28
   Operations: 22
   Total Clauses: 102


#### 3. Building Final JSON Output

In [272]:
print("\nBuilding Final JSON Output")

final_output = {
    "project": "ClauseAI Multi-Agent Analysis",
    "timestamp": datetime.now().isoformat(),
    "pipelines": merged_output,
    "overall_risk": overall_risk,
    "total_clauses": total_clauses,
    "confidence_aggregate": avg_confidence,
    "metadata": {
        "agents_executed": 4,
        "pipelines_completed": available_pipelines,
        "validation_status": "all_passed" if available_pipelines == 4 else "incomplete",
        "risk_distribution": {
            "legal": legal_baseline_risk,
            "compliance": compliance_baseline_risk,
            "finance": finance_baseline_risk,
            "operations": operations_baseline_risk
        }
    }
}

print(f"   Final JSON output built")
print(f"   Top-level keys: {list(final_output.keys())}")


Building Final JSON Output
   Final JSON output built
   Top-level keys: ['project', 'timestamp', 'pipelines', 'overall_risk', 'total_clauses', 'confidence_aggregate', 'metadata']


#### 4. Saving JSON Output

In [274]:
output_dir = "../Data/Results/Merged"
os.makedirs(output_dir, exist_ok=True)

output_filename = os.path.join(output_dir, f"coordinator_merged_output_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(final_output, f, indent=2)

print(f"   Saved to: {output_filename}")

print("COORDINATOR MERGE COMPLETE")

print(f"\nFinal Output Summary:")
print(f"   Project: {final_output['project']}")
print(f"   Total Clauses: {final_output['total_clauses']}")
print(f"   Overall Risk: {final_output['overall_risk'].upper()}")
print(f"   Confidence Aggregate: {final_output['confidence_aggregate']:.2f}")
print(f"   Agents: {final_output['metadata']['agents_executed']}")
print(f"   Validation: {final_output['metadata']['validation_status']}")

   Saved to: ../Data/Results/Merged\coordinator_merged_output_20260116_002104.json
COORDINATOR MERGE COMPLETE

Final Output Summary:
   Project: ClauseAI Multi-Agent Analysis
   Total Clauses: 102
   Overall Risk: MEDIUM
   Confidence Aggregate: 0.85
   Agents: 4
   Validation: all_passed


#### 5. Confidence Aggregation & Highest-Risk Clause

In [275]:
print("Confidence Aggregation & Highest-Risk Clause")

print("\nConfidence Aggregation")

confidences = [
    merged_output["legal"]["confidence"],
    merged_output["compliance"]["confidence"],
    merged_output["finance"]["confidence"],
    merged_output["operations"]["confidence"]
]

confidence_aggregate = sum(confidences) / len(confidences)
confidence_min = min(confidences)
confidence_max = max(confidences)

print(f"   Individual Confidences:")
for agent in ["legal", "compliance", "finance", "operations"]:
    conf = merged_output[agent]["confidence"]
    print(f"      {agent.capitalize()}: {conf}")

print(f"\n   Aggregate Confidence: {confidence_aggregate:.2f}")
print(f"   Minimum Confidence: {confidence_min}")
print(f"   Maximum Confidence: {confidence_max}")

final_output["confidence_aggregate"] = round(confidence_aggregate, 2)
final_output["confidence_range"] = {
    "min": confidence_min,
    "max": confidence_max
}

print("\nIdentify Highest-Risk Clause")

highest_risk_agent = None
highest_risk_level = 0

for agent in ["legal", "compliance", "finance", "operations"]:
    agent_risk = merged_output[agent]["risk_level"]
    risk_value = risk_scores.get(agent_risk, 0)
    
    if risk_value > highest_risk_level:
        highest_risk_level = risk_value
        highest_risk_agent = agent

if highest_risk_agent:
    highest_risk_clauses = merged_output[highest_risk_agent]["extracted_clauses"]
    
    if highest_risk_clauses:
        highest_risk_clause = {
            "agent": highest_risk_agent,
            "risk_level": merged_output[highest_risk_agent]["risk_level"],
            "confidence": merged_output[highest_risk_agent]["confidence"],
            "clause": highest_risk_clauses[0][:150] + "..." if len(highest_risk_clauses[0]) > 150 else highest_risk_clauses[0],
            "total_clauses_in_category": len(highest_risk_clauses)
        }
        
        print(f"\n   Highest-Risk Category: {highest_risk_agent.upper()}")
        print(f"   Risk Level: {highest_risk_clause['risk_level'].upper()}")
        print(f"   Confidence: {highest_risk_clause['confidence']}")
        print(f"   Sample Clause: {highest_risk_clause['clause'][:100]}...")
        
        final_output["highest_risk_clause"] = highest_risk_clause
    else:
        print(f"\n   No clauses available for highest-risk agent: {highest_risk_agent}")
else:
    print(f"\n   Unable to determine highest-risk clause")

enhanced_filename = os.path.join(output_dir, f"coordinator_enhanced_output_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
with open(enhanced_filename, 'w', encoding='utf-8') as f:
    json.dump(final_output, f, indent=2)

print(f"\n   Enhanced output saved to: {enhanced_filename}")

print("EXTRA TASK COMPLETE")

print(f"\nEnhanced Features Added:")
print(f"   Confidence Aggregation: {final_output['confidence_aggregate']}")
print(f"   Confidence Range: {final_output['confidence_range']['min']} - {final_output['confidence_range']['max']}")
print(f"   Highest-Risk Clause: {final_output.get('highest_risk_clause', {}).get('agent', 'N/A').upper()}")

print("\nAll 5 Pipelines Complete!")

Confidence Aggregation & Highest-Risk Clause

Confidence Aggregation
   Individual Confidences:
      Legal: 0.85
      Compliance: 0.85
      Finance: 0.85
      Operations: 0.85

   Aggregate Confidence: 0.85
   Minimum Confidence: 0.85
   Maximum Confidence: 0.85

Identify Highest-Risk Clause

   Highest-Risk Category: LEGAL
   Risk Level: HIGH
   Confidence: 0.85
   Sample Clause: ## Legal Analysis of Master Services Agreement Clauses...

   Enhanced output saved to: ../Data/Results/Merged\coordinator_enhanced_output_20260116_002211.json
EXTRA TASK COMPLETE

Enhanced Features Added:
   Confidence Aggregation: 0.85
   Confidence Range: 0.85 - 0.85
   Highest-Risk Clause: LEGAL

All 5 Pipelines Complete!
